In [ ]:
!pip3 install boto3
!pip3 install pyathena
!pip3 install awswrangler
!pip install xgboost
!pip install kneed
!pip3 install awscli
!pip3 install s3fs
!pip install fsspec

In [8]:
from IPython.core.display import display, HTML

# Adjust notebook to full width
display(HTML("""
<style>
    .container {
        width: 100% !important;
    }
</style>
"""))

In [9]:
# Parameters
team = "fpl_ds_onboarding"
trino = True


In [10]:
from fplathena.init_client import FPLAthena
fplathena = FPLAthena(team)


             _____ ____  _        _   _   _                      
            |  ___|  _ \| |      / \ | |_| |__   ___ _ __   __ _ 
            | |_  | |_) | |     / _ \| __| '_ \ / _ \ '_ \ / _` |
            |  _| |  __/| |___ / ___ \ |_| | | |  __/ | | | (_| |
            |_|   |_|   |_____/_/   \_\__|_| |_|\___|_| |_|\__,_|
        
Open the following URL in browser for the external authentication:
https://trino-critical.eks.getonecard.app/oauth2/token/initiate/5ea5723f5020d603fbea6f4c3a6a015cdf8e05ace13e4d8034838d2ee295fd68


In [12]:
import os
import sys
import time,datetime
import numpy as np
import pandas as pd
import re
from datetime import date
from datetime import datetime
import pandas as pd
import awswrangler as wr

import os
import copy
import textwrap as tw
from datetime import date
from functools import reduce
import pandas as pd
from pandas.tseries.offsets import MonthEnd
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

from pandas.tseries.offsets import MonthEnd

from datetime import datetime
from datetime import timedelta
from datetime import date

import pandas as pd
from sklearn.preprocessing import StandardScaler
import logging
import gc
# import shap

In [13]:
query=fplathena.get_query_result

In [7]:
# !aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_score_simulator_model_oot_data_v1 --recursive

In [8]:
# fplathena.get_query_result(f'''
#         DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_score_simulator_model_oot_data_v1
# ''')

# Bureau accounts data

In [9]:
customer_no=2565455133978658509
y=2025
m=2

In [10]:
df1=query(f"""
SELECT distinct customer_no,
        credit_limit,
        emi_amount,
        account_designator_symbol,
        --CASE WHEN SUBSTRING(pay_status_history,1,1)=',' THEN '0'||pay_status_history ELSE pay_status_history END AS pay_status_history,
        account_number,
        account_type_symbol,
        amount_past_due,
        creditor_name,
        CAST(current_balance AS INT) AS current_balance,
        date_opened,
        date_closed,
        date_reported,
        high_balance,
        interest_rate,
        settlement_amount,
        term_months,
        written_off_amt_total,
        written_off_principal,
        creation_date,
        risk_score,
        app_date,
        utilisation,
        sanctioned_amount,
        active_inactive,
        slice(CAST(repeat(-1, CAST(diff_rep_pull AS INT)) AS ARRAY<INT>) || CAST(SPLIT(replaced_string, ',') AS ARRAY<INT>) || CAST(repeat(-1, 36) AS ARRAY<INT>),1,36) AS final_array
FROM(
SELECT *, (YEAR(DATE(app_date))-YEAR(DATE(date_opened)))*12+(MONTH(DATE(app_date))-MONTH(DATE(date_opened))) AS diff_sin_open, 
(YEAR(DATE(app_date))-YEAR(DATE(date_reported)))*12+(MONTH(DATE(app_date))-MONTH(DATE(date_reported))) AS diff_rep_pull,
REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(
SUBSTRING(pay_status_history, 1, length(pay_status_history)-1), 'XXX', '-1'), 'DBT', '180'),'LSS', '180'), 'SUB', '90'), 'SMA', '60'), 'STD', '0'), ' XXX', '-1'), '   ', '0'), ',', ',0'), '0-', '-') AS replaced_string
FROM(
SELECT distinct customer_no,
        credit_limit,
        emi_amount,
        account_designator_symbol,
        CASE WHEN SUBSTRING(pay_status_history,1,1)=',' THEN '0'||pay_status_history ELSE pay_status_history END AS pay_status_history,
        account_number,
        account_type_symbol,
        amount_past_due,
        creditor_name,
        current_balance,
        date_opened,
        date_closed,
        date_reported,
        high_balance,
        interest_rate,
        settlement_amount,
        term_months,
        written_off_amt_total,
        written_off_principal,
        creation_date,
        gender,
        date_of_birth,
        risk_score,
GREATEST(DATE(DATE(DATE_TRUNC('month', DATE_ADD('month', -2, MAX(DATE(creation_date)) over(partition by customer_no, DATE(creation_date)))))-interval '1' day), 
MAX(DATE(date_reported)) over(partition by customer_no, DATE(creation_date))) AS app_date,
loan_id, IF(GREATEST(high_balance, credit_limit, current_balance)!=0, CAST(current_balance AS DOUBLE)/GREATEST(high_balance, credit_limit, current_balance), NULL) AS utilisation,
GREATEST(high_balance, credit_limit, current_balance) AS sanctioned_amount,
case when date_closed is null and date_diff('month', date_reported, date_trunc('month', creation_date)) < 12 then 1
else 0 end as active_inactive
FROM fpl_bureau.score_cibil_report
WHERE customer_no={customer_no}
AND year={y} AND month={m}
)
)
""")

In [11]:
df1['acct_identifier']=df1['creditor_name']+df1['account_number']


In [12]:
df1

,customer_no,credit_limit,emi_amount,account_designator_symbol,account_number,account_type_symbol,amount_past_due,creditor_name,current_balance,date_opened,date_closed,date_reported,high_balance,interest_rate,settlement_amount,term_months,written_off_amt_total,written_off_principal,creation_date,risk_score,app_date,utilisation,sanctioned_amount,active_inactive,final_array,acct_identifier
0,2565455133978658509,-1,-1,1,######1231,7,778,KOTAK BANK,29778,2024-07-31,None,2025-01-15,29000,-1.00,-1,6,-1,-1,2025-02-02,650,2025-01-16,1.000000,29778,1,"[42, 27, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1...",KOTAK BANK######1231
1,2565455133978658509,-1,-1,1,###########9529,5,0,YES BANK,0,2019-12-31,2021-04-01,2021-04-30,1122661,11.99,-1,70,-1,-1,2025-02-02,650,2025-01-16,0.000000,1122661,0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",YES BANK###########9529
2,2565455133978658509,-1,22744,1,########5168,5,21736,KOTAK BANK,429851,2020-12-15,None,2025-01-15,1036363,-1.00,-1,69,-1,-1,2025-02-02,650,2025-01-16,0.414769,1036363,1,"[11, 0, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0...",KOTAK BANK########5168
3,2565455133978658509,-1,12172,1,################7867,5,12172,INCRED FINANCE,145825,2020-08-19,None,2025-01-15,428577,-1.00,-1,-1,-1,-1,2025-02-02,650,2025-01-16,0.340254,428577,1,"[11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",INCRED FINANCE################7867
4,2565455133978658509,-1,-1,1,##########8943,5,58835,SMICC,269065,2019-12-31,None,2025-01-15,749889,-1.00,-1,-1,-1,-1,2025-02-02,650,2025-01-16,0.358806,749889,1,"[73, 58, 58, 89, 58, 59, 58, 58, 59, 58, 28, 2...",SMICC##########8943
5,2565455133978658509,-1,-1,1,##########1662,5,0,BAJAJ FIN LTD,0,2019-08-31,2019-12-19,2021-03-31,595000,-1.00,-1,-1,-1,-1,2025-02-02,650,2025-01-16,0.000000,595000,0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",BAJAJ FIN LTD##########1662
6,2565455133978658509,-1,-1,1,############6040,5,-1,ICICI BANK,0,2018-07-09,2022-01-13,2022-01-31,500000,-1.00,181500,-1,155887,113285,2025-02-02,650,2025-01-16,0.000000,500000,0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",ICICI BANK############6040
7,2565455133978658509,16000,-1,1,###############8509,10,0,SBMBKINDIA,15170,2021-07-31,None,2025-01-15,15872,-1.00,-1,-1,-1,-1,2025-02-02,650,2025-01-16,0.948125,16000,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",SBMBKINDIA###############8509
8,2565455133978658509,-1,-1,1,##########9073,5,0,BAJAJ FIN LTD,0,2018-12-28,2019-08-31,2021-03-31,300000,-1.00,-1,-1,-1,-1,2025-02-02,650,2025-01-16,0.000000,300000,0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",BAJAJ FIN LTD##########9073
9,2565455133978658509,-1,11895,1,####8316,5,0,HDB,0,2019-12-31,2022-04-30,2023-06-30,500000,-1.00,330000,67,-1,-1,2025-02-02,650,2025-01-16,0.000000,500000,0,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...",HDB####8316


In [13]:
creation_date=df1['creation_date'][0]
creation_date_y=creation_date+timedelta(days=32)
creation_date_y=creation_date_y.strftime('%Y-%m-%d')

In [14]:
creation_date_y

'2025-03-06'

In [15]:
df1[df1['active_inactive']==1]

,customer_no,credit_limit,emi_amount,account_designator_symbol,account_number,account_type_symbol,amount_past_due,creditor_name,current_balance,date_opened,date_closed,date_reported,high_balance,interest_rate,settlement_amount,term_months,written_off_amt_total,written_off_principal,creation_date,risk_score,app_date,utilisation,sanctioned_amount,active_inactive,final_array,acct_identifier
0,2565455133978658509,-1,-1,1,######1231,7,778,KOTAK BANK,29778,2024-07-31,None,2025-01-15,29000,-1.00,-1,6,-1,-1,2025-02-02,650,2025-01-16,1.000000,29778,1,"[42, 27, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1...",KOTAK BANK######1231
2,2565455133978658509,-1,22744,1,########5168,5,21736,KOTAK BANK,429851,2020-12-15,None,2025-01-15,1036363,-1.00,-1,69,-1,-1,2025-02-02,650,2025-01-16,0.414769,1036363,1,"[11, 0, 26, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0...",KOTAK BANK########5168
3,2565455133978658509,-1,12172,1,################7867,5,12172,INCRED FINANCE,145825,2020-08-19,None,2025-01-15,428577,-1.00,-1,-1,-1,-1,2025-02-02,650,2025-01-16,0.340254,428577,1,"[11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",INCRED FINANCE################7867
4,2565455133978658509,-1,-1,1,##########8943,5,58835,SMICC,269065,2019-12-31,None,2025-01-15,749889,-1.00,-1,-1,-1,-1,2025-02-02,650,2025-01-16,0.358806,749889,1,"[73, 58, 58, 89, 58, 59, 58, 58, 59, 58, 28, 2...",SMICC##########8943
7,2565455133978658509,16000,-1,1,###############8509,10,0,SBMBKINDIA,15170,2021-07-31,None,2025-01-15,15872,-1.00,-1,-1,-1,-1,2025-02-02,650,2025-01-16,0.948125,16000,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",SBMBKINDIA###############8509
11,2565455133978658509,-1,11788,1,############8258,5,-1,ICICI BANK,102356,2019-12-30,None,2025-01-15,500000,-1.00,-1,69,-1,-1,2025-02-02,650,2025-01-16,0.204712,500000,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ICICI BANK############8258
12,2565455133978658509,-1,15586,1,####2153,5,0,IDFC FIRST BANK,364778,2019-12-30,None,2025-01-16,898129,-1.00,-1,87,-1,-1,2025-02-02,650,2025-01-16,0.406153,898129,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",IDFC FIRST BANK####2153
13,2565455133978658509,-1,-1,1,###########2578,5,60574,YES BANK,532324,2021-03-26,None,2025-01-15,1046912,11.99,-1,72,-1,-1,2025-02-02,650,2025-01-16,0.508471,1046912,1,"[73, 58, 58, 58, 58, 28, 28, 58, 28, 58, 28, 2...",YES BANK###########2578


In [16]:
 # Process the results as needed
res = wr.s3.to_parquet(
df=df1,
index=False,
path=f"s3://fpl-datalake-global/l2/Anugrah/Score_simulator/os_fow_temp_variables_creation_m1",
dataset=True,
database="fpl_ds_analytics",
table=f"os_fow_temp_variables_creation_m1",
mode="overwrite_partitions"
)

In [17]:
closed_accts=df1[df1['active_inactive']==0]['acct_identifier'].to_list()

In [18]:
active_accts=df1[df1['active_inactive']==1]['acct_identifier'].to_list()

In [19]:
active_accts

['KOTAK BANK######1231',
 'KOTAK BANK########5168',
 'INCRED FINANCE################7867',
 'SMICC##########8943',
 'SBMBKINDIA###############8509',
 'ICICI BANK############8258',
 'IDFC FIRST BANK####2153',
 'YES BANK###########2578']

In [20]:
if m==12:
    m_y=1
    y_y=y+1
else:
    m_y=m+1
    y_y=y

In [21]:
df2=query(f"""
SELECT distinct customer_no,
        credit_limit,
        emi_amount,
        account_designator_symbol,
        --CASE WHEN SUBSTRING(pay_status_history,1,1)=',' THEN '0'||pay_status_history ELSE pay_status_history END AS pay_status_history,
        account_number,
        account_type_symbol,
        amount_past_due,
        creditor_name,
        CAST(current_balance AS INT) AS current_balance,
        date_opened,
        date_closed,
        date_reported,
        high_balance,
        interest_rate,
        settlement_amount,
        term_months,
        written_off_amt_total,
        written_off_principal,
        creation_date,
        risk_score,
        app_date,
        utilisation,
        sanctioned_amount,
        active_inactive,
        slice(CAST(repeat(-1, CAST(diff_rep_pull AS INT)) AS ARRAY<INT>) || CAST(SPLIT(replaced_string, ',') AS ARRAY<INT>) || CAST(repeat(-1, 36) AS ARRAY<INT>),1,36) AS final_array
FROM(
SELECT *, (YEAR(DATE(app_date))-YEAR(DATE(date_opened)))*12+(MONTH(DATE(app_date))-MONTH(DATE(date_opened))) AS diff_sin_open, 
(YEAR(DATE(app_date))-YEAR(DATE(date_reported)))*12+(MONTH(DATE(app_date))-MONTH(DATE(date_reported))) AS diff_rep_pull,
REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(
SUBSTRING(pay_status_history, 1, length(pay_status_history)-1), 'XXX', '-1'), 'DBT', '180'),'LSS', '180'), 'SUB', '90'), 'SMA', '60'), 'STD', '0'), ' XXX', '-1'), '   ', '0'), ',', ',0'), '0-', '-') AS replaced_string
FROM(
SELECT distinct customer_no,
        credit_limit,
        emi_amount,
        account_designator_symbol,
        CASE WHEN SUBSTRING(pay_status_history,1,1)=',' THEN '0'||pay_status_history ELSE pay_status_history END AS pay_status_history,
        account_number,
        account_type_symbol,
        amount_past_due,
        creditor_name,
        current_balance,
        date_opened,
        date_closed,
        date_reported,
        high_balance,
        interest_rate,
        settlement_amount,
        term_months,
        written_off_amt_total,
        written_off_principal,
        creation_date,
        gender,
        date_of_birth,
        risk_score,
GREATEST(DATE(DATE(DATE_TRUNC('month', DATE_ADD('month', -2, MAX(DATE(creation_date)) over(partition by customer_no, DATE(creation_date)))))-interval '1' day), 
MAX(DATE(date_reported)) over(partition by customer_no, DATE(creation_date))) AS app_date,
loan_id, IF(GREATEST(high_balance, credit_limit, current_balance)!=0, CAST(current_balance AS DOUBLE)/GREATEST(high_balance, credit_limit, current_balance), NULL) AS utilisation,
GREATEST(high_balance, credit_limit, current_balance) AS sanctioned_amount,
case when date_closed is null and date_diff('month', date_reported, date_trunc('month', creation_date)) < 12 then 1
else 0 end as active_inactive
FROM fpl_bureau.score_cibil_report
WHERE customer_no={customer_no}
AND year={y_y} AND month={m_y}
)
)
""")

In [22]:
df2['acct_identifier']=df2['creditor_name']+df2['account_number']


In [23]:
 # Process the results as needed
res = wr.s3.to_parquet(
df=df2,
index=False,
path=f"s3://fpl-datalake-global/l2/Anugrah/Score_simulator/os_fow_temp_variables_creation_m2",
dataset=True,
database="fpl_ds_analytics",
table=f"os_fow_temp_variables_creation_m2",
mode="overwrite_partitions"
)

In [176]:
df2

,customer_no,pay_status_history_y,account_number_y,account_type_symbol_y,amount_past_due_y,creditor_name_y,current_balance_y,date_opened_y,date_closed_y,date_reported_y,app_date_y,high_balance_y,interest_rate_y,term_months_y,creation_date_y,risk_score_y,loan_id_y,diff_sin_open_y,diff_rep_pull_y,credit_limit_y,pay_hist_1_y,pay_hist_2_y,pay_hist_3_y,pay_hist_4_y,pay_hist_5_y,pay_hist_6_y,pay_hist_7_y,pay_hist_8_y,pay_hist_9_y,pay_hist_10_y,pay_hist_11_y,pay_hist_12_y,pay_hist_13_y,pay_hist_14_y,pay_hist_15_y,pay_hist_16_y,pay_hist_17_y,pay_hist_18_y,pay_hist_19_y,pay_hist_20_y,pay_hist_21_y,pay_hist_22_y,pay_hist_23_y,pay_hist_24_y,pay_hist_25_y,pay_hist_26_y,pay_hist_27_y,pay_hist_28_y,pay_hist_29_y,pay_hist_30_y,pay_hist_31_y,pay_hist_32_y,pay_hist_33_y,pay_hist_34_y,pay_hist_35_y,pay_hist_36_y,latest_payment_dpd_status_y,latest_payment_dpd_status2_y,latest_payment_dpd_status3_y,string_length_y,lender,category
0,2565455133978658509,"0,XXX,0,0,0,0,0,0,0,",################2993,5,0,INCRED,0,2019-12-30,2020-08-19,2020-08-31,2025-02-28,400000,-1.00,-1,2025-03-07,650,2565455133978658509_8815416908_###############...,62,54,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,0,9,INCRED,NBFC_2
1,2565455133978658509,"0,144,295,568,599,568,568,537,507,507,479,478,...",###########1194,5,0,AXIS BANK,0,2019-12-31,None,2023-11-30,2025-02-28,441000,-1.00,63,2025-03-07,650,2565455133978658509_8815416908_###########1194,62,15,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,144,295,568,599,568,568,537,507,507,479,478,447,417,417,387,356,356,356,325,295,0,144,295,36,AXIS BANK,PVT_1
2,2565455133978658509,"74,59,58,58,89,58,59,58,58,59,58,28,27,28,29,2...",##########8943,5,58797,SMICC,253309,2019-12-31,None,2025-02-15,2025-02-28,749889,-1.00,-1,2025-03-07,650,2565455133978658509_8815416908_##########8943,62,0,-1,74,59,58,58,89,58,59,58,58,59,58,28,27,28,29,27,28,27,28,28,27,29,27,28,25,28,28,27,28,27,27,27,26,28,26,27,74,59,58,36,None,None
3,2565455133978658509,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,27,...",################7867,5,0,INCRED FINANCE,126179,2020-08-19,None,2025-02-28,2025-02-28,428577,-1.00,-1,2025-03-07,650,2565455133978658509_8815416908_###############...,54,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,27,26,27,24,27,27,26,27,26,27,27,26,27,26,27,0,0,0,36,INCRED FINANCE,NBFC_2
4,2565455133978658509,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,",##########1662,5,0,BAJAJ FIN LTD,0,2019-08-31,2019-12-19,2021-03-31,2025-02-28,595000,-1.00,-1,2025-03-07,650,2565455133978658509_8815416908_##########1662,66,47,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,20,BAJAJ FIN LTD,NBFC_1_G
5,2565455133978658509,"0,0,0,0,STD,25,STD,265,235,205,175,145,115,85,...",############6040,5,-1,ICICI BANK,0,2018-07-09,2022-01-13,2022-01-31,2025-02-28,500000,-1.00,-1,2025-03-07,650,2565455133978658509_8815416908_############6040,79,37,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,36,ICICI BANK,PVT_1
6,2565455133978658509,"STD,0,0,0,0,0,0,0,0,0,0,0,STD,0,0,STD,STD,55,5...",############8258,5,12260,ICICI BANK,104065,2019-12-30,None,2025-02-15,2025-02-28,500000,-1.00,69,2025-03-07,650,2565455133978658509_8815416908_############8258,62,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,55,55,55,55,55,55,55,85,55,25,55,25,25,0,0,0,0,0,0,0,0,0,36,ICICI BANK,PVT_1
7,2565455133978658509,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",##########9073,5,0,BAJAJ FIN LTD,0,2018-12-28,2019-08-31,2021-03-31,2025-02-28,300000,-1.00,-1,2025-03-07,650,2565455133978658509_8815416908_##########9073,74,47,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,28,BAJAJ FIN LTD,NBFC_1_G
8,2565455133978658509,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",####2153,5,0,IDFC FIRST BANK,352991,2019-12-30,None,2025-02-28,2

# Enquiries raw data

In [24]:
df_enq=query(f"""
SELECT customer_no, inquiry_date, subscriber_name, inquiry_type, creation_date
FROM fpl_bureau.score_cibil_report_inquiry
WHERE customer_no={customer_no}
AND year={y} AND month={m}
""")

In [25]:
df_enq

,customer_no,inquiry_date,subscriber_name,inquiry_type,creation_date
0,2565455133978658509,2024-07-31,KOTAK BANK,0,2025-02-02
1,2565455133978658509,2022-04-05,KOTAK BANK,31,2025-02-02
2,2565455133978658509,2024-07-31,KOTAK BANK,5,2025-02-02
3,2565455133978658509,2024-04-30,BAJAJ FIN LTD,0,2025-02-02
4,2565455133978658509,2024-04-08,BAJAJ FIN LTD,0,2025-02-02
5,2565455133978658509,2024-11-13,BAJAJ FIN LTD,0,2025-02-02
6,2565455133978658509,2022-02-07,SBI CARD,10,2025-02-02
7,2565455133978658509,2025-01-21,KOTAK BANK,0,2025-02-02
8,2565455133978658509,2024-10-28,ICICI BANK,10,2025-02-02
9,2565455133978658509,2024-02-07,ICICI BANK,10,2025-02-02


In [26]:
res = wr.s3.to_parquet(
df=df_enq,
index=False,
path=f"s3://fpl-datalake-global/l2/Anugrah/Score_simulator/os_fow_inquiry_variables_creation_m1",
dataset=True,
database="fpl_ds_analytics",
table=f"os_fow_inquiry_variables_creation_m1",
mode="overwrite_partitions"
)

In [27]:
df_enq2=query(f"""
SELECT customer_no, inquiry_date, subscriber_name, inquiry_type, creation_date
FROM fpl_bureau.score_cibil_report_inquiry
WHERE customer_no={customer_no}
AND year={y_y} AND month={m_y}
""")

In [28]:
res = wr.s3.to_parquet(
df=df_enq2,
index=False,
path=f"s3://fpl-datalake-global/l2/Anugrah/Score_simulator/os_fow_inquiry_variables_creation_m2",
dataset=True,
database="fpl_ds_analytics",
table=f"os_fow_inquiry_variables_creation_m2",
mode="overwrite_partitions"
)

# M1 variables preparation

In [29]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v1 --recursive

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v1/20250407_112118_22165_iin5u_90990900-ae55-4e27-9788-27196487acf8


In [30]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v1
''')

Table Deleted!


0

In [31]:
df5=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v1
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v1'
)
AS

SELECT *,
final_array[1] AS pay_hist_1,
final_array[2] AS pay_hist_2,
final_array[3] AS pay_hist_3,
final_array[4] AS pay_hist_4,
final_array[5] AS pay_hist_5,
final_array[6] AS pay_hist_6,
final_array[7] AS pay_hist_7,
final_array[8] AS pay_hist_8,
final_array[9] AS pay_hist_9,
final_array[10] AS pay_hist_10,
final_array[11] AS pay_hist_11,
final_array[12] AS pay_hist_12,
final_array[13] AS pay_hist_13,
final_array[14] AS pay_hist_14,
final_array[15] AS pay_hist_15,
final_array[16] AS pay_hist_16,
final_array[17] AS pay_hist_17,
final_array[18] AS pay_hist_18,
final_array[19] AS pay_hist_19,
final_array[20] AS pay_hist_20,
final_array[21] AS pay_hist_21,
final_array[22] AS pay_hist_22,
final_array[23] AS pay_hist_23,
final_array[24] AS pay_hist_24,
final_array[25] AS pay_hist_25,
final_array[26] AS pay_hist_26,
final_array[27] AS pay_hist_27,
final_array[28] AS pay_hist_28,
final_array[29] AS pay_hist_29,
final_array[30] AS pay_hist_30,
final_array[31] AS pay_hist_31,
final_array[32] AS pay_hist_32,
final_array[33] AS pay_hist_33,
final_array[34] AS pay_hist_34,
final_array[35] AS pay_hist_35,
final_array[36] AS pay_hist_36
FROM fpl_ds_analytics.os_fow_temp_variables_creation_m1
""")

Executed!


In [32]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v2 --recursive
    
    

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v2/20250407_112120_22171_iin5u_0a0759f0-a482-4eb2-aa2f-1846933f2128


In [33]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v2
''')

Table Deleted!


0

In [34]:
df2=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v2
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v2'
)
AS

SELECT customer_no, creation_date, 

----Max DPD variables----
MAX(GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)) AS max_dpd_l36m, 
MAX(GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)) AS max_dpd_l24m,
MAX(GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)) AS max_dpd_l18m,
MAX(GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)) AS max_dpd_l12m,
MAX(GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)) AS max_dpd_l6m,
MAX(GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)) AS max_dpd_l3m,
MAX(GREATEST(pay_hist_1
)) AS max_dpd_cm,

MAX(CASE WHEN active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_dpd_l36m, 
MAX(CASE WHEN active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_dpd_l24m,
MAX(CASE WHEN active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_dpd_l18m,
MAX(CASE WHEN active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_dpd_l12m,
MAX(CASE WHEN active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_dpd_l6m,
MAX(CASE WHEN active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_dpd_l3m,
MAX(CASE WHEN active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (10, 31) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_cc_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (10, 31) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_cc_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (10, 31) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_cc_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (10, 31) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_cc_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (10, 31) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_cc_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (10, 31) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_cc_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (10, 31) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_cc_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (10, 31) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_cc_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (10, 31) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_cc_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (10, 31) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_cc_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (10, 31) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_cc_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (10, 31) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_cc_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (10, 31) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_cc_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (10, 31) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_cc_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_pl_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_pl_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_pl_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_pl_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_pl_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_pl_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_pl_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_pl_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_pl_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_pl_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_pl_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_pl_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_pl_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_pl_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_cd_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_cd_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_cd_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_cd_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_cd_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_cd_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_cd_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (6) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_cd_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (6) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_cd_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (6) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_cd_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (6) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_cd_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (6) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_cd_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (6) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_cd_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (6) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_cd_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_gl_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_gl_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_gl_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_gl_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_gl_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_gl_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_gl_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (7) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_gl_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (7) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_gl_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (7) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_gl_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (7) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_gl_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (7) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_gl_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (7) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_gl_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (7) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_gl_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_al_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_al_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_al_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_al_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_al_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_al_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_al_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (1) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_al_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (1) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_al_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (1) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_al_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (1) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_al_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (1) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_al_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (1) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_al_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (1) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_al_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_tw_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_tw_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_tw_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_tw_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_tw_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_tw_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_tw_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (13) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_tw_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (13) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_tw_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (13) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_tw_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (13) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_tw_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (13) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_tw_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (13) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_tw_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (13) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_tw_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (2) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_hl_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (2) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_hl_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (2) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_hl_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (2) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_hl_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (2) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_hl_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (2) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_hl_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (2) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_hl_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (2) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_hl_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (2) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_hl_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (2) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_hl_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (2) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_hl_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (2) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_hl_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (2) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_hl_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (2) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_hl_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_unsecured_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_unsecured_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_unsecured_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_unsecured_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_unsecured_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_unsecured_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_unsecured_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_unsecured_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_unsecured_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_unsecured_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_unsecured_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_unsecured_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_unsecured_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_unsecured_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_secured_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_secured_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_secured_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_secured_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_secured_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_secured_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_secured_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_secured_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_secured_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_secured_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_secured_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_secured_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_secured_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_secured_dpd_cm,

----Overdue amount variables----

SUM(case when date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_l36m, 
SUM(case when date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_l24m, 
SUM(case when date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_l18m, 
SUM(case when date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_l12m, 
SUM(case when date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_l9m, 
SUM(case when date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_l6m, 
SUM(case when date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_l3m, 
SUM(case when date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_cm, 

SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_l36m, 
SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_l24m, 
SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_l18m, 
SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_l12m, 
SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_l9m, 
SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_l6m, 
SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_l3m, 
SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_cm, 

SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_l36m, 
SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_l24m, 
SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_l18m, 
SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_l12m, 
SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_l9m, 
SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_l6m, 
SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_l3m, 
SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_cm, 

SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_l36m, 
SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_l24m, 
SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_l18m, 
SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_l12m, 
SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_l9m, 
SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_l6m, 
SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_l3m, 
SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_cm, 

SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_l36m, 
SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_l24m, 
SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_l18m, 
SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_l12m, 
SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_l9m, 
SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_l6m, 
SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_l3m, 
SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_cm, 

SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_l36m, 
SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_l24m, 
SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_l18m, 
SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_l12m, 
SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_l9m, 
SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_l6m, 
SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_l3m, 
SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_cm, 

SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_l36m, 
SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_l24m, 
SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_l18m, 
SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_l12m, 
SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_l9m, 
SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_l6m, 
SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_l3m, 
SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_cm, 

SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_l36m, 
SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_l24m, 
SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_l18m, 
SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_l12m, 
SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_l9m, 
SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_l6m, 
SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_l3m, 
SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_cm, 

SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_l36m, 
SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_l24m, 
SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_l18m, 
SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_l12m, 
SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_l9m, 
SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_l6m, 
SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_l3m, 
SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_cm, 

SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_l36m, 
SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_l24m, 
SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_l18m, 
SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_l12m, 
SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_l9m, 
SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_l6m, 
SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_l3m, 
SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_cm,

----Number of x+ dpd accounts variables----

COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_x_dpd_l36m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_x_dpd_l24m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_x_dpd_l18m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_x_dpd_l12m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_x_dpd_l6m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_x_dpd_l3m, 
COUNT(CASE WHEN GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_x_dpd_cm,

COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_30_dpd_l36m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_30_dpd_l24m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_30_dpd_l18m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_30_dpd_l12m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_30_dpd_l6m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_30_dpd_l3m, 
COUNT(CASE WHEN GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_30_dpd_cm,

COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_90_dpd_l36m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_90_dpd_l24m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_90_dpd_l18m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_90_dpd_l12m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_90_dpd_l6m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_90_dpd_l3m, 
COUNT(CASE WHEN GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_90_dpd_cm,

COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_x_dpd_l36m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_x_dpd_l24m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_x_dpd_l18m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_x_dpd_l12m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_x_dpd_l6m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_x_dpd_l3m, 
COUNT(CASE WHEN GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_x_dpd_cm,

COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_30_dpd_l36m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_30_dpd_l24m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_30_dpd_l18m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_30_dpd_l12m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_30_dpd_l6m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_30_dpd_l3m, 
COUNT(CASE WHEN GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_30_dpd_cm,

COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_90_dpd_l36m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_90_dpd_l24m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_90_dpd_l18m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_90_dpd_l12m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_90_dpd_l6m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_90_dpd_l3m, 
COUNT(CASE WHEN GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_cc_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_cc_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_cc_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_cc_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_cc_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_cc_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_cc_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_cc_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_cc_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_cc_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_cc_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_cc_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_cc_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_cc_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_cc_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_cc_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_cc_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_cc_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_cc_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_cc_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_cc_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cc_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cc_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cc_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cc_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cc_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cc_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cc_active_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_pl_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_pl_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_pl_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_pl_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_pl_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_pl_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_pl_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_pl_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_pl_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_pl_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_pl_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_pl_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_pl_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_pl_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_pl_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_pl_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_pl_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_pl_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_pl_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_pl_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_pl_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_pl_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_pl_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_pl_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_pl_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_pl_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_pl_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_pl_active_90_dpd_cm,



COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_cd_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_cd_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_cd_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_cd_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_cd_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_cd_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_cd_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_cd_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_cd_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_cd_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_cd_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_cd_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_cd_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_cd_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_cd_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_cd_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_cd_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_cd_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_cd_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_cd_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_cd_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cd_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cd_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cd_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cd_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cd_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cd_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cd_active_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_al_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_al_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_al_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_al_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_al_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_al_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_al_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_al_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_al_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_al_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_al_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_al_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_al_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_al_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_al_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_al_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_al_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_al_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_al_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_al_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_al_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_al_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_al_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_al_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_al_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_al_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_al_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_al_active_90_dpd_cm,



COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_gl_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_gl_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_gl_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_gl_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_gl_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_gl_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_gl_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_gl_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_gl_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_gl_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_gl_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_gl_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_gl_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_gl_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_gl_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_gl_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_gl_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_gl_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_gl_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_gl_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_gl_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_gl_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_gl_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_gl_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_gl_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_gl_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_gl_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_gl_active_90_dpd_cm,


COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_hl_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_hl_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_hl_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_hl_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_hl_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_hl_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_hl_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_hl_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_hl_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_hl_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_hl_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_hl_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_hl_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_hl_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_hl_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_hl_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_hl_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_hl_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_hl_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_hl_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_hl_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_hl_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_hl_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_hl_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_hl_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_hl_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_hl_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_hl_active_90_dpd_cm,



COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_tw_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_tw_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_tw_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_tw_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_tw_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_tw_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_tw_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_tw_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_tw_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_tw_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_tw_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_tw_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_tw_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_tw_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_tw_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_tw_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_tw_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_tw_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_tw_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_tw_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_tw_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_tw_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_tw_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_tw_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_tw_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_tw_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_tw_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_tw_active_90_dpd_cm,



COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_unsecured_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_unsecured_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_unsecured_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_unsecured_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_unsecured_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_unsecured_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_unsecured_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_unsecured_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_unsecured_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_unsecured_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_unsecured_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_unsecured_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_unsecured_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_unsecured_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_unsecured_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_unsecured_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_unsecured_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_unsecured_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_unsecured_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_unsecured_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_unsecured_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_unsecured_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_unsecured_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_unsecured_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_unsecured_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_unsecured_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_unsecured_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_unsecured_active_90_dpd_cm,



COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_secured_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_secured_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_secured_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_secured_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_secured_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_secured_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_secured_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_secured_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_secured_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_secured_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_secured_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_secured_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_secured_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_secured_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_secured_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_secured_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_secured_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_secured_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_secured_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_secured_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_secured_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_secured_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_secured_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_secured_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_secured_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_secured_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_secured_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_secured_active_90_dpd_cm

FROM fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v1
GROUP BY 1,2
""")

Executed!


In [35]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v3 --recursive
    
    

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v3/20250407_112125_22187_iin5u_0eda0677-83e2-4f6a-bfac-4d5f5f4d3f61


In [36]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v3
''')

Table Deleted!


0

In [ ]:
df3=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v3
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v3'
)
AS

SELECT *,

---Utilisation---

IF(total_sanctioned_amount!=0, CAST(total_balance_amount AS DOUBLE)/total_sanctioned_amount, NULL) AS overall_utilisation,
IF(secured_total_sanctioned_amount!=0, CAST(secured_total_balance_amount AS DOUBLE)/secured_total_sanctioned_amount, NULL) AS secured_utilisation,
IF(unsecured_total_sanctioned_amount!=0, CAST(unsecured_total_balance_amount AS DOUBLE)/unsecured_total_sanctioned_amount, NULL) AS unsecured_utilisation,
IF(pl_total_sanctioned_amount!=0, CAST(pl_total_balance_amount AS DOUBLE)/pl_total_sanctioned_amount, NULL) AS pl_utilisation,
IF(hl_total_sanctioned_amount!=0, CAST(hl_total_balance_amount AS DOUBLE)/hl_total_sanctioned_amount, NULL) AS hl_utilisation,
IF(cc_total_sanctioned_amount!=0, CAST(cc_total_balance_amount AS DOUBLE)/cc_total_sanctioned_amount, NULL) AS cc_utilisation,
IF(cd_total_sanctioned_amount!=0, CAST(cd_total_balance_amount AS DOUBLE)/cd_total_sanctioned_amount, NULL) AS cd_utilisation,
IF(al_total_sanctioned_amount!=0, CAST(al_total_balance_amount AS DOUBLE)/al_total_sanctioned_amount, NULL) AS al_utilisation,
IF(gl_total_sanctioned_amount!=0, CAST(gl_total_balance_amount AS DOUBLE)/gl_total_sanctioned_amount, NULL) AS gl_utilisation,
IF(tw_total_sanctioned_amount!=0, CAST(tw_total_balance_amount AS DOUBLE)/tw_total_sanctioned_amount, NULL) AS tw_utilisation
FROM(
SELECT customer_no, creation_date, 

---Sanctioned Amount---

MAX(CASE WHEN active_inactive=1 THEN sanctioned_amount ELSE 0 END) AS max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 THEN sanctioned_amount ELSE 0 END) AS total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN sanctioned_amount ELSE 0 END) AS secured_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN sanctioned_amount ELSE 0 END) AS secured_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN sanctioned_amount ELSE 0 END) AS unsecured_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN sanctioned_amount ELSE 0 END) AS unsecured_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) THEN sanctioned_amount ELSE 0 END) AS pl_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) THEN sanctioned_amount ELSE 0 END) AS pl_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) THEN sanctioned_amount ELSE 0 END) AS hl_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) THEN sanctioned_amount ELSE 0 END) AS hl_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) THEN sanctioned_amount ELSE 0 END) AS cc_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) THEN sanctioned_amount ELSE 0 END) AS cc_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) THEN sanctioned_amount ELSE 0 END) AS cd_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) THEN sanctioned_amount ELSE 0 END) AS cd_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) THEN sanctioned_amount ELSE 0 END) AS al_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) THEN sanctioned_amount ELSE 0 END) AS al_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) THEN sanctioned_amount ELSE 0 END) AS gl_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) THEN sanctioned_amount ELSE 0 END) AS gl_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) THEN sanctioned_amount ELSE 0 END) AS tw_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) THEN sanctioned_amount ELSE 0 END) AS tw_total_sanctioned_amount,

MAX(CASE WHEN active_inactive=1 AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS secured_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS secured_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS unsecured_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS unsecured_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS pl_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS pl_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS hl_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS hl_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS cc_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS cc_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS cd_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS cd_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS al_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS al_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS gl_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS gl_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS tw_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS tw_total_sanctioned_amount_l3m,

MAX(CASE WHEN active_inactive=1 AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS secured_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS secured_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS unsecured_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS unsecured_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS pl_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS pl_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS hl_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS hl_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS cc_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS cc_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS cd_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS cd_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS al_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS al_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS gl_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS gl_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS tw_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS tw_total_sanctioned_amount_l6m,

MAX(CASE WHEN active_inactive=1 AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS secured_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS secured_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS unsecured_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS unsecured_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS pl_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS pl_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS hl_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS hl_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31, 35) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS cc_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31, 35) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS cc_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS cd_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS cd_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS al_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS al_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS gl_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS gl_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS tw_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS tw_total_sanctioned_amount_l12m,

---Balance---

SUM(CASE WHEN active_inactive=1 THEN current_balance ELSE 0 END) AS total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN current_balance ELSE 0 END) AS secured_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN current_balance ELSE 0 END) AS unsecured_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) THEN current_balance ELSE 0 END) AS pl_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) THEN current_balance ELSE 0 END) AS hl_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) THEN current_balance ELSE 0 END) AS cc_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) THEN current_balance ELSE 0 END) AS cd_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) THEN current_balance ELSE 0 END) AS al_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) THEN current_balance ELSE 0 END) AS gl_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) THEN current_balance ELSE 0 END) AS tw_total_balance_amount,

---Product level Utilisation---

AVG(CASE WHEN active_inactive=1 THEN utilisation ELSE NULL END) AS avg_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71)THEN utilisation ELSE NULL END) AS avg_secured_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN utilisation ELSE NULL END) AS avg_unsecured_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) THEN utilisation ELSE NULL END) AS avg_pl_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) THEN utilisation ELSE NULL END) AS avg_hl_tilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) THEN utilisation ELSE NULL END) AS avg_cc_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) THEN utilisation ELSE NULL END) AS avg_cd_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) THEN utilisation ELSE NULL END) AS avg_al_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) THEN utilisation ELSE NULL END) AS avg_gl_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) THEN utilisation ELSE NULL END) AS avg_tw_utilisation

FROM fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v1
GROUP BY 1,2)
""")

Executed!


In [38]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v4 --recursive
    
    

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v4/20250407_112127_22194_iin5u_b378fc9a-c73c-4433-ab95-52be2cb87023


In [39]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v4
''')

Table Deleted!


0

In [40]:
df4=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v4
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v4'
)
AS

SELECT customer_no, creation_date, 

MAX(latest_product) AS latest_product, MAX(first_product) AS first_product,

MAX(DATE_DIFF('month', date_opened, creation_date)) AS max_bureau_vintage,
MIN(DATE_DIFF('month', date_opened, creation_date)) AS min_bureau_vintage,
MAX(CASE WHEN active_inactive=1 THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_bureau_vintage,
MIN(CASE WHEN active_inactive=1 THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_secured_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_secured_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_secured_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_secured_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_unsecured_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_unsecured_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_unsecured_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_unsecured_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_pl_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (5,37,41) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_pl_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_pl_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_pl_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (10,16,31) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_cc_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (10,16,31) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_cc_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (10,16,31) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_cc_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (10,16,31) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_cc_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (6) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_cd_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (6) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_cd_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_cd_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_cd_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (1) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_al_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (1) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_al_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_al_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_al_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (2) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_hl_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (2) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_hl_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_hl_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_hl_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (7) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_gl_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (7) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_gl_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_gl_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_gl_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (13) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_tw_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (13) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_tw_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_tw_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_tw_bureau_vintage
FROM(
SELECT *, first_value(account_type_symbol) over(partition by customer_no, creation_date order by date_opened DESC) AS latest_product,
first_value(account_type_symbol) over(partition by customer_no, creation_date order by date_opened) AS first_product
FROM fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v1
)
GROUP BY 1,2

""")

Executed!


In [41]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v5 --recursive
    
    

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v5/20250407_112129_22201_iin5u_1344df32-5839-4e71-a79f-32539789edfc


In [42]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v5
''')

Table Deleted!


0

In [43]:
df5=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v5
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v5'
)
AS

    select customer_no, creation_date,
        sum(case when date_diff('month', inquiry_date, creation_date) <= 1 then 1 else 0 end) as enq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <= 2 then 1 else 0 end) as enq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <= 3 then 1 else 0 end) as enq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <= 6 then 1 else 0 end) as enq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <= 12 then 1 else 0 end) as enq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) then 1 else 0 end) as unsec_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) then 1 else 0 end) as unsec_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) then 1 else 0 end) as unsec_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) then 1 else 0 end) as unsec_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) then 1 else 0 end) as unsec_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) then 1 else 0 end) as sec_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) then 1 else 0 end) as sec_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) then 1 else 0 end) as sec_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) then 1 else 0 end) as sec_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) then 1 else 0 end) as sec_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (5,37,41) then 1 else 0 end) as pl_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (5,37,41) then 1 else 0 end) as pl_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (5,37,41) then 1 else 0 end) as pl_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (5,37,41) then 1 else 0 end) as pl_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (5,37,41) then 1 else 0 end) as pl_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (10,16,31) then 1 else 0 end) as cc_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (10,16,31) then 1 else 0 end) as cc_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (10,16,31) then 1 else 0 end) as cc_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (10,16,31) then 1 else 0 end) as cc_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (10,16,31) then 1 else 0 end) as cc_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (6) then 1 else 0 end) as cd_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (6) then 1 else 0 end) as cd_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (6) then 1 else 0 end) as cd_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (6) then 1 else 0 end) as cd_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (6) then 1 else 0 end) as cd_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (1) then 1 else 0 end) as al_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (1) then 1 else 0 end) as al_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (1) then 1 else 0 end) as al_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (1) then 1 else 0 end) as al_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (1) then 1 else 0 end) as al_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (7) then 1 else 0 end) as gl_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (7) then 1 else 0 end) as gl_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (7) then 1 else 0 end) as gl_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (7) then 1 else 0 end) as gl_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (7) then 1 else 0 end) as gl_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (2) then 1 else 0 end) as hl_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (2) then 1 else 0 end) as hl_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (2) then 1 else 0 end) as hl_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (2) then 1 else 0 end) as hl_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (2) then 1 else 0 end) as hl_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (13) then 1 else 0 end) as tw_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (13) then 1 else 0 end) as tw_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (13) then 1 else 0 end) as tw_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (13) then 1 else 0 end) as tw_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (13) then 1 else 0 end) as tw_inq_l12m
        
        from fpl_ds_analytics.os_fow_inquiry_variables_creation_m1
        GROUP BY 1,2
""")

Executed!


In [44]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v6 --recursive
    
    

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v6/20250407_112131_22207_iin5u_9c74d9f0-bd5f-4417-a9a5-b60de40b845c


In [45]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v6
''')

Table Deleted!


0

In [46]:
df6=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v6
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_v6'
)
AS

SELECT customer_no, creation_date, MAX(risk_score) AS risk_score,
COUNT(*) AS nbr_ever_accounts,
SUM(CASE WHEN active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_accounts,
SUM(CASE WHEN DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS accts_new_l1m,
SUM(CASE WHEN DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS accts_new_l3m,
SUM(CASE WHEN DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS accts_new_l6m,
SUM(CASE WHEN DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS accts_new_l12m,
SUM(CASE WHEN DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS accts_closed_l1m,
SUM(CASE WHEN DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS accts_closed_l3m,
SUM(CASE WHEN DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS accts_closed_l6m,
SUM(CASE WHEN DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS accts_closed_l12m,

SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN 1 ELSE 0 END) AS nbr_ever_secured_accounts,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN 1 ELSE 0 END) AS nbr_active_secured_accounts,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS secured_accts_new_l1m,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS secured_accts_new_l3m,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS secured_accts_new_l6m,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS secured_accts_new_l12m,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS secured_accts_closed_l1m,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS secured_accts_closed_l3m,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS secured_accts_closed_l6m,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS secured_accts_closed_l12m,

SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN 1 ELSE 0 END) AS nbr_ever_unsecured_accounts,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN 1 ELSE 0 END) AS nbr_active_unsecured_accounts,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS unsecured_accts_new_l1m,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS unsecured_accts_new_l3m,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS unsecured_accts_new_l6m,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS unsecured_accts_new_l12m,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS unsecured_accts_closed_l1m,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS unsecured_accts_closed_l3m,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS unsecured_accts_closed_l6m,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS unsecured_accts_closed_l12m,

MAX(CASE WHEN account_type_symbol=2 THEN 1 ELSE 0 END) AS ever_hl,
SUM(CASE WHEN account_type_symbol=2 THEN 1 ELSE 0 END) AS nbr_ever_hl,
MAX(CASE WHEN account_type_symbol=2 AND active_inactive=1 THEN 1 ELSE 0 END) AS active_hl,
SUM(CASE WHEN account_type_symbol=2 AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_hl,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS hl_new_l1m,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS hl_new_l3m,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS hl_new_l6m,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS hl_new_l12m,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS hl_closed_l1m,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS hl_closed_l3m,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS hl_closed_l6m,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS hl_closed_l12m,

MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN 1 ELSE 0 END) AS ever_pl,
SUM(CASE WHEN account_type_symbol IN (5,37,41) THEN 1 ELSE 0 END) AS nbr_ever_pl,
MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_pl,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_pl,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS pl_new_l1m,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS pl_new_l3m,
SUM(CASE WHEN account_type_symbol IN (5,37,41)AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS pl_new_l6m,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS pl_new_l12m,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS pl_closed_l1m,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS pl_closed_l3m,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS pl_closed_l6m,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS pl_closed_l12m,

MAX(CASE WHEN account_type_symbol IN (10,16,31) THEN 1 ELSE 0 END) AS ever_cc,
SUM(CASE WHEN account_type_symbol IN (10,16,31) THEN 1 ELSE 0 END) AS nbr_ever_cc,
MAX(CASE WHEN account_type_symbol IN (10,16,31) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_cc,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_cc,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS cc_new_l1m,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS cc_new_l3m,
SUM(CASE WHEN account_type_symbol IN (10,16,31)AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS cc_new_l6m,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS cc_new_l12m,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS cc_closed_l1m,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS cc_closed_l3m,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS cc_closed_l6m,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS cc_closed_l12m,

MAX(CASE WHEN account_type_symbol IN (6) THEN 1 ELSE 0 END) AS ever_cd,
SUM(CASE WHEN account_type_symbol IN (6) THEN 1 ELSE 0 END) AS nbr_ever_cd,
MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_cd,
SUM(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_cd,
SUM(CASE WHEN account_type_symbol IN (6) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS cd_new_l1m,
SUM(CASE WHEN account_type_symbol IN (6) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS cd_new_l3m,
SUM(CASE WHEN account_type_symbol IN (6)AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS cd_new_l6m,
SUM(CASE WHEN account_type_symbol IN (6) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS cd_new_l12m,
SUM(CASE WHEN account_type_symbol IN (6) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS cd_closed_l1m,
SUM(CASE WHEN account_type_symbol IN (6) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS cd_closed_l3m,
SUM(CASE WHEN account_type_symbol IN (6) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS cd_closed_l6m,
SUM(CASE WHEN account_type_symbol IN (6) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS cd_closed_l12m,

MAX(CASE WHEN account_type_symbol IN (7) THEN 1 ELSE 0 END) AS ever_gl,
SUM(CASE WHEN account_type_symbol IN (7) THEN 1 ELSE 0 END) AS nbr_ever_gl,
MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_gl,
SUM(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_gl,
SUM(CASE WHEN account_type_symbol IN (7) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS gl_new_l1m,
SUM(CASE WHEN account_type_symbol IN (7) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS gl_new_l3m,
SUM(CASE WHEN account_type_symbol IN (7)AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS gl_new_l6m,
SUM(CASE WHEN account_type_symbol IN (7) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS gl_new_l12m,
SUM(CASE WHEN account_type_symbol IN (7) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS gl_closed_l1m,
SUM(CASE WHEN account_type_symbol IN (7) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS gl_closed_l3m,
SUM(CASE WHEN account_type_symbol IN (7) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS gl_closed_l6m,
SUM(CASE WHEN account_type_symbol IN (7) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS gl_closed_l12m,

MAX(CASE WHEN account_type_symbol IN (1) THEN 1 ELSE 0 END) AS ever_al,
SUM(CASE WHEN account_type_symbol IN (1) THEN 1 ELSE 0 END) AS nbr_ever_al,
MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_al,
SUM(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_al,
SUM(CASE WHEN account_type_symbol IN (1) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS al_new_l1m,
SUM(CASE WHEN account_type_symbol IN (1) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS al_new_l3m,
SUM(CASE WHEN account_type_symbol IN (1)AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS al_new_l6m,
SUM(CASE WHEN account_type_symbol IN (1) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS al_new_l12m,
SUM(CASE WHEN account_type_symbol IN (1) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS al_closed_l1m,
SUM(CASE WHEN account_type_symbol IN (1) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS al_closed_l3m,
SUM(CASE WHEN account_type_symbol IN (1) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS al_closed_l6m,
SUM(CASE WHEN account_type_symbol IN (1) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS al_closed_l12m,

MAX(CASE WHEN account_type_symbol IN (13) THEN 1 ELSE 0 END) AS ever_tw,
SUM(CASE WHEN account_type_symbol IN (13) THEN 1 ELSE 0 END) AS nbr_ever_tw,
MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_tw,
SUM(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_tw,
SUM(CASE WHEN account_type_symbol IN (13) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS tw_new_l1m,
SUM(CASE WHEN account_type_symbol IN (13) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS tw_new_l3m,
SUM(CASE WHEN account_type_symbol IN (13)AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS tw_new_l6m,
SUM(CASE WHEN account_type_symbol IN (13) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS tw_new_l12m,
SUM(CASE WHEN account_type_symbol IN (13) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS tw_closed_l1m,
SUM(CASE WHEN account_type_symbol IN (13) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS tw_closed_l3m,
SUM(CASE WHEN account_type_symbol IN (13) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS tw_closed_l6m,
SUM(CASE WHEN account_type_symbol IN (13) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS tw_closed_l12m,

sum(case when (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as nbr_ever_stpl,
sum(case when active_inactive=1 AND (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as nbr_active_stpl,
sum(case when date_diff('month', date_opened, creation_date)<=1 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_new_l1m,
sum(case when date_diff('month', date_opened, creation_date)<=3 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_new_l3m,
sum(case when date_diff('month', date_opened, creation_date)<=6 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_new_l6m,
sum(case when date_diff('month', date_opened, creation_date)<=12 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_new_l12m,
sum(case when date_diff('month', date_closed, creation_date)<=1 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_closed_l1m,
sum(case when date_diff('month', date_closed, creation_date)<=3 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_closed_l3m,
sum(case when date_diff('month', date_closed, creation_date)<=6 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_closed_l6m,
sum(case when date_diff('month', date_closed, creation_date)<=12 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_closed_l12m,

MAX(CASE WHEN account_type_symbol IN (17) THEN 1 ELSE 0 END) AS ever_cv,
SUM(CASE WHEN account_type_symbol IN (17) THEN 1 ELSE 0 END) AS nbr_ever_cv,
MAX(CASE WHEN account_type_symbol IN (17) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_cv,
SUM(CASE WHEN account_type_symbol IN (17) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_cv,
MAX(CASE WHEN account_type_symbol IN (53,57) THEN 1 ELSE 0 END) AS ever_agri,
SUM(CASE WHEN account_type_symbol IN (53,57) THEN 1 ELSE 0 END) AS nbr_ever_agri,
MAX(CASE WHEN account_type_symbol IN (53,57) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_agri,
SUM(CASE WHEN account_type_symbol IN (53,57) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_agri,
MAX(CASE WHEN account_type_symbol IN (3) THEN 1 ELSE 0 END) AS ever_lap,
SUM(CASE WHEN account_type_symbol IN (3) THEN 1 ELSE 0 END) AS nbr_ever_lap,
MAX(CASE WHEN account_type_symbol IN (3) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_lap,
SUM(CASE WHEN account_type_symbol IN (3) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_lap,
MAX(CASE WHEN account_type_symbol IN (4) THEN 1 ELSE 0 END) AS ever_las,
SUM(CASE WHEN account_type_symbol IN (4) THEN 1 ELSE 0 END) AS nbr_ever_las,
MAX(CASE WHEN account_type_symbol IN (4) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_las,
SUM(CASE WHEN account_type_symbol IN (4) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_las

FROM fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v1
GROUP BY 1,2
""")

Executed!


In [47]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_overall --recursive
    
    

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_overall/20250407_112134_22215_iin5u_b3365bb2-f557-419a-a6bb-0cd4ea0e7df7


In [48]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_overall
''')

Table Deleted!


0

In [49]:
df7=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_overall
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m1_data_overall'
)
AS
SELECT *
FROM fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v2
INNER JOIN fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v3 USING(customer_no, creation_date)
INNER JOIN fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v4 USING(customer_no, creation_date)
INNER JOIN fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v5 USING(customer_no, creation_date)
INNER JOIN fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_v6 USING(customer_no, creation_date)

""")

Executed!


# M2 variables preparation

In [50]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v1 --recursive

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v1/20250407_112139_22230_iin5u_26c36b90-04e9-44cc-a020-fa7adce7e61e


In [51]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v1
''')

Table Deleted!


0

In [52]:
df5=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v1
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v1'
)
AS

SELECT *,
final_array[1] AS pay_hist_1,
final_array[2] AS pay_hist_2,
final_array[3] AS pay_hist_3,
final_array[4] AS pay_hist_4,
final_array[5] AS pay_hist_5,
final_array[6] AS pay_hist_6,
final_array[7] AS pay_hist_7,
final_array[8] AS pay_hist_8,
final_array[9] AS pay_hist_9,
final_array[10] AS pay_hist_10,
final_array[11] AS pay_hist_11,
final_array[12] AS pay_hist_12,
final_array[13] AS pay_hist_13,
final_array[14] AS pay_hist_14,
final_array[15] AS pay_hist_15,
final_array[16] AS pay_hist_16,
final_array[17] AS pay_hist_17,
final_array[18] AS pay_hist_18,
final_array[19] AS pay_hist_19,
final_array[20] AS pay_hist_20,
final_array[21] AS pay_hist_21,
final_array[22] AS pay_hist_22,
final_array[23] AS pay_hist_23,
final_array[24] AS pay_hist_24,
final_array[25] AS pay_hist_25,
final_array[26] AS pay_hist_26,
final_array[27] AS pay_hist_27,
final_array[28] AS pay_hist_28,
final_array[29] AS pay_hist_29,
final_array[30] AS pay_hist_30,
final_array[31] AS pay_hist_31,
final_array[32] AS pay_hist_32,
final_array[33] AS pay_hist_33,
final_array[34] AS pay_hist_34,
final_array[35] AS pay_hist_35,
final_array[36] AS pay_hist_36
FROM fpl_ds_analytics.os_fow_temp_variables_creation_m2
""")

Executed!


In [53]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v2 --recursive
    
    

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v2/20250407_112141_22238_iin5u_4731d351-4688-4ec1-aa28-be5ec15cf97b


In [54]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v2
''')

Table Deleted!


0

In [55]:
df2=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v2
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v2'
)
AS

SELECT customer_no, creation_date, 

----Max DPD variables----
MAX(GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)) AS max_dpd_l36m, 
MAX(GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)) AS max_dpd_l24m,
MAX(GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)) AS max_dpd_l18m,
MAX(GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)) AS max_dpd_l12m,
MAX(GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)) AS max_dpd_l6m,
MAX(GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)) AS max_dpd_l3m,
MAX(GREATEST(pay_hist_1
)) AS max_dpd_cm,

MAX(CASE WHEN active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_dpd_l36m, 
MAX(CASE WHEN active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_dpd_l24m,
MAX(CASE WHEN active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_dpd_l18m,
MAX(CASE WHEN active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_dpd_l12m,
MAX(CASE WHEN active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_dpd_l6m,
MAX(CASE WHEN active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_dpd_l3m,
MAX(CASE WHEN active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (10, 31) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_cc_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (10, 31) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_cc_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (10, 31) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_cc_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (10, 31) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_cc_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (10, 31) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_cc_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (10, 31) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_cc_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (10, 31) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_cc_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (10, 31) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_cc_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (10, 31) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_cc_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (10, 31) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_cc_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (10, 31) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_cc_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (10, 31) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_cc_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (10, 31) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_cc_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (10, 31) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_cc_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_pl_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_pl_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_pl_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_pl_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_pl_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_pl_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_pl_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_pl_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_pl_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_pl_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_pl_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_pl_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_pl_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_pl_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_cd_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_cd_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_cd_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_cd_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_cd_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_cd_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_cd_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (6) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_cd_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (6) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_cd_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (6) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_cd_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (6) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_cd_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (6) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_cd_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (6) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_cd_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (6) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_cd_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_gl_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_gl_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_gl_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_gl_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_gl_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_gl_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_gl_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (7) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_gl_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (7) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_gl_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (7) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_gl_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (7) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_gl_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (7) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_gl_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (7) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_gl_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (7) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_gl_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_al_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_al_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_al_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_al_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_al_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_al_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_al_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (1) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_al_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (1) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_al_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (1) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_al_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (1) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_al_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (1) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_al_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (1) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_al_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (1) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_al_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_tw_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_tw_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_tw_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_tw_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_tw_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_tw_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_tw_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (13) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_tw_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (13) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_tw_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (13) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_tw_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (13) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_tw_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (13) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_tw_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (13) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_tw_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (13) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_tw_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (2) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_hl_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (2) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_hl_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (2) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_hl_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (2) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_hl_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (2) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_hl_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (2) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_hl_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (2) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_hl_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (2) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_hl_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (2) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_hl_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (2) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_hl_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (2) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_hl_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (2) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_hl_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (2) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_hl_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (2) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_hl_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_unsecured_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_unsecured_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_unsecured_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_unsecured_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_unsecured_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_unsecured_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_unsecured_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_unsecured_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_unsecured_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_unsecured_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_unsecured_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_unsecured_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_unsecured_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_unsecured_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_active_secured_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_active_secured_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_active_secured_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_active_secured_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_active_secured_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND active_inactive=1 THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_active_secured_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND active_inactive=1 THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_active_secured_dpd_cm,

MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
) ELSE NULL END) AS max_secured_dpd_l36m, 
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
) ELSE NULL END) AS max_secured_dpd_l24m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
) ELSE NULL END) AS max_secured_dpd_l18m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
) ELSE NULL END) AS max_secured_dpd_l12m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
) ELSE NULL END) AS max_secured_dpd_l6m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
) ELSE NULL END) AS max_secured_dpd_l3m,
MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN GREATEST(pay_hist_1
) ELSE NULL END) AS max_secured_dpd_cm,

----Overdue amount variables----

SUM(case when date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_l36m, 
SUM(case when date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_l24m, 
SUM(case when date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_l18m, 
SUM(case when date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_l12m, 
SUM(case when date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_l9m, 
SUM(case when date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_l6m, 
SUM(case when date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_l3m, 
SUM(case when date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS overdue_cm, 

SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_l36m, 
SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_l24m, 
SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_l18m, 
SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_l12m, 
SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_l9m, 
SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_l6m, 
SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_l3m, 
SUM(case when account_type_symbol IN (10, 31) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cc_overdue_cm, 

SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_l36m, 
SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_l24m, 
SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_l18m, 
SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_l12m, 
SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_l9m, 
SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_l6m, 
SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_l3m, 
SUM(case when account_type_symbol IN (5,37,41) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS pl_overdue_cm, 

SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_l36m, 
SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_l24m, 
SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_l18m, 
SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_l12m, 
SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_l9m, 
SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_l6m, 
SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_l3m, 
SUM(case when account_type_symbol IN (6) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS cd_overdue_cm, 

SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_l36m, 
SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_l24m, 
SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_l18m, 
SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_l12m, 
SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_l9m, 
SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_l6m, 
SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_l3m, 
SUM(case when account_type_symbol IN (7) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS gl_overdue_cm, 

SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_l36m, 
SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_l24m, 
SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_l18m, 
SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_l12m, 
SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_l9m, 
SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_l6m, 
SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_l3m, 
SUM(case when account_type_symbol IN (1) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS al_overdue_cm, 

SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_l36m, 
SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_l24m, 
SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_l18m, 
SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_l12m, 
SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_l9m, 
SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_l6m, 
SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_l3m, 
SUM(case when account_type_symbol IN (13) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS tw_overdue_cm, 

SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_l36m, 
SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_l24m, 
SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_l18m, 
SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_l12m, 
SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_l9m, 
SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_l6m, 
SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_l3m, 
SUM(case when account_type_symbol IN (2) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS hl_overdue_cm, 

SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_l36m, 
SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_l24m, 
SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_l18m, 
SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_l12m, 
SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_l9m, 
SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_l6m, 
SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_l3m, 
SUM(case when account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS unsecured_overdue_cm, 

SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 36 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_l36m, 
SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 24 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_l24m, 
SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 18 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_l18m, 
SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 12 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_l12m, 
SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 9 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_l9m, 
SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 6 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_l6m, 
SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 3 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_l3m, 
SUM(case when account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_reported, creation_date) <= 1 and account_designator_symbol = 1 then greatest(amount_past_due, 0) else 0 end) AS secured_overdue_cm,

----Number of x+ dpd accounts variables----

COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_x_dpd_l36m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_x_dpd_l24m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_x_dpd_l18m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_x_dpd_l12m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_x_dpd_l6m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_x_dpd_l3m, 
COUNT(CASE WHEN GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_x_dpd_cm,

COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_30_dpd_l36m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_30_dpd_l24m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_30_dpd_l18m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_30_dpd_l12m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_30_dpd_l6m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_30_dpd_l3m, 
COUNT(CASE WHEN GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_30_dpd_cm,

COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_90_dpd_l36m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_90_dpd_l24m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_90_dpd_l18m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_90_dpd_l12m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_90_dpd_l6m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_90_dpd_l3m, 
COUNT(CASE WHEN GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_90_dpd_cm,

COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_x_dpd_l36m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_x_dpd_l24m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_x_dpd_l18m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_x_dpd_l12m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_x_dpd_l6m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_x_dpd_l3m, 
COUNT(CASE WHEN GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_x_dpd_cm,

COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_30_dpd_l36m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_30_dpd_l24m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_30_dpd_l18m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_30_dpd_l12m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_30_dpd_l6m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_30_dpd_l3m, 
COUNT(CASE WHEN GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_30_dpd_cm,

COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_90_dpd_l36m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_90_dpd_l24m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_90_dpd_l18m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_90_dpd_l12m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_90_dpd_l6m, 
COUNT(CASE WHEN GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_90_dpd_l3m, 
COUNT(CASE WHEN GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_cc_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_cc_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_cc_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_cc_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_cc_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_cc_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_cc_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_cc_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_cc_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_cc_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_cc_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_cc_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_cc_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_cc_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_cc_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_cc_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_cc_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_cc_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_cc_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_cc_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_cc_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cc_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cc_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cc_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cc_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cc_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cc_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cc_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (10, 31) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cc_active_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_pl_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_pl_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_pl_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_pl_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_pl_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_pl_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_pl_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_pl_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_pl_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_pl_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_pl_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_pl_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_pl_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_pl_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_pl_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_pl_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_pl_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_pl_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_pl_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_pl_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_pl_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_pl_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_pl_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_pl_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_pl_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_pl_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_pl_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_pl_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5,37,41) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_pl_active_90_dpd_cm,



COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_cd_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_cd_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_cd_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_cd_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_cd_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_cd_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_cd_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_cd_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_cd_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_cd_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_cd_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_cd_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_cd_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_cd_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_cd_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_cd_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_cd_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_cd_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_cd_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_cd_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_cd_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_cd_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cd_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cd_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cd_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cd_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cd_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cd_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (6) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_cd_active_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_al_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_al_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_al_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_al_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_al_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_al_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_al_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_al_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_al_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_al_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_al_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_al_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_al_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_al_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_al_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_al_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_al_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_al_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_al_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_al_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_al_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_al_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_al_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_al_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_al_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_al_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_al_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_al_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_al_active_90_dpd_cm,



COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_gl_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_gl_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_gl_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_gl_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_gl_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_gl_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_gl_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_gl_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_gl_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_gl_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_gl_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_gl_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_gl_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_gl_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_gl_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_gl_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_gl_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_gl_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_gl_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_gl_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_gl_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_gl_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_gl_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_gl_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_gl_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_gl_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_gl_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_gl_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (7) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_gl_active_90_dpd_cm,


COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_hl_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_hl_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_hl_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_hl_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_hl_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_hl_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_hl_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_hl_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_hl_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_hl_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_hl_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_hl_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_hl_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_hl_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_hl_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_hl_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_hl_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_hl_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_hl_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_hl_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_hl_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_hl_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_hl_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_hl_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_hl_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_hl_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_hl_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_hl_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (2) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_hl_active_90_dpd_cm,



COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_tw_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_tw_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_tw_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_tw_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_tw_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_tw_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_tw_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_tw_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_tw_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_tw_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_tw_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_tw_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_tw_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_tw_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_tw_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_tw_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_tw_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_tw_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_tw_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_tw_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_tw_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_tw_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_tw_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_tw_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_tw_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_tw_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_tw_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_tw_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (13) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_tw_active_90_dpd_cm,



COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_unsecured_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_unsecured_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_unsecured_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_unsecured_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_unsecured_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_unsecured_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_unsecured_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_unsecured_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_unsecured_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_unsecured_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_unsecured_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_unsecured_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_unsecured_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_unsecured_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_unsecured_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_unsecured_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_unsecured_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_unsecured_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_unsecured_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_unsecured_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_unsecured_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_unsecured_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_unsecured_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_unsecured_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_unsecured_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_unsecured_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_unsecured_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_unsecured_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_unsecured_active_90_dpd_cm,



COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 THEN account_number ELSE NULL END) AS nbr_secured_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 THEN account_number ELSE NULL END) AS nbr_secured_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 THEN account_number ELSE NULL END) AS nbr_secured_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 THEN account_number ELSE NULL END) AS nbr_secured_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 THEN account_number ELSE NULL END) AS nbr_secured_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 THEN account_number ELSE NULL END) AS nbr_secured_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1
)>0 THEN account_number ELSE NULL END) AS nbr_secured_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 THEN account_number ELSE NULL END) AS nbr_secured_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 THEN account_number ELSE NULL END) AS nbr_secured_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 THEN account_number ELSE NULL END) AS nbr_secured_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 THEN account_number ELSE NULL END) AS nbr_secured_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 THEN account_number ELSE NULL END) AS nbr_secured_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 THEN account_number ELSE NULL END) AS nbr_secured_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1
)>=30 THEN account_number ELSE NULL END) AS nbr_secured_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 THEN account_number ELSE NULL END) AS nbr_secured_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 THEN account_number ELSE NULL END) AS nbr_secured_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 THEN account_number ELSE NULL END) AS nbr_secured_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 THEN account_number ELSE NULL END) AS nbr_secured_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 THEN account_number ELSE NULL END) AS nbr_secured_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 THEN account_number ELSE NULL END) AS nbr_secured_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1
)>=90 THEN account_number ELSE NULL END) AS nbr_secured_90_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_x_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_x_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_x_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_x_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_x_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_x_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1
)>0 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_x_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_30_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_30_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_30_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_30_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_30_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_30_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1
)>=30 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_active_secured_30_dpd_cm,

COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24, pay_hist_25, pay_hist_26, pay_hist_27, pay_hist_28, 
pay_hist_29, pay_hist_30, pay_hist_31, pay_hist_32, pay_hist_33, pay_hist_34, pay_hist_35, pay_hist_36
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_secured_active_90_dpd_l36m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18, pay_hist_19, pay_hist_20, pay_hist_21, 
pay_hist_22, pay_hist_23, pay_hist_24
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_secured_active_90_dpd_l24m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12, pay_hist_13, pay_hist_14, 
pay_hist_15, pay_hist_16, pay_hist_17, pay_hist_18
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_secured_active_90_dpd_l18m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6, pay_hist_7, 
pay_hist_8, pay_hist_9, pay_hist_10, pay_hist_11, pay_hist_12
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_secured_active_90_dpd_l12m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3, pay_hist_4, pay_hist_5, pay_hist_6
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_secured_active_90_dpd_l6m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1, pay_hist_2, pay_hist_3
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_secured_active_90_dpd_l3m, 
COUNT(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND GREATEST(pay_hist_1
)>=90 AND active_inactive=1 THEN account_number ELSE NULL END) AS nbr_secured_active_90_dpd_cm

FROM fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v1
GROUP BY 1,2
""")

Executed!


In [56]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v3 --recursive
    
    

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v3/20250407_112146_22253_iin5u_b6c964c0-2ae2-47f0-9483-c15e911fa3cc


In [57]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v3
''')

Table Deleted!


0

In [58]:
df3=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v3
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v3'
)
AS

SELECT *,

---Utilisation---

IF(total_sanctioned_amount!=0, CAST(total_balance_amount AS DOUBLE)/total_sanctioned_amount, NULL) AS overall_utilisation,
IF(secured_total_sanctioned_amount!=0, CAST(secured_total_balance_amount AS DOUBLE)/secured_total_sanctioned_amount, NULL) AS secured_utilisation,
IF(unsecured_total_sanctioned_amount!=0, CAST(unsecured_total_balance_amount AS DOUBLE)/unsecured_total_sanctioned_amount, NULL) AS unsecured_utilisation,
IF(pl_total_sanctioned_amount!=0, CAST(pl_total_balance_amount AS DOUBLE)/pl_total_sanctioned_amount, NULL) AS pl_utilisation,
IF(hl_total_sanctioned_amount!=0, CAST(hl_total_balance_amount AS DOUBLE)/hl_total_sanctioned_amount, NULL) AS hl_utilisation,
IF(cc_total_sanctioned_amount!=0, CAST(cc_total_balance_amount AS DOUBLE)/cc_total_sanctioned_amount, NULL) AS cc_utilisation,
IF(cd_total_sanctioned_amount!=0, CAST(cd_total_balance_amount AS DOUBLE)/cd_total_sanctioned_amount, NULL) AS cd_utilisation,
IF(al_total_sanctioned_amount!=0, CAST(al_total_balance_amount AS DOUBLE)/al_total_sanctioned_amount, NULL) AS al_utilisation,
IF(gl_total_sanctioned_amount!=0, CAST(gl_total_balance_amount AS DOUBLE)/gl_total_sanctioned_amount, NULL) AS gl_utilisation,
IF(tw_total_sanctioned_amount!=0, CAST(tw_total_balance_amount AS DOUBLE)/tw_total_sanctioned_amount, NULL) AS tw_utilisation
FROM(
SELECT customer_no, creation_date, 

---Sanctioned Amount---

MAX(CASE WHEN active_inactive=1 THEN sanctioned_amount ELSE 0 END) AS max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 THEN sanctioned_amount ELSE 0 END) AS total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN sanctioned_amount ELSE 0 END) AS secured_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN sanctioned_amount ELSE 0 END) AS secured_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN sanctioned_amount ELSE 0 END) AS unsecured_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN sanctioned_amount ELSE 0 END) AS unsecured_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) THEN sanctioned_amount ELSE 0 END) AS pl_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) THEN sanctioned_amount ELSE 0 END) AS pl_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) THEN sanctioned_amount ELSE 0 END) AS hl_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) THEN sanctioned_amount ELSE 0 END) AS hl_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) THEN sanctioned_amount ELSE 0 END) AS cc_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) THEN sanctioned_amount ELSE 0 END) AS cc_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) THEN sanctioned_amount ELSE 0 END) AS cd_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) THEN sanctioned_amount ELSE 0 END) AS cd_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) THEN sanctioned_amount ELSE 0 END) AS al_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) THEN sanctioned_amount ELSE 0 END) AS al_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) THEN sanctioned_amount ELSE 0 END) AS gl_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) THEN sanctioned_amount ELSE 0 END) AS gl_total_sanctioned_amount,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) THEN sanctioned_amount ELSE 0 END) AS tw_max_sanctioned_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) THEN sanctioned_amount ELSE 0 END) AS tw_total_sanctioned_amount,

MAX(CASE WHEN active_inactive=1 AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS secured_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS secured_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS unsecured_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS unsecured_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS pl_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS pl_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS hl_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS hl_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS cc_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS cc_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS cd_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS cd_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS al_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS al_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS gl_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS gl_total_sanctioned_amount_l3m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS tw_max_sanctioned_amount_l3m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) AND date_diff('month', date_opened, creation_date)<=3 THEN sanctioned_amount ELSE 0 END) AS tw_total_sanctioned_amount_l3m,

MAX(CASE WHEN active_inactive=1 AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS secured_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS secured_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS unsecured_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS unsecured_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS pl_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS pl_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS hl_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS hl_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS cc_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS cc_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS cd_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS cd_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS al_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS al_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS gl_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS gl_total_sanctioned_amount_l6m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS tw_max_sanctioned_amount_l6m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) AND date_diff('month', date_opened, creation_date)<=6 THEN sanctioned_amount ELSE 0 END) AS tw_total_sanctioned_amount_l6m,

MAX(CASE WHEN active_inactive=1 AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS secured_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS secured_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS unsecured_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS unsecured_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS pl_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS pl_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS hl_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS hl_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31, 35) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS cc_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31, 35) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS cc_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS cd_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS cd_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS al_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS al_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS gl_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS gl_total_sanctioned_amount_l12m,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS tw_max_sanctioned_amount_l12m,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) AND date_diff('month', date_opened, creation_date)<=12 THEN sanctioned_amount ELSE 0 END) AS tw_total_sanctioned_amount_l12m,

---Balance---

SUM(CASE WHEN active_inactive=1 THEN current_balance ELSE 0 END) AS total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN current_balance ELSE 0 END) AS secured_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN current_balance ELSE 0 END) AS unsecured_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) THEN current_balance ELSE 0 END) AS pl_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) THEN current_balance ELSE 0 END) AS hl_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) THEN current_balance ELSE 0 END) AS cc_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) THEN current_balance ELSE 0 END) AS cd_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) THEN current_balance ELSE 0 END) AS al_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) THEN current_balance ELSE 0 END) AS gl_total_balance_amount,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) THEN current_balance ELSE 0 END) AS tw_total_balance_amount,

---Product level Utilisation---

AVG(CASE WHEN active_inactive=1 THEN utilisation ELSE NULL END) AS avg_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71)THEN utilisation ELSE NULL END) AS avg_secured_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN utilisation ELSE NULL END) AS avg_unsecured_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) THEN utilisation ELSE NULL END) AS avg_pl_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) THEN utilisation ELSE NULL END) AS avg_hl_tilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (10, 16 ,31) THEN utilisation ELSE NULL END) AS avg_cc_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) THEN utilisation ELSE NULL END) AS avg_cd_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) THEN utilisation ELSE NULL END) AS avg_al_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) THEN utilisation ELSE NULL END) AS avg_gl_utilisation,
AVG(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) THEN utilisation ELSE NULL END) AS avg_tw_utilisation

FROM fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v1
GROUP BY 1,2)
""")

Executed!


In [59]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v4 --recursive
    
    

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v4/20250407_112148_22260_iin5u_b4000f76-e703-4c03-a84d-5177bb36f9d2


In [60]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v4
''')

Table Deleted!


0

In [61]:
df4=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v4
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v4'
)
AS

SELECT customer_no, creation_date, 

MAX(latest_product) AS latest_product, MAX(first_product) AS first_product,

MAX(DATE_DIFF('month', date_opened, creation_date)) AS max_bureau_vintage,
MIN(DATE_DIFF('month', date_opened, creation_date)) AS min_bureau_vintage,
MAX(CASE WHEN active_inactive=1 THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_bureau_vintage,
MIN(CASE WHEN active_inactive=1 THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_secured_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_secured_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_secured_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_secured_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_unsecured_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_unsecured_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_unsecured_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_unsecured_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_pl_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (5,37,41) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_pl_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_pl_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (5,37,41) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_pl_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (10,16,31) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_cc_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (10,16,31) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_cc_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (10,16,31) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_cc_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (10,16,31) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_cc_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (6) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_cd_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (6) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_cd_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_cd_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (6) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_cd_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (1) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_al_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (1) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_al_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_al_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (1) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_al_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (2) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_hl_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (2) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_hl_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_hl_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (2) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_hl_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (7) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_gl_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (7) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_gl_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_gl_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (7) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_gl_bureau_vintage,

MAX(CASE WHEN account_type_symbol IN (13) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_tw_bureau_vintage,
MIN(CASE WHEN account_type_symbol IN (13) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_tw_bureau_vintage,
MAX(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS max_active_tw_bureau_vintage,
MIN(CASE WHEN active_inactive=1 AND account_type_symbol IN (13) THEN DATE_DIFF('month', date_opened, creation_date) ELSE NULL END) AS min_active_tw_bureau_vintage
FROM(
SELECT *, first_value(account_type_symbol) over(partition by customer_no, creation_date order by date_opened DESC) AS latest_product,
first_value(account_type_symbol) over(partition by customer_no, creation_date order by date_opened) AS first_product
FROM fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v1
)
GROUP BY 1,2

""")

Executed!


In [62]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v5 --recursive
    
    

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v5/20250407_112150_22266_iin5u_0af7ea0e-f44d-458b-af1d-57bb34180da7


In [63]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v5
''')

Table Deleted!


0

In [64]:
df5=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v5
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v5'
)
AS

    select customer_no, creation_date,
        sum(case when date_diff('month', inquiry_date, creation_date) <= 1 then 1 else 0 end) as enq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <= 2 then 1 else 0 end) as enq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <= 3 then 1 else 0 end) as enq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <= 6 then 1 else 0 end) as enq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <= 12 then 1 else 0 end) as enq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) then 1 else 0 end) as unsec_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) then 1 else 0 end) as unsec_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) then 1 else 0 end) as unsec_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) then 1 else 0 end) as unsec_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) then 1 else 0 end) as unsec_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) then 1 else 0 end) as sec_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) then 1 else 0 end) as sec_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) then 1 else 0 end) as sec_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) then 1 else 0 end) as sec_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) then 1 else 0 end) as sec_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (5,37,41) then 1 else 0 end) as pl_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (5,37,41) then 1 else 0 end) as pl_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (5,37,41) then 1 else 0 end) as pl_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (5,37,41) then 1 else 0 end) as pl_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (5,37,41) then 1 else 0 end) as pl_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (10,16,31) then 1 else 0 end) as cc_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (10,16,31) then 1 else 0 end) as cc_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (10,16,31) then 1 else 0 end) as cc_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (10,16,31) then 1 else 0 end) as cc_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (10,16,31) then 1 else 0 end) as cc_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (6) then 1 else 0 end) as cd_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (6) then 1 else 0 end) as cd_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (6) then 1 else 0 end) as cd_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (6) then 1 else 0 end) as cd_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (6) then 1 else 0 end) as cd_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (1) then 1 else 0 end) as al_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (1) then 1 else 0 end) as al_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (1) then 1 else 0 end) as al_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (1) then 1 else 0 end) as al_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (1) then 1 else 0 end) as al_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (7) then 1 else 0 end) as gl_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (7) then 1 else 0 end) as gl_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (7) then 1 else 0 end) as gl_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (7) then 1 else 0 end) as gl_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (7) then 1 else 0 end) as gl_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (2) then 1 else 0 end) as hl_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (2) then 1 else 0 end) as hl_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (2) then 1 else 0 end) as hl_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (2) then 1 else 0 end) as hl_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (2) then 1 else 0 end) as hl_inq_l12m,
        
        sum(case when date_diff('month', inquiry_date, creation_date) <=1 and inquiry_type in (13) then 1 else 0 end) as tw_inq_l1m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=2 and inquiry_type in (13) then 1 else 0 end) as tw_inq_l2m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=3 and inquiry_type in (13) then 1 else 0 end) as tw_inq_l3m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=6 and inquiry_type in (13) then 1 else 0 end) as tw_inq_l6m,
        sum(case when date_diff('month', inquiry_date, creation_date) <=12 and inquiry_type in (13) then 1 else 0 end) as tw_inq_l12m
        
        from fpl_ds_analytics.os_fow_inquiry_variables_creation_m2
        GROUP BY 1,2
""")

Executed!


In [65]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v6 --recursive
    
    

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v6/20250407_112152_22272_iin5u_ca945965-6a9f-4bc1-afa3-a58f342837f1


In [66]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v6
''')

Table Deleted!


0

In [67]:
df6=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v6
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_v6'
)
AS

SELECT customer_no, creation_date, 
COUNT(*) AS nbr_ever_accounts,
SUM(CASE WHEN active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_accounts,
SUM(CASE WHEN DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS accts_new_l1m,
SUM(CASE WHEN DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS accts_new_l3m,
SUM(CASE WHEN DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS accts_new_l6m,
SUM(CASE WHEN DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS accts_new_l12m,
SUM(CASE WHEN DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS accts_closed_l1m,
SUM(CASE WHEN DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS accts_closed_l3m,
SUM(CASE WHEN DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS accts_closed_l6m,
SUM(CASE WHEN DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS accts_closed_l12m,

SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN 1 ELSE 0 END) AS nbr_ever_secured_accounts,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) THEN 1 ELSE 0 END) AS nbr_active_secured_accounts,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS secured_accts_new_l1m,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS secured_accts_new_l3m,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS secured_accts_new_l6m,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS secured_accts_new_l12m,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS secured_accts_closed_l1m,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS secured_accts_closed_l3m,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS secured_accts_closed_l6m,
SUM(CASE WHEN account_type_symbol in (1,2,3,4,7,13,15,17,21,23,31,32,33,34,42,44,59,70,71) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS secured_accts_closed_l12m,

SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN 1 ELSE 0 END) AS nbr_ever_unsecured_accounts,
SUM(CASE WHEN active_inactive=1 AND account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) THEN 1 ELSE 0 END) AS nbr_active_unsecured_accounts,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS unsecured_accts_new_l1m,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS unsecured_accts_new_l3m,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS unsecured_accts_new_l6m,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS unsecured_accts_new_l12m,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS unsecured_accts_closed_l1m,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS unsecured_accts_closed_l3m,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS unsecured_accts_closed_l6m,
SUM(CASE WHEN account_type_symbol in (5, 8, 9, 10, 11, 12, 14, 16, 31, 35, 36, 37, 38, 39, 40, 41, 43, 50, 51, 52, 53, 54, 55, 56, 57, 58, 61, 60, 24, 45, 46, 6, 47, 69) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS unsecured_accts_closed_l12m,

MAX(CASE WHEN account_type_symbol=2 THEN 1 ELSE 0 END) AS ever_hl,
SUM(CASE WHEN account_type_symbol=2 THEN 1 ELSE 0 END) AS nbr_ever_hl,
MAX(CASE WHEN account_type_symbol=2 AND active_inactive=1 THEN 1 ELSE 0 END) AS active_hl,
SUM(CASE WHEN account_type_symbol=2 AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_hl,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS hl_new_l1m,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS hl_new_l3m,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS hl_new_l6m,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS hl_new_l12m,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS hl_closed_l1m,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS hl_closed_l3m,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS hl_closed_l6m,
SUM(CASE WHEN account_type_symbol=2 AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS hl_closed_l12m,

MAX(CASE WHEN account_type_symbol IN (5,37,41) THEN 1 ELSE 0 END) AS ever_pl,
SUM(CASE WHEN account_type_symbol IN (5,37,41) THEN 1 ELSE 0 END) AS nbr_ever_pl,
MAX(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_pl,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_pl,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS pl_new_l1m,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS pl_new_l3m,
SUM(CASE WHEN account_type_symbol IN (5,37,41)AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS pl_new_l6m,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS pl_new_l12m,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS pl_closed_l1m,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS pl_closed_l3m,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS pl_closed_l6m,
SUM(CASE WHEN account_type_symbol IN (5,37,41) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS pl_closed_l12m,

MAX(CASE WHEN account_type_symbol IN (10,16,31) THEN 1 ELSE 0 END) AS ever_cc,
SUM(CASE WHEN account_type_symbol IN (10,16,31) THEN 1 ELSE 0 END) AS nbr_ever_cc,
MAX(CASE WHEN account_type_symbol IN (10,16,31) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_cc,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_cc,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS cc_new_l1m,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS cc_new_l3m,
SUM(CASE WHEN account_type_symbol IN (10,16,31)AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS cc_new_l6m,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS cc_new_l12m,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS cc_closed_l1m,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS cc_closed_l3m,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS cc_closed_l6m,
SUM(CASE WHEN account_type_symbol IN (10,16,31) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS cc_closed_l12m,

MAX(CASE WHEN account_type_symbol IN (6) THEN 1 ELSE 0 END) AS ever_cd,
SUM(CASE WHEN account_type_symbol IN (6) THEN 1 ELSE 0 END) AS nbr_ever_cd,
MAX(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_cd,
SUM(CASE WHEN account_type_symbol IN (6) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_cd,
SUM(CASE WHEN account_type_symbol IN (6) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS cd_new_l1m,
SUM(CASE WHEN account_type_symbol IN (6) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS cd_new_l3m,
SUM(CASE WHEN account_type_symbol IN (6)AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS cd_new_l6m,
SUM(CASE WHEN account_type_symbol IN (6) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS cd_new_l12m,
SUM(CASE WHEN account_type_symbol IN (6) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS cd_closed_l1m,
SUM(CASE WHEN account_type_symbol IN (6) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS cd_closed_l3m,
SUM(CASE WHEN account_type_symbol IN (6) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS cd_closed_l6m,
SUM(CASE WHEN account_type_symbol IN (6) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS cd_closed_l12m,

MAX(CASE WHEN account_type_symbol IN (7) THEN 1 ELSE 0 END) AS ever_gl,
SUM(CASE WHEN account_type_symbol IN (7) THEN 1 ELSE 0 END) AS nbr_ever_gl,
MAX(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_gl,
SUM(CASE WHEN account_type_symbol IN (7) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_gl,
SUM(CASE WHEN account_type_symbol IN (7) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS gl_new_l1m,
SUM(CASE WHEN account_type_symbol IN (7) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS gl_new_l3m,
SUM(CASE WHEN account_type_symbol IN (7)AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS gl_new_l6m,
SUM(CASE WHEN account_type_symbol IN (7) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS gl_new_l12m,
SUM(CASE WHEN account_type_symbol IN (7) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS gl_closed_l1m,
SUM(CASE WHEN account_type_symbol IN (7) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS gl_closed_l3m,
SUM(CASE WHEN account_type_symbol IN (7) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS gl_closed_l6m,
SUM(CASE WHEN account_type_symbol IN (7) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS gl_closed_l12m,

MAX(CASE WHEN account_type_symbol IN (1) THEN 1 ELSE 0 END) AS ever_al,
SUM(CASE WHEN account_type_symbol IN (1) THEN 1 ELSE 0 END) AS nbr_ever_al,
MAX(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_al,
SUM(CASE WHEN account_type_symbol IN (1) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_al,
SUM(CASE WHEN account_type_symbol IN (1) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS al_new_l1m,
SUM(CASE WHEN account_type_symbol IN (1) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS al_new_l3m,
SUM(CASE WHEN account_type_symbol IN (1)AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS al_new_l6m,
SUM(CASE WHEN account_type_symbol IN (1) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS al_new_l12m,
SUM(CASE WHEN account_type_symbol IN (1) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS al_closed_l1m,
SUM(CASE WHEN account_type_symbol IN (1) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS al_closed_l3m,
SUM(CASE WHEN account_type_symbol IN (1) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS al_closed_l6m,
SUM(CASE WHEN account_type_symbol IN (1) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS al_closed_l12m,

MAX(CASE WHEN account_type_symbol IN (13) THEN 1 ELSE 0 END) AS ever_tw,
SUM(CASE WHEN account_type_symbol IN (13) THEN 1 ELSE 0 END) AS nbr_ever_tw,
MAX(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_tw,
SUM(CASE WHEN account_type_symbol IN (13) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_tw,
SUM(CASE WHEN account_type_symbol IN (13) AND DATE_DIFF('month', date_opened, creation_date)<=1 THEN 1 ELSE 0 END) AS tw_new_l1m,
SUM(CASE WHEN account_type_symbol IN (13) AND DATE_DIFF('month', date_opened, creation_date)<=3 THEN 1 ELSE 0 END) AS tw_new_l3m,
SUM(CASE WHEN account_type_symbol IN (13)AND DATE_DIFF('month', date_opened, creation_date)<=6 THEN 1 ELSE 0 END) AS tw_new_l6m,
SUM(CASE WHEN account_type_symbol IN (13) AND DATE_DIFF('month', date_opened, creation_date)<=12 THEN 1 ELSE 0 END) AS tw_new_l12m,
SUM(CASE WHEN account_type_symbol IN (13) AND DATE_DIFF('month', date_closed, creation_date)<=1 THEN 1 ELSE 0 END) AS tw_closed_l1m,
SUM(CASE WHEN account_type_symbol IN (13) AND DATE_DIFF('month', date_closed, creation_date)<=3 THEN 1 ELSE 0 END) AS tw_closed_l3m,
SUM(CASE WHEN account_type_symbol IN (13) AND DATE_DIFF('month', date_closed, creation_date)<=6 THEN 1 ELSE 0 END) AS tw_closed_l6m,
SUM(CASE WHEN account_type_symbol IN (13) AND DATE_DIFF('month', date_closed, creation_date)<=12 THEN 1 ELSE 0 END) AS tw_closed_l12m,

sum(case when (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as nbr_ever_stpl,
sum(case when active_inactive=1 AND (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as nbr_active_stpl,
sum(case when date_diff('month', date_opened, creation_date)<=1 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_new_l1m,
sum(case when date_diff('month', date_opened, creation_date)<=3 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_new_l3m,
sum(case when date_diff('month', date_opened, creation_date)<=6 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_new_l6m,
sum(case when date_diff('month', date_opened, creation_date)<=12 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_new_l12m,
sum(case when date_diff('month', date_closed, creation_date)<=1 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_closed_l1m,
sum(case when date_diff('month', date_closed, creation_date)<=3 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_closed_l3m,
sum(case when date_diff('month', date_closed, creation_date)<=6 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_closed_l6m,
sum(case when date_diff('month', date_closed, creation_date)<=12 and (account_type_symbol = 69 or (account_type_symbol in (5,37,41) and greatest(high_balance, current_balance)<50000)) then 1 else 0 end) as stpl_closed_l12m,

MAX(CASE WHEN account_type_symbol IN (17) THEN 1 ELSE 0 END) AS ever_cv,
SUM(CASE WHEN account_type_symbol IN (17) THEN 1 ELSE 0 END) AS nbr_ever_cv,
MAX(CASE WHEN account_type_symbol IN (17) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_cv,
SUM(CASE WHEN account_type_symbol IN (17) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_cv,
MAX(CASE WHEN account_type_symbol IN (53,57) THEN 1 ELSE 0 END) AS ever_agri,
SUM(CASE WHEN account_type_symbol IN (53,57) THEN 1 ELSE 0 END) AS nbr_ever_agri,
MAX(CASE WHEN account_type_symbol IN (53,57) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_agri,
SUM(CASE WHEN account_type_symbol IN (53,57) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_agri,
MAX(CASE WHEN account_type_symbol IN (3) THEN 1 ELSE 0 END) AS ever_lap,
SUM(CASE WHEN account_type_symbol IN (3) THEN 1 ELSE 0 END) AS nbr_ever_lap,
MAX(CASE WHEN account_type_symbol IN (3) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_lap,
SUM(CASE WHEN account_type_symbol IN (3) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_lap,
MAX(CASE WHEN account_type_symbol IN (4) THEN 1 ELSE 0 END) AS ever_las,
SUM(CASE WHEN account_type_symbol IN (4) THEN 1 ELSE 0 END) AS nbr_ever_las,
MAX(CASE WHEN account_type_symbol IN (4) AND active_inactive=1 THEN 1 ELSE 0 END) AS active_las,
SUM(CASE WHEN account_type_symbol IN (4) AND active_inactive=1 THEN 1 ELSE 0 END) AS nbr_active_las

FROM fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v1
GROUP BY 1,2
""")

Executed!


In [68]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_overall --recursive
    
    

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_overall/20250407_112155_22281_iin5u_cba3dedd-60b5-4683-895d-869a7ad9e3df


In [69]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_overall
''')

Table Deleted!


0

In [70]:
df7=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_overall
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_data_overall'
)
AS
SELECT *
FROM fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v2
INNER JOIN fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v3 USING(customer_no, creation_date)
INNER JOIN fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v4 USING(customer_no, creation_date)
INNER JOIN fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v5 USING(customer_no, creation_date)
INNER JOIN fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_v6 USING(customer_no, creation_date)

""")

Executed!


In [71]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_m1_data_overall --recursive
    
    

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_m1_data_overall/20250407_112200_22296_iin5u_fd18b95b-fd61-4e32-a30f-baf0c58a4cd1


In [72]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_m1_data_overall
''')

Table Deleted!


0

In [73]:
df8=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_m1_data_overall
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_m1_data_overall'
)
AS

SELECT *
FROM(
SELECT customer_no,
risk_score AS risk_score_x,
creation_date	creation_date_x	,
max_dpd_l36m	max_dpd_l36m_x	,
max_dpd_l24m	max_dpd_l24m_x	,
max_dpd_l18m	max_dpd_l18m_x	,
max_dpd_l12m	max_dpd_l12m_x	,
max_dpd_l6m	max_dpd_l6m_x	,
max_dpd_l3m	max_dpd_l3m_x	,
max_dpd_cm	max_dpd_cm_x	,
max_active_dpd_l36m	max_active_dpd_l36m_x	,
max_active_dpd_l24m	max_active_dpd_l24m_x	,
max_active_dpd_l18m	max_active_dpd_l18m_x	,
max_active_dpd_l12m	max_active_dpd_l12m_x	,
max_active_dpd_l6m	max_active_dpd_l6m_x	,
max_active_dpd_l3m	max_active_dpd_l3m_x	,
max_active_dpd_cm	max_active_dpd_cm_x	,
max_active_cc_dpd_l36m	max_active_cc_dpd_l36m_x	,
max_active_cc_dpd_l24m	max_active_cc_dpd_l24m_x	,
max_active_cc_dpd_l18m	max_active_cc_dpd_l18m_x	,
max_active_cc_dpd_l12m	max_active_cc_dpd_l12m_x	,
max_active_cc_dpd_l6m	max_active_cc_dpd_l6m_x	,
max_active_cc_dpd_l3m	max_active_cc_dpd_l3m_x	,
max_active_cc_dpd_cm	max_active_cc_dpd_cm_x	,
max_cc_dpd_l36m	max_cc_dpd_l36m_x	,
max_cc_dpd_l24m	max_cc_dpd_l24m_x	,
max_cc_dpd_l18m	max_cc_dpd_l18m_x	,
max_cc_dpd_l12m	max_cc_dpd_l12m_x	,
max_cc_dpd_l6m	max_cc_dpd_l6m_x	,
max_cc_dpd_l3m	max_cc_dpd_l3m_x	,
max_cc_dpd_cm	max_cc_dpd_cm_x	,
max_active_pl_dpd_l36m	max_active_pl_dpd_l36m_x	,
max_active_pl_dpd_l24m	max_active_pl_dpd_l24m_x	,
max_active_pl_dpd_l18m	max_active_pl_dpd_l18m_x	,
max_active_pl_dpd_l12m	max_active_pl_dpd_l12m_x	,
max_active_pl_dpd_l6m	max_active_pl_dpd_l6m_x	,
max_active_pl_dpd_l3m	max_active_pl_dpd_l3m_x	,
max_active_pl_dpd_cm	max_active_pl_dpd_cm_x	,
max_pl_dpd_l36m	max_pl_dpd_l36m_x	,
max_pl_dpd_l24m	max_pl_dpd_l24m_x	,
max_pl_dpd_l18m	max_pl_dpd_l18m_x	,
max_pl_dpd_l12m	max_pl_dpd_l12m_x	,
max_pl_dpd_l6m	max_pl_dpd_l6m_x	,
max_pl_dpd_l3m	max_pl_dpd_l3m_x	,
max_pl_dpd_cm	max_pl_dpd_cm_x	,
max_active_cd_dpd_l36m	max_active_cd_dpd_l36m_x	,
max_active_cd_dpd_l24m	max_active_cd_dpd_l24m_x	,
max_active_cd_dpd_l18m	max_active_cd_dpd_l18m_x	,
max_active_cd_dpd_l12m	max_active_cd_dpd_l12m_x	,
max_active_cd_dpd_l6m	max_active_cd_dpd_l6m_x	,
max_active_cd_dpd_l3m	max_active_cd_dpd_l3m_x	,
max_active_cd_dpd_cm	max_active_cd_dpd_cm_x	,
max_cd_dpd_l36m	max_cd_dpd_l36m_x	,
max_cd_dpd_l24m	max_cd_dpd_l24m_x	,
max_cd_dpd_l18m	max_cd_dpd_l18m_x	,
max_cd_dpd_l12m	max_cd_dpd_l12m_x	,
max_cd_dpd_l6m	max_cd_dpd_l6m_x	,
max_cd_dpd_l3m	max_cd_dpd_l3m_x	,
max_cd_dpd_cm	max_cd_dpd_cm_x	,
max_active_gl_dpd_l36m	max_active_gl_dpd_l36m_x	,
max_active_gl_dpd_l24m	max_active_gl_dpd_l24m_x	,
max_active_gl_dpd_l18m	max_active_gl_dpd_l18m_x	,
max_active_gl_dpd_l12m	max_active_gl_dpd_l12m_x	,
max_active_gl_dpd_l6m	max_active_gl_dpd_l6m_x	,
max_active_gl_dpd_l3m	max_active_gl_dpd_l3m_x	,
max_active_gl_dpd_cm	max_active_gl_dpd_cm_x	,
max_gl_dpd_l36m	max_gl_dpd_l36m_x	,
max_gl_dpd_l24m	max_gl_dpd_l24m_x	,
max_gl_dpd_l18m	max_gl_dpd_l18m_x	,
max_gl_dpd_l12m	max_gl_dpd_l12m_x	,
max_gl_dpd_l6m	max_gl_dpd_l6m_x	,
max_gl_dpd_l3m	max_gl_dpd_l3m_x	,
max_gl_dpd_cm	max_gl_dpd_cm_x	,
max_active_al_dpd_l36m	max_active_al_dpd_l36m_x	,
max_active_al_dpd_l24m	max_active_al_dpd_l24m_x	,
max_active_al_dpd_l18m	max_active_al_dpd_l18m_x	,
max_active_al_dpd_l12m	max_active_al_dpd_l12m_x	,
max_active_al_dpd_l6m	max_active_al_dpd_l6m_x	,
max_active_al_dpd_l3m	max_active_al_dpd_l3m_x	,
max_active_al_dpd_cm	max_active_al_dpd_cm_x	,
max_al_dpd_l36m	max_al_dpd_l36m_x	,
max_al_dpd_l24m	max_al_dpd_l24m_x	,
max_al_dpd_l18m	max_al_dpd_l18m_x	,
max_al_dpd_l12m	max_al_dpd_l12m_x	,
max_al_dpd_l6m	max_al_dpd_l6m_x	,
max_al_dpd_l3m	max_al_dpd_l3m_x	,
max_al_dpd_cm	max_al_dpd_cm_x	,
max_active_tw_dpd_l36m	max_active_tw_dpd_l36m_x	,
max_active_tw_dpd_l24m	max_active_tw_dpd_l24m_x	,
max_active_tw_dpd_l18m	max_active_tw_dpd_l18m_x	,
max_active_tw_dpd_l12m	max_active_tw_dpd_l12m_x	,
max_active_tw_dpd_l6m	max_active_tw_dpd_l6m_x	,
max_active_tw_dpd_l3m	max_active_tw_dpd_l3m_x	,
max_active_tw_dpd_cm	max_active_tw_dpd_cm_x	,
max_tw_dpd_l36m	max_tw_dpd_l36m_x	,
max_tw_dpd_l24m	max_tw_dpd_l24m_x	,
max_tw_dpd_l18m	max_tw_dpd_l18m_x	,
max_tw_dpd_l12m	max_tw_dpd_l12m_x	,
max_tw_dpd_l6m	max_tw_dpd_l6m_x	,
max_tw_dpd_l3m	max_tw_dpd_l3m_x	,
max_tw_dpd_cm	max_tw_dpd_cm_x	,
max_active_hl_dpd_l36m	max_active_hl_dpd_l36m_x	,
max_active_hl_dpd_l24m	max_active_hl_dpd_l24m_x	,
max_active_hl_dpd_l18m	max_active_hl_dpd_l18m_x	,
max_active_hl_dpd_l12m	max_active_hl_dpd_l12m_x	,
max_active_hl_dpd_l6m	max_active_hl_dpd_l6m_x	,
max_active_hl_dpd_l3m	max_active_hl_dpd_l3m_x	,
max_active_hl_dpd_cm	max_active_hl_dpd_cm_x	,
max_hl_dpd_l36m	max_hl_dpd_l36m_x	,
max_hl_dpd_l24m	max_hl_dpd_l24m_x	,
max_hl_dpd_l18m	max_hl_dpd_l18m_x	,
max_hl_dpd_l12m	max_hl_dpd_l12m_x	,
max_hl_dpd_l6m	max_hl_dpd_l6m_x	,
max_hl_dpd_l3m	max_hl_dpd_l3m_x	,
max_hl_dpd_cm	max_hl_dpd_cm_x	,
max_active_unsecured_dpd_l36m	max_active_unsecured_dpd_l36m_x	,
max_active_unsecured_dpd_l24m	max_active_unsecured_dpd_l24m_x	,
max_active_unsecured_dpd_l18m	max_active_unsecured_dpd_l18m_x	,
max_active_unsecured_dpd_l12m	max_active_unsecured_dpd_l12m_x	,
max_active_unsecured_dpd_l6m	max_active_unsecured_dpd_l6m_x	,
max_active_unsecured_dpd_l3m	max_active_unsecured_dpd_l3m_x	,
max_active_unsecured_dpd_cm	max_active_unsecured_dpd_cm_x	,
max_unsecured_dpd_l36m	max_unsecured_dpd_l36m_x	,
max_unsecured_dpd_l24m	max_unsecured_dpd_l24m_x	,
max_unsecured_dpd_l18m	max_unsecured_dpd_l18m_x	,
max_unsecured_dpd_l12m	max_unsecured_dpd_l12m_x	,
max_unsecured_dpd_l6m	max_unsecured_dpd_l6m_x	,
max_unsecured_dpd_l3m	max_unsecured_dpd_l3m_x	,
max_unsecured_dpd_cm	max_unsecured_dpd_cm_x	,
max_active_secured_dpd_l36m	max_active_secured_dpd_l36m_x	,
max_active_secured_dpd_l24m	max_active_secured_dpd_l24m_x	,
max_active_secured_dpd_l18m	max_active_secured_dpd_l18m_x	,
max_active_secured_dpd_l12m	max_active_secured_dpd_l12m_x	,
max_active_secured_dpd_l6m	max_active_secured_dpd_l6m_x	,
max_active_secured_dpd_l3m	max_active_secured_dpd_l3m_x	,
max_active_secured_dpd_cm	max_active_secured_dpd_cm_x	,
max_secured_dpd_l36m	max_secured_dpd_l36m_x	,
max_secured_dpd_l24m	max_secured_dpd_l24m_x	,
max_secured_dpd_l18m	max_secured_dpd_l18m_x	,
max_secured_dpd_l12m	max_secured_dpd_l12m_x	,
max_secured_dpd_l6m	max_secured_dpd_l6m_x	,
max_secured_dpd_l3m	max_secured_dpd_l3m_x	,
max_secured_dpd_cm	max_secured_dpd_cm_x	,
overdue_l36m	overdue_l36m_x	,
overdue_l24m	overdue_l24m_x	,
overdue_l18m	overdue_l18m_x	,
overdue_l12m	overdue_l12m_x	,
overdue_l9m	overdue_l9m_x	,
overdue_l6m	overdue_l6m_x	,
overdue_l3m	overdue_l3m_x	,
overdue_cm	overdue_cm_x	,
cc_overdue_l36m	cc_overdue_l36m_x	,
cc_overdue_l24m	cc_overdue_l24m_x	,
cc_overdue_l18m	cc_overdue_l18m_x	,
cc_overdue_l12m	cc_overdue_l12m_x	,
cc_overdue_l9m	cc_overdue_l9m_x	,
cc_overdue_l6m	cc_overdue_l6m_x	,
cc_overdue_l3m	cc_overdue_l3m_x	,
cc_overdue_cm	cc_overdue_cm_x	,
pl_overdue_l36m	pl_overdue_l36m_x	,
pl_overdue_l24m	pl_overdue_l24m_x	,
pl_overdue_l18m	pl_overdue_l18m_x	,
pl_overdue_l12m	pl_overdue_l12m_x	,
pl_overdue_l9m	pl_overdue_l9m_x	,
pl_overdue_l6m	pl_overdue_l6m_x	,
pl_overdue_l3m	pl_overdue_l3m_x	,
pl_overdue_cm	pl_overdue_cm_x	,
cd_overdue_l36m	cd_overdue_l36m_x	,
cd_overdue_l24m	cd_overdue_l24m_x	,
cd_overdue_l18m	cd_overdue_l18m_x	,
cd_overdue_l12m	cd_overdue_l12m_x	,
cd_overdue_l9m	cd_overdue_l9m_x	,
cd_overdue_l6m	cd_overdue_l6m_x	,
cd_overdue_l3m	cd_overdue_l3m_x	,
cd_overdue_cm	cd_overdue_cm_x	,
gl_overdue_l36m	gl_overdue_l36m_x	,
gl_overdue_l24m	gl_overdue_l24m_x	,
gl_overdue_l18m	gl_overdue_l18m_x	,
gl_overdue_l12m	gl_overdue_l12m_x	,
gl_overdue_l9m	gl_overdue_l9m_x	,
gl_overdue_l6m	gl_overdue_l6m_x	,
gl_overdue_l3m	gl_overdue_l3m_x	,
gl_overdue_cm	gl_overdue_cm_x	,
al_overdue_l36m	al_overdue_l36m_x	,
al_overdue_l24m	al_overdue_l24m_x	,
al_overdue_l18m	al_overdue_l18m_x	,
al_overdue_l12m	al_overdue_l12m_x	,
al_overdue_l9m	al_overdue_l9m_x	,
al_overdue_l6m	al_overdue_l6m_x	,
al_overdue_l3m	al_overdue_l3m_x	,
al_overdue_cm	al_overdue_cm_x	,
tw_overdue_l36m	tw_overdue_l36m_x	,
tw_overdue_l24m	tw_overdue_l24m_x	,
tw_overdue_l18m	tw_overdue_l18m_x	,
tw_overdue_l12m	tw_overdue_l12m_x	,
tw_overdue_l9m	tw_overdue_l9m_x	,
tw_overdue_l6m	tw_overdue_l6m_x	,
tw_overdue_l3m	tw_overdue_l3m_x	,
tw_overdue_cm	tw_overdue_cm_x	,
hl_overdue_l36m	hl_overdue_l36m_x	,
hl_overdue_l24m	hl_overdue_l24m_x	,
hl_overdue_l18m	hl_overdue_l18m_x	,
hl_overdue_l12m	hl_overdue_l12m_x	,
hl_overdue_l9m	hl_overdue_l9m_x	,
hl_overdue_l6m	hl_overdue_l6m_x	,
hl_overdue_l3m	hl_overdue_l3m_x	,
hl_overdue_cm	hl_overdue_cm_x	,
unsecured_overdue_l36m	unsecured_overdue_l36m_x	,
unsecured_overdue_l24m	unsecured_overdue_l24m_x	,
unsecured_overdue_l18m	unsecured_overdue_l18m_x	,
unsecured_overdue_l12m	unsecured_overdue_l12m_x	,
unsecured_overdue_l9m	unsecured_overdue_l9m_x	,
unsecured_overdue_l6m	unsecured_overdue_l6m_x	,
unsecured_overdue_l3m	unsecured_overdue_l3m_x	,
unsecured_overdue_cm	unsecured_overdue_cm_x	,
secured_overdue_l36m	secured_overdue_l36m_x	,
secured_overdue_l24m	secured_overdue_l24m_x	,
secured_overdue_l18m	secured_overdue_l18m_x	,
secured_overdue_l12m	secured_overdue_l12m_x	,
secured_overdue_l9m	secured_overdue_l9m_x	,
secured_overdue_l6m	secured_overdue_l6m_x	,
secured_overdue_l3m	secured_overdue_l3m_x	,
secured_overdue_cm	secured_overdue_cm_x	,
nbr_x_dpd_l36m	nbr_x_dpd_l36m_x	,
nbr_x_dpd_l24m	nbr_x_dpd_l24m_x	,
nbr_x_dpd_l18m	nbr_x_dpd_l18m_x	,
nbr_x_dpd_l12m	nbr_x_dpd_l12m_x	,
nbr_x_dpd_l6m	nbr_x_dpd_l6m_x	,
nbr_x_dpd_l3m	nbr_x_dpd_l3m_x	,
nbr_x_dpd_cm	nbr_x_dpd_cm_x	,
nbr_30_dpd_l36m	nbr_30_dpd_l36m_x	,
nbr_30_dpd_l24m	nbr_30_dpd_l24m_x	,
nbr_30_dpd_l18m	nbr_30_dpd_l18m_x	,
nbr_30_dpd_l12m	nbr_30_dpd_l12m_x	,
nbr_30_dpd_l6m	nbr_30_dpd_l6m_x	,
nbr_30_dpd_l3m	nbr_30_dpd_l3m_x	,
nbr_30_dpd_cm	nbr_30_dpd_cm_x	,
nbr_90_dpd_l36m	nbr_90_dpd_l36m_x	,
nbr_90_dpd_l24m	nbr_90_dpd_l24m_x	,
nbr_90_dpd_l18m	nbr_90_dpd_l18m_x	,
nbr_90_dpd_l12m	nbr_90_dpd_l12m_x	,
nbr_90_dpd_l6m	nbr_90_dpd_l6m_x	,
nbr_90_dpd_l3m	nbr_90_dpd_l3m_x	,
nbr_90_dpd_cm	nbr_90_dpd_cm_x	,
nbr_active_x_dpd_l36m	nbr_active_x_dpd_l36m_x	,
nbr_active_x_dpd_l24m	nbr_active_x_dpd_l24m_x	,
nbr_active_x_dpd_l18m	nbr_active_x_dpd_l18m_x	,
nbr_active_x_dpd_l12m	nbr_active_x_dpd_l12m_x	,
nbr_active_x_dpd_l6m	nbr_active_x_dpd_l6m_x	,
nbr_active_x_dpd_l3m	nbr_active_x_dpd_l3m_x	,
nbr_active_x_dpd_cm	nbr_active_x_dpd_cm_x	,
nbr_active_30_dpd_l36m	nbr_active_30_dpd_l36m_x	,
nbr_active_30_dpd_l24m	nbr_active_30_dpd_l24m_x	,
nbr_active_30_dpd_l18m	nbr_active_30_dpd_l18m_x	,
nbr_active_30_dpd_l12m	nbr_active_30_dpd_l12m_x	,
nbr_active_30_dpd_l6m	nbr_active_30_dpd_l6m_x	,
nbr_active_30_dpd_l3m	nbr_active_30_dpd_l3m_x	,
nbr_active_30_dpd_cm	nbr_active_30_dpd_cm_x	,
nbr_active_90_dpd_l36m	nbr_active_90_dpd_l36m_x	,
nbr_active_90_dpd_l24m	nbr_active_90_dpd_l24m_x	,
nbr_active_90_dpd_l18m	nbr_active_90_dpd_l18m_x	,
nbr_active_90_dpd_l12m	nbr_active_90_dpd_l12m_x	,
nbr_active_90_dpd_l6m	nbr_active_90_dpd_l6m_x	,
nbr_active_90_dpd_l3m	nbr_active_90_dpd_l3m_x	,
nbr_active_90_dpd_cm	nbr_active_90_dpd_cm_x	,
nbr_cc_x_dpd_l36m	nbr_cc_x_dpd_l36m_x	,
nbr_cc_x_dpd_l24m	nbr_cc_x_dpd_l24m_x	,
nbr_cc_x_dpd_l18m	nbr_cc_x_dpd_l18m_x	,
nbr_cc_x_dpd_l12m	nbr_cc_x_dpd_l12m_x	,
nbr_cc_x_dpd_l6m	nbr_cc_x_dpd_l6m_x	,
nbr_cc_x_dpd_l3m	nbr_cc_x_dpd_l3m_x	,
nbr_cc_x_dpd_cm	nbr_cc_x_dpd_cm_x	,
nbr_cc_30_dpd_l36m	nbr_cc_30_dpd_l36m_x	,
nbr_cc_30_dpd_l24m	nbr_cc_30_dpd_l24m_x	,
nbr_cc_30_dpd_l18m	nbr_cc_30_dpd_l18m_x	,
nbr_cc_30_dpd_l12m	nbr_cc_30_dpd_l12m_x	,
nbr_cc_30_dpd_l6m	nbr_cc_30_dpd_l6m_x	,
nbr_cc_30_dpd_l3m	nbr_cc_30_dpd_l3m_x	,
nbr_cc_30_dpd_cm	nbr_cc_30_dpd_cm_x	,
nbr_cc_90_dpd_l36m	nbr_cc_90_dpd_l36m_x	,
nbr_cc_90_dpd_l24m	nbr_cc_90_dpd_l24m_x	,
nbr_cc_90_dpd_l18m	nbr_cc_90_dpd_l18m_x	,
nbr_cc_90_dpd_l12m	nbr_cc_90_dpd_l12m_x	,
nbr_cc_90_dpd_l6m	nbr_cc_90_dpd_l6m_x	,
nbr_cc_90_dpd_l3m	nbr_cc_90_dpd_l3m_x	,
nbr_cc_90_dpd_cm	nbr_cc_90_dpd_cm_x	,
nbr_active_cc_x_dpd_l36m	nbr_active_cc_x_dpd_l36m_x	,
nbr_active_cc_x_dpd_l24m	nbr_active_cc_x_dpd_l24m_x	,
nbr_active_cc_x_dpd_l18m	nbr_active_cc_x_dpd_l18m_x	,
nbr_active_cc_x_dpd_l12m	nbr_active_cc_x_dpd_l12m_x	,
nbr_active_cc_x_dpd_l6m	nbr_active_cc_x_dpd_l6m_x	,
nbr_active_cc_x_dpd_l3m	nbr_active_cc_x_dpd_l3m_x	,
nbr_active_cc_x_dpd_cm	nbr_active_cc_x_dpd_cm_x	,
nbr_active_cc_30_dpd_l36m	nbr_active_cc_30_dpd_l36m_x	,
nbr_active_cc_30_dpd_l24m	nbr_active_cc_30_dpd_l24m_x	,
nbr_active_cc_30_dpd_l18m	nbr_active_cc_30_dpd_l18m_x	,
nbr_active_cc_30_dpd_l12m	nbr_active_cc_30_dpd_l12m_x	,
nbr_active_cc_30_dpd_l6m	nbr_active_cc_30_dpd_l6m_x	,
nbr_active_cc_30_dpd_l3m	nbr_active_cc_30_dpd_l3m_x	,
nbr_active_cc_30_dpd_cm	nbr_active_cc_30_dpd_cm_x	,
nbr_cc_active_90_dpd_l36m	nbr_cc_active_90_dpd_l36m_x	,
nbr_cc_active_90_dpd_l24m	nbr_cc_active_90_dpd_l24m_x	,
nbr_cc_active_90_dpd_l18m	nbr_cc_active_90_dpd_l18m_x	,
nbr_cc_active_90_dpd_l12m	nbr_cc_active_90_dpd_l12m_x	,
nbr_cc_active_90_dpd_l6m	nbr_cc_active_90_dpd_l6m_x	,
nbr_cc_active_90_dpd_l3m	nbr_cc_active_90_dpd_l3m_x	,
nbr_cc_active_90_dpd_cm	nbr_cc_active_90_dpd_cm_x	,
nbr_pl_x_dpd_l36m	nbr_pl_x_dpd_l36m_x	,
nbr_pl_x_dpd_l24m	nbr_pl_x_dpd_l24m_x	,
nbr_pl_x_dpd_l18m	nbr_pl_x_dpd_l18m_x	,
nbr_pl_x_dpd_l12m	nbr_pl_x_dpd_l12m_x	,
nbr_pl_x_dpd_l6m	nbr_pl_x_dpd_l6m_x	,
nbr_pl_x_dpd_l3m	nbr_pl_x_dpd_l3m_x	,
nbr_pl_x_dpd_cm	nbr_pl_x_dpd_cm_x	,
nbr_pl_30_dpd_l36m	nbr_pl_30_dpd_l36m_x	,
nbr_pl_30_dpd_l24m	nbr_pl_30_dpd_l24m_x	,
nbr_pl_30_dpd_l18m	nbr_pl_30_dpd_l18m_x	,
nbr_pl_30_dpd_l12m	nbr_pl_30_dpd_l12m_x	,
nbr_pl_30_dpd_l6m	nbr_pl_30_dpd_l6m_x	,
nbr_pl_30_dpd_l3m	nbr_pl_30_dpd_l3m_x	,
nbr_pl_30_dpd_cm	nbr_pl_30_dpd_cm_x	,
nbr_pl_90_dpd_l36m	nbr_pl_90_dpd_l36m_x	,
nbr_pl_90_dpd_l24m	nbr_pl_90_dpd_l24m_x	,
nbr_pl_90_dpd_l18m	nbr_pl_90_dpd_l18m_x	,
nbr_pl_90_dpd_l12m	nbr_pl_90_dpd_l12m_x	,
nbr_pl_90_dpd_l6m	nbr_pl_90_dpd_l6m_x	,
nbr_pl_90_dpd_l3m	nbr_pl_90_dpd_l3m_x	,
nbr_pl_90_dpd_cm	nbr_pl_90_dpd_cm_x	,
nbr_active_pl_x_dpd_l36m	nbr_active_pl_x_dpd_l36m_x	,
nbr_active_pl_x_dpd_l24m	nbr_active_pl_x_dpd_l24m_x	,
nbr_active_pl_x_dpd_l18m	nbr_active_pl_x_dpd_l18m_x	,
nbr_active_pl_x_dpd_l12m	nbr_active_pl_x_dpd_l12m_x	,
nbr_active_pl_x_dpd_l6m	nbr_active_pl_x_dpd_l6m_x	,
nbr_active_pl_x_dpd_l3m	nbr_active_pl_x_dpd_l3m_x	,
nbr_active_pl_x_dpd_cm	nbr_active_pl_x_dpd_cm_x	,
nbr_active_pl_30_dpd_l36m	nbr_active_pl_30_dpd_l36m_x	,
nbr_active_pl_30_dpd_l24m	nbr_active_pl_30_dpd_l24m_x	,
nbr_active_pl_30_dpd_l18m	nbr_active_pl_30_dpd_l18m_x	,
nbr_active_pl_30_dpd_l12m	nbr_active_pl_30_dpd_l12m_x	,
nbr_active_pl_30_dpd_l6m	nbr_active_pl_30_dpd_l6m_x	,
nbr_active_pl_30_dpd_l3m	nbr_active_pl_30_dpd_l3m_x	,
nbr_active_pl_30_dpd_cm	nbr_active_pl_30_dpd_cm_x	,
nbr_pl_active_90_dpd_l36m	nbr_pl_active_90_dpd_l36m_x	,
nbr_pl_active_90_dpd_l24m	nbr_pl_active_90_dpd_l24m_x	,
nbr_pl_active_90_dpd_l18m	nbr_pl_active_90_dpd_l18m_x	,
nbr_pl_active_90_dpd_l12m	nbr_pl_active_90_dpd_l12m_x	,
nbr_pl_active_90_dpd_l6m	nbr_pl_active_90_dpd_l6m_x	,
nbr_pl_active_90_dpd_l3m	nbr_pl_active_90_dpd_l3m_x	,
nbr_pl_active_90_dpd_cm	nbr_pl_active_90_dpd_cm_x	,
nbr_cd_x_dpd_l36m	nbr_cd_x_dpd_l36m_x	,
nbr_cd_x_dpd_l24m	nbr_cd_x_dpd_l24m_x	,
nbr_cd_x_dpd_l18m	nbr_cd_x_dpd_l18m_x	,
nbr_cd_x_dpd_l12m	nbr_cd_x_dpd_l12m_x	,
nbr_cd_x_dpd_l6m	nbr_cd_x_dpd_l6m_x	,
nbr_cd_x_dpd_l3m	nbr_cd_x_dpd_l3m_x	,
nbr_cd_x_dpd_cm	nbr_cd_x_dpd_cm_x	,
nbr_cd_30_dpd_l36m	nbr_cd_30_dpd_l36m_x	,
nbr_cd_30_dpd_l24m	nbr_cd_30_dpd_l24m_x	,
nbr_cd_30_dpd_l18m	nbr_cd_30_dpd_l18m_x	,
nbr_cd_30_dpd_l12m	nbr_cd_30_dpd_l12m_x	,
nbr_cd_30_dpd_l6m	nbr_cd_30_dpd_l6m_x	,
nbr_cd_30_dpd_l3m	nbr_cd_30_dpd_l3m_x	,
nbr_cd_30_dpd_cm	nbr_cd_30_dpd_cm_x	,
nbr_cd_90_dpd_l36m	nbr_cd_90_dpd_l36m_x	,
nbr_cd_90_dpd_l24m	nbr_cd_90_dpd_l24m_x	,
nbr_cd_90_dpd_l18m	nbr_cd_90_dpd_l18m_x	,
nbr_cd_90_dpd_l12m	nbr_cd_90_dpd_l12m_x	,
nbr_cd_90_dpd_l6m	nbr_cd_90_dpd_l6m_x	,
nbr_cd_90_dpd_l3m	nbr_cd_90_dpd_l3m_x	,
nbr_cd_90_dpd_cm	nbr_cd_90_dpd_cm_x	,
nbr_active_cd_x_dpd_l36m	nbr_active_cd_x_dpd_l36m_x	,
nbr_active_cd_x_dpd_l24m	nbr_active_cd_x_dpd_l24m_x	,
nbr_active_cd_x_dpd_l18m	nbr_active_cd_x_dpd_l18m_x	,
nbr_active_cd_x_dpd_l12m	nbr_active_cd_x_dpd_l12m_x	,
nbr_active_cd_x_dpd_l6m	nbr_active_cd_x_dpd_l6m_x	,
nbr_active_cd_x_dpd_l3m	nbr_active_cd_x_dpd_l3m_x	,
nbr_active_cd_x_dpd_cm	nbr_active_cd_x_dpd_cm_x	,
nbr_active_cd_30_dpd_l36m	nbr_active_cd_30_dpd_l36m_x	,
nbr_active_cd_30_dpd_l24m	nbr_active_cd_30_dpd_l24m_x	,
nbr_active_cd_30_dpd_l18m	nbr_active_cd_30_dpd_l18m_x	,
nbr_active_cd_30_dpd_l12m	nbr_active_cd_30_dpd_l12m_x	,
nbr_active_cd_30_dpd_l6m	nbr_active_cd_30_dpd_l6m_x	,
nbr_active_cd_30_dpd_l3m	nbr_active_cd_30_dpd_l3m_x	,
nbr_active_cd_30_dpd_cm	nbr_active_cd_30_dpd_cm_x	,
nbr_cd_active_90_dpd_l36m	nbr_cd_active_90_dpd_l36m_x	,
nbr_cd_active_90_dpd_l24m	nbr_cd_active_90_dpd_l24m_x	,
nbr_cd_active_90_dpd_l18m	nbr_cd_active_90_dpd_l18m_x	,
nbr_cd_active_90_dpd_l12m	nbr_cd_active_90_dpd_l12m_x	,
nbr_cd_active_90_dpd_l6m	nbr_cd_active_90_dpd_l6m_x	,
nbr_cd_active_90_dpd_l3m	nbr_cd_active_90_dpd_l3m_x	,
nbr_cd_active_90_dpd_cm	nbr_cd_active_90_dpd_cm_x	,
nbr_al_x_dpd_l36m	nbr_al_x_dpd_l36m_x	,
nbr_al_x_dpd_l24m	nbr_al_x_dpd_l24m_x	,
nbr_al_x_dpd_l18m	nbr_al_x_dpd_l18m_x	,
nbr_al_x_dpd_l12m	nbr_al_x_dpd_l12m_x	,
nbr_al_x_dpd_l6m	nbr_al_x_dpd_l6m_x	,
nbr_al_x_dpd_l3m	nbr_al_x_dpd_l3m_x	,
nbr_al_x_dpd_cm	nbr_al_x_dpd_cm_x	,
nbr_al_30_dpd_l36m	nbr_al_30_dpd_l36m_x	,
nbr_al_30_dpd_l24m	nbr_al_30_dpd_l24m_x	,
nbr_al_30_dpd_l18m	nbr_al_30_dpd_l18m_x	,
nbr_al_30_dpd_l12m	nbr_al_30_dpd_l12m_x	,
nbr_al_30_dpd_l6m	nbr_al_30_dpd_l6m_x	,
nbr_al_30_dpd_l3m	nbr_al_30_dpd_l3m_x	,
nbr_al_30_dpd_cm	nbr_al_30_dpd_cm_x	,
nbr_al_90_dpd_l36m	nbr_al_90_dpd_l36m_x	,
nbr_al_90_dpd_l24m	nbr_al_90_dpd_l24m_x	,
nbr_al_90_dpd_l18m	nbr_al_90_dpd_l18m_x	,
nbr_al_90_dpd_l12m	nbr_al_90_dpd_l12m_x	,
nbr_al_90_dpd_l6m	nbr_al_90_dpd_l6m_x	,
nbr_al_90_dpd_l3m	nbr_al_90_dpd_l3m_x	,
nbr_al_90_dpd_cm	nbr_al_90_dpd_cm_x	,
nbr_active_al_x_dpd_l36m	nbr_active_al_x_dpd_l36m_x	,
nbr_active_al_x_dpd_l24m	nbr_active_al_x_dpd_l24m_x	,
nbr_active_al_x_dpd_l18m	nbr_active_al_x_dpd_l18m_x	,
nbr_active_al_x_dpd_l12m	nbr_active_al_x_dpd_l12m_x	,
nbr_active_al_x_dpd_l6m	nbr_active_al_x_dpd_l6m_x	,
nbr_active_al_x_dpd_l3m	nbr_active_al_x_dpd_l3m_x	,
nbr_active_al_x_dpd_cm	nbr_active_al_x_dpd_cm_x	,
nbr_active_al_30_dpd_l36m	nbr_active_al_30_dpd_l36m_x	,
nbr_active_al_30_dpd_l24m	nbr_active_al_30_dpd_l24m_x	,
nbr_active_al_30_dpd_l18m	nbr_active_al_30_dpd_l18m_x	,
nbr_active_al_30_dpd_l12m	nbr_active_al_30_dpd_l12m_x	,
nbr_active_al_30_dpd_l6m	nbr_active_al_30_dpd_l6m_x	,
nbr_active_al_30_dpd_l3m	nbr_active_al_30_dpd_l3m_x	,
nbr_active_al_30_dpd_cm	nbr_active_al_30_dpd_cm_x	,
nbr_al_active_90_dpd_l36m	nbr_al_active_90_dpd_l36m_x	,
nbr_al_active_90_dpd_l24m	nbr_al_active_90_dpd_l24m_x	,
nbr_al_active_90_dpd_l18m	nbr_al_active_90_dpd_l18m_x	,
nbr_al_active_90_dpd_l12m	nbr_al_active_90_dpd_l12m_x	,
nbr_al_active_90_dpd_l6m	nbr_al_active_90_dpd_l6m_x	,
nbr_al_active_90_dpd_l3m	nbr_al_active_90_dpd_l3m_x	,
nbr_al_active_90_dpd_cm	nbr_al_active_90_dpd_cm_x	,
nbr_gl_x_dpd_l36m	nbr_gl_x_dpd_l36m_x	,
nbr_gl_x_dpd_l24m	nbr_gl_x_dpd_l24m_x	,
nbr_gl_x_dpd_l18m	nbr_gl_x_dpd_l18m_x	,
nbr_gl_x_dpd_l12m	nbr_gl_x_dpd_l12m_x	,
nbr_gl_x_dpd_l6m	nbr_gl_x_dpd_l6m_x	,
nbr_gl_x_dpd_l3m	nbr_gl_x_dpd_l3m_x	,
nbr_gl_x_dpd_cm	nbr_gl_x_dpd_cm_x	,
nbr_gl_30_dpd_l36m	nbr_gl_30_dpd_l36m_x	,
nbr_gl_30_dpd_l24m	nbr_gl_30_dpd_l24m_x	,
nbr_gl_30_dpd_l18m	nbr_gl_30_dpd_l18m_x	,
nbr_gl_30_dpd_l12m	nbr_gl_30_dpd_l12m_x	,
nbr_gl_30_dpd_l6m	nbr_gl_30_dpd_l6m_x	,
nbr_gl_30_dpd_l3m	nbr_gl_30_dpd_l3m_x	,
nbr_gl_30_dpd_cm	nbr_gl_30_dpd_cm_x	,
nbr_gl_90_dpd_l36m	nbr_gl_90_dpd_l36m_x	,
nbr_gl_90_dpd_l24m	nbr_gl_90_dpd_l24m_x	,
nbr_gl_90_dpd_l18m	nbr_gl_90_dpd_l18m_x	,
nbr_gl_90_dpd_l12m	nbr_gl_90_dpd_l12m_x	,
nbr_gl_90_dpd_l6m	nbr_gl_90_dpd_l6m_x	,
nbr_gl_90_dpd_l3m	nbr_gl_90_dpd_l3m_x	,
nbr_gl_90_dpd_cm	nbr_gl_90_dpd_cm_x	,
nbr_active_gl_x_dpd_l36m	nbr_active_gl_x_dpd_l36m_x	,
nbr_active_gl_x_dpd_l24m	nbr_active_gl_x_dpd_l24m_x	,
nbr_active_gl_x_dpd_l18m	nbr_active_gl_x_dpd_l18m_x	,
nbr_active_gl_x_dpd_l12m	nbr_active_gl_x_dpd_l12m_x	,
nbr_active_gl_x_dpd_l6m	nbr_active_gl_x_dpd_l6m_x	,
nbr_active_gl_x_dpd_l3m	nbr_active_gl_x_dpd_l3m_x	,
nbr_active_gl_x_dpd_cm	nbr_active_gl_x_dpd_cm_x	,
nbr_active_gl_30_dpd_l36m	nbr_active_gl_30_dpd_l36m_x	,
nbr_active_gl_30_dpd_l24m	nbr_active_gl_30_dpd_l24m_x	,
nbr_active_gl_30_dpd_l18m	nbr_active_gl_30_dpd_l18m_x	,
nbr_active_gl_30_dpd_l12m	nbr_active_gl_30_dpd_l12m_x	,
nbr_active_gl_30_dpd_l6m	nbr_active_gl_30_dpd_l6m_x	,
nbr_active_gl_30_dpd_l3m	nbr_active_gl_30_dpd_l3m_x	,
nbr_active_gl_30_dpd_cm	nbr_active_gl_30_dpd_cm_x	,
nbr_gl_active_90_dpd_l36m	nbr_gl_active_90_dpd_l36m_x	,
nbr_gl_active_90_dpd_l24m	nbr_gl_active_90_dpd_l24m_x	,
nbr_gl_active_90_dpd_l18m	nbr_gl_active_90_dpd_l18m_x	,
nbr_gl_active_90_dpd_l12m	nbr_gl_active_90_dpd_l12m_x	,
nbr_gl_active_90_dpd_l6m	nbr_gl_active_90_dpd_l6m_x	,
nbr_gl_active_90_dpd_l3m	nbr_gl_active_90_dpd_l3m_x	,
nbr_gl_active_90_dpd_cm	nbr_gl_active_90_dpd_cm_x	,
nbr_hl_x_dpd_l36m	nbr_hl_x_dpd_l36m_x	,
nbr_hl_x_dpd_l24m	nbr_hl_x_dpd_l24m_x	,
nbr_hl_x_dpd_l18m	nbr_hl_x_dpd_l18m_x	,
nbr_hl_x_dpd_l12m	nbr_hl_x_dpd_l12m_x	,
nbr_hl_x_dpd_l6m	nbr_hl_x_dpd_l6m_x	,
nbr_hl_x_dpd_l3m	nbr_hl_x_dpd_l3m_x	,
nbr_hl_x_dpd_cm	nbr_hl_x_dpd_cm_x	,
nbr_hl_30_dpd_l36m	nbr_hl_30_dpd_l36m_x	,
nbr_hl_30_dpd_l24m	nbr_hl_30_dpd_l24m_x	,
nbr_hl_30_dpd_l18m	nbr_hl_30_dpd_l18m_x	,
nbr_hl_30_dpd_l12m	nbr_hl_30_dpd_l12m_x	,
nbr_hl_30_dpd_l6m	nbr_hl_30_dpd_l6m_x	,
nbr_hl_30_dpd_l3m	nbr_hl_30_dpd_l3m_x	,
nbr_hl_30_dpd_cm	nbr_hl_30_dpd_cm_x	,
nbr_hl_90_dpd_l36m	nbr_hl_90_dpd_l36m_x	,
nbr_hl_90_dpd_l24m	nbr_hl_90_dpd_l24m_x	,
nbr_hl_90_dpd_l18m	nbr_hl_90_dpd_l18m_x	,
nbr_hl_90_dpd_l12m	nbr_hl_90_dpd_l12m_x	,
nbr_hl_90_dpd_l6m	nbr_hl_90_dpd_l6m_x	,
nbr_hl_90_dpd_l3m	nbr_hl_90_dpd_l3m_x	,
nbr_hl_90_dpd_cm	nbr_hl_90_dpd_cm_x	,
nbr_active_hl_x_dpd_l36m	nbr_active_hl_x_dpd_l36m_x	,
nbr_active_hl_x_dpd_l24m	nbr_active_hl_x_dpd_l24m_x	,
nbr_active_hl_x_dpd_l18m	nbr_active_hl_x_dpd_l18m_x	,
nbr_active_hl_x_dpd_l12m	nbr_active_hl_x_dpd_l12m_x	,
nbr_active_hl_x_dpd_l6m	nbr_active_hl_x_dpd_l6m_x	,
nbr_active_hl_x_dpd_l3m	nbr_active_hl_x_dpd_l3m_x	,
nbr_active_hl_x_dpd_cm	nbr_active_hl_x_dpd_cm_x	,
nbr_active_hl_30_dpd_l36m	nbr_active_hl_30_dpd_l36m_x	,
nbr_active_hl_30_dpd_l24m	nbr_active_hl_30_dpd_l24m_x	,
nbr_active_hl_30_dpd_l18m	nbr_active_hl_30_dpd_l18m_x	,
nbr_active_hl_30_dpd_l12m	nbr_active_hl_30_dpd_l12m_x	,
nbr_active_hl_30_dpd_l6m	nbr_active_hl_30_dpd_l6m_x	,
nbr_active_hl_30_dpd_l3m	nbr_active_hl_30_dpd_l3m_x	,
nbr_active_hl_30_dpd_cm	nbr_active_hl_30_dpd_cm_x	,
nbr_hl_active_90_dpd_l36m	nbr_hl_active_90_dpd_l36m_x	,
nbr_hl_active_90_dpd_l24m	nbr_hl_active_90_dpd_l24m_x	,
nbr_hl_active_90_dpd_l18m	nbr_hl_active_90_dpd_l18m_x	,
nbr_hl_active_90_dpd_l12m	nbr_hl_active_90_dpd_l12m_x	,
nbr_hl_active_90_dpd_l6m	nbr_hl_active_90_dpd_l6m_x	,
nbr_hl_active_90_dpd_l3m	nbr_hl_active_90_dpd_l3m_x	,
nbr_hl_active_90_dpd_cm	nbr_hl_active_90_dpd_cm_x	,
nbr_tw_x_dpd_l36m	nbr_tw_x_dpd_l36m_x	,
nbr_tw_x_dpd_l24m	nbr_tw_x_dpd_l24m_x	,
nbr_tw_x_dpd_l18m	nbr_tw_x_dpd_l18m_x	,
nbr_tw_x_dpd_l12m	nbr_tw_x_dpd_l12m_x	,
nbr_tw_x_dpd_l6m	nbr_tw_x_dpd_l6m_x	,
nbr_tw_x_dpd_l3m	nbr_tw_x_dpd_l3m_x	,
nbr_tw_x_dpd_cm	nbr_tw_x_dpd_cm_x	,
nbr_tw_30_dpd_l36m	nbr_tw_30_dpd_l36m_x	,
nbr_tw_30_dpd_l24m	nbr_tw_30_dpd_l24m_x	,
nbr_tw_30_dpd_l18m	nbr_tw_30_dpd_l18m_x	,
nbr_tw_30_dpd_l12m	nbr_tw_30_dpd_l12m_x	,
nbr_tw_30_dpd_l6m	nbr_tw_30_dpd_l6m_x	,
nbr_tw_30_dpd_l3m	nbr_tw_30_dpd_l3m_x	,
nbr_tw_30_dpd_cm	nbr_tw_30_dpd_cm_x	,
nbr_tw_90_dpd_l36m	nbr_tw_90_dpd_l36m_x	,
nbr_tw_90_dpd_l24m	nbr_tw_90_dpd_l24m_x	,
nbr_tw_90_dpd_l18m	nbr_tw_90_dpd_l18m_x	,
nbr_tw_90_dpd_l12m	nbr_tw_90_dpd_l12m_x	,
nbr_tw_90_dpd_l6m	nbr_tw_90_dpd_l6m_x	,
nbr_tw_90_dpd_l3m	nbr_tw_90_dpd_l3m_x	,
nbr_tw_90_dpd_cm	nbr_tw_90_dpd_cm_x	,
nbr_active_tw_x_dpd_l36m	nbr_active_tw_x_dpd_l36m_x	,
nbr_active_tw_x_dpd_l24m	nbr_active_tw_x_dpd_l24m_x	,
nbr_active_tw_x_dpd_l18m	nbr_active_tw_x_dpd_l18m_x	,
nbr_active_tw_x_dpd_l12m	nbr_active_tw_x_dpd_l12m_x	,
nbr_active_tw_x_dpd_l6m	nbr_active_tw_x_dpd_l6m_x	,
nbr_active_tw_x_dpd_l3m	nbr_active_tw_x_dpd_l3m_x	,
nbr_active_tw_x_dpd_cm	nbr_active_tw_x_dpd_cm_x	,
nbr_active_tw_30_dpd_l36m	nbr_active_tw_30_dpd_l36m_x	,
nbr_active_tw_30_dpd_l24m	nbr_active_tw_30_dpd_l24m_x	,
nbr_active_tw_30_dpd_l18m	nbr_active_tw_30_dpd_l18m_x	,
nbr_active_tw_30_dpd_l12m	nbr_active_tw_30_dpd_l12m_x	,
nbr_active_tw_30_dpd_l6m	nbr_active_tw_30_dpd_l6m_x	,
nbr_active_tw_30_dpd_l3m	nbr_active_tw_30_dpd_l3m_x	,
nbr_active_tw_30_dpd_cm	nbr_active_tw_30_dpd_cm_x	,
nbr_tw_active_90_dpd_l36m	nbr_tw_active_90_dpd_l36m_x	,
nbr_tw_active_90_dpd_l24m	nbr_tw_active_90_dpd_l24m_x	,
nbr_tw_active_90_dpd_l18m	nbr_tw_active_90_dpd_l18m_x	,
nbr_tw_active_90_dpd_l12m	nbr_tw_active_90_dpd_l12m_x	,
nbr_tw_active_90_dpd_l6m	nbr_tw_active_90_dpd_l6m_x	,
nbr_tw_active_90_dpd_l3m	nbr_tw_active_90_dpd_l3m_x	,
nbr_tw_active_90_dpd_cm	nbr_tw_active_90_dpd_cm_x	,
nbr_unsecured_x_dpd_l36m	nbr_unsecured_x_dpd_l36m_x	,
nbr_unsecured_x_dpd_l24m	nbr_unsecured_x_dpd_l24m_x	,
nbr_unsecured_x_dpd_l18m	nbr_unsecured_x_dpd_l18m_x	,
nbr_unsecured_x_dpd_l12m	nbr_unsecured_x_dpd_l12m_x	,
nbr_unsecured_x_dpd_l6m	nbr_unsecured_x_dpd_l6m_x	,
nbr_unsecured_x_dpd_l3m	nbr_unsecured_x_dpd_l3m_x	,
nbr_unsecured_x_dpd_cm	nbr_unsecured_x_dpd_cm_x	,
nbr_unsecured_30_dpd_l36m	nbr_unsecured_30_dpd_l36m_x	,
nbr_unsecured_30_dpd_l24m	nbr_unsecured_30_dpd_l24m_x	,
nbr_unsecured_30_dpd_l18m	nbr_unsecured_30_dpd_l18m_x	,
nbr_unsecured_30_dpd_l12m	nbr_unsecured_30_dpd_l12m_x	,
nbr_unsecured_30_dpd_l6m	nbr_unsecured_30_dpd_l6m_x	,
nbr_unsecured_30_dpd_l3m	nbr_unsecured_30_dpd_l3m_x	,
nbr_unsecured_30_dpd_cm	nbr_unsecured_30_dpd_cm_x	,
nbr_unsecured_90_dpd_l36m	nbr_unsecured_90_dpd_l36m_x	,
nbr_unsecured_90_dpd_l24m	nbr_unsecured_90_dpd_l24m_x	,
nbr_unsecured_90_dpd_l18m	nbr_unsecured_90_dpd_l18m_x	,
nbr_unsecured_90_dpd_l12m	nbr_unsecured_90_dpd_l12m_x	,
nbr_unsecured_90_dpd_l6m	nbr_unsecured_90_dpd_l6m_x	,
nbr_unsecured_90_dpd_l3m	nbr_unsecured_90_dpd_l3m_x	,
nbr_unsecured_90_dpd_cm	nbr_unsecured_90_dpd_cm_x	,
nbr_active_unsecured_x_dpd_l36m	nbr_active_unsecured_x_dpd_l36m_x	,
nbr_active_unsecured_x_dpd_l24m	nbr_active_unsecured_x_dpd_l24m_x	,
nbr_active_unsecured_x_dpd_l18m	nbr_active_unsecured_x_dpd_l18m_x	,
nbr_active_unsecured_x_dpd_l12m	nbr_active_unsecured_x_dpd_l12m_x	,
nbr_active_unsecured_x_dpd_l6m	nbr_active_unsecured_x_dpd_l6m_x	,
nbr_active_unsecured_x_dpd_l3m	nbr_active_unsecured_x_dpd_l3m_x	,
nbr_active_unsecured_x_dpd_cm	nbr_active_unsecured_x_dpd_cm_x	,
nbr_active_unsecured_30_dpd_l36m	nbr_active_unsecured_30_dpd_l36m_x	,
nbr_active_unsecured_30_dpd_l24m	nbr_active_unsecured_30_dpd_l24m_x	,
nbr_active_unsecured_30_dpd_l18m	nbr_active_unsecured_30_dpd_l18m_x	,
nbr_active_unsecured_30_dpd_l12m	nbr_active_unsecured_30_dpd_l12m_x	,
nbr_active_unsecured_30_dpd_l6m	nbr_active_unsecured_30_dpd_l6m_x	,
nbr_active_unsecured_30_dpd_l3m	nbr_active_unsecured_30_dpd_l3m_x	,
nbr_active_unsecured_30_dpd_cm	nbr_active_unsecured_30_dpd_cm_x	,
nbr_unsecured_active_90_dpd_l36m	nbr_unsecured_active_90_dpd_l36m_x	,
nbr_unsecured_active_90_dpd_l24m	nbr_unsecured_active_90_dpd_l24m_x	,
nbr_unsecured_active_90_dpd_l18m	nbr_unsecured_active_90_dpd_l18m_x	,
nbr_unsecured_active_90_dpd_l12m	nbr_unsecured_active_90_dpd_l12m_x	,
nbr_unsecured_active_90_dpd_l6m	nbr_unsecured_active_90_dpd_l6m_x	,
nbr_unsecured_active_90_dpd_l3m	nbr_unsecured_active_90_dpd_l3m_x	,
nbr_unsecured_active_90_dpd_cm	nbr_unsecured_active_90_dpd_cm_x	,
nbr_secured_x_dpd_l36m	nbr_secured_x_dpd_l36m_x	,
nbr_secured_x_dpd_l24m	nbr_secured_x_dpd_l24m_x	,
nbr_secured_x_dpd_l18m	nbr_secured_x_dpd_l18m_x	,
nbr_secured_x_dpd_l12m	nbr_secured_x_dpd_l12m_x	,
nbr_secured_x_dpd_l6m	nbr_secured_x_dpd_l6m_x	,
nbr_secured_x_dpd_l3m	nbr_secured_x_dpd_l3m_x	,
nbr_secured_x_dpd_cm	nbr_secured_x_dpd_cm_x	,
nbr_secured_30_dpd_l36m	nbr_secured_30_dpd_l36m_x	,
nbr_secured_30_dpd_l24m	nbr_secured_30_dpd_l24m_x	,
nbr_secured_30_dpd_l18m	nbr_secured_30_dpd_l18m_x	,
nbr_secured_30_dpd_l12m	nbr_secured_30_dpd_l12m_x	,
nbr_secured_30_dpd_l6m	nbr_secured_30_dpd_l6m_x	,
nbr_secured_30_dpd_l3m	nbr_secured_30_dpd_l3m_x	,
nbr_secured_30_dpd_cm	nbr_secured_30_dpd_cm_x	,
nbr_secured_90_dpd_l36m	nbr_secured_90_dpd_l36m_x	,
nbr_secured_90_dpd_l24m	nbr_secured_90_dpd_l24m_x	,
nbr_secured_90_dpd_l18m	nbr_secured_90_dpd_l18m_x	,
nbr_secured_90_dpd_l12m	nbr_secured_90_dpd_l12m_x	,
nbr_secured_90_dpd_l6m	nbr_secured_90_dpd_l6m_x	,
nbr_secured_90_dpd_l3m	nbr_secured_90_dpd_l3m_x	,
nbr_secured_90_dpd_cm	nbr_secured_90_dpd_cm_x	,
nbr_active_secured_x_dpd_l36m	nbr_active_secured_x_dpd_l36m_x	,
nbr_active_secured_x_dpd_l24m	nbr_active_secured_x_dpd_l24m_x	,
nbr_active_secured_x_dpd_l18m	nbr_active_secured_x_dpd_l18m_x	,
nbr_active_secured_x_dpd_l12m	nbr_active_secured_x_dpd_l12m_x	,
nbr_active_secured_x_dpd_l6m	nbr_active_secured_x_dpd_l6m_x	,
nbr_active_secured_x_dpd_l3m	nbr_active_secured_x_dpd_l3m_x	,
nbr_active_secured_x_dpd_cm	nbr_active_secured_x_dpd_cm_x	,
nbr_active_secured_30_dpd_l36m	nbr_active_secured_30_dpd_l36m_x	,
nbr_active_secured_30_dpd_l24m	nbr_active_secured_30_dpd_l24m_x	,
nbr_active_secured_30_dpd_l18m	nbr_active_secured_30_dpd_l18m_x	,
nbr_active_secured_30_dpd_l12m	nbr_active_secured_30_dpd_l12m_x	,
nbr_active_secured_30_dpd_l6m	nbr_active_secured_30_dpd_l6m_x	,
nbr_active_secured_30_dpd_l3m	nbr_active_secured_30_dpd_l3m_x	,
nbr_active_secured_30_dpd_cm	nbr_active_secured_30_dpd_cm_x	,
nbr_secured_active_90_dpd_l36m	nbr_secured_active_90_dpd_l36m_x	,
nbr_secured_active_90_dpd_l24m	nbr_secured_active_90_dpd_l24m_x	,
nbr_secured_active_90_dpd_l18m	nbr_secured_active_90_dpd_l18m_x	,
nbr_secured_active_90_dpd_l12m	nbr_secured_active_90_dpd_l12m_x	,
nbr_secured_active_90_dpd_l6m	nbr_secured_active_90_dpd_l6m_x	,
nbr_secured_active_90_dpd_l3m	nbr_secured_active_90_dpd_l3m_x	,
nbr_secured_active_90_dpd_cm	nbr_secured_active_90_dpd_cm_x	,
max_sanctioned_amount	max_sanctioned_amount_x	,
total_sanctioned_amount	total_sanctioned_amount_x	,
secured_max_sanctioned_amount	secured_max_sanctioned_amount_x	,
secured_total_sanctioned_amount	secured_total_sanctioned_amount_x	,
unsecured_max_sanctioned_amount	unsecured_max_sanctioned_amount_x	,
unsecured_total_sanctioned_amount	unsecured_total_sanctioned_amount_x	,
pl_max_sanctioned_amount	pl_max_sanctioned_amount_x	,
pl_total_sanctioned_amount	pl_total_sanctioned_amount_x	,
hl_max_sanctioned_amount	hl_max_sanctioned_amount_x	,
hl_total_sanctioned_amount	hl_total_sanctioned_amount_x	,
cc_max_sanctioned_amount	cc_max_sanctioned_amount_x	,
cc_total_sanctioned_amount	cc_total_sanctioned_amount_x	,
cd_max_sanctioned_amount	cd_max_sanctioned_amount_x	,
cd_total_sanctioned_amount	cd_total_sanctioned_amount_x	,
al_max_sanctioned_amount	al_max_sanctioned_amount_x	,
al_total_sanctioned_amount	al_total_sanctioned_amount_x	,
gl_max_sanctioned_amount	gl_max_sanctioned_amount_x	,
gl_total_sanctioned_amount	gl_total_sanctioned_amount_x	,
tw_max_sanctioned_amount	tw_max_sanctioned_amount_x	,
tw_total_sanctioned_amount	tw_total_sanctioned_amount_x	,
max_sanctioned_amount_l3m	max_sanctioned_amount_l3m_x	,
total_sanctioned_amount_l3m	total_sanctioned_amount_l3m_x	,
secured_max_sanctioned_amount_l3m	secured_max_sanctioned_amount_l3m_x	,
secured_total_sanctioned_amount_l3m	secured_total_sanctioned_amount_l3m_x	,
unsecured_max_sanctioned_amount_l3m	unsecured_max_sanctioned_amount_l3m_x	,
unsecured_total_sanctioned_amount_l3m	unsecured_total_sanctioned_amount_l3m_x	,
pl_max_sanctioned_amount_l3m	pl_max_sanctioned_amount_l3m_x	,
pl_total_sanctioned_amount_l3m	pl_total_sanctioned_amount_l3m_x	,
hl_max_sanctioned_amount_l3m	hl_max_sanctioned_amount_l3m_x	,
hl_total_sanctioned_amount_l3m	hl_total_sanctioned_amount_l3m_x	,
cc_max_sanctioned_amount_l3m	cc_max_sanctioned_amount_l3m_x	,
cc_total_sanctioned_amount_l3m	cc_total_sanctioned_amount_l3m_x	,
cd_max_sanctioned_amount_l3m	cd_max_sanctioned_amount_l3m_x	,
cd_total_sanctioned_amount_l3m	cd_total_sanctioned_amount_l3m_x	,
al_max_sanctioned_amount_l3m	al_max_sanctioned_amount_l3m_x	,
al_total_sanctioned_amount_l3m	al_total_sanctioned_amount_l3m_x	,
gl_max_sanctioned_amount_l3m	gl_max_sanctioned_amount_l3m_x	,
gl_total_sanctioned_amount_l3m	gl_total_sanctioned_amount_l3m_x	,
tw_max_sanctioned_amount_l3m	tw_max_sanctioned_amount_l3m_x	,
tw_total_sanctioned_amount_l3m	tw_total_sanctioned_amount_l3m_x	,
max_sanctioned_amount_l6m	max_sanctioned_amount_l6m_x	,
total_sanctioned_amount_l6m	total_sanctioned_amount_l6m_x	,
secured_max_sanctioned_amount_l6m	secured_max_sanctioned_amount_l6m_x	,
secured_total_sanctioned_amount_l6m	secured_total_sanctioned_amount_l6m_x	,
unsecured_max_sanctioned_amount_l6m	unsecured_max_sanctioned_amount_l6m_x	,
unsecured_total_sanctioned_amount_l6m	unsecured_total_sanctioned_amount_l6m_x	,
pl_max_sanctioned_amount_l6m	pl_max_sanctioned_amount_l6m_x	,
pl_total_sanctioned_amount_l6m	pl_total_sanctioned_amount_l6m_x	,
hl_max_sanctioned_amount_l6m	hl_max_sanctioned_amount_l6m_x	,
hl_total_sanctioned_amount_l6m	hl_total_sanctioned_amount_l6m_x	,
cc_max_sanctioned_amount_l6m	cc_max_sanctioned_amount_l6m_x	,
cc_total_sanctioned_amount_l6m	cc_total_sanctioned_amount_l6m_x	,
cd_max_sanctioned_amount_l6m	cd_max_sanctioned_amount_l6m_x	,
cd_total_sanctioned_amount_l6m	cd_total_sanctioned_amount_l6m_x	,
al_max_sanctioned_amount_l6m	al_max_sanctioned_amount_l6m_x	,
al_total_sanctioned_amount_l6m	al_total_sanctioned_amount_l6m_x	,
gl_max_sanctioned_amount_l6m	gl_max_sanctioned_amount_l6m_x	,
gl_total_sanctioned_amount_l6m	gl_total_sanctioned_amount_l6m_x	,
tw_max_sanctioned_amount_l6m	tw_max_sanctioned_amount_l6m_x	,
tw_total_sanctioned_amount_l6m	tw_total_sanctioned_amount_l6m_x	,
max_sanctioned_amount_l12m	max_sanctioned_amount_l12m_x	,
total_sanctioned_amount_l12m	total_sanctioned_amount_l12m_x	,
secured_max_sanctioned_amount_l12m	secured_max_sanctioned_amount_l12m_x	,
secured_total_sanctioned_amount_l12m	secured_total_sanctioned_amount_l12m_x	,
unsecured_max_sanctioned_amount_l12m	unsecured_max_sanctioned_amount_l12m_x	,
unsecured_total_sanctioned_amount_l12m	unsecured_total_sanctioned_amount_l12m_x	,
pl_max_sanctioned_amount_l12m	pl_max_sanctioned_amount_l12m_x	,
pl_total_sanctioned_amount_l12m	pl_total_sanctioned_amount_l12m_x	,
hl_max_sanctioned_amount_l12m	hl_max_sanctioned_amount_l12m_x	,
hl_total_sanctioned_amount_l12m	hl_total_sanctioned_amount_l12m_x	,
cc_max_sanctioned_amount_l12m	cc_max_sanctioned_amount_l12m_x	,
cc_total_sanctioned_amount_l12m	cc_total_sanctioned_amount_l12m_x	,
cd_max_sanctioned_amount_l12m	cd_max_sanctioned_amount_l12m_x	,
cd_total_sanctioned_amount_l12m	cd_total_sanctioned_amount_l12m_x	,
al_max_sanctioned_amount_l12m	al_max_sanctioned_amount_l12m_x	,
al_total_sanctioned_amount_l12m	al_total_sanctioned_amount_l12m_x	,
gl_max_sanctioned_amount_l12m	gl_max_sanctioned_amount_l12m_x	,
gl_total_sanctioned_amount_l12m	gl_total_sanctioned_amount_l12m_x	,
tw_max_sanctioned_amount_l12m	tw_max_sanctioned_amount_l12m_x	,
tw_total_sanctioned_amount_l12m	tw_total_sanctioned_amount_l12m_x	,
total_balance_amount	total_balance_amount_x	,
secured_total_balance_amount	secured_total_balance_amount_x	,
unsecured_total_balance_amount	unsecured_total_balance_amount_x	,
pl_total_balance_amount	pl_total_balance_amount_x	,
hl_total_balance_amount	hl_total_balance_amount_x	,
cc_total_balance_amount	cc_total_balance_amount_x	,
cd_total_balance_amount	cd_total_balance_amount_x	,
al_total_balance_amount	al_total_balance_amount_x	,
gl_total_balance_amount	gl_total_balance_amount_x	,
tw_total_balance_amount	tw_total_balance_amount_x	,
avg_utilisation	avg_utilisation_x	,
avg_secured_utilisation	avg_secured_utilisation_x	,
avg_unsecured_utilisation	avg_unsecured_utilisation_x	,
avg_pl_utilisation	avg_pl_utilisation_x	,
avg_hl_tilisation	avg_hl_tilisation_x	,
avg_cc_utilisation	avg_cc_utilisation_x	,
avg_cd_utilisation	avg_cd_utilisation_x	,
avg_al_utilisation	avg_al_utilisation_x	,
avg_gl_utilisation	avg_gl_utilisation_x	,
avg_tw_utilisation	avg_tw_utilisation_x	,
overall_utilisation	overall_utilisation_x	,
secured_utilisation	secured_utilisation_x	,
unsecured_utilisation	unsecured_utilisation_x	,
pl_utilisation	pl_utilisation_x	,
hl_utilisation	hl_utilisation_x	,
cc_utilisation	cc_utilisation_x	,
cd_utilisation	cd_utilisation_x	,
al_utilisation	al_utilisation_x	,
gl_utilisation	gl_utilisation_x	,
tw_utilisation	tw_utilisation_x	,
latest_product	latest_product_x	,
first_product	first_product_x	,
max_bureau_vintage	max_bureau_vintage_x	,
min_bureau_vintage	min_bureau_vintage_x	,
max_active_bureau_vintage	max_active_bureau_vintage_x	,
min_active_bureau_vintage	min_active_bureau_vintage_x	,
max_secured_bureau_vintage	max_secured_bureau_vintage_x	,
min_secured_bureau_vintage	min_secured_bureau_vintage_x	,
max_active_secured_bureau_vintage	max_active_secured_bureau_vintage_x	,
min_active_secured_bureau_vintage	min_active_secured_bureau_vintage_x	,
max_unsecured_bureau_vintage	max_unsecured_bureau_vintage_x	,
min_unsecured_bureau_vintage	min_unsecured_bureau_vintage_x	,
max_active_unsecured_bureau_vintage	max_active_unsecured_bureau_vintage_x	,
min_active_unsecured_bureau_vintage	min_active_unsecured_bureau_vintage_x	,
max_pl_bureau_vintage	max_pl_bureau_vintage_x	,
min_pl_bureau_vintage	min_pl_bureau_vintage_x	,
max_active_pl_bureau_vintage	max_active_pl_bureau_vintage_x	,
min_active_pl_bureau_vintage	min_active_pl_bureau_vintage_x	,
max_cc_bureau_vintage	max_cc_bureau_vintage_x	,
min_cc_bureau_vintage	min_cc_bureau_vintage_x	,
max_active_cc_bureau_vintage	max_active_cc_bureau_vintage_x	,
min_active_cc_bureau_vintage	min_active_cc_bureau_vintage_x	,
max_cd_bureau_vintage	max_cd_bureau_vintage_x	,
min_cd_bureau_vintage	min_cd_bureau_vintage_x	,
max_active_cd_bureau_vintage	max_active_cd_bureau_vintage_x	,
min_active_cd_bureau_vintage	min_active_cd_bureau_vintage_x	,
max_al_bureau_vintage	max_al_bureau_vintage_x	,
min_al_bureau_vintage	min_al_bureau_vintage_x	,
max_active_al_bureau_vintage	max_active_al_bureau_vintage_x	,
min_active_al_bureau_vintage	min_active_al_bureau_vintage_x	,
max_hl_bureau_vintage	max_hl_bureau_vintage_x	,
min_hl_bureau_vintage	min_hl_bureau_vintage_x	,
max_active_hl_bureau_vintage	max_active_hl_bureau_vintage_x	,
min_active_hl_bureau_vintage	min_active_hl_bureau_vintage_x	,
max_gl_bureau_vintage	max_gl_bureau_vintage_x	,
min_gl_bureau_vintage	min_gl_bureau_vintage_x	,
max_active_gl_bureau_vintage	max_active_gl_bureau_vintage_x	,
min_active_gl_bureau_vintage	min_active_gl_bureau_vintage_x	,
max_tw_bureau_vintage	max_tw_bureau_vintage_x	,
min_tw_bureau_vintage	min_tw_bureau_vintage_x	,
max_active_tw_bureau_vintage	max_active_tw_bureau_vintage_x	,
min_active_tw_bureau_vintage	min_active_tw_bureau_vintage_x	,
enq_l1m	enq_l1m_x	,
enq_l2m	enq_l2m_x	,
enq_l3m	enq_l3m_x	,
enq_l6m	enq_l6m_x	,
enq_l12m	enq_l12m_x	,
unsec_inq_l1m	unsec_inq_l1m_x	,
unsec_inq_l2m	unsec_inq_l2m_x	,
unsec_inq_l3m	unsec_inq_l3m_x	,
unsec_inq_l6m	unsec_inq_l6m_x	,
unsec_inq_l12m	unsec_inq_l12m_x	,
sec_inq_l1m	sec_inq_l1m_x	,
sec_inq_l2m	sec_inq_l2m_x	,
sec_inq_l3m	sec_inq_l3m_x	,
sec_inq_l6m	sec_inq_l6m_x	,
sec_inq_l12m	sec_inq_l12m_x	,
pl_inq_l1m	pl_inq_l1m_x	,
pl_inq_l2m	pl_inq_l2m_x	,
pl_inq_l3m	pl_inq_l3m_x	,
pl_inq_l6m	pl_inq_l6m_x	,
pl_inq_l12m	pl_inq_l12m_x	,
cc_inq_l1m	cc_inq_l1m_x	,
cc_inq_l2m	cc_inq_l2m_x	,
cc_inq_l3m	cc_inq_l3m_x	,
cc_inq_l6m	cc_inq_l6m_x	,
cc_inq_l12m	cc_inq_l12m_x	,
cd_inq_l1m	cd_inq_l1m_x	,
cd_inq_l2m	cd_inq_l2m_x	,
cd_inq_l3m	cd_inq_l3m_x	,
cd_inq_l6m	cd_inq_l6m_x	,
cd_inq_l12m	cd_inq_l12m_x	,
al_inq_l1m	al_inq_l1m_x	,
al_inq_l2m	al_inq_l2m_x	,
al_inq_l3m	al_inq_l3m_x	,
al_inq_l6m	al_inq_l6m_x	,
al_inq_l12m	al_inq_l12m_x	,
gl_inq_l1m	gl_inq_l1m_x	,
gl_inq_l2m	gl_inq_l2m_x	,
gl_inq_l3m	gl_inq_l3m_x	,
gl_inq_l6m	gl_inq_l6m_x	,
gl_inq_l12m	gl_inq_l12m_x	,
hl_inq_l1m	hl_inq_l1m_x	,
hl_inq_l2m	hl_inq_l2m_x	,
hl_inq_l3m	hl_inq_l3m_x	,
hl_inq_l6m	hl_inq_l6m_x	,
hl_inq_l12m	hl_inq_l12m_x	,
tw_inq_l1m	tw_inq_l1m_x	,
tw_inq_l2m	tw_inq_l2m_x	,
tw_inq_l3m	tw_inq_l3m_x	,
tw_inq_l6m	tw_inq_l6m_x	,
tw_inq_l12m	tw_inq_l12m_x	,
nbr_ever_accounts	nbr_ever_accounts_x	,
nbr_active_accounts	nbr_active_accounts_x	,
accts_new_l1m	accts_new_l1m_x	,
accts_new_l3m	accts_new_l3m_x	,
accts_new_l6m	accts_new_l6m_x	,
accts_new_l12m	accts_new_l12m_x	,
accts_closed_l1m	accts_closed_l1m_x	,
accts_closed_l3m	accts_closed_l3m_x	,
accts_closed_l6m	accts_closed_l6m_x	,
accts_closed_l12m	accts_closed_l12m_x	,
nbr_ever_secured_accounts	nbr_ever_secured_accounts_x	,
nbr_active_secured_accounts	nbr_active_secured_accounts_x	,
secured_accts_new_l1m	secured_accts_new_l1m_x	,
secured_accts_new_l3m	secured_accts_new_l3m_x	,
secured_accts_new_l6m	secured_accts_new_l6m_x	,
secured_accts_new_l12m	secured_accts_new_l12m_x	,
secured_accts_closed_l1m	secured_accts_closed_l1m_x	,
secured_accts_closed_l3m	secured_accts_closed_l3m_x	,
secured_accts_closed_l6m	secured_accts_closed_l6m_x	,
secured_accts_closed_l12m	secured_accts_closed_l12m_x	,
nbr_ever_unsecured_accounts	nbr_ever_unsecured_accounts_x	,
nbr_active_unsecured_accounts	nbr_active_unsecured_accounts_x	,
unsecured_accts_new_l1m	unsecured_accts_new_l1m_x	,
unsecured_accts_new_l3m	unsecured_accts_new_l3m_x	,
unsecured_accts_new_l6m	unsecured_accts_new_l6m_x	,
unsecured_accts_new_l12m	unsecured_accts_new_l12m_x	,
unsecured_accts_closed_l1m	unsecured_accts_closed_l1m_x	,
unsecured_accts_closed_l3m	unsecured_accts_closed_l3m_x	,
unsecured_accts_closed_l6m	unsecured_accts_closed_l6m_x	,
unsecured_accts_closed_l12m	unsecured_accts_closed_l12m_x	,
ever_hl	ever_hl_x	,
nbr_ever_hl	nbr_ever_hl_x	,
active_hl	active_hl_x	,
nbr_active_hl	nbr_active_hl_x	,
hl_new_l1m	hl_new_l1m_x	,
hl_new_l3m	hl_new_l3m_x	,
hl_new_l6m	hl_new_l6m_x	,
hl_new_l12m	hl_new_l12m_x	,
hl_closed_l1m	hl_closed_l1m_x	,
hl_closed_l3m	hl_closed_l3m_x	,
hl_closed_l6m	hl_closed_l6m_x	,
hl_closed_l12m	hl_closed_l12m_x	,
ever_pl	ever_pl_x	,
nbr_ever_pl	nbr_ever_pl_x	,
active_pl	active_pl_x	,
nbr_active_pl	nbr_active_pl_x	,
pl_new_l1m	pl_new_l1m_x	,
pl_new_l3m	pl_new_l3m_x	,
pl_new_l6m	pl_new_l6m_x	,
pl_new_l12m	pl_new_l12m_x	,
pl_closed_l1m	pl_closed_l1m_x	,
pl_closed_l3m	pl_closed_l3m_x	,
pl_closed_l6m	pl_closed_l6m_x	,
pl_closed_l12m	pl_closed_l12m_x	,
ever_cc	ever_cc_x	,
nbr_ever_cc	nbr_ever_cc_x	,
active_cc	active_cc_x	,
nbr_active_cc	nbr_active_cc_x	,
cc_new_l1m	cc_new_l1m_x	,
cc_new_l3m	cc_new_l3m_x	,
cc_new_l6m	cc_new_l6m_x	,
cc_new_l12m	cc_new_l12m_x	,
cc_closed_l1m	cc_closed_l1m_x	,
cc_closed_l3m	cc_closed_l3m_x	,
cc_closed_l6m	cc_closed_l6m_x	,
cc_closed_l12m	cc_closed_l12m_x	,
ever_cd	ever_cd_x	,
nbr_ever_cd	nbr_ever_cd_x	,
active_cd	active_cd_x	,
nbr_active_cd	nbr_active_cd_x	,
cd_new_l1m	cd_new_l1m_x	,
cd_new_l3m	cd_new_l3m_x	,
cd_new_l6m	cd_new_l6m_x	,
cd_new_l12m	cd_new_l12m_x	,
cd_closed_l1m	cd_closed_l1m_x	,
cd_closed_l3m	cd_closed_l3m_x	,
cd_closed_l6m	cd_closed_l6m_x	,
cd_closed_l12m	cd_closed_l12m_x	,
ever_gl	ever_gl_x	,
nbr_ever_gl	nbr_ever_gl_x	,
active_gl	active_gl_x	,
nbr_active_gl	nbr_active_gl_x	,
gl_new_l1m	gl_new_l1m_x	,
gl_new_l3m	gl_new_l3m_x	,
gl_new_l6m	gl_new_l6m_x	,
gl_new_l12m	gl_new_l12m_x	,
gl_closed_l1m	gl_closed_l1m_x	,
gl_closed_l3m	gl_closed_l3m_x	,
gl_closed_l6m	gl_closed_l6m_x	,
gl_closed_l12m	gl_closed_l12m_x	,
ever_al	ever_al_x	,
nbr_ever_al	nbr_ever_al_x	,
active_al	active_al_x	,
nbr_active_al	nbr_active_al_x	,
al_new_l1m	al_new_l1m_x	,
al_new_l3m	al_new_l3m_x	,
al_new_l6m	al_new_l6m_x	,
al_new_l12m	al_new_l12m_x	,
al_closed_l1m	al_closed_l1m_x	,
al_closed_l3m	al_closed_l3m_x	,
al_closed_l6m	al_closed_l6m_x	,
al_closed_l12m	al_closed_l12m_x	,
ever_tw	ever_tw_x	,
nbr_ever_tw	nbr_ever_tw_x	,
active_tw	active_tw_x	,
nbr_active_tw	nbr_active_tw_x	,
tw_new_l1m	tw_new_l1m_x	,
tw_new_l3m	tw_new_l3m_x	,
tw_new_l6m	tw_new_l6m_x	,
tw_new_l12m	tw_new_l12m_x	,
tw_closed_l1m	tw_closed_l1m_x	,
tw_closed_l3m	tw_closed_l3m_x	,
tw_closed_l6m	tw_closed_l6m_x	,
tw_closed_l12m	tw_closed_l12m_x	,
nbr_ever_stpl	nbr_ever_stpl_x	,
nbr_active_stpl	nbr_active_stpl_x	,
stpl_new_l1m	stpl_new_l1m_x	,
stpl_new_l3m	stpl_new_l3m_x	,
stpl_new_l6m	stpl_new_l6m_x	,
stpl_new_l12m	stpl_new_l12m_x	,
stpl_closed_l1m	stpl_closed_l1m_x	,
stpl_closed_l3m	stpl_closed_l3m_x	,
stpl_closed_l6m	stpl_closed_l6m_x	,
stpl_closed_l12m	stpl_closed_l12m_x	,
ever_cv	ever_cv_x	,
nbr_ever_cv	nbr_ever_cv_x	,
active_cv	active_cv_x	,
nbr_active_cv	nbr_active_cv_x	,
ever_agri	ever_agri_x	,
nbr_ever_agri	nbr_ever_agri_x	,
active_agri	active_agri_x	,
nbr_active_agri	nbr_active_agri_x	,
ever_lap	ever_lap_x	,
nbr_ever_lap	nbr_ever_lap_x	,
active_lap	active_lap_x	,
nbr_active_lap	nbr_active_lap_x	,
ever_las	ever_las_x	,
nbr_ever_las	nbr_ever_las_x	,
active_las	active_las_x	,
nbr_active_las	nbr_active_las_x	
FROM fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m1_data_overall
)
INNER JOIN(
SELECT customer_no,
creation_date	creation_date_y	,
max_dpd_l36m	max_dpd_l36m_y	,
max_dpd_l24m	max_dpd_l24m_y	,
max_dpd_l18m	max_dpd_l18m_y	,
max_dpd_l12m	max_dpd_l12m_y	,
max_dpd_l6m	max_dpd_l6m_y	,
max_dpd_l3m	max_dpd_l3m_y	,
max_dpd_cm	max_dpd_cm_y	,
max_active_dpd_l36m	max_active_dpd_l36m_y	,
max_active_dpd_l24m	max_active_dpd_l24m_y	,
max_active_dpd_l18m	max_active_dpd_l18m_y	,
max_active_dpd_l12m	max_active_dpd_l12m_y	,
max_active_dpd_l6m	max_active_dpd_l6m_y	,
max_active_dpd_l3m	max_active_dpd_l3m_y	,
max_active_dpd_cm	max_active_dpd_cm_y	,
max_active_cc_dpd_l36m	max_active_cc_dpd_l36m_y	,
max_active_cc_dpd_l24m	max_active_cc_dpd_l24m_y	,
max_active_cc_dpd_l18m	max_active_cc_dpd_l18m_y	,
max_active_cc_dpd_l12m	max_active_cc_dpd_l12m_y	,
max_active_cc_dpd_l6m	max_active_cc_dpd_l6m_y	,
max_active_cc_dpd_l3m	max_active_cc_dpd_l3m_y	,
max_active_cc_dpd_cm	max_active_cc_dpd_cm_y	,
max_cc_dpd_l36m	max_cc_dpd_l36m_y	,
max_cc_dpd_l24m	max_cc_dpd_l24m_y	,
max_cc_dpd_l18m	max_cc_dpd_l18m_y	,
max_cc_dpd_l12m	max_cc_dpd_l12m_y	,
max_cc_dpd_l6m	max_cc_dpd_l6m_y	,
max_cc_dpd_l3m	max_cc_dpd_l3m_y	,
max_cc_dpd_cm	max_cc_dpd_cm_y	,
max_active_pl_dpd_l36m	max_active_pl_dpd_l36m_y	,
max_active_pl_dpd_l24m	max_active_pl_dpd_l24m_y	,
max_active_pl_dpd_l18m	max_active_pl_dpd_l18m_y	,
max_active_pl_dpd_l12m	max_active_pl_dpd_l12m_y	,
max_active_pl_dpd_l6m	max_active_pl_dpd_l6m_y	,
max_active_pl_dpd_l3m	max_active_pl_dpd_l3m_y	,
max_active_pl_dpd_cm	max_active_pl_dpd_cm_y	,
max_pl_dpd_l36m	max_pl_dpd_l36m_y	,
max_pl_dpd_l24m	max_pl_dpd_l24m_y	,
max_pl_dpd_l18m	max_pl_dpd_l18m_y	,
max_pl_dpd_l12m	max_pl_dpd_l12m_y	,
max_pl_dpd_l6m	max_pl_dpd_l6m_y	,
max_pl_dpd_l3m	max_pl_dpd_l3m_y	,
max_pl_dpd_cm	max_pl_dpd_cm_y	,
max_active_cd_dpd_l36m	max_active_cd_dpd_l36m_y	,
max_active_cd_dpd_l24m	max_active_cd_dpd_l24m_y	,
max_active_cd_dpd_l18m	max_active_cd_dpd_l18m_y	,
max_active_cd_dpd_l12m	max_active_cd_dpd_l12m_y	,
max_active_cd_dpd_l6m	max_active_cd_dpd_l6m_y	,
max_active_cd_dpd_l3m	max_active_cd_dpd_l3m_y	,
max_active_cd_dpd_cm	max_active_cd_dpd_cm_y	,
max_cd_dpd_l36m	max_cd_dpd_l36m_y	,
max_cd_dpd_l24m	max_cd_dpd_l24m_y	,
max_cd_dpd_l18m	max_cd_dpd_l18m_y	,
max_cd_dpd_l12m	max_cd_dpd_l12m_y	,
max_cd_dpd_l6m	max_cd_dpd_l6m_y	,
max_cd_dpd_l3m	max_cd_dpd_l3m_y	,
max_cd_dpd_cm	max_cd_dpd_cm_y	,
max_active_gl_dpd_l36m	max_active_gl_dpd_l36m_y	,
max_active_gl_dpd_l24m	max_active_gl_dpd_l24m_y	,
max_active_gl_dpd_l18m	max_active_gl_dpd_l18m_y	,
max_active_gl_dpd_l12m	max_active_gl_dpd_l12m_y	,
max_active_gl_dpd_l6m	max_active_gl_dpd_l6m_y	,
max_active_gl_dpd_l3m	max_active_gl_dpd_l3m_y	,
max_active_gl_dpd_cm	max_active_gl_dpd_cm_y	,
max_gl_dpd_l36m	max_gl_dpd_l36m_y	,
max_gl_dpd_l24m	max_gl_dpd_l24m_y	,
max_gl_dpd_l18m	max_gl_dpd_l18m_y	,
max_gl_dpd_l12m	max_gl_dpd_l12m_y	,
max_gl_dpd_l6m	max_gl_dpd_l6m_y	,
max_gl_dpd_l3m	max_gl_dpd_l3m_y	,
max_gl_dpd_cm	max_gl_dpd_cm_y	,
max_active_al_dpd_l36m	max_active_al_dpd_l36m_y	,
max_active_al_dpd_l24m	max_active_al_dpd_l24m_y	,
max_active_al_dpd_l18m	max_active_al_dpd_l18m_y	,
max_active_al_dpd_l12m	max_active_al_dpd_l12m_y	,
max_active_al_dpd_l6m	max_active_al_dpd_l6m_y	,
max_active_al_dpd_l3m	max_active_al_dpd_l3m_y	,
max_active_al_dpd_cm	max_active_al_dpd_cm_y	,
max_al_dpd_l36m	max_al_dpd_l36m_y	,
max_al_dpd_l24m	max_al_dpd_l24m_y	,
max_al_dpd_l18m	max_al_dpd_l18m_y	,
max_al_dpd_l12m	max_al_dpd_l12m_y	,
max_al_dpd_l6m	max_al_dpd_l6m_y	,
max_al_dpd_l3m	max_al_dpd_l3m_y	,
max_al_dpd_cm	max_al_dpd_cm_y	,
max_active_tw_dpd_l36m	max_active_tw_dpd_l36m_y	,
max_active_tw_dpd_l24m	max_active_tw_dpd_l24m_y	,
max_active_tw_dpd_l18m	max_active_tw_dpd_l18m_y	,
max_active_tw_dpd_l12m	max_active_tw_dpd_l12m_y	,
max_active_tw_dpd_l6m	max_active_tw_dpd_l6m_y	,
max_active_tw_dpd_l3m	max_active_tw_dpd_l3m_y	,
max_active_tw_dpd_cm	max_active_tw_dpd_cm_y	,
max_tw_dpd_l36m	max_tw_dpd_l36m_y	,
max_tw_dpd_l24m	max_tw_dpd_l24m_y	,
max_tw_dpd_l18m	max_tw_dpd_l18m_y	,
max_tw_dpd_l12m	max_tw_dpd_l12m_y	,
max_tw_dpd_l6m	max_tw_dpd_l6m_y	,
max_tw_dpd_l3m	max_tw_dpd_l3m_y	,
max_tw_dpd_cm	max_tw_dpd_cm_y	,
max_active_hl_dpd_l36m	max_active_hl_dpd_l36m_y	,
max_active_hl_dpd_l24m	max_active_hl_dpd_l24m_y	,
max_active_hl_dpd_l18m	max_active_hl_dpd_l18m_y	,
max_active_hl_dpd_l12m	max_active_hl_dpd_l12m_y	,
max_active_hl_dpd_l6m	max_active_hl_dpd_l6m_y	,
max_active_hl_dpd_l3m	max_active_hl_dpd_l3m_y	,
max_active_hl_dpd_cm	max_active_hl_dpd_cm_y	,
max_hl_dpd_l36m	max_hl_dpd_l36m_y	,
max_hl_dpd_l24m	max_hl_dpd_l24m_y	,
max_hl_dpd_l18m	max_hl_dpd_l18m_y	,
max_hl_dpd_l12m	max_hl_dpd_l12m_y	,
max_hl_dpd_l6m	max_hl_dpd_l6m_y	,
max_hl_dpd_l3m	max_hl_dpd_l3m_y	,
max_hl_dpd_cm	max_hl_dpd_cm_y	,
max_active_unsecured_dpd_l36m	max_active_unsecured_dpd_l36m_y	,
max_active_unsecured_dpd_l24m	max_active_unsecured_dpd_l24m_y	,
max_active_unsecured_dpd_l18m	max_active_unsecured_dpd_l18m_y	,
max_active_unsecured_dpd_l12m	max_active_unsecured_dpd_l12m_y	,
max_active_unsecured_dpd_l6m	max_active_unsecured_dpd_l6m_y	,
max_active_unsecured_dpd_l3m	max_active_unsecured_dpd_l3m_y	,
max_active_unsecured_dpd_cm	max_active_unsecured_dpd_cm_y	,
max_unsecured_dpd_l36m	max_unsecured_dpd_l36m_y	,
max_unsecured_dpd_l24m	max_unsecured_dpd_l24m_y	,
max_unsecured_dpd_l18m	max_unsecured_dpd_l18m_y	,
max_unsecured_dpd_l12m	max_unsecured_dpd_l12m_y	,
max_unsecured_dpd_l6m	max_unsecured_dpd_l6m_y	,
max_unsecured_dpd_l3m	max_unsecured_dpd_l3m_y	,
max_unsecured_dpd_cm	max_unsecured_dpd_cm_y	,
max_active_secured_dpd_l36m	max_active_secured_dpd_l36m_y	,
max_active_secured_dpd_l24m	max_active_secured_dpd_l24m_y	,
max_active_secured_dpd_l18m	max_active_secured_dpd_l18m_y	,
max_active_secured_dpd_l12m	max_active_secured_dpd_l12m_y	,
max_active_secured_dpd_l6m	max_active_secured_dpd_l6m_y	,
max_active_secured_dpd_l3m	max_active_secured_dpd_l3m_y	,
max_active_secured_dpd_cm	max_active_secured_dpd_cm_y	,
max_secured_dpd_l36m	max_secured_dpd_l36m_y	,
max_secured_dpd_l24m	max_secured_dpd_l24m_y	,
max_secured_dpd_l18m	max_secured_dpd_l18m_y	,
max_secured_dpd_l12m	max_secured_dpd_l12m_y	,
max_secured_dpd_l6m	max_secured_dpd_l6m_y	,
max_secured_dpd_l3m	max_secured_dpd_l3m_y	,
max_secured_dpd_cm	max_secured_dpd_cm_y	,
overdue_l36m	overdue_l36m_y	,
overdue_l24m	overdue_l24m_y	,
overdue_l18m	overdue_l18m_y	,
overdue_l12m	overdue_l12m_y	,
overdue_l9m	overdue_l9m_y	,
overdue_l6m	overdue_l6m_y	,
overdue_l3m	overdue_l3m_y	,
overdue_cm	overdue_cm_y	,
cc_overdue_l36m	cc_overdue_l36m_y	,
cc_overdue_l24m	cc_overdue_l24m_y	,
cc_overdue_l18m	cc_overdue_l18m_y	,
cc_overdue_l12m	cc_overdue_l12m_y	,
cc_overdue_l9m	cc_overdue_l9m_y	,
cc_overdue_l6m	cc_overdue_l6m_y	,
cc_overdue_l3m	cc_overdue_l3m_y	,
cc_overdue_cm	cc_overdue_cm_y	,
pl_overdue_l36m	pl_overdue_l36m_y	,
pl_overdue_l24m	pl_overdue_l24m_y	,
pl_overdue_l18m	pl_overdue_l18m_y	,
pl_overdue_l12m	pl_overdue_l12m_y	,
pl_overdue_l9m	pl_overdue_l9m_y	,
pl_overdue_l6m	pl_overdue_l6m_y	,
pl_overdue_l3m	pl_overdue_l3m_y	,
pl_overdue_cm	pl_overdue_cm_y	,
cd_overdue_l36m	cd_overdue_l36m_y	,
cd_overdue_l24m	cd_overdue_l24m_y	,
cd_overdue_l18m	cd_overdue_l18m_y	,
cd_overdue_l12m	cd_overdue_l12m_y	,
cd_overdue_l9m	cd_overdue_l9m_y	,
cd_overdue_l6m	cd_overdue_l6m_y	,
cd_overdue_l3m	cd_overdue_l3m_y	,
cd_overdue_cm	cd_overdue_cm_y	,
gl_overdue_l36m	gl_overdue_l36m_y	,
gl_overdue_l24m	gl_overdue_l24m_y	,
gl_overdue_l18m	gl_overdue_l18m_y	,
gl_overdue_l12m	gl_overdue_l12m_y	,
gl_overdue_l9m	gl_overdue_l9m_y	,
gl_overdue_l6m	gl_overdue_l6m_y	,
gl_overdue_l3m	gl_overdue_l3m_y	,
gl_overdue_cm	gl_overdue_cm_y	,
al_overdue_l36m	al_overdue_l36m_y	,
al_overdue_l24m	al_overdue_l24m_y	,
al_overdue_l18m	al_overdue_l18m_y	,
al_overdue_l12m	al_overdue_l12m_y	,
al_overdue_l9m	al_overdue_l9m_y	,
al_overdue_l6m	al_overdue_l6m_y	,
al_overdue_l3m	al_overdue_l3m_y	,
al_overdue_cm	al_overdue_cm_y	,
tw_overdue_l36m	tw_overdue_l36m_y	,
tw_overdue_l24m	tw_overdue_l24m_y	,
tw_overdue_l18m	tw_overdue_l18m_y	,
tw_overdue_l12m	tw_overdue_l12m_y	,
tw_overdue_l9m	tw_overdue_l9m_y	,
tw_overdue_l6m	tw_overdue_l6m_y	,
tw_overdue_l3m	tw_overdue_l3m_y	,
tw_overdue_cm	tw_overdue_cm_y	,
hl_overdue_l36m	hl_overdue_l36m_y	,
hl_overdue_l24m	hl_overdue_l24m_y	,
hl_overdue_l18m	hl_overdue_l18m_y	,
hl_overdue_l12m	hl_overdue_l12m_y	,
hl_overdue_l9m	hl_overdue_l9m_y	,
hl_overdue_l6m	hl_overdue_l6m_y	,
hl_overdue_l3m	hl_overdue_l3m_y	,
hl_overdue_cm	hl_overdue_cm_y	,
unsecured_overdue_l36m	unsecured_overdue_l36m_y	,
unsecured_overdue_l24m	unsecured_overdue_l24m_y	,
unsecured_overdue_l18m	unsecured_overdue_l18m_y	,
unsecured_overdue_l12m	unsecured_overdue_l12m_y	,
unsecured_overdue_l9m	unsecured_overdue_l9m_y	,
unsecured_overdue_l6m	unsecured_overdue_l6m_y	,
unsecured_overdue_l3m	unsecured_overdue_l3m_y	,
unsecured_overdue_cm	unsecured_overdue_cm_y	,
secured_overdue_l36m	secured_overdue_l36m_y	,
secured_overdue_l24m	secured_overdue_l24m_y	,
secured_overdue_l18m	secured_overdue_l18m_y	,
secured_overdue_l12m	secured_overdue_l12m_y	,
secured_overdue_l9m	secured_overdue_l9m_y	,
secured_overdue_l6m	secured_overdue_l6m_y	,
secured_overdue_l3m	secured_overdue_l3m_y	,
secured_overdue_cm	secured_overdue_cm_y	,
nbr_x_dpd_l36m	nbr_x_dpd_l36m_y	,
nbr_x_dpd_l24m	nbr_x_dpd_l24m_y	,
nbr_x_dpd_l18m	nbr_x_dpd_l18m_y	,
nbr_x_dpd_l12m	nbr_x_dpd_l12m_y	,
nbr_x_dpd_l6m	nbr_x_dpd_l6m_y	,
nbr_x_dpd_l3m	nbr_x_dpd_l3m_y	,
nbr_x_dpd_cm	nbr_x_dpd_cm_y	,
nbr_30_dpd_l36m	nbr_30_dpd_l36m_y	,
nbr_30_dpd_l24m	nbr_30_dpd_l24m_y	,
nbr_30_dpd_l18m	nbr_30_dpd_l18m_y	,
nbr_30_dpd_l12m	nbr_30_dpd_l12m_y	,
nbr_30_dpd_l6m	nbr_30_dpd_l6m_y	,
nbr_30_dpd_l3m	nbr_30_dpd_l3m_y	,
nbr_30_dpd_cm	nbr_30_dpd_cm_y	,
nbr_90_dpd_l36m	nbr_90_dpd_l36m_y	,
nbr_90_dpd_l24m	nbr_90_dpd_l24m_y	,
nbr_90_dpd_l18m	nbr_90_dpd_l18m_y	,
nbr_90_dpd_l12m	nbr_90_dpd_l12m_y	,
nbr_90_dpd_l6m	nbr_90_dpd_l6m_y	,
nbr_90_dpd_l3m	nbr_90_dpd_l3m_y	,
nbr_90_dpd_cm	nbr_90_dpd_cm_y	,
nbr_active_x_dpd_l36m	nbr_active_x_dpd_l36m_y	,
nbr_active_x_dpd_l24m	nbr_active_x_dpd_l24m_y	,
nbr_active_x_dpd_l18m	nbr_active_x_dpd_l18m_y	,
nbr_active_x_dpd_l12m	nbr_active_x_dpd_l12m_y	,
nbr_active_x_dpd_l6m	nbr_active_x_dpd_l6m_y	,
nbr_active_x_dpd_l3m	nbr_active_x_dpd_l3m_y	,
nbr_active_x_dpd_cm	nbr_active_x_dpd_cm_y	,
nbr_active_30_dpd_l36m	nbr_active_30_dpd_l36m_y	,
nbr_active_30_dpd_l24m	nbr_active_30_dpd_l24m_y	,
nbr_active_30_dpd_l18m	nbr_active_30_dpd_l18m_y	,
nbr_active_30_dpd_l12m	nbr_active_30_dpd_l12m_y	,
nbr_active_30_dpd_l6m	nbr_active_30_dpd_l6m_y	,
nbr_active_30_dpd_l3m	nbr_active_30_dpd_l3m_y	,
nbr_active_30_dpd_cm	nbr_active_30_dpd_cm_y	,
nbr_active_90_dpd_l36m	nbr_active_90_dpd_l36m_y	,
nbr_active_90_dpd_l24m	nbr_active_90_dpd_l24m_y	,
nbr_active_90_dpd_l18m	nbr_active_90_dpd_l18m_y	,
nbr_active_90_dpd_l12m	nbr_active_90_dpd_l12m_y	,
nbr_active_90_dpd_l6m	nbr_active_90_dpd_l6m_y	,
nbr_active_90_dpd_l3m	nbr_active_90_dpd_l3m_y	,
nbr_active_90_dpd_cm	nbr_active_90_dpd_cm_y	,
nbr_cc_x_dpd_l36m	nbr_cc_x_dpd_l36m_y	,
nbr_cc_x_dpd_l24m	nbr_cc_x_dpd_l24m_y	,
nbr_cc_x_dpd_l18m	nbr_cc_x_dpd_l18m_y	,
nbr_cc_x_dpd_l12m	nbr_cc_x_dpd_l12m_y	,
nbr_cc_x_dpd_l6m	nbr_cc_x_dpd_l6m_y	,
nbr_cc_x_dpd_l3m	nbr_cc_x_dpd_l3m_y	,
nbr_cc_x_dpd_cm	nbr_cc_x_dpd_cm_y	,
nbr_cc_30_dpd_l36m	nbr_cc_30_dpd_l36m_y	,
nbr_cc_30_dpd_l24m	nbr_cc_30_dpd_l24m_y	,
nbr_cc_30_dpd_l18m	nbr_cc_30_dpd_l18m_y	,
nbr_cc_30_dpd_l12m	nbr_cc_30_dpd_l12m_y	,
nbr_cc_30_dpd_l6m	nbr_cc_30_dpd_l6m_y	,
nbr_cc_30_dpd_l3m	nbr_cc_30_dpd_l3m_y	,
nbr_cc_30_dpd_cm	nbr_cc_30_dpd_cm_y	,
nbr_cc_90_dpd_l36m	nbr_cc_90_dpd_l36m_y	,
nbr_cc_90_dpd_l24m	nbr_cc_90_dpd_l24m_y	,
nbr_cc_90_dpd_l18m	nbr_cc_90_dpd_l18m_y	,
nbr_cc_90_dpd_l12m	nbr_cc_90_dpd_l12m_y	,
nbr_cc_90_dpd_l6m	nbr_cc_90_dpd_l6m_y	,
nbr_cc_90_dpd_l3m	nbr_cc_90_dpd_l3m_y	,
nbr_cc_90_dpd_cm	nbr_cc_90_dpd_cm_y	,
nbr_active_cc_x_dpd_l36m	nbr_active_cc_x_dpd_l36m_y	,
nbr_active_cc_x_dpd_l24m	nbr_active_cc_x_dpd_l24m_y	,
nbr_active_cc_x_dpd_l18m	nbr_active_cc_x_dpd_l18m_y	,
nbr_active_cc_x_dpd_l12m	nbr_active_cc_x_dpd_l12m_y	,
nbr_active_cc_x_dpd_l6m	nbr_active_cc_x_dpd_l6m_y	,
nbr_active_cc_x_dpd_l3m	nbr_active_cc_x_dpd_l3m_y	,
nbr_active_cc_x_dpd_cm	nbr_active_cc_x_dpd_cm_y	,
nbr_active_cc_30_dpd_l36m	nbr_active_cc_30_dpd_l36m_y	,
nbr_active_cc_30_dpd_l24m	nbr_active_cc_30_dpd_l24m_y	,
nbr_active_cc_30_dpd_l18m	nbr_active_cc_30_dpd_l18m_y	,
nbr_active_cc_30_dpd_l12m	nbr_active_cc_30_dpd_l12m_y	,
nbr_active_cc_30_dpd_l6m	nbr_active_cc_30_dpd_l6m_y	,
nbr_active_cc_30_dpd_l3m	nbr_active_cc_30_dpd_l3m_y	,
nbr_active_cc_30_dpd_cm	nbr_active_cc_30_dpd_cm_y	,
nbr_cc_active_90_dpd_l36m	nbr_cc_active_90_dpd_l36m_y	,
nbr_cc_active_90_dpd_l24m	nbr_cc_active_90_dpd_l24m_y	,
nbr_cc_active_90_dpd_l18m	nbr_cc_active_90_dpd_l18m_y	,
nbr_cc_active_90_dpd_l12m	nbr_cc_active_90_dpd_l12m_y	,
nbr_cc_active_90_dpd_l6m	nbr_cc_active_90_dpd_l6m_y	,
nbr_cc_active_90_dpd_l3m	nbr_cc_active_90_dpd_l3m_y	,
nbr_cc_active_90_dpd_cm	nbr_cc_active_90_dpd_cm_y	,
nbr_pl_x_dpd_l36m	nbr_pl_x_dpd_l36m_y	,
nbr_pl_x_dpd_l24m	nbr_pl_x_dpd_l24m_y	,
nbr_pl_x_dpd_l18m	nbr_pl_x_dpd_l18m_y	,
nbr_pl_x_dpd_l12m	nbr_pl_x_dpd_l12m_y	,
nbr_pl_x_dpd_l6m	nbr_pl_x_dpd_l6m_y	,
nbr_pl_x_dpd_l3m	nbr_pl_x_dpd_l3m_y	,
nbr_pl_x_dpd_cm	nbr_pl_x_dpd_cm_y	,
nbr_pl_30_dpd_l36m	nbr_pl_30_dpd_l36m_y	,
nbr_pl_30_dpd_l24m	nbr_pl_30_dpd_l24m_y	,
nbr_pl_30_dpd_l18m	nbr_pl_30_dpd_l18m_y	,
nbr_pl_30_dpd_l12m	nbr_pl_30_dpd_l12m_y	,
nbr_pl_30_dpd_l6m	nbr_pl_30_dpd_l6m_y	,
nbr_pl_30_dpd_l3m	nbr_pl_30_dpd_l3m_y	,
nbr_pl_30_dpd_cm	nbr_pl_30_dpd_cm_y	,
nbr_pl_90_dpd_l36m	nbr_pl_90_dpd_l36m_y	,
nbr_pl_90_dpd_l24m	nbr_pl_90_dpd_l24m_y	,
nbr_pl_90_dpd_l18m	nbr_pl_90_dpd_l18m_y	,
nbr_pl_90_dpd_l12m	nbr_pl_90_dpd_l12m_y	,
nbr_pl_90_dpd_l6m	nbr_pl_90_dpd_l6m_y	,
nbr_pl_90_dpd_l3m	nbr_pl_90_dpd_l3m_y	,
nbr_pl_90_dpd_cm	nbr_pl_90_dpd_cm_y	,
nbr_active_pl_x_dpd_l36m	nbr_active_pl_x_dpd_l36m_y	,
nbr_active_pl_x_dpd_l24m	nbr_active_pl_x_dpd_l24m_y	,
nbr_active_pl_x_dpd_l18m	nbr_active_pl_x_dpd_l18m_y	,
nbr_active_pl_x_dpd_l12m	nbr_active_pl_x_dpd_l12m_y	,
nbr_active_pl_x_dpd_l6m	nbr_active_pl_x_dpd_l6m_y	,
nbr_active_pl_x_dpd_l3m	nbr_active_pl_x_dpd_l3m_y	,
nbr_active_pl_x_dpd_cm	nbr_active_pl_x_dpd_cm_y	,
nbr_active_pl_30_dpd_l36m	nbr_active_pl_30_dpd_l36m_y	,
nbr_active_pl_30_dpd_l24m	nbr_active_pl_30_dpd_l24m_y	,
nbr_active_pl_30_dpd_l18m	nbr_active_pl_30_dpd_l18m_y	,
nbr_active_pl_30_dpd_l12m	nbr_active_pl_30_dpd_l12m_y	,
nbr_active_pl_30_dpd_l6m	nbr_active_pl_30_dpd_l6m_y	,
nbr_active_pl_30_dpd_l3m	nbr_active_pl_30_dpd_l3m_y	,
nbr_active_pl_30_dpd_cm	nbr_active_pl_30_dpd_cm_y	,
nbr_pl_active_90_dpd_l36m	nbr_pl_active_90_dpd_l36m_y	,
nbr_pl_active_90_dpd_l24m	nbr_pl_active_90_dpd_l24m_y	,
nbr_pl_active_90_dpd_l18m	nbr_pl_active_90_dpd_l18m_y	,
nbr_pl_active_90_dpd_l12m	nbr_pl_active_90_dpd_l12m_y	,
nbr_pl_active_90_dpd_l6m	nbr_pl_active_90_dpd_l6m_y	,
nbr_pl_active_90_dpd_l3m	nbr_pl_active_90_dpd_l3m_y	,
nbr_pl_active_90_dpd_cm	nbr_pl_active_90_dpd_cm_y	,
nbr_cd_x_dpd_l36m	nbr_cd_x_dpd_l36m_y	,
nbr_cd_x_dpd_l24m	nbr_cd_x_dpd_l24m_y	,
nbr_cd_x_dpd_l18m	nbr_cd_x_dpd_l18m_y	,
nbr_cd_x_dpd_l12m	nbr_cd_x_dpd_l12m_y	,
nbr_cd_x_dpd_l6m	nbr_cd_x_dpd_l6m_y	,
nbr_cd_x_dpd_l3m	nbr_cd_x_dpd_l3m_y	,
nbr_cd_x_dpd_cm	nbr_cd_x_dpd_cm_y	,
nbr_cd_30_dpd_l36m	nbr_cd_30_dpd_l36m_y	,
nbr_cd_30_dpd_l24m	nbr_cd_30_dpd_l24m_y	,
nbr_cd_30_dpd_l18m	nbr_cd_30_dpd_l18m_y	,
nbr_cd_30_dpd_l12m	nbr_cd_30_dpd_l12m_y	,
nbr_cd_30_dpd_l6m	nbr_cd_30_dpd_l6m_y	,
nbr_cd_30_dpd_l3m	nbr_cd_30_dpd_l3m_y	,
nbr_cd_30_dpd_cm	nbr_cd_30_dpd_cm_y	,
nbr_cd_90_dpd_l36m	nbr_cd_90_dpd_l36m_y	,
nbr_cd_90_dpd_l24m	nbr_cd_90_dpd_l24m_y	,
nbr_cd_90_dpd_l18m	nbr_cd_90_dpd_l18m_y	,
nbr_cd_90_dpd_l12m	nbr_cd_90_dpd_l12m_y	,
nbr_cd_90_dpd_l6m	nbr_cd_90_dpd_l6m_y	,
nbr_cd_90_dpd_l3m	nbr_cd_90_dpd_l3m_y	,
nbr_cd_90_dpd_cm	nbr_cd_90_dpd_cm_y	,
nbr_active_cd_x_dpd_l36m	nbr_active_cd_x_dpd_l36m_y	,
nbr_active_cd_x_dpd_l24m	nbr_active_cd_x_dpd_l24m_y	,
nbr_active_cd_x_dpd_l18m	nbr_active_cd_x_dpd_l18m_y	,
nbr_active_cd_x_dpd_l12m	nbr_active_cd_x_dpd_l12m_y	,
nbr_active_cd_x_dpd_l6m	nbr_active_cd_x_dpd_l6m_y	,
nbr_active_cd_x_dpd_l3m	nbr_active_cd_x_dpd_l3m_y	,
nbr_active_cd_x_dpd_cm	nbr_active_cd_x_dpd_cm_y	,
nbr_active_cd_30_dpd_l36m	nbr_active_cd_30_dpd_l36m_y	,
nbr_active_cd_30_dpd_l24m	nbr_active_cd_30_dpd_l24m_y	,
nbr_active_cd_30_dpd_l18m	nbr_active_cd_30_dpd_l18m_y	,
nbr_active_cd_30_dpd_l12m	nbr_active_cd_30_dpd_l12m_y	,
nbr_active_cd_30_dpd_l6m	nbr_active_cd_30_dpd_l6m_y	,
nbr_active_cd_30_dpd_l3m	nbr_active_cd_30_dpd_l3m_y	,
nbr_active_cd_30_dpd_cm	nbr_active_cd_30_dpd_cm_y	,
nbr_cd_active_90_dpd_l36m	nbr_cd_active_90_dpd_l36m_y	,
nbr_cd_active_90_dpd_l24m	nbr_cd_active_90_dpd_l24m_y	,
nbr_cd_active_90_dpd_l18m	nbr_cd_active_90_dpd_l18m_y	,
nbr_cd_active_90_dpd_l12m	nbr_cd_active_90_dpd_l12m_y	,
nbr_cd_active_90_dpd_l6m	nbr_cd_active_90_dpd_l6m_y	,
nbr_cd_active_90_dpd_l3m	nbr_cd_active_90_dpd_l3m_y	,
nbr_cd_active_90_dpd_cm	nbr_cd_active_90_dpd_cm_y	,
nbr_al_x_dpd_l36m	nbr_al_x_dpd_l36m_y	,
nbr_al_x_dpd_l24m	nbr_al_x_dpd_l24m_y	,
nbr_al_x_dpd_l18m	nbr_al_x_dpd_l18m_y	,
nbr_al_x_dpd_l12m	nbr_al_x_dpd_l12m_y	,
nbr_al_x_dpd_l6m	nbr_al_x_dpd_l6m_y	,
nbr_al_x_dpd_l3m	nbr_al_x_dpd_l3m_y	,
nbr_al_x_dpd_cm	nbr_al_x_dpd_cm_y	,
nbr_al_30_dpd_l36m	nbr_al_30_dpd_l36m_y	,
nbr_al_30_dpd_l24m	nbr_al_30_dpd_l24m_y	,
nbr_al_30_dpd_l18m	nbr_al_30_dpd_l18m_y	,
nbr_al_30_dpd_l12m	nbr_al_30_dpd_l12m_y	,
nbr_al_30_dpd_l6m	nbr_al_30_dpd_l6m_y	,
nbr_al_30_dpd_l3m	nbr_al_30_dpd_l3m_y	,
nbr_al_30_dpd_cm	nbr_al_30_dpd_cm_y	,
nbr_al_90_dpd_l36m	nbr_al_90_dpd_l36m_y	,
nbr_al_90_dpd_l24m	nbr_al_90_dpd_l24m_y	,
nbr_al_90_dpd_l18m	nbr_al_90_dpd_l18m_y	,
nbr_al_90_dpd_l12m	nbr_al_90_dpd_l12m_y	,
nbr_al_90_dpd_l6m	nbr_al_90_dpd_l6m_y	,
nbr_al_90_dpd_l3m	nbr_al_90_dpd_l3m_y	,
nbr_al_90_dpd_cm	nbr_al_90_dpd_cm_y	,
nbr_active_al_x_dpd_l36m	nbr_active_al_x_dpd_l36m_y	,
nbr_active_al_x_dpd_l24m	nbr_active_al_x_dpd_l24m_y	,
nbr_active_al_x_dpd_l18m	nbr_active_al_x_dpd_l18m_y	,
nbr_active_al_x_dpd_l12m	nbr_active_al_x_dpd_l12m_y	,
nbr_active_al_x_dpd_l6m	nbr_active_al_x_dpd_l6m_y	,
nbr_active_al_x_dpd_l3m	nbr_active_al_x_dpd_l3m_y	,
nbr_active_al_x_dpd_cm	nbr_active_al_x_dpd_cm_y	,
nbr_active_al_30_dpd_l36m	nbr_active_al_30_dpd_l36m_y	,
nbr_active_al_30_dpd_l24m	nbr_active_al_30_dpd_l24m_y	,
nbr_active_al_30_dpd_l18m	nbr_active_al_30_dpd_l18m_y	,
nbr_active_al_30_dpd_l12m	nbr_active_al_30_dpd_l12m_y	,
nbr_active_al_30_dpd_l6m	nbr_active_al_30_dpd_l6m_y	,
nbr_active_al_30_dpd_l3m	nbr_active_al_30_dpd_l3m_y	,
nbr_active_al_30_dpd_cm	nbr_active_al_30_dpd_cm_y	,
nbr_al_active_90_dpd_l36m	nbr_al_active_90_dpd_l36m_y	,
nbr_al_active_90_dpd_l24m	nbr_al_active_90_dpd_l24m_y	,
nbr_al_active_90_dpd_l18m	nbr_al_active_90_dpd_l18m_y	,
nbr_al_active_90_dpd_l12m	nbr_al_active_90_dpd_l12m_y	,
nbr_al_active_90_dpd_l6m	nbr_al_active_90_dpd_l6m_y	,
nbr_al_active_90_dpd_l3m	nbr_al_active_90_dpd_l3m_y	,
nbr_al_active_90_dpd_cm	nbr_al_active_90_dpd_cm_y	,
nbr_gl_x_dpd_l36m	nbr_gl_x_dpd_l36m_y	,
nbr_gl_x_dpd_l24m	nbr_gl_x_dpd_l24m_y	,
nbr_gl_x_dpd_l18m	nbr_gl_x_dpd_l18m_y	,
nbr_gl_x_dpd_l12m	nbr_gl_x_dpd_l12m_y	,
nbr_gl_x_dpd_l6m	nbr_gl_x_dpd_l6m_y	,
nbr_gl_x_dpd_l3m	nbr_gl_x_dpd_l3m_y	,
nbr_gl_x_dpd_cm	nbr_gl_x_dpd_cm_y	,
nbr_gl_30_dpd_l36m	nbr_gl_30_dpd_l36m_y	,
nbr_gl_30_dpd_l24m	nbr_gl_30_dpd_l24m_y	,
nbr_gl_30_dpd_l18m	nbr_gl_30_dpd_l18m_y	,
nbr_gl_30_dpd_l12m	nbr_gl_30_dpd_l12m_y	,
nbr_gl_30_dpd_l6m	nbr_gl_30_dpd_l6m_y	,
nbr_gl_30_dpd_l3m	nbr_gl_30_dpd_l3m_y	,
nbr_gl_30_dpd_cm	nbr_gl_30_dpd_cm_y	,
nbr_gl_90_dpd_l36m	nbr_gl_90_dpd_l36m_y	,
nbr_gl_90_dpd_l24m	nbr_gl_90_dpd_l24m_y	,
nbr_gl_90_dpd_l18m	nbr_gl_90_dpd_l18m_y	,
nbr_gl_90_dpd_l12m	nbr_gl_90_dpd_l12m_y	,
nbr_gl_90_dpd_l6m	nbr_gl_90_dpd_l6m_y	,
nbr_gl_90_dpd_l3m	nbr_gl_90_dpd_l3m_y	,
nbr_gl_90_dpd_cm	nbr_gl_90_dpd_cm_y	,
nbr_active_gl_x_dpd_l36m	nbr_active_gl_x_dpd_l36m_y	,
nbr_active_gl_x_dpd_l24m	nbr_active_gl_x_dpd_l24m_y	,
nbr_active_gl_x_dpd_l18m	nbr_active_gl_x_dpd_l18m_y	,
nbr_active_gl_x_dpd_l12m	nbr_active_gl_x_dpd_l12m_y	,
nbr_active_gl_x_dpd_l6m	nbr_active_gl_x_dpd_l6m_y	,
nbr_active_gl_x_dpd_l3m	nbr_active_gl_x_dpd_l3m_y	,
nbr_active_gl_x_dpd_cm	nbr_active_gl_x_dpd_cm_y	,
nbr_active_gl_30_dpd_l36m	nbr_active_gl_30_dpd_l36m_y	,
nbr_active_gl_30_dpd_l24m	nbr_active_gl_30_dpd_l24m_y	,
nbr_active_gl_30_dpd_l18m	nbr_active_gl_30_dpd_l18m_y	,
nbr_active_gl_30_dpd_l12m	nbr_active_gl_30_dpd_l12m_y	,
nbr_active_gl_30_dpd_l6m	nbr_active_gl_30_dpd_l6m_y	,
nbr_active_gl_30_dpd_l3m	nbr_active_gl_30_dpd_l3m_y	,
nbr_active_gl_30_dpd_cm	nbr_active_gl_30_dpd_cm_y	,
nbr_gl_active_90_dpd_l36m	nbr_gl_active_90_dpd_l36m_y	,
nbr_gl_active_90_dpd_l24m	nbr_gl_active_90_dpd_l24m_y	,
nbr_gl_active_90_dpd_l18m	nbr_gl_active_90_dpd_l18m_y	,
nbr_gl_active_90_dpd_l12m	nbr_gl_active_90_dpd_l12m_y	,
nbr_gl_active_90_dpd_l6m	nbr_gl_active_90_dpd_l6m_y	,
nbr_gl_active_90_dpd_l3m	nbr_gl_active_90_dpd_l3m_y	,
nbr_gl_active_90_dpd_cm	nbr_gl_active_90_dpd_cm_y	,
nbr_hl_x_dpd_l36m	nbr_hl_x_dpd_l36m_y	,
nbr_hl_x_dpd_l24m	nbr_hl_x_dpd_l24m_y	,
nbr_hl_x_dpd_l18m	nbr_hl_x_dpd_l18m_y	,
nbr_hl_x_dpd_l12m	nbr_hl_x_dpd_l12m_y	,
nbr_hl_x_dpd_l6m	nbr_hl_x_dpd_l6m_y	,
nbr_hl_x_dpd_l3m	nbr_hl_x_dpd_l3m_y	,
nbr_hl_x_dpd_cm	nbr_hl_x_dpd_cm_y	,
nbr_hl_30_dpd_l36m	nbr_hl_30_dpd_l36m_y	,
nbr_hl_30_dpd_l24m	nbr_hl_30_dpd_l24m_y	,
nbr_hl_30_dpd_l18m	nbr_hl_30_dpd_l18m_y	,
nbr_hl_30_dpd_l12m	nbr_hl_30_dpd_l12m_y	,
nbr_hl_30_dpd_l6m	nbr_hl_30_dpd_l6m_y	,
nbr_hl_30_dpd_l3m	nbr_hl_30_dpd_l3m_y	,
nbr_hl_30_dpd_cm	nbr_hl_30_dpd_cm_y	,
nbr_hl_90_dpd_l36m	nbr_hl_90_dpd_l36m_y	,
nbr_hl_90_dpd_l24m	nbr_hl_90_dpd_l24m_y	,
nbr_hl_90_dpd_l18m	nbr_hl_90_dpd_l18m_y	,
nbr_hl_90_dpd_l12m	nbr_hl_90_dpd_l12m_y	,
nbr_hl_90_dpd_l6m	nbr_hl_90_dpd_l6m_y	,
nbr_hl_90_dpd_l3m	nbr_hl_90_dpd_l3m_y	,
nbr_hl_90_dpd_cm	nbr_hl_90_dpd_cm_y	,
nbr_active_hl_x_dpd_l36m	nbr_active_hl_x_dpd_l36m_y	,
nbr_active_hl_x_dpd_l24m	nbr_active_hl_x_dpd_l24m_y	,
nbr_active_hl_x_dpd_l18m	nbr_active_hl_x_dpd_l18m_y	,
nbr_active_hl_x_dpd_l12m	nbr_active_hl_x_dpd_l12m_y	,
nbr_active_hl_x_dpd_l6m	nbr_active_hl_x_dpd_l6m_y	,
nbr_active_hl_x_dpd_l3m	nbr_active_hl_x_dpd_l3m_y	,
nbr_active_hl_x_dpd_cm	nbr_active_hl_x_dpd_cm_y	,
nbr_active_hl_30_dpd_l36m	nbr_active_hl_30_dpd_l36m_y	,
nbr_active_hl_30_dpd_l24m	nbr_active_hl_30_dpd_l24m_y	,
nbr_active_hl_30_dpd_l18m	nbr_active_hl_30_dpd_l18m_y	,
nbr_active_hl_30_dpd_l12m	nbr_active_hl_30_dpd_l12m_y	,
nbr_active_hl_30_dpd_l6m	nbr_active_hl_30_dpd_l6m_y	,
nbr_active_hl_30_dpd_l3m	nbr_active_hl_30_dpd_l3m_y	,
nbr_active_hl_30_dpd_cm	nbr_active_hl_30_dpd_cm_y	,
nbr_hl_active_90_dpd_l36m	nbr_hl_active_90_dpd_l36m_y	,
nbr_hl_active_90_dpd_l24m	nbr_hl_active_90_dpd_l24m_y	,
nbr_hl_active_90_dpd_l18m	nbr_hl_active_90_dpd_l18m_y	,
nbr_hl_active_90_dpd_l12m	nbr_hl_active_90_dpd_l12m_y	,
nbr_hl_active_90_dpd_l6m	nbr_hl_active_90_dpd_l6m_y	,
nbr_hl_active_90_dpd_l3m	nbr_hl_active_90_dpd_l3m_y	,
nbr_hl_active_90_dpd_cm	nbr_hl_active_90_dpd_cm_y	,
nbr_tw_x_dpd_l36m	nbr_tw_x_dpd_l36m_y	,
nbr_tw_x_dpd_l24m	nbr_tw_x_dpd_l24m_y	,
nbr_tw_x_dpd_l18m	nbr_tw_x_dpd_l18m_y	,
nbr_tw_x_dpd_l12m	nbr_tw_x_dpd_l12m_y	,
nbr_tw_x_dpd_l6m	nbr_tw_x_dpd_l6m_y	,
nbr_tw_x_dpd_l3m	nbr_tw_x_dpd_l3m_y	,
nbr_tw_x_dpd_cm	nbr_tw_x_dpd_cm_y	,
nbr_tw_30_dpd_l36m	nbr_tw_30_dpd_l36m_y	,
nbr_tw_30_dpd_l24m	nbr_tw_30_dpd_l24m_y	,
nbr_tw_30_dpd_l18m	nbr_tw_30_dpd_l18m_y	,
nbr_tw_30_dpd_l12m	nbr_tw_30_dpd_l12m_y	,
nbr_tw_30_dpd_l6m	nbr_tw_30_dpd_l6m_y	,
nbr_tw_30_dpd_l3m	nbr_tw_30_dpd_l3m_y	,
nbr_tw_30_dpd_cm	nbr_tw_30_dpd_cm_y	,
nbr_tw_90_dpd_l36m	nbr_tw_90_dpd_l36m_y	,
nbr_tw_90_dpd_l24m	nbr_tw_90_dpd_l24m_y	,
nbr_tw_90_dpd_l18m	nbr_tw_90_dpd_l18m_y	,
nbr_tw_90_dpd_l12m	nbr_tw_90_dpd_l12m_y	,
nbr_tw_90_dpd_l6m	nbr_tw_90_dpd_l6m_y	,
nbr_tw_90_dpd_l3m	nbr_tw_90_dpd_l3m_y	,
nbr_tw_90_dpd_cm	nbr_tw_90_dpd_cm_y	,
nbr_active_tw_x_dpd_l36m	nbr_active_tw_x_dpd_l36m_y	,
nbr_active_tw_x_dpd_l24m	nbr_active_tw_x_dpd_l24m_y	,
nbr_active_tw_x_dpd_l18m	nbr_active_tw_x_dpd_l18m_y	,
nbr_active_tw_x_dpd_l12m	nbr_active_tw_x_dpd_l12m_y	,
nbr_active_tw_x_dpd_l6m	nbr_active_tw_x_dpd_l6m_y	,
nbr_active_tw_x_dpd_l3m	nbr_active_tw_x_dpd_l3m_y	,
nbr_active_tw_x_dpd_cm	nbr_active_tw_x_dpd_cm_y	,
nbr_active_tw_30_dpd_l36m	nbr_active_tw_30_dpd_l36m_y	,
nbr_active_tw_30_dpd_l24m	nbr_active_tw_30_dpd_l24m_y	,
nbr_active_tw_30_dpd_l18m	nbr_active_tw_30_dpd_l18m_y	,
nbr_active_tw_30_dpd_l12m	nbr_active_tw_30_dpd_l12m_y	,
nbr_active_tw_30_dpd_l6m	nbr_active_tw_30_dpd_l6m_y	,
nbr_active_tw_30_dpd_l3m	nbr_active_tw_30_dpd_l3m_y	,
nbr_active_tw_30_dpd_cm	nbr_active_tw_30_dpd_cm_y	,
nbr_tw_active_90_dpd_l36m	nbr_tw_active_90_dpd_l36m_y	,
nbr_tw_active_90_dpd_l24m	nbr_tw_active_90_dpd_l24m_y	,
nbr_tw_active_90_dpd_l18m	nbr_tw_active_90_dpd_l18m_y	,
nbr_tw_active_90_dpd_l12m	nbr_tw_active_90_dpd_l12m_y	,
nbr_tw_active_90_dpd_l6m	nbr_tw_active_90_dpd_l6m_y	,
nbr_tw_active_90_dpd_l3m	nbr_tw_active_90_dpd_l3m_y	,
nbr_tw_active_90_dpd_cm	nbr_tw_active_90_dpd_cm_y	,
nbr_unsecured_x_dpd_l36m	nbr_unsecured_x_dpd_l36m_y	,
nbr_unsecured_x_dpd_l24m	nbr_unsecured_x_dpd_l24m_y	,
nbr_unsecured_x_dpd_l18m	nbr_unsecured_x_dpd_l18m_y	,
nbr_unsecured_x_dpd_l12m	nbr_unsecured_x_dpd_l12m_y	,
nbr_unsecured_x_dpd_l6m	nbr_unsecured_x_dpd_l6m_y	,
nbr_unsecured_x_dpd_l3m	nbr_unsecured_x_dpd_l3m_y	,
nbr_unsecured_x_dpd_cm	nbr_unsecured_x_dpd_cm_y	,
nbr_unsecured_30_dpd_l36m	nbr_unsecured_30_dpd_l36m_y	,
nbr_unsecured_30_dpd_l24m	nbr_unsecured_30_dpd_l24m_y	,
nbr_unsecured_30_dpd_l18m	nbr_unsecured_30_dpd_l18m_y	,
nbr_unsecured_30_dpd_l12m	nbr_unsecured_30_dpd_l12m_y	,
nbr_unsecured_30_dpd_l6m	nbr_unsecured_30_dpd_l6m_y	,
nbr_unsecured_30_dpd_l3m	nbr_unsecured_30_dpd_l3m_y	,
nbr_unsecured_30_dpd_cm	nbr_unsecured_30_dpd_cm_y	,
nbr_unsecured_90_dpd_l36m	nbr_unsecured_90_dpd_l36m_y	,
nbr_unsecured_90_dpd_l24m	nbr_unsecured_90_dpd_l24m_y	,
nbr_unsecured_90_dpd_l18m	nbr_unsecured_90_dpd_l18m_y	,
nbr_unsecured_90_dpd_l12m	nbr_unsecured_90_dpd_l12m_y	,
nbr_unsecured_90_dpd_l6m	nbr_unsecured_90_dpd_l6m_y	,
nbr_unsecured_90_dpd_l3m	nbr_unsecured_90_dpd_l3m_y	,
nbr_unsecured_90_dpd_cm	nbr_unsecured_90_dpd_cm_y	,
nbr_active_unsecured_x_dpd_l36m	nbr_active_unsecured_x_dpd_l36m_y	,
nbr_active_unsecured_x_dpd_l24m	nbr_active_unsecured_x_dpd_l24m_y	,
nbr_active_unsecured_x_dpd_l18m	nbr_active_unsecured_x_dpd_l18m_y	,
nbr_active_unsecured_x_dpd_l12m	nbr_active_unsecured_x_dpd_l12m_y	,
nbr_active_unsecured_x_dpd_l6m	nbr_active_unsecured_x_dpd_l6m_y	,
nbr_active_unsecured_x_dpd_l3m	nbr_active_unsecured_x_dpd_l3m_y	,
nbr_active_unsecured_x_dpd_cm	nbr_active_unsecured_x_dpd_cm_y	,
nbr_active_unsecured_30_dpd_l36m	nbr_active_unsecured_30_dpd_l36m_y	,
nbr_active_unsecured_30_dpd_l24m	nbr_active_unsecured_30_dpd_l24m_y	,
nbr_active_unsecured_30_dpd_l18m	nbr_active_unsecured_30_dpd_l18m_y	,
nbr_active_unsecured_30_dpd_l12m	nbr_active_unsecured_30_dpd_l12m_y	,
nbr_active_unsecured_30_dpd_l6m	nbr_active_unsecured_30_dpd_l6m_y	,
nbr_active_unsecured_30_dpd_l3m	nbr_active_unsecured_30_dpd_l3m_y	,
nbr_active_unsecured_30_dpd_cm	nbr_active_unsecured_30_dpd_cm_y	,
nbr_unsecured_active_90_dpd_l36m	nbr_unsecured_active_90_dpd_l36m_y	,
nbr_unsecured_active_90_dpd_l24m	nbr_unsecured_active_90_dpd_l24m_y	,
nbr_unsecured_active_90_dpd_l18m	nbr_unsecured_active_90_dpd_l18m_y	,
nbr_unsecured_active_90_dpd_l12m	nbr_unsecured_active_90_dpd_l12m_y	,
nbr_unsecured_active_90_dpd_l6m	nbr_unsecured_active_90_dpd_l6m_y	,
nbr_unsecured_active_90_dpd_l3m	nbr_unsecured_active_90_dpd_l3m_y	,
nbr_unsecured_active_90_dpd_cm	nbr_unsecured_active_90_dpd_cm_y	,
nbr_secured_x_dpd_l36m	nbr_secured_x_dpd_l36m_y	,
nbr_secured_x_dpd_l24m	nbr_secured_x_dpd_l24m_y	,
nbr_secured_x_dpd_l18m	nbr_secured_x_dpd_l18m_y	,
nbr_secured_x_dpd_l12m	nbr_secured_x_dpd_l12m_y	,
nbr_secured_x_dpd_l6m	nbr_secured_x_dpd_l6m_y	,
nbr_secured_x_dpd_l3m	nbr_secured_x_dpd_l3m_y	,
nbr_secured_x_dpd_cm	nbr_secured_x_dpd_cm_y	,
nbr_secured_30_dpd_l36m	nbr_secured_30_dpd_l36m_y	,
nbr_secured_30_dpd_l24m	nbr_secured_30_dpd_l24m_y	,
nbr_secured_30_dpd_l18m	nbr_secured_30_dpd_l18m_y	,
nbr_secured_30_dpd_l12m	nbr_secured_30_dpd_l12m_y	,
nbr_secured_30_dpd_l6m	nbr_secured_30_dpd_l6m_y	,
nbr_secured_30_dpd_l3m	nbr_secured_30_dpd_l3m_y	,
nbr_secured_30_dpd_cm	nbr_secured_30_dpd_cm_y	,
nbr_secured_90_dpd_l36m	nbr_secured_90_dpd_l36m_y	,
nbr_secured_90_dpd_l24m	nbr_secured_90_dpd_l24m_y	,
nbr_secured_90_dpd_l18m	nbr_secured_90_dpd_l18m_y	,
nbr_secured_90_dpd_l12m	nbr_secured_90_dpd_l12m_y	,
nbr_secured_90_dpd_l6m	nbr_secured_90_dpd_l6m_y	,
nbr_secured_90_dpd_l3m	nbr_secured_90_dpd_l3m_y	,
nbr_secured_90_dpd_cm	nbr_secured_90_dpd_cm_y	,
nbr_active_secured_x_dpd_l36m	nbr_active_secured_x_dpd_l36m_y	,
nbr_active_secured_x_dpd_l24m	nbr_active_secured_x_dpd_l24m_y	,
nbr_active_secured_x_dpd_l18m	nbr_active_secured_x_dpd_l18m_y	,
nbr_active_secured_x_dpd_l12m	nbr_active_secured_x_dpd_l12m_y	,
nbr_active_secured_x_dpd_l6m	nbr_active_secured_x_dpd_l6m_y	,
nbr_active_secured_x_dpd_l3m	nbr_active_secured_x_dpd_l3m_y	,
nbr_active_secured_x_dpd_cm	nbr_active_secured_x_dpd_cm_y	,
nbr_active_secured_30_dpd_l36m	nbr_active_secured_30_dpd_l36m_y	,
nbr_active_secured_30_dpd_l24m	nbr_active_secured_30_dpd_l24m_y	,
nbr_active_secured_30_dpd_l18m	nbr_active_secured_30_dpd_l18m_y	,
nbr_active_secured_30_dpd_l12m	nbr_active_secured_30_dpd_l12m_y	,
nbr_active_secured_30_dpd_l6m	nbr_active_secured_30_dpd_l6m_y	,
nbr_active_secured_30_dpd_l3m	nbr_active_secured_30_dpd_l3m_y	,
nbr_active_secured_30_dpd_cm	nbr_active_secured_30_dpd_cm_y	,
nbr_secured_active_90_dpd_l36m	nbr_secured_active_90_dpd_l36m_y	,
nbr_secured_active_90_dpd_l24m	nbr_secured_active_90_dpd_l24m_y	,
nbr_secured_active_90_dpd_l18m	nbr_secured_active_90_dpd_l18m_y	,
nbr_secured_active_90_dpd_l12m	nbr_secured_active_90_dpd_l12m_y	,
nbr_secured_active_90_dpd_l6m	nbr_secured_active_90_dpd_l6m_y	,
nbr_secured_active_90_dpd_l3m	nbr_secured_active_90_dpd_l3m_y	,
nbr_secured_active_90_dpd_cm	nbr_secured_active_90_dpd_cm_y	,
max_sanctioned_amount	max_sanctioned_amount_y	,
total_sanctioned_amount	total_sanctioned_amount_y	,
secured_max_sanctioned_amount	secured_max_sanctioned_amount_y	,
secured_total_sanctioned_amount	secured_total_sanctioned_amount_y	,
unsecured_max_sanctioned_amount	unsecured_max_sanctioned_amount_y	,
unsecured_total_sanctioned_amount	unsecured_total_sanctioned_amount_y	,
pl_max_sanctioned_amount	pl_max_sanctioned_amount_y	,
pl_total_sanctioned_amount	pl_total_sanctioned_amount_y	,
hl_max_sanctioned_amount	hl_max_sanctioned_amount_y	,
hl_total_sanctioned_amount	hl_total_sanctioned_amount_y	,
cc_max_sanctioned_amount	cc_max_sanctioned_amount_y	,
cc_total_sanctioned_amount	cc_total_sanctioned_amount_y	,
cd_max_sanctioned_amount	cd_max_sanctioned_amount_y	,
cd_total_sanctioned_amount	cd_total_sanctioned_amount_y	,
al_max_sanctioned_amount	al_max_sanctioned_amount_y	,
al_total_sanctioned_amount	al_total_sanctioned_amount_y	,
gl_max_sanctioned_amount	gl_max_sanctioned_amount_y	,
gl_total_sanctioned_amount	gl_total_sanctioned_amount_y	,
tw_max_sanctioned_amount	tw_max_sanctioned_amount_y	,
tw_total_sanctioned_amount	tw_total_sanctioned_amount_y	,
max_sanctioned_amount_l3m	max_sanctioned_amount_l3m_y	,
total_sanctioned_amount_l3m	total_sanctioned_amount_l3m_y	,
secured_max_sanctioned_amount_l3m	secured_max_sanctioned_amount_l3m_y	,
secured_total_sanctioned_amount_l3m	secured_total_sanctioned_amount_l3m_y	,
unsecured_max_sanctioned_amount_l3m	unsecured_max_sanctioned_amount_l3m_y	,
unsecured_total_sanctioned_amount_l3m	unsecured_total_sanctioned_amount_l3m_y	,
pl_max_sanctioned_amount_l3m	pl_max_sanctioned_amount_l3m_y	,
pl_total_sanctioned_amount_l3m	pl_total_sanctioned_amount_l3m_y	,
hl_max_sanctioned_amount_l3m	hl_max_sanctioned_amount_l3m_y	,
hl_total_sanctioned_amount_l3m	hl_total_sanctioned_amount_l3m_y	,
cc_max_sanctioned_amount_l3m	cc_max_sanctioned_amount_l3m_y	,
cc_total_sanctioned_amount_l3m	cc_total_sanctioned_amount_l3m_y	,
cd_max_sanctioned_amount_l3m	cd_max_sanctioned_amount_l3m_y	,
cd_total_sanctioned_amount_l3m	cd_total_sanctioned_amount_l3m_y	,
al_max_sanctioned_amount_l3m	al_max_sanctioned_amount_l3m_y	,
al_total_sanctioned_amount_l3m	al_total_sanctioned_amount_l3m_y	,
gl_max_sanctioned_amount_l3m	gl_max_sanctioned_amount_l3m_y	,
gl_total_sanctioned_amount_l3m	gl_total_sanctioned_amount_l3m_y	,
tw_max_sanctioned_amount_l3m	tw_max_sanctioned_amount_l3m_y	,
tw_total_sanctioned_amount_l3m	tw_total_sanctioned_amount_l3m_y	,
max_sanctioned_amount_l6m	max_sanctioned_amount_l6m_y	,
total_sanctioned_amount_l6m	total_sanctioned_amount_l6m_y	,
secured_max_sanctioned_amount_l6m	secured_max_sanctioned_amount_l6m_y	,
secured_total_sanctioned_amount_l6m	secured_total_sanctioned_amount_l6m_y	,
unsecured_max_sanctioned_amount_l6m	unsecured_max_sanctioned_amount_l6m_y	,
unsecured_total_sanctioned_amount_l6m	unsecured_total_sanctioned_amount_l6m_y	,
pl_max_sanctioned_amount_l6m	pl_max_sanctioned_amount_l6m_y	,
pl_total_sanctioned_amount_l6m	pl_total_sanctioned_amount_l6m_y	,
hl_max_sanctioned_amount_l6m	hl_max_sanctioned_amount_l6m_y	,
hl_total_sanctioned_amount_l6m	hl_total_sanctioned_amount_l6m_y	,
cc_max_sanctioned_amount_l6m	cc_max_sanctioned_amount_l6m_y	,
cc_total_sanctioned_amount_l6m	cc_total_sanctioned_amount_l6m_y	,
cd_max_sanctioned_amount_l6m	cd_max_sanctioned_amount_l6m_y	,
cd_total_sanctioned_amount_l6m	cd_total_sanctioned_amount_l6m_y	,
al_max_sanctioned_amount_l6m	al_max_sanctioned_amount_l6m_y	,
al_total_sanctioned_amount_l6m	al_total_sanctioned_amount_l6m_y	,
gl_max_sanctioned_amount_l6m	gl_max_sanctioned_amount_l6m_y	,
gl_total_sanctioned_amount_l6m	gl_total_sanctioned_amount_l6m_y	,
tw_max_sanctioned_amount_l6m	tw_max_sanctioned_amount_l6m_y	,
tw_total_sanctioned_amount_l6m	tw_total_sanctioned_amount_l6m_y	,
max_sanctioned_amount_l12m	max_sanctioned_amount_l12m_y	,
total_sanctioned_amount_l12m	total_sanctioned_amount_l12m_y	,
secured_max_sanctioned_amount_l12m	secured_max_sanctioned_amount_l12m_y	,
secured_total_sanctioned_amount_l12m	secured_total_sanctioned_amount_l12m_y	,
unsecured_max_sanctioned_amount_l12m	unsecured_max_sanctioned_amount_l12m_y	,
unsecured_total_sanctioned_amount_l12m	unsecured_total_sanctioned_amount_l12m_y	,
pl_max_sanctioned_amount_l12m	pl_max_sanctioned_amount_l12m_y	,
pl_total_sanctioned_amount_l12m	pl_total_sanctioned_amount_l12m_y	,
hl_max_sanctioned_amount_l12m	hl_max_sanctioned_amount_l12m_y	,
hl_total_sanctioned_amount_l12m	hl_total_sanctioned_amount_l12m_y	,
cc_max_sanctioned_amount_l12m	cc_max_sanctioned_amount_l12m_y	,
cc_total_sanctioned_amount_l12m	cc_total_sanctioned_amount_l12m_y	,
cd_max_sanctioned_amount_l12m	cd_max_sanctioned_amount_l12m_y	,
cd_total_sanctioned_amount_l12m	cd_total_sanctioned_amount_l12m_y	,
al_max_sanctioned_amount_l12m	al_max_sanctioned_amount_l12m_y	,
al_total_sanctioned_amount_l12m	al_total_sanctioned_amount_l12m_y	,
gl_max_sanctioned_amount_l12m	gl_max_sanctioned_amount_l12m_y	,
gl_total_sanctioned_amount_l12m	gl_total_sanctioned_amount_l12m_y	,
tw_max_sanctioned_amount_l12m	tw_max_sanctioned_amount_l12m_y	,
tw_total_sanctioned_amount_l12m	tw_total_sanctioned_amount_l12m_y	,
total_balance_amount	total_balance_amount_y	,
secured_total_balance_amount	secured_total_balance_amount_y	,
unsecured_total_balance_amount	unsecured_total_balance_amount_y	,
pl_total_balance_amount	pl_total_balance_amount_y	,
hl_total_balance_amount	hl_total_balance_amount_y	,
cc_total_balance_amount	cc_total_balance_amount_y	,
cd_total_balance_amount	cd_total_balance_amount_y	,
al_total_balance_amount	al_total_balance_amount_y	,
gl_total_balance_amount	gl_total_balance_amount_y	,
tw_total_balance_amount	tw_total_balance_amount_y	,
avg_utilisation	avg_utilisation_y	,
avg_secured_utilisation	avg_secured_utilisation_y	,
avg_unsecured_utilisation	avg_unsecured_utilisation_y	,
avg_pl_utilisation	avg_pl_utilisation_y	,
avg_hl_tilisation	avg_hl_tilisation_y	,
avg_cc_utilisation	avg_cc_utilisation_y	,
avg_cd_utilisation	avg_cd_utilisation_y	,
avg_al_utilisation	avg_al_utilisation_y	,
avg_gl_utilisation	avg_gl_utilisation_y	,
avg_tw_utilisation	avg_tw_utilisation_y	,
overall_utilisation	overall_utilisation_y	,
secured_utilisation	secured_utilisation_y	,
unsecured_utilisation	unsecured_utilisation_y	,
pl_utilisation	pl_utilisation_y	,
hl_utilisation	hl_utilisation_y	,
cc_utilisation	cc_utilisation_y	,
cd_utilisation	cd_utilisation_y	,
al_utilisation	al_utilisation_y	,
gl_utilisation	gl_utilisation_y	,
tw_utilisation	tw_utilisation_y	,
latest_product	latest_product_y	,
first_product	first_product_y	,
max_bureau_vintage	max_bureau_vintage_y	,
min_bureau_vintage	min_bureau_vintage_y	,
max_active_bureau_vintage	max_active_bureau_vintage_y	,
min_active_bureau_vintage	min_active_bureau_vintage_y	,
max_secured_bureau_vintage	max_secured_bureau_vintage_y	,
min_secured_bureau_vintage	min_secured_bureau_vintage_y	,
max_active_secured_bureau_vintage	max_active_secured_bureau_vintage_y	,
min_active_secured_bureau_vintage	min_active_secured_bureau_vintage_y	,
max_unsecured_bureau_vintage	max_unsecured_bureau_vintage_y	,
min_unsecured_bureau_vintage	min_unsecured_bureau_vintage_y	,
max_active_unsecured_bureau_vintage	max_active_unsecured_bureau_vintage_y	,
min_active_unsecured_bureau_vintage	min_active_unsecured_bureau_vintage_y	,
max_pl_bureau_vintage	max_pl_bureau_vintage_y	,
min_pl_bureau_vintage	min_pl_bureau_vintage_y	,
max_active_pl_bureau_vintage	max_active_pl_bureau_vintage_y	,
min_active_pl_bureau_vintage	min_active_pl_bureau_vintage_y	,
max_cc_bureau_vintage	max_cc_bureau_vintage_y	,
min_cc_bureau_vintage	min_cc_bureau_vintage_y	,
max_active_cc_bureau_vintage	max_active_cc_bureau_vintage_y	,
min_active_cc_bureau_vintage	min_active_cc_bureau_vintage_y	,
max_cd_bureau_vintage	max_cd_bureau_vintage_y	,
min_cd_bureau_vintage	min_cd_bureau_vintage_y	,
max_active_cd_bureau_vintage	max_active_cd_bureau_vintage_y	,
min_active_cd_bureau_vintage	min_active_cd_bureau_vintage_y	,
max_al_bureau_vintage	max_al_bureau_vintage_y	,
min_al_bureau_vintage	min_al_bureau_vintage_y	,
max_active_al_bureau_vintage	max_active_al_bureau_vintage_y	,
min_active_al_bureau_vintage	min_active_al_bureau_vintage_y	,
max_hl_bureau_vintage	max_hl_bureau_vintage_y	,
min_hl_bureau_vintage	min_hl_bureau_vintage_y	,
max_active_hl_bureau_vintage	max_active_hl_bureau_vintage_y	,
min_active_hl_bureau_vintage	min_active_hl_bureau_vintage_y	,
max_gl_bureau_vintage	max_gl_bureau_vintage_y	,
min_gl_bureau_vintage	min_gl_bureau_vintage_y	,
max_active_gl_bureau_vintage	max_active_gl_bureau_vintage_y	,
min_active_gl_bureau_vintage	min_active_gl_bureau_vintage_y	,
max_tw_bureau_vintage	max_tw_bureau_vintage_y	,
min_tw_bureau_vintage	min_tw_bureau_vintage_y	,
max_active_tw_bureau_vintage	max_active_tw_bureau_vintage_y	,
min_active_tw_bureau_vintage	min_active_tw_bureau_vintage_y	,
enq_l1m	enq_l1m_y	,
enq_l2m	enq_l2m_y	,
enq_l3m	enq_l3m_y	,
enq_l6m	enq_l6m_y	,
enq_l12m	enq_l12m_y	,
unsec_inq_l1m	unsec_inq_l1m_y	,
unsec_inq_l2m	unsec_inq_l2m_y	,
unsec_inq_l3m	unsec_inq_l3m_y	,
unsec_inq_l6m	unsec_inq_l6m_y	,
unsec_inq_l12m	unsec_inq_l12m_y	,
sec_inq_l1m	sec_inq_l1m_y	,
sec_inq_l2m	sec_inq_l2m_y	,
sec_inq_l3m	sec_inq_l3m_y	,
sec_inq_l6m	sec_inq_l6m_y	,
sec_inq_l12m	sec_inq_l12m_y	,
pl_inq_l1m	pl_inq_l1m_y	,
pl_inq_l2m	pl_inq_l2m_y	,
pl_inq_l3m	pl_inq_l3m_y	,
pl_inq_l6m	pl_inq_l6m_y	,
pl_inq_l12m	pl_inq_l12m_y	,
cc_inq_l1m	cc_inq_l1m_y	,
cc_inq_l2m	cc_inq_l2m_y	,
cc_inq_l3m	cc_inq_l3m_y	,
cc_inq_l6m	cc_inq_l6m_y	,
cc_inq_l12m	cc_inq_l12m_y	,
cd_inq_l1m	cd_inq_l1m_y	,
cd_inq_l2m	cd_inq_l2m_y	,
cd_inq_l3m	cd_inq_l3m_y	,
cd_inq_l6m	cd_inq_l6m_y	,
cd_inq_l12m	cd_inq_l12m_y	,
al_inq_l1m	al_inq_l1m_y	,
al_inq_l2m	al_inq_l2m_y	,
al_inq_l3m	al_inq_l3m_y	,
al_inq_l6m	al_inq_l6m_y	,
al_inq_l12m	al_inq_l12m_y	,
gl_inq_l1m	gl_inq_l1m_y	,
gl_inq_l2m	gl_inq_l2m_y	,
gl_inq_l3m	gl_inq_l3m_y	,
gl_inq_l6m	gl_inq_l6m_y	,
gl_inq_l12m	gl_inq_l12m_y	,
hl_inq_l1m	hl_inq_l1m_y	,
hl_inq_l2m	hl_inq_l2m_y	,
hl_inq_l3m	hl_inq_l3m_y	,
hl_inq_l6m	hl_inq_l6m_y	,
hl_inq_l12m	hl_inq_l12m_y	,
tw_inq_l1m	tw_inq_l1m_y	,
tw_inq_l2m	tw_inq_l2m_y	,
tw_inq_l3m	tw_inq_l3m_y	,
tw_inq_l6m	tw_inq_l6m_y	,
tw_inq_l12m	tw_inq_l12m_y	,
nbr_ever_accounts	nbr_ever_accounts_y	,
nbr_active_accounts	nbr_active_accounts_y	,
accts_new_l1m	accts_new_l1m_y	,
accts_new_l3m	accts_new_l3m_y	,
accts_new_l6m	accts_new_l6m_y	,
accts_new_l12m	accts_new_l12m_y	,
accts_closed_l1m	accts_closed_l1m_y	,
accts_closed_l3m	accts_closed_l3m_y	,
accts_closed_l6m	accts_closed_l6m_y	,
accts_closed_l12m	accts_closed_l12m_y	,
nbr_ever_secured_accounts	nbr_ever_secured_accounts_y	,
nbr_active_secured_accounts	nbr_active_secured_accounts_y	,
secured_accts_new_l1m	secured_accts_new_l1m_y	,
secured_accts_new_l3m	secured_accts_new_l3m_y	,
secured_accts_new_l6m	secured_accts_new_l6m_y	,
secured_accts_new_l12m	secured_accts_new_l12m_y	,
secured_accts_closed_l1m	secured_accts_closed_l1m_y	,
secured_accts_closed_l3m	secured_accts_closed_l3m_y	,
secured_accts_closed_l6m	secured_accts_closed_l6m_y	,
secured_accts_closed_l12m	secured_accts_closed_l12m_y	,
nbr_ever_unsecured_accounts	nbr_ever_unsecured_accounts_y	,
nbr_active_unsecured_accounts	nbr_active_unsecured_accounts_y	,
unsecured_accts_new_l1m	unsecured_accts_new_l1m_y	,
unsecured_accts_new_l3m	unsecured_accts_new_l3m_y	,
unsecured_accts_new_l6m	unsecured_accts_new_l6m_y	,
unsecured_accts_new_l12m	unsecured_accts_new_l12m_y	,
unsecured_accts_closed_l1m	unsecured_accts_closed_l1m_y	,
unsecured_accts_closed_l3m	unsecured_accts_closed_l3m_y	,
unsecured_accts_closed_l6m	unsecured_accts_closed_l6m_y	,
unsecured_accts_closed_l12m	unsecured_accts_closed_l12m_y	,
ever_hl	ever_hl_y	,
nbr_ever_hl	nbr_ever_hl_y	,
active_hl	active_hl_y	,
nbr_active_hl	nbr_active_hl_y	,
hl_new_l1m	hl_new_l1m_y	,
hl_new_l3m	hl_new_l3m_y	,
hl_new_l6m	hl_new_l6m_y	,
hl_new_l12m	hl_new_l12m_y	,
hl_closed_l1m	hl_closed_l1m_y	,
hl_closed_l3m	hl_closed_l3m_y	,
hl_closed_l6m	hl_closed_l6m_y	,
hl_closed_l12m	hl_closed_l12m_y	,
ever_pl	ever_pl_y	,
nbr_ever_pl	nbr_ever_pl_y	,
active_pl	active_pl_y	,
nbr_active_pl	nbr_active_pl_y	,
pl_new_l1m	pl_new_l1m_y	,
pl_new_l3m	pl_new_l3m_y	,
pl_new_l6m	pl_new_l6m_y	,
pl_new_l12m	pl_new_l12m_y	,
pl_closed_l1m	pl_closed_l1m_y	,
pl_closed_l3m	pl_closed_l3m_y	,
pl_closed_l6m	pl_closed_l6m_y	,
pl_closed_l12m	pl_closed_l12m_y	,
ever_cc	ever_cc_y	,
nbr_ever_cc	nbr_ever_cc_y	,
active_cc	active_cc_y	,
nbr_active_cc	nbr_active_cc_y	,
cc_new_l1m	cc_new_l1m_y	,
cc_new_l3m	cc_new_l3m_y	,
cc_new_l6m	cc_new_l6m_y	,
cc_new_l12m	cc_new_l12m_y	,
cc_closed_l1m	cc_closed_l1m_y	,
cc_closed_l3m	cc_closed_l3m_y	,
cc_closed_l6m	cc_closed_l6m_y	,
cc_closed_l12m	cc_closed_l12m_y	,
ever_cd	ever_cd_y	,
nbr_ever_cd	nbr_ever_cd_y	,
active_cd	active_cd_y	,
nbr_active_cd	nbr_active_cd_y	,
cd_new_l1m	cd_new_l1m_y	,
cd_new_l3m	cd_new_l3m_y	,
cd_new_l6m	cd_new_l6m_y	,
cd_new_l12m	cd_new_l12m_y	,
cd_closed_l1m	cd_closed_l1m_y	,
cd_closed_l3m	cd_closed_l3m_y	,
cd_closed_l6m	cd_closed_l6m_y	,
cd_closed_l12m	cd_closed_l12m_y	,
ever_gl	ever_gl_y	,
nbr_ever_gl	nbr_ever_gl_y	,
active_gl	active_gl_y	,
nbr_active_gl	nbr_active_gl_y	,
gl_new_l1m	gl_new_l1m_y	,
gl_new_l3m	gl_new_l3m_y	,
gl_new_l6m	gl_new_l6m_y	,
gl_new_l12m	gl_new_l12m_y	,
gl_closed_l1m	gl_closed_l1m_y	,
gl_closed_l3m	gl_closed_l3m_y	,
gl_closed_l6m	gl_closed_l6m_y	,
gl_closed_l12m	gl_closed_l12m_y	,
ever_al	ever_al_y	,
nbr_ever_al	nbr_ever_al_y	,
active_al	active_al_y	,
nbr_active_al	nbr_active_al_y	,
al_new_l1m	al_new_l1m_y	,
al_new_l3m	al_new_l3m_y	,
al_new_l6m	al_new_l6m_y	,
al_new_l12m	al_new_l12m_y	,
al_closed_l1m	al_closed_l1m_y	,
al_closed_l3m	al_closed_l3m_y	,
al_closed_l6m	al_closed_l6m_y	,
al_closed_l12m	al_closed_l12m_y	,
ever_tw	ever_tw_y	,
nbr_ever_tw	nbr_ever_tw_y	,
active_tw	active_tw_y	,
nbr_active_tw	nbr_active_tw_y	,
tw_new_l1m	tw_new_l1m_y	,
tw_new_l3m	tw_new_l3m_y	,
tw_new_l6m	tw_new_l6m_y	,
tw_new_l12m	tw_new_l12m_y	,
tw_closed_l1m	tw_closed_l1m_y	,
tw_closed_l3m	tw_closed_l3m_y	,
tw_closed_l6m	tw_closed_l6m_y	,
tw_closed_l12m	tw_closed_l12m_y	,
nbr_ever_stpl	nbr_ever_stpl_y	,
nbr_active_stpl	nbr_active_stpl_y	,
stpl_new_l1m	stpl_new_l1m_y	,
stpl_new_l3m	stpl_new_l3m_y	,
stpl_new_l6m	stpl_new_l6m_y	,
stpl_new_l12m	stpl_new_l12m_y	,
stpl_closed_l1m	stpl_closed_l1m_y	,
stpl_closed_l3m	stpl_closed_l3m_y	,
stpl_closed_l6m	stpl_closed_l6m_y	,
stpl_closed_l12m	stpl_closed_l12m_y	,
ever_cv	ever_cv_y	,
nbr_ever_cv	nbr_ever_cv_y	,
active_cv	active_cv_y	,
nbr_active_cv	nbr_active_cv_y	,
ever_agri	ever_agri_y	,
nbr_ever_agri	nbr_ever_agri_y	,
active_agri	active_agri_y	,
nbr_active_agri	nbr_active_agri_y	,
ever_lap	ever_lap_y	,
nbr_ever_lap	nbr_ever_lap_y	,
active_lap	active_lap_y	,
nbr_active_lap	nbr_active_lap_y	,
ever_las	ever_las_y	,
nbr_ever_las	nbr_ever_las_y	,
active_las	active_las_y	,
nbr_active_las	nbr_active_las_y	
FROM fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_data_overall
)
USING (customer_no)
""")

Executed!


In [74]:
!aws s3 rm s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_m1_diff_data_overall --recursive
    
    

delete: s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_m1_diff_data_overall/20250407_112205_22313_iin5u_e1e55c54-2731-45d6-a6a6-9d534bf9cbe3


In [75]:
fplathena.get_query_result(f'''
        DROP TABLE IF EXISTS hive.fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_m1_diff_data_overall
''')

Table Deleted!


0

In [ ]:
df9=query("""
CREATE TABLE fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_m1_diff_data_overall
WITH (
    format = 'PARQUET',
    external_location = 's3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_m1_diff_data_overall'
)
AS

SELECT * ,
max_dpd_l36m_y-max_dpd_l36m_x AS max_dpd_l36m_diff,
max_dpd_l24m_y-max_dpd_l24m_x AS max_dpd_l24m_diff,
max_dpd_l18m_y-max_dpd_l18m_x AS max_dpd_l18m_diff,
max_dpd_l12m_y-max_dpd_l12m_x AS max_dpd_l12m_diff,
max_dpd_l6m_y-max_dpd_l6m_x AS max_dpd_l6m_diff,
max_dpd_l3m_y-max_dpd_l3m_x AS max_dpd_l3m_diff,
max_dpd_cm_y-max_dpd_cm_x AS max_dpd_cm_diff,
max_active_dpd_l36m_y-max_active_dpd_l36m_x AS max_active_dpd_l36m_diff,
max_active_dpd_l24m_y-max_active_dpd_l24m_x AS max_active_dpd_l24m_diff,
max_active_dpd_l18m_y-max_active_dpd_l18m_x AS max_active_dpd_l18m_diff,
max_active_dpd_l12m_y-max_active_dpd_l12m_x AS max_active_dpd_l12m_diff,
max_active_dpd_l6m_y-max_active_dpd_l6m_x AS max_active_dpd_l6m_diff,
max_active_dpd_l3m_y-max_active_dpd_l3m_x AS max_active_dpd_l3m_diff,
max_active_dpd_cm_y-max_active_dpd_cm_x AS max_active_dpd_cm_diff,
max_active_cc_dpd_l36m_y-max_active_cc_dpd_l36m_x AS max_active_cc_dpd_l36m_diff,
max_active_cc_dpd_l24m_y-max_active_cc_dpd_l24m_x AS max_active_cc_dpd_l24m_diff,
max_active_cc_dpd_l18m_y-max_active_cc_dpd_l18m_x AS max_active_cc_dpd_l18m_diff,
max_active_cc_dpd_l12m_y-max_active_cc_dpd_l12m_x AS max_active_cc_dpd_l12m_diff,
max_active_cc_dpd_l6m_y-max_active_cc_dpd_l6m_x AS max_active_cc_dpd_l6m_diff,
max_active_cc_dpd_l3m_y-max_active_cc_dpd_l3m_x AS max_active_cc_dpd_l3m_diff,
max_active_cc_dpd_cm_y-max_active_cc_dpd_cm_x AS max_active_cc_dpd_cm_diff,
max_cc_dpd_l36m_y-max_cc_dpd_l36m_x AS max_cc_dpd_l36m_diff,
max_cc_dpd_l24m_y-max_cc_dpd_l24m_x AS max_cc_dpd_l24m_diff,
max_cc_dpd_l18m_y-max_cc_dpd_l18m_x AS max_cc_dpd_l18m_diff,
max_cc_dpd_l12m_y-max_cc_dpd_l12m_x AS max_cc_dpd_l12m_diff,
max_cc_dpd_l6m_y-max_cc_dpd_l6m_x AS max_cc_dpd_l6m_diff,
max_cc_dpd_l3m_y-max_cc_dpd_l3m_x AS max_cc_dpd_l3m_diff,
max_cc_dpd_cm_y-max_cc_dpd_cm_x AS max_cc_dpd_cm_diff,
max_active_pl_dpd_l36m_y-max_active_pl_dpd_l36m_x AS max_active_pl_dpd_l36m_diff,
max_active_pl_dpd_l24m_y-max_active_pl_dpd_l24m_x AS max_active_pl_dpd_l24m_diff,
max_active_pl_dpd_l18m_y-max_active_pl_dpd_l18m_x AS max_active_pl_dpd_l18m_diff,
max_active_pl_dpd_l12m_y-max_active_pl_dpd_l12m_x AS max_active_pl_dpd_l12m_diff,
max_active_pl_dpd_l6m_y-max_active_pl_dpd_l6m_x AS max_active_pl_dpd_l6m_diff,
max_active_pl_dpd_l3m_y-max_active_pl_dpd_l3m_x AS max_active_pl_dpd_l3m_diff,
max_active_pl_dpd_cm_y-max_active_pl_dpd_cm_x AS max_active_pl_dpd_cm_diff,
max_pl_dpd_l36m_y-max_pl_dpd_l36m_x AS max_pl_dpd_l36m_diff,
max_pl_dpd_l24m_y-max_pl_dpd_l24m_x AS max_pl_dpd_l24m_diff,
max_pl_dpd_l18m_y-max_pl_dpd_l18m_x AS max_pl_dpd_l18m_diff,
max_pl_dpd_l12m_y-max_pl_dpd_l12m_x AS max_pl_dpd_l12m_diff,
max_pl_dpd_l6m_y-max_pl_dpd_l6m_x AS max_pl_dpd_l6m_diff,
max_pl_dpd_l3m_y-max_pl_dpd_l3m_x AS max_pl_dpd_l3m_diff,
max_pl_dpd_cm_y-max_pl_dpd_cm_x AS max_pl_dpd_cm_diff,
max_active_cd_dpd_l36m_y-max_active_cd_dpd_l36m_x AS max_active_cd_dpd_l36m_diff,
max_active_cd_dpd_l24m_y-max_active_cd_dpd_l24m_x AS max_active_cd_dpd_l24m_diff,
max_active_cd_dpd_l18m_y-max_active_cd_dpd_l18m_x AS max_active_cd_dpd_l18m_diff,
max_active_cd_dpd_l12m_y-max_active_cd_dpd_l12m_x AS max_active_cd_dpd_l12m_diff,
max_active_cd_dpd_l6m_y-max_active_cd_dpd_l6m_x AS max_active_cd_dpd_l6m_diff,
max_active_cd_dpd_l3m_y-max_active_cd_dpd_l3m_x AS max_active_cd_dpd_l3m_diff,
max_active_cd_dpd_cm_y-max_active_cd_dpd_cm_x AS max_active_cd_dpd_cm_diff,
max_cd_dpd_l36m_y-max_cd_dpd_l36m_x AS max_cd_dpd_l36m_diff,
max_cd_dpd_l24m_y-max_cd_dpd_l24m_x AS max_cd_dpd_l24m_diff,
max_cd_dpd_l18m_y-max_cd_dpd_l18m_x AS max_cd_dpd_l18m_diff,
max_cd_dpd_l12m_y-max_cd_dpd_l12m_x AS max_cd_dpd_l12m_diff,
max_cd_dpd_l6m_y-max_cd_dpd_l6m_x AS max_cd_dpd_l6m_diff,
max_cd_dpd_l3m_y-max_cd_dpd_l3m_x AS max_cd_dpd_l3m_diff,
max_cd_dpd_cm_y-max_cd_dpd_cm_x AS max_cd_dpd_cm_diff,
max_active_gl_dpd_l36m_y-max_active_gl_dpd_l36m_x AS max_active_gl_dpd_l36m_diff,
max_active_gl_dpd_l24m_y-max_active_gl_dpd_l24m_x AS max_active_gl_dpd_l24m_diff,
max_active_gl_dpd_l18m_y-max_active_gl_dpd_l18m_x AS max_active_gl_dpd_l18m_diff,
max_active_gl_dpd_l12m_y-max_active_gl_dpd_l12m_x AS max_active_gl_dpd_l12m_diff,
max_active_gl_dpd_l6m_y-max_active_gl_dpd_l6m_x AS max_active_gl_dpd_l6m_diff,
max_active_gl_dpd_l3m_y-max_active_gl_dpd_l3m_x AS max_active_gl_dpd_l3m_diff,
max_active_gl_dpd_cm_y-max_active_gl_dpd_cm_x AS max_active_gl_dpd_cm_diff,
max_gl_dpd_l36m_y-max_gl_dpd_l36m_x AS max_gl_dpd_l36m_diff,
max_gl_dpd_l24m_y-max_gl_dpd_l24m_x AS max_gl_dpd_l24m_diff,
max_gl_dpd_l18m_y-max_gl_dpd_l18m_x AS max_gl_dpd_l18m_diff,
max_gl_dpd_l12m_y-max_gl_dpd_l12m_x AS max_gl_dpd_l12m_diff,
max_gl_dpd_l6m_y-max_gl_dpd_l6m_x AS max_gl_dpd_l6m_diff,
max_gl_dpd_l3m_y-max_gl_dpd_l3m_x AS max_gl_dpd_l3m_diff,
max_gl_dpd_cm_y-max_gl_dpd_cm_x AS max_gl_dpd_cm_diff,
max_active_al_dpd_l36m_y-max_active_al_dpd_l36m_x AS max_active_al_dpd_l36m_diff,
max_active_al_dpd_l24m_y-max_active_al_dpd_l24m_x AS max_active_al_dpd_l24m_diff,
max_active_al_dpd_l18m_y-max_active_al_dpd_l18m_x AS max_active_al_dpd_l18m_diff,
max_active_al_dpd_l12m_y-max_active_al_dpd_l12m_x AS max_active_al_dpd_l12m_diff,
max_active_al_dpd_l6m_y-max_active_al_dpd_l6m_x AS max_active_al_dpd_l6m_diff,
max_active_al_dpd_l3m_y-max_active_al_dpd_l3m_x AS max_active_al_dpd_l3m_diff,
max_active_al_dpd_cm_y-max_active_al_dpd_cm_x AS max_active_al_dpd_cm_diff,
max_al_dpd_l36m_y-max_al_dpd_l36m_x AS max_al_dpd_l36m_diff,
max_al_dpd_l24m_y-max_al_dpd_l24m_x AS max_al_dpd_l24m_diff,
max_al_dpd_l18m_y-max_al_dpd_l18m_x AS max_al_dpd_l18m_diff,
max_al_dpd_l12m_y-max_al_dpd_l12m_x AS max_al_dpd_l12m_diff,
max_al_dpd_l6m_y-max_al_dpd_l6m_x AS max_al_dpd_l6m_diff,
max_al_dpd_l3m_y-max_al_dpd_l3m_x AS max_al_dpd_l3m_diff,
max_al_dpd_cm_y-max_al_dpd_cm_x AS max_al_dpd_cm_diff,
max_active_tw_dpd_l36m_y-max_active_tw_dpd_l36m_x AS max_active_tw_dpd_l36m_diff,
max_active_tw_dpd_l24m_y-max_active_tw_dpd_l24m_x AS max_active_tw_dpd_l24m_diff,
max_active_tw_dpd_l18m_y-max_active_tw_dpd_l18m_x AS max_active_tw_dpd_l18m_diff,
max_active_tw_dpd_l12m_y-max_active_tw_dpd_l12m_x AS max_active_tw_dpd_l12m_diff,
max_active_tw_dpd_l6m_y-max_active_tw_dpd_l6m_x AS max_active_tw_dpd_l6m_diff,
max_active_tw_dpd_l3m_y-max_active_tw_dpd_l3m_x AS max_active_tw_dpd_l3m_diff,
max_active_tw_dpd_cm_y-max_active_tw_dpd_cm_x AS max_active_tw_dpd_cm_diff,
max_tw_dpd_l36m_y-max_tw_dpd_l36m_x AS max_tw_dpd_l36m_diff,
max_tw_dpd_l24m_y-max_tw_dpd_l24m_x AS max_tw_dpd_l24m_diff,
max_tw_dpd_l18m_y-max_tw_dpd_l18m_x AS max_tw_dpd_l18m_diff,
max_tw_dpd_l12m_y-max_tw_dpd_l12m_x AS max_tw_dpd_l12m_diff,
max_tw_dpd_l6m_y-max_tw_dpd_l6m_x AS max_tw_dpd_l6m_diff,
max_tw_dpd_l3m_y-max_tw_dpd_l3m_x AS max_tw_dpd_l3m_diff,
max_tw_dpd_cm_y-max_tw_dpd_cm_x AS max_tw_dpd_cm_diff,
max_active_hl_dpd_l36m_y-max_active_hl_dpd_l36m_x AS max_active_hl_dpd_l36m_diff,
max_active_hl_dpd_l24m_y-max_active_hl_dpd_l24m_x AS max_active_hl_dpd_l24m_diff,
max_active_hl_dpd_l18m_y-max_active_hl_dpd_l18m_x AS max_active_hl_dpd_l18m_diff,
max_active_hl_dpd_l12m_y-max_active_hl_dpd_l12m_x AS max_active_hl_dpd_l12m_diff,
max_active_hl_dpd_l6m_y-max_active_hl_dpd_l6m_x AS max_active_hl_dpd_l6m_diff,
max_active_hl_dpd_l3m_y-max_active_hl_dpd_l3m_x AS max_active_hl_dpd_l3m_diff,
max_active_hl_dpd_cm_y-max_active_hl_dpd_cm_x AS max_active_hl_dpd_cm_diff,
max_hl_dpd_l36m_y-max_hl_dpd_l36m_x AS max_hl_dpd_l36m_diff,
max_hl_dpd_l24m_y-max_hl_dpd_l24m_x AS max_hl_dpd_l24m_diff,
max_hl_dpd_l18m_y-max_hl_dpd_l18m_x AS max_hl_dpd_l18m_diff,
max_hl_dpd_l12m_y-max_hl_dpd_l12m_x AS max_hl_dpd_l12m_diff,
max_hl_dpd_l6m_y-max_hl_dpd_l6m_x AS max_hl_dpd_l6m_diff,
max_hl_dpd_l3m_y-max_hl_dpd_l3m_x AS max_hl_dpd_l3m_diff,
max_hl_dpd_cm_y-max_hl_dpd_cm_x AS max_hl_dpd_cm_diff,
max_active_unsecured_dpd_l36m_y-max_active_unsecured_dpd_l36m_x AS max_active_unsecured_dpd_l36m_diff,
max_active_unsecured_dpd_l24m_y-max_active_unsecured_dpd_l24m_x AS max_active_unsecured_dpd_l24m_diff,
max_active_unsecured_dpd_l18m_y-max_active_unsecured_dpd_l18m_x AS max_active_unsecured_dpd_l18m_diff,
max_active_unsecured_dpd_l12m_y-max_active_unsecured_dpd_l12m_x AS max_active_unsecured_dpd_l12m_diff,
max_active_unsecured_dpd_l6m_y-max_active_unsecured_dpd_l6m_x AS max_active_unsecured_dpd_l6m_diff,
max_active_unsecured_dpd_l3m_y-max_active_unsecured_dpd_l3m_x AS max_active_unsecured_dpd_l3m_diff,
max_active_unsecured_dpd_cm_y-max_active_unsecured_dpd_cm_x AS max_active_unsecured_dpd_cm_diff,
max_unsecured_dpd_l36m_y-max_unsecured_dpd_l36m_x AS max_unsecured_dpd_l36m_diff,
max_unsecured_dpd_l24m_y-max_unsecured_dpd_l24m_x AS max_unsecured_dpd_l24m_diff,
max_unsecured_dpd_l18m_y-max_unsecured_dpd_l18m_x AS max_unsecured_dpd_l18m_diff,
max_unsecured_dpd_l12m_y-max_unsecured_dpd_l12m_x AS max_unsecured_dpd_l12m_diff,
max_unsecured_dpd_l6m_y-max_unsecured_dpd_l6m_x AS max_unsecured_dpd_l6m_diff,
max_unsecured_dpd_l3m_y-max_unsecured_dpd_l3m_x AS max_unsecured_dpd_l3m_diff,
max_unsecured_dpd_cm_y-max_unsecured_dpd_cm_x AS max_unsecured_dpd_cm_diff,
max_active_secured_dpd_l36m_y-max_active_secured_dpd_l36m_x AS max_active_secured_dpd_l36m_diff,
max_active_secured_dpd_l24m_y-max_active_secured_dpd_l24m_x AS max_active_secured_dpd_l24m_diff,
max_active_secured_dpd_l18m_y-max_active_secured_dpd_l18m_x AS max_active_secured_dpd_l18m_diff,
max_active_secured_dpd_l12m_y-max_active_secured_dpd_l12m_x AS max_active_secured_dpd_l12m_diff,
max_active_secured_dpd_l6m_y-max_active_secured_dpd_l6m_x AS max_active_secured_dpd_l6m_diff,
max_active_secured_dpd_l3m_y-max_active_secured_dpd_l3m_x AS max_active_secured_dpd_l3m_diff,
max_active_secured_dpd_cm_y-max_active_secured_dpd_cm_x AS max_active_secured_dpd_cm_diff,
max_secured_dpd_l36m_y-max_secured_dpd_l36m_x AS max_secured_dpd_l36m_diff,
max_secured_dpd_l24m_y-max_secured_dpd_l24m_x AS max_secured_dpd_l24m_diff,
max_secured_dpd_l18m_y-max_secured_dpd_l18m_x AS max_secured_dpd_l18m_diff,
max_secured_dpd_l12m_y-max_secured_dpd_l12m_x AS max_secured_dpd_l12m_diff,
max_secured_dpd_l6m_y-max_secured_dpd_l6m_x AS max_secured_dpd_l6m_diff,
max_secured_dpd_l3m_y-max_secured_dpd_l3m_x AS max_secured_dpd_l3m_diff,
max_secured_dpd_cm_y-max_secured_dpd_cm_x AS max_secured_dpd_cm_diff,
overdue_l36m_y-overdue_l36m_x AS overdue_l36m_diff,
overdue_l24m_y-overdue_l24m_x AS overdue_l24m_diff,
overdue_l18m_y-overdue_l18m_x AS overdue_l18m_diff,
overdue_l12m_y-overdue_l12m_x AS overdue_l12m_diff,
overdue_l9m_y-overdue_l9m_x AS overdue_l9m_diff,
overdue_l6m_y-overdue_l6m_x AS overdue_l6m_diff,
overdue_l3m_y-overdue_l3m_x AS overdue_l3m_diff,
overdue_cm_y-overdue_cm_x AS overdue_cm_diff,
cc_overdue_l36m_y-cc_overdue_l36m_x AS cc_overdue_l36m_diff,
cc_overdue_l24m_y-cc_overdue_l24m_x AS cc_overdue_l24m_diff,
cc_overdue_l18m_y-cc_overdue_l18m_x AS cc_overdue_l18m_diff,
cc_overdue_l12m_y-cc_overdue_l12m_x AS cc_overdue_l12m_diff,
cc_overdue_l9m_y-cc_overdue_l9m_x AS cc_overdue_l9m_diff,
cc_overdue_l6m_y-cc_overdue_l6m_x AS cc_overdue_l6m_diff,
cc_overdue_l3m_y-cc_overdue_l3m_x AS cc_overdue_l3m_diff,
cc_overdue_cm_y-cc_overdue_cm_x AS cc_overdue_cm_diff,
pl_overdue_l36m_y-pl_overdue_l36m_x AS pl_overdue_l36m_diff,
pl_overdue_l24m_y-pl_overdue_l24m_x AS pl_overdue_l24m_diff,
pl_overdue_l18m_y-pl_overdue_l18m_x AS pl_overdue_l18m_diff,
pl_overdue_l12m_y-pl_overdue_l12m_x AS pl_overdue_l12m_diff,
pl_overdue_l9m_y-pl_overdue_l9m_x AS pl_overdue_l9m_diff,
pl_overdue_l6m_y-pl_overdue_l6m_x AS pl_overdue_l6m_diff,
pl_overdue_l3m_y-pl_overdue_l3m_x AS pl_overdue_l3m_diff,
pl_overdue_cm_y-pl_overdue_cm_x AS pl_overdue_cm_diff,
cd_overdue_l36m_y-cd_overdue_l36m_x AS cd_overdue_l36m_diff,
cd_overdue_l24m_y-cd_overdue_l24m_x AS cd_overdue_l24m_diff,
cd_overdue_l18m_y-cd_overdue_l18m_x AS cd_overdue_l18m_diff,
cd_overdue_l12m_y-cd_overdue_l12m_x AS cd_overdue_l12m_diff,
cd_overdue_l9m_y-cd_overdue_l9m_x AS cd_overdue_l9m_diff,
cd_overdue_l6m_y-cd_overdue_l6m_x AS cd_overdue_l6m_diff,
cd_overdue_l3m_y-cd_overdue_l3m_x AS cd_overdue_l3m_diff,
cd_overdue_cm_y-cd_overdue_cm_x AS cd_overdue_cm_diff,
gl_overdue_l36m_y-gl_overdue_l36m_x AS gl_overdue_l36m_diff,
gl_overdue_l24m_y-gl_overdue_l24m_x AS gl_overdue_l24m_diff,
gl_overdue_l18m_y-gl_overdue_l18m_x AS gl_overdue_l18m_diff,
gl_overdue_l12m_y-gl_overdue_l12m_x AS gl_overdue_l12m_diff,
gl_overdue_l9m_y-gl_overdue_l9m_x AS gl_overdue_l9m_diff,
gl_overdue_l6m_y-gl_overdue_l6m_x AS gl_overdue_l6m_diff,
gl_overdue_l3m_y-gl_overdue_l3m_x AS gl_overdue_l3m_diff,
gl_overdue_cm_y-gl_overdue_cm_x AS gl_overdue_cm_diff,
al_overdue_l36m_y-al_overdue_l36m_x AS al_overdue_l36m_diff,
al_overdue_l24m_y-al_overdue_l24m_x AS al_overdue_l24m_diff,
al_overdue_l18m_y-al_overdue_l18m_x AS al_overdue_l18m_diff,
al_overdue_l12m_y-al_overdue_l12m_x AS al_overdue_l12m_diff,
al_overdue_l9m_y-al_overdue_l9m_x AS al_overdue_l9m_diff,
al_overdue_l6m_y-al_overdue_l6m_x AS al_overdue_l6m_diff,
al_overdue_l3m_y-al_overdue_l3m_x AS al_overdue_l3m_diff,
al_overdue_cm_y-al_overdue_cm_x AS al_overdue_cm_diff,
tw_overdue_l36m_y-tw_overdue_l36m_x AS tw_overdue_l36m_diff,
tw_overdue_l24m_y-tw_overdue_l24m_x AS tw_overdue_l24m_diff,
tw_overdue_l18m_y-tw_overdue_l18m_x AS tw_overdue_l18m_diff,
tw_overdue_l12m_y-tw_overdue_l12m_x AS tw_overdue_l12m_diff,
tw_overdue_l9m_y-tw_overdue_l9m_x AS tw_overdue_l9m_diff,
tw_overdue_l6m_y-tw_overdue_l6m_x AS tw_overdue_l6m_diff,
tw_overdue_l3m_y-tw_overdue_l3m_x AS tw_overdue_l3m_diff,
tw_overdue_cm_y-tw_overdue_cm_x AS tw_overdue_cm_diff,
hl_overdue_l36m_y-hl_overdue_l36m_x AS hl_overdue_l36m_diff,
hl_overdue_l24m_y-hl_overdue_l24m_x AS hl_overdue_l24m_diff,
hl_overdue_l18m_y-hl_overdue_l18m_x AS hl_overdue_l18m_diff,
hl_overdue_l12m_y-hl_overdue_l12m_x AS hl_overdue_l12m_diff,
hl_overdue_l9m_y-hl_overdue_l9m_x AS hl_overdue_l9m_diff,
hl_overdue_l6m_y-hl_overdue_l6m_x AS hl_overdue_l6m_diff,
hl_overdue_l3m_y-hl_overdue_l3m_x AS hl_overdue_l3m_diff,
hl_overdue_cm_y-hl_overdue_cm_x AS hl_overdue_cm_diff,
unsecured_overdue_l36m_y-unsecured_overdue_l36m_x AS unsecured_overdue_l36m_diff,
unsecured_overdue_l24m_y-unsecured_overdue_l24m_x AS unsecured_overdue_l24m_diff,
unsecured_overdue_l18m_y-unsecured_overdue_l18m_x AS unsecured_overdue_l18m_diff,
unsecured_overdue_l12m_y-unsecured_overdue_l12m_x AS unsecured_overdue_l12m_diff,
unsecured_overdue_l9m_y-unsecured_overdue_l9m_x AS unsecured_overdue_l9m_diff,
unsecured_overdue_l6m_y-unsecured_overdue_l6m_x AS unsecured_overdue_l6m_diff,
unsecured_overdue_l3m_y-unsecured_overdue_l3m_x AS unsecured_overdue_l3m_diff,
unsecured_overdue_cm_y-unsecured_overdue_cm_x AS unsecured_overdue_cm_diff,
secured_overdue_l36m_y-secured_overdue_l36m_x AS secured_overdue_l36m_diff,
secured_overdue_l24m_y-secured_overdue_l24m_x AS secured_overdue_l24m_diff,
secured_overdue_l18m_y-secured_overdue_l18m_x AS secured_overdue_l18m_diff,
secured_overdue_l12m_y-secured_overdue_l12m_x AS secured_overdue_l12m_diff,
secured_overdue_l9m_y-secured_overdue_l9m_x AS secured_overdue_l9m_diff,
secured_overdue_l6m_y-secured_overdue_l6m_x AS secured_overdue_l6m_diff,
secured_overdue_l3m_y-secured_overdue_l3m_x AS secured_overdue_l3m_diff,
secured_overdue_cm_y-secured_overdue_cm_x AS secured_overdue_cm_diff,
nbr_x_dpd_l36m_y-nbr_x_dpd_l36m_x AS nbr_x_dpd_l36m_diff,
nbr_x_dpd_l24m_y-nbr_x_dpd_l24m_x AS nbr_x_dpd_l24m_diff,
nbr_x_dpd_l18m_y-nbr_x_dpd_l18m_x AS nbr_x_dpd_l18m_diff,
nbr_x_dpd_l12m_y-nbr_x_dpd_l12m_x AS nbr_x_dpd_l12m_diff,
nbr_x_dpd_l6m_y-nbr_x_dpd_l6m_x AS nbr_x_dpd_l6m_diff,
nbr_x_dpd_l3m_y-nbr_x_dpd_l3m_x AS nbr_x_dpd_l3m_diff,
nbr_x_dpd_cm_y-nbr_x_dpd_cm_x AS nbr_x_dpd_cm_diff,
nbr_30_dpd_l36m_y-nbr_30_dpd_l36m_x AS nbr_30_dpd_l36m_diff,
nbr_30_dpd_l24m_y-nbr_30_dpd_l24m_x AS nbr_30_dpd_l24m_diff,
nbr_30_dpd_l18m_y-nbr_30_dpd_l18m_x AS nbr_30_dpd_l18m_diff,
nbr_30_dpd_l12m_y-nbr_30_dpd_l12m_x AS nbr_30_dpd_l12m_diff,
nbr_30_dpd_l6m_y-nbr_30_dpd_l6m_x AS nbr_30_dpd_l6m_diff,
nbr_30_dpd_l3m_y-nbr_30_dpd_l3m_x AS nbr_30_dpd_l3m_diff,
nbr_30_dpd_cm_y-nbr_30_dpd_cm_x AS nbr_30_dpd_cm_diff,
nbr_90_dpd_l36m_y-nbr_90_dpd_l36m_x AS nbr_90_dpd_l36m_diff,
nbr_90_dpd_l24m_y-nbr_90_dpd_l24m_x AS nbr_90_dpd_l24m_diff,
nbr_90_dpd_l18m_y-nbr_90_dpd_l18m_x AS nbr_90_dpd_l18m_diff,
nbr_90_dpd_l12m_y-nbr_90_dpd_l12m_x AS nbr_90_dpd_l12m_diff,
nbr_90_dpd_l6m_y-nbr_90_dpd_l6m_x AS nbr_90_dpd_l6m_diff,
nbr_90_dpd_l3m_y-nbr_90_dpd_l3m_x AS nbr_90_dpd_l3m_diff,
nbr_90_dpd_cm_y-nbr_90_dpd_cm_x AS nbr_90_dpd_cm_diff,
nbr_active_x_dpd_l36m_y-nbr_active_x_dpd_l36m_x AS nbr_active_x_dpd_l36m_diff,
nbr_active_x_dpd_l24m_y-nbr_active_x_dpd_l24m_x AS nbr_active_x_dpd_l24m_diff,
nbr_active_x_dpd_l18m_y-nbr_active_x_dpd_l18m_x AS nbr_active_x_dpd_l18m_diff,
nbr_active_x_dpd_l12m_y-nbr_active_x_dpd_l12m_x AS nbr_active_x_dpd_l12m_diff,
nbr_active_x_dpd_l6m_y-nbr_active_x_dpd_l6m_x AS nbr_active_x_dpd_l6m_diff,
nbr_active_x_dpd_l3m_y-nbr_active_x_dpd_l3m_x AS nbr_active_x_dpd_l3m_diff,
nbr_active_x_dpd_cm_y-nbr_active_x_dpd_cm_x AS nbr_active_x_dpd_cm_diff,
nbr_active_30_dpd_l36m_y-nbr_active_30_dpd_l36m_x AS nbr_active_30_dpd_l36m_diff,
nbr_active_30_dpd_l24m_y-nbr_active_30_dpd_l24m_x AS nbr_active_30_dpd_l24m_diff,
nbr_active_30_dpd_l18m_y-nbr_active_30_dpd_l18m_x AS nbr_active_30_dpd_l18m_diff,
nbr_active_30_dpd_l12m_y-nbr_active_30_dpd_l12m_x AS nbr_active_30_dpd_l12m_diff,
nbr_active_30_dpd_l6m_y-nbr_active_30_dpd_l6m_x AS nbr_active_30_dpd_l6m_diff,
nbr_active_30_dpd_l3m_y-nbr_active_30_dpd_l3m_x AS nbr_active_30_dpd_l3m_diff,
nbr_active_30_dpd_cm_y-nbr_active_30_dpd_cm_x AS nbr_active_30_dpd_cm_diff,
nbr_active_90_dpd_l36m_y-nbr_active_90_dpd_l36m_x AS nbr_active_90_dpd_l36m_diff,
nbr_active_90_dpd_l24m_y-nbr_active_90_dpd_l24m_x AS nbr_active_90_dpd_l24m_diff,
nbr_active_90_dpd_l18m_y-nbr_active_90_dpd_l18m_x AS nbr_active_90_dpd_l18m_diff,
nbr_active_90_dpd_l12m_y-nbr_active_90_dpd_l12m_x AS nbr_active_90_dpd_l12m_diff,
nbr_active_90_dpd_l6m_y-nbr_active_90_dpd_l6m_x AS nbr_active_90_dpd_l6m_diff,
nbr_active_90_dpd_l3m_y-nbr_active_90_dpd_l3m_x AS nbr_active_90_dpd_l3m_diff,
nbr_active_90_dpd_cm_y-nbr_active_90_dpd_cm_x AS nbr_active_90_dpd_cm_diff,
nbr_cc_x_dpd_l36m_y-nbr_cc_x_dpd_l36m_x AS nbr_cc_x_dpd_l36m_diff,
nbr_cc_x_dpd_l24m_y-nbr_cc_x_dpd_l24m_x AS nbr_cc_x_dpd_l24m_diff,
nbr_cc_x_dpd_l18m_y-nbr_cc_x_dpd_l18m_x AS nbr_cc_x_dpd_l18m_diff,
nbr_cc_x_dpd_l12m_y-nbr_cc_x_dpd_l12m_x AS nbr_cc_x_dpd_l12m_diff,
nbr_cc_x_dpd_l6m_y-nbr_cc_x_dpd_l6m_x AS nbr_cc_x_dpd_l6m_diff,
nbr_cc_x_dpd_l3m_y-nbr_cc_x_dpd_l3m_x AS nbr_cc_x_dpd_l3m_diff,
nbr_cc_x_dpd_cm_y-nbr_cc_x_dpd_cm_x AS nbr_cc_x_dpd_cm_diff,
nbr_cc_30_dpd_l36m_y-nbr_cc_30_dpd_l36m_x AS nbr_cc_30_dpd_l36m_diff,
nbr_cc_30_dpd_l24m_y-nbr_cc_30_dpd_l24m_x AS nbr_cc_30_dpd_l24m_diff,
nbr_cc_30_dpd_l18m_y-nbr_cc_30_dpd_l18m_x AS nbr_cc_30_dpd_l18m_diff,
nbr_cc_30_dpd_l12m_y-nbr_cc_30_dpd_l12m_x AS nbr_cc_30_dpd_l12m_diff,
nbr_cc_30_dpd_l6m_y-nbr_cc_30_dpd_l6m_x AS nbr_cc_30_dpd_l6m_diff,
nbr_cc_30_dpd_l3m_y-nbr_cc_30_dpd_l3m_x AS nbr_cc_30_dpd_l3m_diff,
nbr_cc_30_dpd_cm_y-nbr_cc_30_dpd_cm_x AS nbr_cc_30_dpd_cm_diff,
nbr_cc_90_dpd_l36m_y-nbr_cc_90_dpd_l36m_x AS nbr_cc_90_dpd_l36m_diff,
nbr_cc_90_dpd_l24m_y-nbr_cc_90_dpd_l24m_x AS nbr_cc_90_dpd_l24m_diff,
nbr_cc_90_dpd_l18m_y-nbr_cc_90_dpd_l18m_x AS nbr_cc_90_dpd_l18m_diff,
nbr_cc_90_dpd_l12m_y-nbr_cc_90_dpd_l12m_x AS nbr_cc_90_dpd_l12m_diff,
nbr_cc_90_dpd_l6m_y-nbr_cc_90_dpd_l6m_x AS nbr_cc_90_dpd_l6m_diff,
nbr_cc_90_dpd_l3m_y-nbr_cc_90_dpd_l3m_x AS nbr_cc_90_dpd_l3m_diff,
nbr_cc_90_dpd_cm_y-nbr_cc_90_dpd_cm_x AS nbr_cc_90_dpd_cm_diff,
nbr_active_cc_x_dpd_l36m_y-nbr_active_cc_x_dpd_l36m_x AS nbr_active_cc_x_dpd_l36m_diff,
nbr_active_cc_x_dpd_l24m_y-nbr_active_cc_x_dpd_l24m_x AS nbr_active_cc_x_dpd_l24m_diff,
nbr_active_cc_x_dpd_l18m_y-nbr_active_cc_x_dpd_l18m_x AS nbr_active_cc_x_dpd_l18m_diff,
nbr_active_cc_x_dpd_l12m_y-nbr_active_cc_x_dpd_l12m_x AS nbr_active_cc_x_dpd_l12m_diff,
nbr_active_cc_x_dpd_l6m_y-nbr_active_cc_x_dpd_l6m_x AS nbr_active_cc_x_dpd_l6m_diff,
nbr_active_cc_x_dpd_l3m_y-nbr_active_cc_x_dpd_l3m_x AS nbr_active_cc_x_dpd_l3m_diff,
nbr_active_cc_x_dpd_cm_y-nbr_active_cc_x_dpd_cm_x AS nbr_active_cc_x_dpd_cm_diff,
nbr_active_cc_30_dpd_l36m_y-nbr_active_cc_30_dpd_l36m_x AS nbr_active_cc_30_dpd_l36m_diff,
nbr_active_cc_30_dpd_l24m_y-nbr_active_cc_30_dpd_l24m_x AS nbr_active_cc_30_dpd_l24m_diff,
nbr_active_cc_30_dpd_l18m_y-nbr_active_cc_30_dpd_l18m_x AS nbr_active_cc_30_dpd_l18m_diff,
nbr_active_cc_30_dpd_l12m_y-nbr_active_cc_30_dpd_l12m_x AS nbr_active_cc_30_dpd_l12m_diff,
nbr_active_cc_30_dpd_l6m_y-nbr_active_cc_30_dpd_l6m_x AS nbr_active_cc_30_dpd_l6m_diff,
nbr_active_cc_30_dpd_l3m_y-nbr_active_cc_30_dpd_l3m_x AS nbr_active_cc_30_dpd_l3m_diff,
nbr_active_cc_30_dpd_cm_y-nbr_active_cc_30_dpd_cm_x AS nbr_active_cc_30_dpd_cm_diff,
nbr_cc_active_90_dpd_l36m_y-nbr_cc_active_90_dpd_l36m_x AS nbr_cc_active_90_dpd_l36m_diff,
nbr_cc_active_90_dpd_l24m_y-nbr_cc_active_90_dpd_l24m_x AS nbr_cc_active_90_dpd_l24m_diff,
nbr_cc_active_90_dpd_l18m_y-nbr_cc_active_90_dpd_l18m_x AS nbr_cc_active_90_dpd_l18m_diff,
nbr_cc_active_90_dpd_l12m_y-nbr_cc_active_90_dpd_l12m_x AS nbr_cc_active_90_dpd_l12m_diff,
nbr_cc_active_90_dpd_l6m_y-nbr_cc_active_90_dpd_l6m_x AS nbr_cc_active_90_dpd_l6m_diff,
nbr_cc_active_90_dpd_l3m_y-nbr_cc_active_90_dpd_l3m_x AS nbr_cc_active_90_dpd_l3m_diff,
nbr_cc_active_90_dpd_cm_y-nbr_cc_active_90_dpd_cm_x AS nbr_cc_active_90_dpd_cm_diff,
nbr_pl_x_dpd_l36m_y-nbr_pl_x_dpd_l36m_x AS nbr_pl_x_dpd_l36m_diff,
nbr_pl_x_dpd_l24m_y-nbr_pl_x_dpd_l24m_x AS nbr_pl_x_dpd_l24m_diff,
nbr_pl_x_dpd_l18m_y-nbr_pl_x_dpd_l18m_x AS nbr_pl_x_dpd_l18m_diff,
nbr_pl_x_dpd_l12m_y-nbr_pl_x_dpd_l12m_x AS nbr_pl_x_dpd_l12m_diff,
nbr_pl_x_dpd_l6m_y-nbr_pl_x_dpd_l6m_x AS nbr_pl_x_dpd_l6m_diff,
nbr_pl_x_dpd_l3m_y-nbr_pl_x_dpd_l3m_x AS nbr_pl_x_dpd_l3m_diff,
nbr_pl_x_dpd_cm_y-nbr_pl_x_dpd_cm_x AS nbr_pl_x_dpd_cm_diff,
nbr_pl_30_dpd_l36m_y-nbr_pl_30_dpd_l36m_x AS nbr_pl_30_dpd_l36m_diff,
nbr_pl_30_dpd_l24m_y-nbr_pl_30_dpd_l24m_x AS nbr_pl_30_dpd_l24m_diff,
nbr_pl_30_dpd_l18m_y-nbr_pl_30_dpd_l18m_x AS nbr_pl_30_dpd_l18m_diff,
nbr_pl_30_dpd_l12m_y-nbr_pl_30_dpd_l12m_x AS nbr_pl_30_dpd_l12m_diff,
nbr_pl_30_dpd_l6m_y-nbr_pl_30_dpd_l6m_x AS nbr_pl_30_dpd_l6m_diff,
nbr_pl_30_dpd_l3m_y-nbr_pl_30_dpd_l3m_x AS nbr_pl_30_dpd_l3m_diff,
nbr_pl_30_dpd_cm_y-nbr_pl_30_dpd_cm_x AS nbr_pl_30_dpd_cm_diff,
nbr_pl_90_dpd_l36m_y-nbr_pl_90_dpd_l36m_x AS nbr_pl_90_dpd_l36m_diff,
nbr_pl_90_dpd_l24m_y-nbr_pl_90_dpd_l24m_x AS nbr_pl_90_dpd_l24m_diff,
nbr_pl_90_dpd_l18m_y-nbr_pl_90_dpd_l18m_x AS nbr_pl_90_dpd_l18m_diff,
nbr_pl_90_dpd_l12m_y-nbr_pl_90_dpd_l12m_x AS nbr_pl_90_dpd_l12m_diff,
nbr_pl_90_dpd_l6m_y-nbr_pl_90_dpd_l6m_x AS nbr_pl_90_dpd_l6m_diff,
nbr_pl_90_dpd_l3m_y-nbr_pl_90_dpd_l3m_x AS nbr_pl_90_dpd_l3m_diff,
nbr_pl_90_dpd_cm_y-nbr_pl_90_dpd_cm_x AS nbr_pl_90_dpd_cm_diff,
nbr_active_pl_x_dpd_l36m_y-nbr_active_pl_x_dpd_l36m_x AS nbr_active_pl_x_dpd_l36m_diff,
nbr_active_pl_x_dpd_l24m_y-nbr_active_pl_x_dpd_l24m_x AS nbr_active_pl_x_dpd_l24m_diff,
nbr_active_pl_x_dpd_l18m_y-nbr_active_pl_x_dpd_l18m_x AS nbr_active_pl_x_dpd_l18m_diff,
nbr_active_pl_x_dpd_l12m_y-nbr_active_pl_x_dpd_l12m_x AS nbr_active_pl_x_dpd_l12m_diff,
nbr_active_pl_x_dpd_l6m_y-nbr_active_pl_x_dpd_l6m_x AS nbr_active_pl_x_dpd_l6m_diff,
nbr_active_pl_x_dpd_l3m_y-nbr_active_pl_x_dpd_l3m_x AS nbr_active_pl_x_dpd_l3m_diff,
nbr_active_pl_x_dpd_cm_y-nbr_active_pl_x_dpd_cm_x AS nbr_active_pl_x_dpd_cm_diff,
nbr_active_pl_30_dpd_l36m_y-nbr_active_pl_30_dpd_l36m_x AS nbr_active_pl_30_dpd_l36m_diff,
nbr_active_pl_30_dpd_l24m_y-nbr_active_pl_30_dpd_l24m_x AS nbr_active_pl_30_dpd_l24m_diff,
nbr_active_pl_30_dpd_l18m_y-nbr_active_pl_30_dpd_l18m_x AS nbr_active_pl_30_dpd_l18m_diff,
nbr_active_pl_30_dpd_l12m_y-nbr_active_pl_30_dpd_l12m_x AS nbr_active_pl_30_dpd_l12m_diff,
nbr_active_pl_30_dpd_l6m_y-nbr_active_pl_30_dpd_l6m_x AS nbr_active_pl_30_dpd_l6m_diff,
nbr_active_pl_30_dpd_l3m_y-nbr_active_pl_30_dpd_l3m_x AS nbr_active_pl_30_dpd_l3m_diff,
nbr_active_pl_30_dpd_cm_y-nbr_active_pl_30_dpd_cm_x AS nbr_active_pl_30_dpd_cm_diff,
nbr_pl_active_90_dpd_l36m_y-nbr_pl_active_90_dpd_l36m_x AS nbr_pl_active_90_dpd_l36m_diff,
nbr_pl_active_90_dpd_l24m_y-nbr_pl_active_90_dpd_l24m_x AS nbr_pl_active_90_dpd_l24m_diff,
nbr_pl_active_90_dpd_l18m_y-nbr_pl_active_90_dpd_l18m_x AS nbr_pl_active_90_dpd_l18m_diff,
nbr_pl_active_90_dpd_l12m_y-nbr_pl_active_90_dpd_l12m_x AS nbr_pl_active_90_dpd_l12m_diff,
nbr_pl_active_90_dpd_l6m_y-nbr_pl_active_90_dpd_l6m_x AS nbr_pl_active_90_dpd_l6m_diff,
nbr_pl_active_90_dpd_l3m_y-nbr_pl_active_90_dpd_l3m_x AS nbr_pl_active_90_dpd_l3m_diff,
nbr_pl_active_90_dpd_cm_y-nbr_pl_active_90_dpd_cm_x AS nbr_pl_active_90_dpd_cm_diff,
nbr_cd_x_dpd_l36m_y-nbr_cd_x_dpd_l36m_x AS nbr_cd_x_dpd_l36m_diff,
nbr_cd_x_dpd_l24m_y-nbr_cd_x_dpd_l24m_x AS nbr_cd_x_dpd_l24m_diff,
nbr_cd_x_dpd_l18m_y-nbr_cd_x_dpd_l18m_x AS nbr_cd_x_dpd_l18m_diff,
nbr_cd_x_dpd_l12m_y-nbr_cd_x_dpd_l12m_x AS nbr_cd_x_dpd_l12m_diff,
nbr_cd_x_dpd_l6m_y-nbr_cd_x_dpd_l6m_x AS nbr_cd_x_dpd_l6m_diff,
nbr_cd_x_dpd_l3m_y-nbr_cd_x_dpd_l3m_x AS nbr_cd_x_dpd_l3m_diff,
nbr_cd_x_dpd_cm_y-nbr_cd_x_dpd_cm_x AS nbr_cd_x_dpd_cm_diff,
nbr_cd_30_dpd_l36m_y-nbr_cd_30_dpd_l36m_x AS nbr_cd_30_dpd_l36m_diff,
nbr_cd_30_dpd_l24m_y-nbr_cd_30_dpd_l24m_x AS nbr_cd_30_dpd_l24m_diff,
nbr_cd_30_dpd_l18m_y-nbr_cd_30_dpd_l18m_x AS nbr_cd_30_dpd_l18m_diff,
nbr_cd_30_dpd_l12m_y-nbr_cd_30_dpd_l12m_x AS nbr_cd_30_dpd_l12m_diff,
nbr_cd_30_dpd_l6m_y-nbr_cd_30_dpd_l6m_x AS nbr_cd_30_dpd_l6m_diff,
nbr_cd_30_dpd_l3m_y-nbr_cd_30_dpd_l3m_x AS nbr_cd_30_dpd_l3m_diff,
nbr_cd_30_dpd_cm_y-nbr_cd_30_dpd_cm_x AS nbr_cd_30_dpd_cm_diff,
nbr_cd_90_dpd_l36m_y-nbr_cd_90_dpd_l36m_x AS nbr_cd_90_dpd_l36m_diff,
nbr_cd_90_dpd_l24m_y-nbr_cd_90_dpd_l24m_x AS nbr_cd_90_dpd_l24m_diff,
nbr_cd_90_dpd_l18m_y-nbr_cd_90_dpd_l18m_x AS nbr_cd_90_dpd_l18m_diff,
nbr_cd_90_dpd_l12m_y-nbr_cd_90_dpd_l12m_x AS nbr_cd_90_dpd_l12m_diff,
nbr_cd_90_dpd_l6m_y-nbr_cd_90_dpd_l6m_x AS nbr_cd_90_dpd_l6m_diff,
nbr_cd_90_dpd_l3m_y-nbr_cd_90_dpd_l3m_x AS nbr_cd_90_dpd_l3m_diff,
nbr_cd_90_dpd_cm_y-nbr_cd_90_dpd_cm_x AS nbr_cd_90_dpd_cm_diff,
nbr_active_cd_x_dpd_l36m_y-nbr_active_cd_x_dpd_l36m_x AS nbr_active_cd_x_dpd_l36m_diff,
nbr_active_cd_x_dpd_l24m_y-nbr_active_cd_x_dpd_l24m_x AS nbr_active_cd_x_dpd_l24m_diff,
nbr_active_cd_x_dpd_l18m_y-nbr_active_cd_x_dpd_l18m_x AS nbr_active_cd_x_dpd_l18m_diff,
nbr_active_cd_x_dpd_l12m_y-nbr_active_cd_x_dpd_l12m_x AS nbr_active_cd_x_dpd_l12m_diff,
nbr_active_cd_x_dpd_l6m_y-nbr_active_cd_x_dpd_l6m_x AS nbr_active_cd_x_dpd_l6m_diff,
nbr_active_cd_x_dpd_l3m_y-nbr_active_cd_x_dpd_l3m_x AS nbr_active_cd_x_dpd_l3m_diff,
nbr_active_cd_x_dpd_cm_y-nbr_active_cd_x_dpd_cm_x AS nbr_active_cd_x_dpd_cm_diff,
nbr_active_cd_30_dpd_l36m_y-nbr_active_cd_30_dpd_l36m_x AS nbr_active_cd_30_dpd_l36m_diff,
nbr_active_cd_30_dpd_l24m_y-nbr_active_cd_30_dpd_l24m_x AS nbr_active_cd_30_dpd_l24m_diff,
nbr_active_cd_30_dpd_l18m_y-nbr_active_cd_30_dpd_l18m_x AS nbr_active_cd_30_dpd_l18m_diff,
nbr_active_cd_30_dpd_l12m_y-nbr_active_cd_30_dpd_l12m_x AS nbr_active_cd_30_dpd_l12m_diff,
nbr_active_cd_30_dpd_l6m_y-nbr_active_cd_30_dpd_l6m_x AS nbr_active_cd_30_dpd_l6m_diff,
nbr_active_cd_30_dpd_l3m_y-nbr_active_cd_30_dpd_l3m_x AS nbr_active_cd_30_dpd_l3m_diff,
nbr_active_cd_30_dpd_cm_y-nbr_active_cd_30_dpd_cm_x AS nbr_active_cd_30_dpd_cm_diff,
nbr_cd_active_90_dpd_l36m_y-nbr_cd_active_90_dpd_l36m_x AS nbr_cd_active_90_dpd_l36m_diff,
nbr_cd_active_90_dpd_l24m_y-nbr_cd_active_90_dpd_l24m_x AS nbr_cd_active_90_dpd_l24m_diff,
nbr_cd_active_90_dpd_l18m_y-nbr_cd_active_90_dpd_l18m_x AS nbr_cd_active_90_dpd_l18m_diff,
nbr_cd_active_90_dpd_l12m_y-nbr_cd_active_90_dpd_l12m_x AS nbr_cd_active_90_dpd_l12m_diff,
nbr_cd_active_90_dpd_l6m_y-nbr_cd_active_90_dpd_l6m_x AS nbr_cd_active_90_dpd_l6m_diff,
nbr_cd_active_90_dpd_l3m_y-nbr_cd_active_90_dpd_l3m_x AS nbr_cd_active_90_dpd_l3m_diff,
nbr_cd_active_90_dpd_cm_y-nbr_cd_active_90_dpd_cm_x AS nbr_cd_active_90_dpd_cm_diff,
nbr_al_x_dpd_l36m_y-nbr_al_x_dpd_l36m_x AS nbr_al_x_dpd_l36m_diff,
nbr_al_x_dpd_l24m_y-nbr_al_x_dpd_l24m_x AS nbr_al_x_dpd_l24m_diff,
nbr_al_x_dpd_l18m_y-nbr_al_x_dpd_l18m_x AS nbr_al_x_dpd_l18m_diff,
nbr_al_x_dpd_l12m_y-nbr_al_x_dpd_l12m_x AS nbr_al_x_dpd_l12m_diff,
nbr_al_x_dpd_l6m_y-nbr_al_x_dpd_l6m_x AS nbr_al_x_dpd_l6m_diff,
nbr_al_x_dpd_l3m_y-nbr_al_x_dpd_l3m_x AS nbr_al_x_dpd_l3m_diff,
nbr_al_x_dpd_cm_y-nbr_al_x_dpd_cm_x AS nbr_al_x_dpd_cm_diff,
nbr_al_30_dpd_l36m_y-nbr_al_30_dpd_l36m_x AS nbr_al_30_dpd_l36m_diff,
nbr_al_30_dpd_l24m_y-nbr_al_30_dpd_l24m_x AS nbr_al_30_dpd_l24m_diff,
nbr_al_30_dpd_l18m_y-nbr_al_30_dpd_l18m_x AS nbr_al_30_dpd_l18m_diff,
nbr_al_30_dpd_l12m_y-nbr_al_30_dpd_l12m_x AS nbr_al_30_dpd_l12m_diff,
nbr_al_30_dpd_l6m_y-nbr_al_30_dpd_l6m_x AS nbr_al_30_dpd_l6m_diff,
nbr_al_30_dpd_l3m_y-nbr_al_30_dpd_l3m_x AS nbr_al_30_dpd_l3m_diff,
nbr_al_30_dpd_cm_y-nbr_al_30_dpd_cm_x AS nbr_al_30_dpd_cm_diff,
nbr_al_90_dpd_l36m_y-nbr_al_90_dpd_l36m_x AS nbr_al_90_dpd_l36m_diff,
nbr_al_90_dpd_l24m_y-nbr_al_90_dpd_l24m_x AS nbr_al_90_dpd_l24m_diff,
nbr_al_90_dpd_l18m_y-nbr_al_90_dpd_l18m_x AS nbr_al_90_dpd_l18m_diff,
nbr_al_90_dpd_l12m_y-nbr_al_90_dpd_l12m_x AS nbr_al_90_dpd_l12m_diff,
nbr_al_90_dpd_l6m_y-nbr_al_90_dpd_l6m_x AS nbr_al_90_dpd_l6m_diff,
nbr_al_90_dpd_l3m_y-nbr_al_90_dpd_l3m_x AS nbr_al_90_dpd_l3m_diff,
nbr_al_90_dpd_cm_y-nbr_al_90_dpd_cm_x AS nbr_al_90_dpd_cm_diff,
nbr_active_al_x_dpd_l36m_y-nbr_active_al_x_dpd_l36m_x AS nbr_active_al_x_dpd_l36m_diff,
nbr_active_al_x_dpd_l24m_y-nbr_active_al_x_dpd_l24m_x AS nbr_active_al_x_dpd_l24m_diff,
nbr_active_al_x_dpd_l18m_y-nbr_active_al_x_dpd_l18m_x AS nbr_active_al_x_dpd_l18m_diff,
nbr_active_al_x_dpd_l12m_y-nbr_active_al_x_dpd_l12m_x AS nbr_active_al_x_dpd_l12m_diff,
nbr_active_al_x_dpd_l6m_y-nbr_active_al_x_dpd_l6m_x AS nbr_active_al_x_dpd_l6m_diff,
nbr_active_al_x_dpd_l3m_y-nbr_active_al_x_dpd_l3m_x AS nbr_active_al_x_dpd_l3m_diff,
nbr_active_al_x_dpd_cm_y-nbr_active_al_x_dpd_cm_x AS nbr_active_al_x_dpd_cm_diff,
nbr_active_al_30_dpd_l36m_y-nbr_active_al_30_dpd_l36m_x AS nbr_active_al_30_dpd_l36m_diff,
nbr_active_al_30_dpd_l24m_y-nbr_active_al_30_dpd_l24m_x AS nbr_active_al_30_dpd_l24m_diff,
nbr_active_al_30_dpd_l18m_y-nbr_active_al_30_dpd_l18m_x AS nbr_active_al_30_dpd_l18m_diff,
nbr_active_al_30_dpd_l12m_y-nbr_active_al_30_dpd_l12m_x AS nbr_active_al_30_dpd_l12m_diff,
nbr_active_al_30_dpd_l6m_y-nbr_active_al_30_dpd_l6m_x AS nbr_active_al_30_dpd_l6m_diff,
nbr_active_al_30_dpd_l3m_y-nbr_active_al_30_dpd_l3m_x AS nbr_active_al_30_dpd_l3m_diff,
nbr_active_al_30_dpd_cm_y-nbr_active_al_30_dpd_cm_x AS nbr_active_al_30_dpd_cm_diff,
nbr_al_active_90_dpd_l36m_y-nbr_al_active_90_dpd_l36m_x AS nbr_al_active_90_dpd_l36m_diff,
nbr_al_active_90_dpd_l24m_y-nbr_al_active_90_dpd_l24m_x AS nbr_al_active_90_dpd_l24m_diff,
nbr_al_active_90_dpd_l18m_y-nbr_al_active_90_dpd_l18m_x AS nbr_al_active_90_dpd_l18m_diff,
nbr_al_active_90_dpd_l12m_y-nbr_al_active_90_dpd_l12m_x AS nbr_al_active_90_dpd_l12m_diff,
nbr_al_active_90_dpd_l6m_y-nbr_al_active_90_dpd_l6m_x AS nbr_al_active_90_dpd_l6m_diff,
nbr_al_active_90_dpd_l3m_y-nbr_al_active_90_dpd_l3m_x AS nbr_al_active_90_dpd_l3m_diff,
nbr_al_active_90_dpd_cm_y-nbr_al_active_90_dpd_cm_x AS nbr_al_active_90_dpd_cm_diff,
nbr_gl_x_dpd_l36m_y-nbr_gl_x_dpd_l36m_x AS nbr_gl_x_dpd_l36m_diff,
nbr_gl_x_dpd_l24m_y-nbr_gl_x_dpd_l24m_x AS nbr_gl_x_dpd_l24m_diff,
nbr_gl_x_dpd_l18m_y-nbr_gl_x_dpd_l18m_x AS nbr_gl_x_dpd_l18m_diff,
nbr_gl_x_dpd_l12m_y-nbr_gl_x_dpd_l12m_x AS nbr_gl_x_dpd_l12m_diff,
nbr_gl_x_dpd_l6m_y-nbr_gl_x_dpd_l6m_x AS nbr_gl_x_dpd_l6m_diff,
nbr_gl_x_dpd_l3m_y-nbr_gl_x_dpd_l3m_x AS nbr_gl_x_dpd_l3m_diff,
nbr_gl_x_dpd_cm_y-nbr_gl_x_dpd_cm_x AS nbr_gl_x_dpd_cm_diff,
nbr_gl_30_dpd_l36m_y-nbr_gl_30_dpd_l36m_x AS nbr_gl_30_dpd_l36m_diff,
nbr_gl_30_dpd_l24m_y-nbr_gl_30_dpd_l24m_x AS nbr_gl_30_dpd_l24m_diff,
nbr_gl_30_dpd_l18m_y-nbr_gl_30_dpd_l18m_x AS nbr_gl_30_dpd_l18m_diff,
nbr_gl_30_dpd_l12m_y-nbr_gl_30_dpd_l12m_x AS nbr_gl_30_dpd_l12m_diff,
nbr_gl_30_dpd_l6m_y-nbr_gl_30_dpd_l6m_x AS nbr_gl_30_dpd_l6m_diff,
nbr_gl_30_dpd_l3m_y-nbr_gl_30_dpd_l3m_x AS nbr_gl_30_dpd_l3m_diff,
nbr_gl_30_dpd_cm_y-nbr_gl_30_dpd_cm_x AS nbr_gl_30_dpd_cm_diff,
nbr_gl_90_dpd_l36m_y-nbr_gl_90_dpd_l36m_x AS nbr_gl_90_dpd_l36m_diff,
nbr_gl_90_dpd_l24m_y-nbr_gl_90_dpd_l24m_x AS nbr_gl_90_dpd_l24m_diff,
nbr_gl_90_dpd_l18m_y-nbr_gl_90_dpd_l18m_x AS nbr_gl_90_dpd_l18m_diff,
nbr_gl_90_dpd_l12m_y-nbr_gl_90_dpd_l12m_x AS nbr_gl_90_dpd_l12m_diff,
nbr_gl_90_dpd_l6m_y-nbr_gl_90_dpd_l6m_x AS nbr_gl_90_dpd_l6m_diff,
nbr_gl_90_dpd_l3m_y-nbr_gl_90_dpd_l3m_x AS nbr_gl_90_dpd_l3m_diff,
nbr_gl_90_dpd_cm_y-nbr_gl_90_dpd_cm_x AS nbr_gl_90_dpd_cm_diff,
nbr_active_gl_x_dpd_l36m_y-nbr_active_gl_x_dpd_l36m_x AS nbr_active_gl_x_dpd_l36m_diff,
nbr_active_gl_x_dpd_l24m_y-nbr_active_gl_x_dpd_l24m_x AS nbr_active_gl_x_dpd_l24m_diff,
nbr_active_gl_x_dpd_l18m_y-nbr_active_gl_x_dpd_l18m_x AS nbr_active_gl_x_dpd_l18m_diff,
nbr_active_gl_x_dpd_l12m_y-nbr_active_gl_x_dpd_l12m_x AS nbr_active_gl_x_dpd_l12m_diff,
nbr_active_gl_x_dpd_l6m_y-nbr_active_gl_x_dpd_l6m_x AS nbr_active_gl_x_dpd_l6m_diff,
nbr_active_gl_x_dpd_l3m_y-nbr_active_gl_x_dpd_l3m_x AS nbr_active_gl_x_dpd_l3m_diff,
nbr_active_gl_x_dpd_cm_y-nbr_active_gl_x_dpd_cm_x AS nbr_active_gl_x_dpd_cm_diff,
nbr_active_gl_30_dpd_l36m_y-nbr_active_gl_30_dpd_l36m_x AS nbr_active_gl_30_dpd_l36m_diff,
nbr_active_gl_30_dpd_l24m_y-nbr_active_gl_30_dpd_l24m_x AS nbr_active_gl_30_dpd_l24m_diff,
nbr_active_gl_30_dpd_l18m_y-nbr_active_gl_30_dpd_l18m_x AS nbr_active_gl_30_dpd_l18m_diff,
nbr_active_gl_30_dpd_l12m_y-nbr_active_gl_30_dpd_l12m_x AS nbr_active_gl_30_dpd_l12m_diff,
nbr_active_gl_30_dpd_l6m_y-nbr_active_gl_30_dpd_l6m_x AS nbr_active_gl_30_dpd_l6m_diff,
nbr_active_gl_30_dpd_l3m_y-nbr_active_gl_30_dpd_l3m_x AS nbr_active_gl_30_dpd_l3m_diff,
nbr_active_gl_30_dpd_cm_y-nbr_active_gl_30_dpd_cm_x AS nbr_active_gl_30_dpd_cm_diff,
nbr_gl_active_90_dpd_l36m_y-nbr_gl_active_90_dpd_l36m_x AS nbr_gl_active_90_dpd_l36m_diff,
nbr_gl_active_90_dpd_l24m_y-nbr_gl_active_90_dpd_l24m_x AS nbr_gl_active_90_dpd_l24m_diff,
nbr_gl_active_90_dpd_l18m_y-nbr_gl_active_90_dpd_l18m_x AS nbr_gl_active_90_dpd_l18m_diff,
nbr_gl_active_90_dpd_l12m_y-nbr_gl_active_90_dpd_l12m_x AS nbr_gl_active_90_dpd_l12m_diff,
nbr_gl_active_90_dpd_l6m_y-nbr_gl_active_90_dpd_l6m_x AS nbr_gl_active_90_dpd_l6m_diff,
nbr_gl_active_90_dpd_l3m_y-nbr_gl_active_90_dpd_l3m_x AS nbr_gl_active_90_dpd_l3m_diff,
nbr_gl_active_90_dpd_cm_y-nbr_gl_active_90_dpd_cm_x AS nbr_gl_active_90_dpd_cm_diff,
nbr_hl_x_dpd_l36m_y-nbr_hl_x_dpd_l36m_x AS nbr_hl_x_dpd_l36m_diff,
nbr_hl_x_dpd_l24m_y-nbr_hl_x_dpd_l24m_x AS nbr_hl_x_dpd_l24m_diff,
nbr_hl_x_dpd_l18m_y-nbr_hl_x_dpd_l18m_x AS nbr_hl_x_dpd_l18m_diff,
nbr_hl_x_dpd_l12m_y-nbr_hl_x_dpd_l12m_x AS nbr_hl_x_dpd_l12m_diff,
nbr_hl_x_dpd_l6m_y-nbr_hl_x_dpd_l6m_x AS nbr_hl_x_dpd_l6m_diff,
nbr_hl_x_dpd_l3m_y-nbr_hl_x_dpd_l3m_x AS nbr_hl_x_dpd_l3m_diff,
nbr_hl_x_dpd_cm_y-nbr_hl_x_dpd_cm_x AS nbr_hl_x_dpd_cm_diff,
nbr_hl_30_dpd_l36m_y-nbr_hl_30_dpd_l36m_x AS nbr_hl_30_dpd_l36m_diff,
nbr_hl_30_dpd_l24m_y-nbr_hl_30_dpd_l24m_x AS nbr_hl_30_dpd_l24m_diff,
nbr_hl_30_dpd_l18m_y-nbr_hl_30_dpd_l18m_x AS nbr_hl_30_dpd_l18m_diff,
nbr_hl_30_dpd_l12m_y-nbr_hl_30_dpd_l12m_x AS nbr_hl_30_dpd_l12m_diff,
nbr_hl_30_dpd_l6m_y-nbr_hl_30_dpd_l6m_x AS nbr_hl_30_dpd_l6m_diff,
nbr_hl_30_dpd_l3m_y-nbr_hl_30_dpd_l3m_x AS nbr_hl_30_dpd_l3m_diff,
nbr_hl_30_dpd_cm_y-nbr_hl_30_dpd_cm_x AS nbr_hl_30_dpd_cm_diff,
nbr_hl_90_dpd_l36m_y-nbr_hl_90_dpd_l36m_x AS nbr_hl_90_dpd_l36m_diff,
nbr_hl_90_dpd_l24m_y-nbr_hl_90_dpd_l24m_x AS nbr_hl_90_dpd_l24m_diff,
nbr_hl_90_dpd_l18m_y-nbr_hl_90_dpd_l18m_x AS nbr_hl_90_dpd_l18m_diff,
nbr_hl_90_dpd_l12m_y-nbr_hl_90_dpd_l12m_x AS nbr_hl_90_dpd_l12m_diff,
nbr_hl_90_dpd_l6m_y-nbr_hl_90_dpd_l6m_x AS nbr_hl_90_dpd_l6m_diff,
nbr_hl_90_dpd_l3m_y-nbr_hl_90_dpd_l3m_x AS nbr_hl_90_dpd_l3m_diff,
nbr_hl_90_dpd_cm_y-nbr_hl_90_dpd_cm_x AS nbr_hl_90_dpd_cm_diff,
nbr_active_hl_x_dpd_l36m_y-nbr_active_hl_x_dpd_l36m_x AS nbr_active_hl_x_dpd_l36m_diff,
nbr_active_hl_x_dpd_l24m_y-nbr_active_hl_x_dpd_l24m_x AS nbr_active_hl_x_dpd_l24m_diff,
nbr_active_hl_x_dpd_l18m_y-nbr_active_hl_x_dpd_l18m_x AS nbr_active_hl_x_dpd_l18m_diff,
nbr_active_hl_x_dpd_l12m_y-nbr_active_hl_x_dpd_l12m_x AS nbr_active_hl_x_dpd_l12m_diff,
nbr_active_hl_x_dpd_l6m_y-nbr_active_hl_x_dpd_l6m_x AS nbr_active_hl_x_dpd_l6m_diff,
nbr_active_hl_x_dpd_l3m_y-nbr_active_hl_x_dpd_l3m_x AS nbr_active_hl_x_dpd_l3m_diff,
nbr_active_hl_x_dpd_cm_y-nbr_active_hl_x_dpd_cm_x AS nbr_active_hl_x_dpd_cm_diff,
nbr_active_hl_30_dpd_l36m_y-nbr_active_hl_30_dpd_l36m_x AS nbr_active_hl_30_dpd_l36m_diff,
nbr_active_hl_30_dpd_l24m_y-nbr_active_hl_30_dpd_l24m_x AS nbr_active_hl_30_dpd_l24m_diff,
nbr_active_hl_30_dpd_l18m_y-nbr_active_hl_30_dpd_l18m_x AS nbr_active_hl_30_dpd_l18m_diff,
nbr_active_hl_30_dpd_l12m_y-nbr_active_hl_30_dpd_l12m_x AS nbr_active_hl_30_dpd_l12m_diff,
nbr_active_hl_30_dpd_l6m_y-nbr_active_hl_30_dpd_l6m_x AS nbr_active_hl_30_dpd_l6m_diff,
nbr_active_hl_30_dpd_l3m_y-nbr_active_hl_30_dpd_l3m_x AS nbr_active_hl_30_dpd_l3m_diff,
nbr_active_hl_30_dpd_cm_y-nbr_active_hl_30_dpd_cm_x AS nbr_active_hl_30_dpd_cm_diff,
nbr_hl_active_90_dpd_l36m_y-nbr_hl_active_90_dpd_l36m_x AS nbr_hl_active_90_dpd_l36m_diff,
nbr_hl_active_90_dpd_l24m_y-nbr_hl_active_90_dpd_l24m_x AS nbr_hl_active_90_dpd_l24m_diff,
nbr_hl_active_90_dpd_l18m_y-nbr_hl_active_90_dpd_l18m_x AS nbr_hl_active_90_dpd_l18m_diff,
nbr_hl_active_90_dpd_l12m_y-nbr_hl_active_90_dpd_l12m_x AS nbr_hl_active_90_dpd_l12m_diff,
nbr_hl_active_90_dpd_l6m_y-nbr_hl_active_90_dpd_l6m_x AS nbr_hl_active_90_dpd_l6m_diff,
nbr_hl_active_90_dpd_l3m_y-nbr_hl_active_90_dpd_l3m_x AS nbr_hl_active_90_dpd_l3m_diff,
nbr_hl_active_90_dpd_cm_y-nbr_hl_active_90_dpd_cm_x AS nbr_hl_active_90_dpd_cm_diff,
nbr_tw_x_dpd_l36m_y-nbr_tw_x_dpd_l36m_x AS nbr_tw_x_dpd_l36m_diff,
nbr_tw_x_dpd_l24m_y-nbr_tw_x_dpd_l24m_x AS nbr_tw_x_dpd_l24m_diff,
nbr_tw_x_dpd_l18m_y-nbr_tw_x_dpd_l18m_x AS nbr_tw_x_dpd_l18m_diff,
nbr_tw_x_dpd_l12m_y-nbr_tw_x_dpd_l12m_x AS nbr_tw_x_dpd_l12m_diff,
nbr_tw_x_dpd_l6m_y-nbr_tw_x_dpd_l6m_x AS nbr_tw_x_dpd_l6m_diff,
nbr_tw_x_dpd_l3m_y-nbr_tw_x_dpd_l3m_x AS nbr_tw_x_dpd_l3m_diff,
nbr_tw_x_dpd_cm_y-nbr_tw_x_dpd_cm_x AS nbr_tw_x_dpd_cm_diff,
nbr_tw_30_dpd_l36m_y-nbr_tw_30_dpd_l36m_x AS nbr_tw_30_dpd_l36m_diff,
nbr_tw_30_dpd_l24m_y-nbr_tw_30_dpd_l24m_x AS nbr_tw_30_dpd_l24m_diff,
nbr_tw_30_dpd_l18m_y-nbr_tw_30_dpd_l18m_x AS nbr_tw_30_dpd_l18m_diff,
nbr_tw_30_dpd_l12m_y-nbr_tw_30_dpd_l12m_x AS nbr_tw_30_dpd_l12m_diff,
nbr_tw_30_dpd_l6m_y-nbr_tw_30_dpd_l6m_x AS nbr_tw_30_dpd_l6m_diff,
nbr_tw_30_dpd_l3m_y-nbr_tw_30_dpd_l3m_x AS nbr_tw_30_dpd_l3m_diff,
nbr_tw_30_dpd_cm_y-nbr_tw_30_dpd_cm_x AS nbr_tw_30_dpd_cm_diff,
nbr_tw_90_dpd_l36m_y-nbr_tw_90_dpd_l36m_x AS nbr_tw_90_dpd_l36m_diff,
nbr_tw_90_dpd_l24m_y-nbr_tw_90_dpd_l24m_x AS nbr_tw_90_dpd_l24m_diff,
nbr_tw_90_dpd_l18m_y-nbr_tw_90_dpd_l18m_x AS nbr_tw_90_dpd_l18m_diff,
nbr_tw_90_dpd_l12m_y-nbr_tw_90_dpd_l12m_x AS nbr_tw_90_dpd_l12m_diff,
nbr_tw_90_dpd_l6m_y-nbr_tw_90_dpd_l6m_x AS nbr_tw_90_dpd_l6m_diff,
nbr_tw_90_dpd_l3m_y-nbr_tw_90_dpd_l3m_x AS nbr_tw_90_dpd_l3m_diff,
nbr_tw_90_dpd_cm_y-nbr_tw_90_dpd_cm_x AS nbr_tw_90_dpd_cm_diff,
nbr_active_tw_x_dpd_l36m_y-nbr_active_tw_x_dpd_l36m_x AS nbr_active_tw_x_dpd_l36m_diff,
nbr_active_tw_x_dpd_l24m_y-nbr_active_tw_x_dpd_l24m_x AS nbr_active_tw_x_dpd_l24m_diff,
nbr_active_tw_x_dpd_l18m_y-nbr_active_tw_x_dpd_l18m_x AS nbr_active_tw_x_dpd_l18m_diff,
nbr_active_tw_x_dpd_l12m_y-nbr_active_tw_x_dpd_l12m_x AS nbr_active_tw_x_dpd_l12m_diff,
nbr_active_tw_x_dpd_l6m_y-nbr_active_tw_x_dpd_l6m_x AS nbr_active_tw_x_dpd_l6m_diff,
nbr_active_tw_x_dpd_l3m_y-nbr_active_tw_x_dpd_l3m_x AS nbr_active_tw_x_dpd_l3m_diff,
nbr_active_tw_x_dpd_cm_y-nbr_active_tw_x_dpd_cm_x AS nbr_active_tw_x_dpd_cm_diff,
nbr_active_tw_30_dpd_l36m_y-nbr_active_tw_30_dpd_l36m_x AS nbr_active_tw_30_dpd_l36m_diff,
nbr_active_tw_30_dpd_l24m_y-nbr_active_tw_30_dpd_l24m_x AS nbr_active_tw_30_dpd_l24m_diff,
nbr_active_tw_30_dpd_l18m_y-nbr_active_tw_30_dpd_l18m_x AS nbr_active_tw_30_dpd_l18m_diff,
nbr_active_tw_30_dpd_l12m_y-nbr_active_tw_30_dpd_l12m_x AS nbr_active_tw_30_dpd_l12m_diff,
nbr_active_tw_30_dpd_l6m_y-nbr_active_tw_30_dpd_l6m_x AS nbr_active_tw_30_dpd_l6m_diff,
nbr_active_tw_30_dpd_l3m_y-nbr_active_tw_30_dpd_l3m_x AS nbr_active_tw_30_dpd_l3m_diff,
nbr_active_tw_30_dpd_cm_y-nbr_active_tw_30_dpd_cm_x AS nbr_active_tw_30_dpd_cm_diff,
nbr_tw_active_90_dpd_l36m_y-nbr_tw_active_90_dpd_l36m_x AS nbr_tw_active_90_dpd_l36m_diff,
nbr_tw_active_90_dpd_l24m_y-nbr_tw_active_90_dpd_l24m_x AS nbr_tw_active_90_dpd_l24m_diff,
nbr_tw_active_90_dpd_l18m_y-nbr_tw_active_90_dpd_l18m_x AS nbr_tw_active_90_dpd_l18m_diff,
nbr_tw_active_90_dpd_l12m_y-nbr_tw_active_90_dpd_l12m_x AS nbr_tw_active_90_dpd_l12m_diff,
nbr_tw_active_90_dpd_l6m_y-nbr_tw_active_90_dpd_l6m_x AS nbr_tw_active_90_dpd_l6m_diff,
nbr_tw_active_90_dpd_l3m_y-nbr_tw_active_90_dpd_l3m_x AS nbr_tw_active_90_dpd_l3m_diff,
nbr_tw_active_90_dpd_cm_y-nbr_tw_active_90_dpd_cm_x AS nbr_tw_active_90_dpd_cm_diff,
nbr_unsecured_x_dpd_l36m_y-nbr_unsecured_x_dpd_l36m_x AS nbr_unsecured_x_dpd_l36m_diff,
nbr_unsecured_x_dpd_l24m_y-nbr_unsecured_x_dpd_l24m_x AS nbr_unsecured_x_dpd_l24m_diff,
nbr_unsecured_x_dpd_l18m_y-nbr_unsecured_x_dpd_l18m_x AS nbr_unsecured_x_dpd_l18m_diff,
nbr_unsecured_x_dpd_l12m_y-nbr_unsecured_x_dpd_l12m_x AS nbr_unsecured_x_dpd_l12m_diff,
nbr_unsecured_x_dpd_l6m_y-nbr_unsecured_x_dpd_l6m_x AS nbr_unsecured_x_dpd_l6m_diff,
nbr_unsecured_x_dpd_l3m_y-nbr_unsecured_x_dpd_l3m_x AS nbr_unsecured_x_dpd_l3m_diff,
nbr_unsecured_x_dpd_cm_y-nbr_unsecured_x_dpd_cm_x AS nbr_unsecured_x_dpd_cm_diff,
nbr_unsecured_30_dpd_l36m_y-nbr_unsecured_30_dpd_l36m_x AS nbr_unsecured_30_dpd_l36m_diff,
nbr_unsecured_30_dpd_l24m_y-nbr_unsecured_30_dpd_l24m_x AS nbr_unsecured_30_dpd_l24m_diff,
nbr_unsecured_30_dpd_l18m_y-nbr_unsecured_30_dpd_l18m_x AS nbr_unsecured_30_dpd_l18m_diff,
nbr_unsecured_30_dpd_l12m_y-nbr_unsecured_30_dpd_l12m_x AS nbr_unsecured_30_dpd_l12m_diff,
nbr_unsecured_30_dpd_l6m_y-nbr_unsecured_30_dpd_l6m_x AS nbr_unsecured_30_dpd_l6m_diff,
nbr_unsecured_30_dpd_l3m_y-nbr_unsecured_30_dpd_l3m_x AS nbr_unsecured_30_dpd_l3m_diff,
nbr_unsecured_30_dpd_cm_y-nbr_unsecured_30_dpd_cm_x AS nbr_unsecured_30_dpd_cm_diff,
nbr_unsecured_90_dpd_l36m_y-nbr_unsecured_90_dpd_l36m_x AS nbr_unsecured_90_dpd_l36m_diff,
nbr_unsecured_90_dpd_l24m_y-nbr_unsecured_90_dpd_l24m_x AS nbr_unsecured_90_dpd_l24m_diff,
nbr_unsecured_90_dpd_l18m_y-nbr_unsecured_90_dpd_l18m_x AS nbr_unsecured_90_dpd_l18m_diff,
nbr_unsecured_90_dpd_l12m_y-nbr_unsecured_90_dpd_l12m_x AS nbr_unsecured_90_dpd_l12m_diff,
nbr_unsecured_90_dpd_l6m_y-nbr_unsecured_90_dpd_l6m_x AS nbr_unsecured_90_dpd_l6m_diff,
nbr_unsecured_90_dpd_l3m_y-nbr_unsecured_90_dpd_l3m_x AS nbr_unsecured_90_dpd_l3m_diff,
nbr_unsecured_90_dpd_cm_y-nbr_unsecured_90_dpd_cm_x AS nbr_unsecured_90_dpd_cm_diff,
nbr_active_unsecured_x_dpd_l36m_y-nbr_active_unsecured_x_dpd_l36m_x AS nbr_active_unsecured_x_dpd_l36m_diff,
nbr_active_unsecured_x_dpd_l24m_y-nbr_active_unsecured_x_dpd_l24m_x AS nbr_active_unsecured_x_dpd_l24m_diff,
nbr_active_unsecured_x_dpd_l18m_y-nbr_active_unsecured_x_dpd_l18m_x AS nbr_active_unsecured_x_dpd_l18m_diff,
nbr_active_unsecured_x_dpd_l12m_y-nbr_active_unsecured_x_dpd_l12m_x AS nbr_active_unsecured_x_dpd_l12m_diff,
nbr_active_unsecured_x_dpd_l6m_y-nbr_active_unsecured_x_dpd_l6m_x AS nbr_active_unsecured_x_dpd_l6m_diff,
nbr_active_unsecured_x_dpd_l3m_y-nbr_active_unsecured_x_dpd_l3m_x AS nbr_active_unsecured_x_dpd_l3m_diff,
nbr_active_unsecured_x_dpd_cm_y-nbr_active_unsecured_x_dpd_cm_x AS nbr_active_unsecured_x_dpd_cm_diff,
nbr_active_unsecured_30_dpd_l36m_y-nbr_active_unsecured_30_dpd_l36m_x AS nbr_active_unsecured_30_dpd_l36m_diff,
nbr_active_unsecured_30_dpd_l24m_y-nbr_active_unsecured_30_dpd_l24m_x AS nbr_active_unsecured_30_dpd_l24m_diff,
nbr_active_unsecured_30_dpd_l18m_y-nbr_active_unsecured_30_dpd_l18m_x AS nbr_active_unsecured_30_dpd_l18m_diff,
nbr_active_unsecured_30_dpd_l12m_y-nbr_active_unsecured_30_dpd_l12m_x AS nbr_active_unsecured_30_dpd_l12m_diff,
nbr_active_unsecured_30_dpd_l6m_y-nbr_active_unsecured_30_dpd_l6m_x AS nbr_active_unsecured_30_dpd_l6m_diff,
nbr_active_unsecured_30_dpd_l3m_y-nbr_active_unsecured_30_dpd_l3m_x AS nbr_active_unsecured_30_dpd_l3m_diff,
nbr_active_unsecured_30_dpd_cm_y-nbr_active_unsecured_30_dpd_cm_x AS nbr_active_unsecured_30_dpd_cm_diff,
nbr_unsecured_active_90_dpd_l36m_y-nbr_unsecured_active_90_dpd_l36m_x AS nbr_unsecured_active_90_dpd_l36m_diff,
nbr_unsecured_active_90_dpd_l24m_y-nbr_unsecured_active_90_dpd_l24m_x AS nbr_unsecured_active_90_dpd_l24m_diff,
nbr_unsecured_active_90_dpd_l18m_y-nbr_unsecured_active_90_dpd_l18m_x AS nbr_unsecured_active_90_dpd_l18m_diff,
nbr_unsecured_active_90_dpd_l12m_y-nbr_unsecured_active_90_dpd_l12m_x AS nbr_unsecured_active_90_dpd_l12m_diff,
nbr_unsecured_active_90_dpd_l6m_y-nbr_unsecured_active_90_dpd_l6m_x AS nbr_unsecured_active_90_dpd_l6m_diff,
nbr_unsecured_active_90_dpd_l3m_y-nbr_unsecured_active_90_dpd_l3m_x AS nbr_unsecured_active_90_dpd_l3m_diff,
nbr_unsecured_active_90_dpd_cm_y-nbr_unsecured_active_90_dpd_cm_x AS nbr_unsecured_active_90_dpd_cm_diff,
nbr_secured_x_dpd_l36m_y-nbr_secured_x_dpd_l36m_x AS nbr_secured_x_dpd_l36m_diff,
nbr_secured_x_dpd_l24m_y-nbr_secured_x_dpd_l24m_x AS nbr_secured_x_dpd_l24m_diff,
nbr_secured_x_dpd_l18m_y-nbr_secured_x_dpd_l18m_x AS nbr_secured_x_dpd_l18m_diff,
nbr_secured_x_dpd_l12m_y-nbr_secured_x_dpd_l12m_x AS nbr_secured_x_dpd_l12m_diff,
nbr_secured_x_dpd_l6m_y-nbr_secured_x_dpd_l6m_x AS nbr_secured_x_dpd_l6m_diff,
nbr_secured_x_dpd_l3m_y-nbr_secured_x_dpd_l3m_x AS nbr_secured_x_dpd_l3m_diff,
nbr_secured_x_dpd_cm_y-nbr_secured_x_dpd_cm_x AS nbr_secured_x_dpd_cm_diff,
nbr_secured_30_dpd_l36m_y-nbr_secured_30_dpd_l36m_x AS nbr_secured_30_dpd_l36m_diff,
nbr_secured_30_dpd_l24m_y-nbr_secured_30_dpd_l24m_x AS nbr_secured_30_dpd_l24m_diff,
nbr_secured_30_dpd_l18m_y-nbr_secured_30_dpd_l18m_x AS nbr_secured_30_dpd_l18m_diff,
nbr_secured_30_dpd_l12m_y-nbr_secured_30_dpd_l12m_x AS nbr_secured_30_dpd_l12m_diff,
nbr_secured_30_dpd_l6m_y-nbr_secured_30_dpd_l6m_x AS nbr_secured_30_dpd_l6m_diff,
nbr_secured_30_dpd_l3m_y-nbr_secured_30_dpd_l3m_x AS nbr_secured_30_dpd_l3m_diff,
nbr_secured_30_dpd_cm_y-nbr_secured_30_dpd_cm_x AS nbr_secured_30_dpd_cm_diff,
nbr_secured_90_dpd_l36m_y-nbr_secured_90_dpd_l36m_x AS nbr_secured_90_dpd_l36m_diff,
nbr_secured_90_dpd_l24m_y-nbr_secured_90_dpd_l24m_x AS nbr_secured_90_dpd_l24m_diff,
nbr_secured_90_dpd_l18m_y-nbr_secured_90_dpd_l18m_x AS nbr_secured_90_dpd_l18m_diff,
nbr_secured_90_dpd_l12m_y-nbr_secured_90_dpd_l12m_x AS nbr_secured_90_dpd_l12m_diff,
nbr_secured_90_dpd_l6m_y-nbr_secured_90_dpd_l6m_x AS nbr_secured_90_dpd_l6m_diff,
nbr_secured_90_dpd_l3m_y-nbr_secured_90_dpd_l3m_x AS nbr_secured_90_dpd_l3m_diff,
nbr_secured_90_dpd_cm_y-nbr_secured_90_dpd_cm_x AS nbr_secured_90_dpd_cm_diff,
nbr_active_secured_x_dpd_l36m_y-nbr_active_secured_x_dpd_l36m_x AS nbr_active_secured_x_dpd_l36m_diff,
nbr_active_secured_x_dpd_l24m_y-nbr_active_secured_x_dpd_l24m_x AS nbr_active_secured_x_dpd_l24m_diff,
nbr_active_secured_x_dpd_l18m_y-nbr_active_secured_x_dpd_l18m_x AS nbr_active_secured_x_dpd_l18m_diff,
nbr_active_secured_x_dpd_l12m_y-nbr_active_secured_x_dpd_l12m_x AS nbr_active_secured_x_dpd_l12m_diff,
nbr_active_secured_x_dpd_l6m_y-nbr_active_secured_x_dpd_l6m_x AS nbr_active_secured_x_dpd_l6m_diff,
nbr_active_secured_x_dpd_l3m_y-nbr_active_secured_x_dpd_l3m_x AS nbr_active_secured_x_dpd_l3m_diff,
nbr_active_secured_x_dpd_cm_y-nbr_active_secured_x_dpd_cm_x AS nbr_active_secured_x_dpd_cm_diff,
nbr_active_secured_30_dpd_l36m_y-nbr_active_secured_30_dpd_l36m_x AS nbr_active_secured_30_dpd_l36m_diff,
nbr_active_secured_30_dpd_l24m_y-nbr_active_secured_30_dpd_l24m_x AS nbr_active_secured_30_dpd_l24m_diff,
nbr_active_secured_30_dpd_l18m_y-nbr_active_secured_30_dpd_l18m_x AS nbr_active_secured_30_dpd_l18m_diff,
nbr_active_secured_30_dpd_l12m_y-nbr_active_secured_30_dpd_l12m_x AS nbr_active_secured_30_dpd_l12m_diff,
nbr_active_secured_30_dpd_l6m_y-nbr_active_secured_30_dpd_l6m_x AS nbr_active_secured_30_dpd_l6m_diff,
nbr_active_secured_30_dpd_l3m_y-nbr_active_secured_30_dpd_l3m_x AS nbr_active_secured_30_dpd_l3m_diff,
nbr_active_secured_30_dpd_cm_y-nbr_active_secured_30_dpd_cm_x AS nbr_active_secured_30_dpd_cm_diff,
nbr_secured_active_90_dpd_l36m_y-nbr_secured_active_90_dpd_l36m_x AS nbr_secured_active_90_dpd_l36m_diff,
nbr_secured_active_90_dpd_l24m_y-nbr_secured_active_90_dpd_l24m_x AS nbr_secured_active_90_dpd_l24m_diff,
nbr_secured_active_90_dpd_l18m_y-nbr_secured_active_90_dpd_l18m_x AS nbr_secured_active_90_dpd_l18m_diff,
nbr_secured_active_90_dpd_l12m_y-nbr_secured_active_90_dpd_l12m_x AS nbr_secured_active_90_dpd_l12m_diff,
nbr_secured_active_90_dpd_l6m_y-nbr_secured_active_90_dpd_l6m_x AS nbr_secured_active_90_dpd_l6m_diff,
nbr_secured_active_90_dpd_l3m_y-nbr_secured_active_90_dpd_l3m_x AS nbr_secured_active_90_dpd_l3m_diff,
nbr_secured_active_90_dpd_cm_y-nbr_secured_active_90_dpd_cm_x AS nbr_secured_active_90_dpd_cm_diff,
max_sanctioned_amount_y-max_sanctioned_amount_x AS max_sanctioned_amount_diff,
total_sanctioned_amount_y-total_sanctioned_amount_x AS total_sanctioned_amount_diff,
secured_max_sanctioned_amount_y-secured_max_sanctioned_amount_x AS secured_max_sanctioned_amount_diff,
secured_total_sanctioned_amount_y-secured_total_sanctioned_amount_x AS secured_total_sanctioned_amount_diff,
unsecured_max_sanctioned_amount_y-unsecured_max_sanctioned_amount_x AS unsecured_max_sanctioned_amount_diff,
unsecured_total_sanctioned_amount_y-unsecured_total_sanctioned_amount_x AS unsecured_total_sanctioned_amount_diff,
pl_max_sanctioned_amount_y-pl_max_sanctioned_amount_x AS pl_max_sanctioned_amount_diff,
pl_total_sanctioned_amount_y-pl_total_sanctioned_amount_x AS pl_total_sanctioned_amount_diff,
hl_max_sanctioned_amount_y-hl_max_sanctioned_amount_x AS hl_max_sanctioned_amount_diff,
hl_total_sanctioned_amount_y-hl_total_sanctioned_amount_x AS hl_total_sanctioned_amount_diff,
cc_max_sanctioned_amount_y-cc_max_sanctioned_amount_x AS cc_max_sanctioned_amount_diff,
cc_total_sanctioned_amount_y-cc_total_sanctioned_amount_x AS cc_total_sanctioned_amount_diff,
cd_max_sanctioned_amount_y-cd_max_sanctioned_amount_x AS cd_max_sanctioned_amount_diff,
cd_total_sanctioned_amount_y-cd_total_sanctioned_amount_x AS cd_total_sanctioned_amount_diff,
al_max_sanctioned_amount_y-al_max_sanctioned_amount_x AS al_max_sanctioned_amount_diff,
al_total_sanctioned_amount_y-al_total_sanctioned_amount_x AS al_total_sanctioned_amount_diff,
gl_max_sanctioned_amount_y-gl_max_sanctioned_amount_x AS gl_max_sanctioned_amount_diff,
gl_total_sanctioned_amount_y-gl_total_sanctioned_amount_x AS gl_total_sanctioned_amount_diff,
tw_max_sanctioned_amount_y-tw_max_sanctioned_amount_x AS tw_max_sanctioned_amount_diff,
tw_total_sanctioned_amount_y-tw_total_sanctioned_amount_x AS tw_total_sanctioned_amount_diff,
max_sanctioned_amount_l3m_y-max_sanctioned_amount_l3m_x AS max_sanctioned_amount_l3m_diff,
total_sanctioned_amount_l3m_y-total_sanctioned_amount_l3m_x AS total_sanctioned_amount_l3m_diff,
secured_max_sanctioned_amount_l3m_y-secured_max_sanctioned_amount_l3m_x AS secured_max_sanctioned_amount_l3m_diff,
secured_total_sanctioned_amount_l3m_y-secured_total_sanctioned_amount_l3m_x AS secured_total_sanctioned_amount_l3m_diff,
unsecured_max_sanctioned_amount_l3m_y-unsecured_max_sanctioned_amount_l3m_x AS unsecured_max_sanctioned_amount_l3m_diff,
unsecured_total_sanctioned_amount_l3m_y-unsecured_total_sanctioned_amount_l3m_x AS unsecured_total_sanctioned_amount_l3m_diff,
pl_max_sanctioned_amount_l3m_y-pl_max_sanctioned_amount_l3m_x AS pl_max_sanctioned_amount_l3m_diff,
pl_total_sanctioned_amount_l3m_y-pl_total_sanctioned_amount_l3m_x AS pl_total_sanctioned_amount_l3m_diff,
hl_max_sanctioned_amount_l3m_y-hl_max_sanctioned_amount_l3m_x AS hl_max_sanctioned_amount_l3m_diff,
hl_total_sanctioned_amount_l3m_y-hl_total_sanctioned_amount_l3m_x AS hl_total_sanctioned_amount_l3m_diff,
cc_max_sanctioned_amount_l3m_y-cc_max_sanctioned_amount_l3m_x AS cc_max_sanctioned_amount_l3m_diff,
cc_total_sanctioned_amount_l3m_y-cc_total_sanctioned_amount_l3m_x AS cc_total_sanctioned_amount_l3m_diff,
cd_max_sanctioned_amount_l3m_y-cd_max_sanctioned_amount_l3m_x AS cd_max_sanctioned_amount_l3m_diff,
cd_total_sanctioned_amount_l3m_y-cd_total_sanctioned_amount_l3m_x AS cd_total_sanctioned_amount_l3m_diff,
al_max_sanctioned_amount_l3m_y-al_max_sanctioned_amount_l3m_x AS al_max_sanctioned_amount_l3m_diff,
al_total_sanctioned_amount_l3m_y-al_total_sanctioned_amount_l3m_x AS al_total_sanctioned_amount_l3m_diff,
gl_max_sanctioned_amount_l3m_y-gl_max_sanctioned_amount_l3m_x AS gl_max_sanctioned_amount_l3m_diff,
gl_total_sanctioned_amount_l3m_y-gl_total_sanctioned_amount_l3m_x AS gl_total_sanctioned_amount_l3m_diff,
tw_max_sanctioned_amount_l3m_y-tw_max_sanctioned_amount_l3m_x AS tw_max_sanctioned_amount_l3m_diff,
tw_total_sanctioned_amount_l3m_y-tw_total_sanctioned_amount_l3m_x AS tw_total_sanctioned_amount_l3m_diff,
max_sanctioned_amount_l6m_y-max_sanctioned_amount_l6m_x AS max_sanctioned_amount_l6m_diff,
total_sanctioned_amount_l6m_y-total_sanctioned_amount_l6m_x AS total_sanctioned_amount_l6m_diff,
secured_max_sanctioned_amount_l6m_y-secured_max_sanctioned_amount_l6m_x AS secured_max_sanctioned_amount_l6m_diff,
secured_total_sanctioned_amount_l6m_y-secured_total_sanctioned_amount_l6m_x AS secured_total_sanctioned_amount_l6m_diff,
unsecured_max_sanctioned_amount_l6m_y-unsecured_max_sanctioned_amount_l6m_x AS unsecured_max_sanctioned_amount_l6m_diff,
unsecured_total_sanctioned_amount_l6m_y-unsecured_total_sanctioned_amount_l6m_x AS unsecured_total_sanctioned_amount_l6m_diff,
pl_max_sanctioned_amount_l6m_y-pl_max_sanctioned_amount_l6m_x AS pl_max_sanctioned_amount_l6m_diff,
pl_total_sanctioned_amount_l6m_y-pl_total_sanctioned_amount_l6m_x AS pl_total_sanctioned_amount_l6m_diff,
hl_max_sanctioned_amount_l6m_y-hl_max_sanctioned_amount_l6m_x AS hl_max_sanctioned_amount_l6m_diff,
hl_total_sanctioned_amount_l6m_y-hl_total_sanctioned_amount_l6m_x AS hl_total_sanctioned_amount_l6m_diff,
cc_max_sanctioned_amount_l6m_y-cc_max_sanctioned_amount_l6m_x AS cc_max_sanctioned_amount_l6m_diff,
cc_total_sanctioned_amount_l6m_y-cc_total_sanctioned_amount_l6m_x AS cc_total_sanctioned_amount_l6m_diff,
cd_max_sanctioned_amount_l6m_y-cd_max_sanctioned_amount_l6m_x AS cd_max_sanctioned_amount_l6m_diff,
cd_total_sanctioned_amount_l6m_y-cd_total_sanctioned_amount_l6m_x AS cd_total_sanctioned_amount_l6m_diff,
al_max_sanctioned_amount_l6m_y-al_max_sanctioned_amount_l6m_x AS al_max_sanctioned_amount_l6m_diff,
al_total_sanctioned_amount_l6m_y-al_total_sanctioned_amount_l6m_x AS al_total_sanctioned_amount_l6m_diff,
gl_max_sanctioned_amount_l6m_y-gl_max_sanctioned_amount_l6m_x AS gl_max_sanctioned_amount_l6m_diff,
gl_total_sanctioned_amount_l6m_y-gl_total_sanctioned_amount_l6m_x AS gl_total_sanctioned_amount_l6m_diff,
tw_max_sanctioned_amount_l6m_y-tw_max_sanctioned_amount_l6m_x AS tw_max_sanctioned_amount_l6m_diff,
tw_total_sanctioned_amount_l6m_y-tw_total_sanctioned_amount_l6m_x AS tw_total_sanctioned_amount_l6m_diff,
max_sanctioned_amount_l12m_y-max_sanctioned_amount_l12m_x AS max_sanctioned_amount_l12m_diff,
total_sanctioned_amount_l12m_y-total_sanctioned_amount_l12m_x AS total_sanctioned_amount_l12m_diff,
secured_max_sanctioned_amount_l12m_y-secured_max_sanctioned_amount_l12m_x AS secured_max_sanctioned_amount_l12m_diff,
secured_total_sanctioned_amount_l12m_y-secured_total_sanctioned_amount_l12m_x AS secured_total_sanctioned_amount_l12m_diff,
unsecured_max_sanctioned_amount_l12m_y-unsecured_max_sanctioned_amount_l12m_x AS unsecured_max_sanctioned_amount_l12m_diff,
unsecured_total_sanctioned_amount_l12m_y-unsecured_total_sanctioned_amount_l12m_x AS unsecured_total_sanctioned_amount_l12m_diff,
pl_max_sanctioned_amount_l12m_y-pl_max_sanctioned_amount_l12m_x AS pl_max_sanctioned_amount_l12m_diff,
pl_total_sanctioned_amount_l12m_y-pl_total_sanctioned_amount_l12m_x AS pl_total_sanctioned_amount_l12m_diff,
hl_max_sanctioned_amount_l12m_y-hl_max_sanctioned_amount_l12m_x AS hl_max_sanctioned_amount_l12m_diff,
hl_total_sanctioned_amount_l12m_y-hl_total_sanctioned_amount_l12m_x AS hl_total_sanctioned_amount_l12m_diff,
cc_max_sanctioned_amount_l12m_y-cc_max_sanctioned_amount_l12m_x AS cc_max_sanctioned_amount_l12m_diff,
cc_total_sanctioned_amount_l12m_y-cc_total_sanctioned_amount_l12m_x AS cc_total_sanctioned_amount_l12m_diff,
cd_max_sanctioned_amount_l12m_y-cd_max_sanctioned_amount_l12m_x AS cd_max_sanctioned_amount_l12m_diff,
cd_total_sanctioned_amount_l12m_y-cd_total_sanctioned_amount_l12m_x AS cd_total_sanctioned_amount_l12m_diff,
al_max_sanctioned_amount_l12m_y-al_max_sanctioned_amount_l12m_x AS al_max_sanctioned_amount_l12m_diff,
al_total_sanctioned_amount_l12m_y-al_total_sanctioned_amount_l12m_x AS al_total_sanctioned_amount_l12m_diff,
gl_max_sanctioned_amount_l12m_y-gl_max_sanctioned_amount_l12m_x AS gl_max_sanctioned_amount_l12m_diff,
gl_total_sanctioned_amount_l12m_y-gl_total_sanctioned_amount_l12m_x AS gl_total_sanctioned_amount_l12m_diff,
tw_max_sanctioned_amount_l12m_y-tw_max_sanctioned_amount_l12m_x AS tw_max_sanctioned_amount_l12m_diff,
tw_total_sanctioned_amount_l12m_y-tw_total_sanctioned_amount_l12m_x AS tw_total_sanctioned_amount_l12m_diff,
total_balance_amount_y-total_balance_amount_x AS total_balance_amount_diff,
secured_total_balance_amount_y-secured_total_balance_amount_x AS secured_total_balance_amount_diff,
unsecured_total_balance_amount_y-unsecured_total_balance_amount_x AS unsecured_total_balance_amount_diff,
pl_total_balance_amount_y-pl_total_balance_amount_x AS pl_total_balance_amount_diff,
hl_total_balance_amount_y-hl_total_balance_amount_x AS hl_total_balance_amount_diff,
cc_total_balance_amount_y-cc_total_balance_amount_x AS cc_total_balance_amount_diff,
cd_total_balance_amount_y-cd_total_balance_amount_x AS cd_total_balance_amount_diff,
al_total_balance_amount_y-al_total_balance_amount_x AS al_total_balance_amount_diff,
gl_total_balance_amount_y-gl_total_balance_amount_x AS gl_total_balance_amount_diff,
tw_total_balance_amount_y-tw_total_balance_amount_x AS tw_total_balance_amount_diff,
avg_utilisation_y-avg_utilisation_x AS avg_utilisation_diff,
avg_secured_utilisation_y-avg_secured_utilisation_x AS avg_secured_utilisation_diff,
avg_unsecured_utilisation_y-avg_unsecured_utilisation_x AS avg_unsecured_utilisation_diff,
avg_pl_utilisation_y-avg_pl_utilisation_x AS avg_pl_utilisation_diff,
avg_hl_tilisation_y-avg_hl_tilisation_x AS avg_hl_tilisation_diff,
avg_cc_utilisation_y-avg_cc_utilisation_x AS avg_cc_utilisation_diff,
avg_cd_utilisation_y-avg_cd_utilisation_x AS avg_cd_utilisation_diff,
avg_al_utilisation_y-avg_al_utilisation_x AS avg_al_utilisation_diff,
avg_gl_utilisation_y-avg_gl_utilisation_x AS avg_gl_utilisation_diff,
avg_tw_utilisation_y-avg_tw_utilisation_x AS avg_tw_utilisation_diff,
overall_utilisation_y-overall_utilisation_x AS overall_utilisation_diff,
secured_utilisation_y-secured_utilisation_x AS secured_utilisation_diff,
unsecured_utilisation_y-unsecured_utilisation_x AS unsecured_utilisation_diff,
pl_utilisation_y-pl_utilisation_x AS pl_utilisation_diff,
hl_utilisation_y-hl_utilisation_x AS hl_utilisation_diff,
cc_utilisation_y-cc_utilisation_x AS cc_utilisation_diff,
cd_utilisation_y-cd_utilisation_x AS cd_utilisation_diff,
al_utilisation_y-al_utilisation_x AS al_utilisation_diff,
gl_utilisation_y-gl_utilisation_x AS gl_utilisation_diff,
tw_utilisation_y-tw_utilisation_x AS tw_utilisation_diff,
max_bureau_vintage_y-max_bureau_vintage_x AS max_bureau_vintage_diff,
min_bureau_vintage_y-min_bureau_vintage_x AS min_bureau_vintage_diff,
max_active_bureau_vintage_y-max_active_bureau_vintage_x AS max_active_bureau_vintage_diff,
min_active_bureau_vintage_y-min_active_bureau_vintage_x AS min_active_bureau_vintage_diff,
max_secured_bureau_vintage_y-max_secured_bureau_vintage_x AS max_secured_bureau_vintage_diff,
min_secured_bureau_vintage_y-min_secured_bureau_vintage_x AS min_secured_bureau_vintage_diff,
max_active_secured_bureau_vintage_y-max_active_secured_bureau_vintage_x AS max_active_secured_bureau_vintage_diff,
min_active_secured_bureau_vintage_y-min_active_secured_bureau_vintage_x AS min_active_secured_bureau_vintage_diff,
max_unsecured_bureau_vintage_y-max_unsecured_bureau_vintage_x AS max_unsecured_bureau_vintage_diff,
min_unsecured_bureau_vintage_y-min_unsecured_bureau_vintage_x AS min_unsecured_bureau_vintage_diff,
max_active_unsecured_bureau_vintage_y-max_active_unsecured_bureau_vintage_x AS max_active_unsecured_bureau_vintage_diff,
min_active_unsecured_bureau_vintage_y-min_active_unsecured_bureau_vintage_x AS min_active_unsecured_bureau_vintage_diff,
max_pl_bureau_vintage_y-max_pl_bureau_vintage_x AS max_pl_bureau_vintage_diff,
min_pl_bureau_vintage_y-min_pl_bureau_vintage_x AS min_pl_bureau_vintage_diff,
max_active_pl_bureau_vintage_y-max_active_pl_bureau_vintage_x AS max_active_pl_bureau_vintage_diff,
min_active_pl_bureau_vintage_y-min_active_pl_bureau_vintage_x AS min_active_pl_bureau_vintage_diff,
max_cc_bureau_vintage_y-max_cc_bureau_vintage_x AS max_cc_bureau_vintage_diff,
min_cc_bureau_vintage_y-min_cc_bureau_vintage_x AS min_cc_bureau_vintage_diff,
max_active_cc_bureau_vintage_y-max_active_cc_bureau_vintage_x AS max_active_cc_bureau_vintage_diff,
min_active_cc_bureau_vintage_y-min_active_cc_bureau_vintage_x AS min_active_cc_bureau_vintage_diff,
max_cd_bureau_vintage_y-max_cd_bureau_vintage_x AS max_cd_bureau_vintage_diff,
min_cd_bureau_vintage_y-min_cd_bureau_vintage_x AS min_cd_bureau_vintage_diff,
max_active_cd_bureau_vintage_y-max_active_cd_bureau_vintage_x AS max_active_cd_bureau_vintage_diff,
min_active_cd_bureau_vintage_y-min_active_cd_bureau_vintage_x AS min_active_cd_bureau_vintage_diff,
max_al_bureau_vintage_y-max_al_bureau_vintage_x AS max_al_bureau_vintage_diff,
min_al_bureau_vintage_y-min_al_bureau_vintage_x AS min_al_bureau_vintage_diff,
max_active_al_bureau_vintage_y-max_active_al_bureau_vintage_x AS max_active_al_bureau_vintage_diff,
min_active_al_bureau_vintage_y-min_active_al_bureau_vintage_x AS min_active_al_bureau_vintage_diff,
max_hl_bureau_vintage_y-max_hl_bureau_vintage_x AS max_hl_bureau_vintage_diff,
min_hl_bureau_vintage_y-min_hl_bureau_vintage_x AS min_hl_bureau_vintage_diff,
max_active_hl_bureau_vintage_y-max_active_hl_bureau_vintage_x AS max_active_hl_bureau_vintage_diff,
min_active_hl_bureau_vintage_y-min_active_hl_bureau_vintage_x AS min_active_hl_bureau_vintage_diff,
max_gl_bureau_vintage_y-max_gl_bureau_vintage_x AS max_gl_bureau_vintage_diff,
min_gl_bureau_vintage_y-min_gl_bureau_vintage_x AS min_gl_bureau_vintage_diff,
max_active_gl_bureau_vintage_y-max_active_gl_bureau_vintage_x AS max_active_gl_bureau_vintage_diff,
min_active_gl_bureau_vintage_y-min_active_gl_bureau_vintage_x AS min_active_gl_bureau_vintage_diff,
max_tw_bureau_vintage_y-max_tw_bureau_vintage_x AS max_tw_bureau_vintage_diff,
min_tw_bureau_vintage_y-min_tw_bureau_vintage_x AS min_tw_bureau_vintage_diff,
max_active_tw_bureau_vintage_y-max_active_tw_bureau_vintage_x AS max_active_tw_bureau_vintage_diff,
min_active_tw_bureau_vintage_y-min_active_tw_bureau_vintage_x AS min_active_tw_bureau_vintage_diff,
enq_l1m_y-enq_l1m_x AS enq_l1m_diff,
enq_l2m_y-enq_l2m_x AS enq_l2m_diff,
enq_l3m_y-enq_l3m_x AS enq_l3m_diff,
enq_l6m_y-enq_l6m_x AS enq_l6m_diff,
enq_l12m_y-enq_l12m_x AS enq_l12m_diff,
unsec_inq_l1m_y-unsec_inq_l1m_x AS unsec_inq_l1m_diff,
unsec_inq_l2m_y-unsec_inq_l2m_x AS unsec_inq_l2m_diff,
unsec_inq_l3m_y-unsec_inq_l3m_x AS unsec_inq_l3m_diff,
unsec_inq_l6m_y-unsec_inq_l6m_x AS unsec_inq_l6m_diff,
unsec_inq_l12m_y-unsec_inq_l12m_x AS unsec_inq_l12m_diff,
sec_inq_l1m_y-sec_inq_l1m_x AS sec_inq_l1m_diff,
sec_inq_l2m_y-sec_inq_l2m_x AS sec_inq_l2m_diff,
sec_inq_l3m_y-sec_inq_l3m_x AS sec_inq_l3m_diff,
sec_inq_l6m_y-sec_inq_l6m_x AS sec_inq_l6m_diff,
sec_inq_l12m_y-sec_inq_l12m_x AS sec_inq_l12m_diff,
pl_inq_l1m_y-pl_inq_l1m_x AS pl_inq_l1m_diff,
pl_inq_l2m_y-pl_inq_l2m_x AS pl_inq_l2m_diff,
pl_inq_l3m_y-pl_inq_l3m_x AS pl_inq_l3m_diff,
pl_inq_l6m_y-pl_inq_l6m_x AS pl_inq_l6m_diff,
pl_inq_l12m_y-pl_inq_l12m_x AS pl_inq_l12m_diff,
cc_inq_l1m_y-cc_inq_l1m_x AS cc_inq_l1m_diff,
cc_inq_l2m_y-cc_inq_l2m_x AS cc_inq_l2m_diff,
cc_inq_l3m_y-cc_inq_l3m_x AS cc_inq_l3m_diff,
cc_inq_l6m_y-cc_inq_l6m_x AS cc_inq_l6m_diff,
cc_inq_l12m_y-cc_inq_l12m_x AS cc_inq_l12m_diff,
cd_inq_l1m_y-cd_inq_l1m_x AS cd_inq_l1m_diff,
cd_inq_l2m_y-cd_inq_l2m_x AS cd_inq_l2m_diff,
cd_inq_l3m_y-cd_inq_l3m_x AS cd_inq_l3m_diff,
cd_inq_l6m_y-cd_inq_l6m_x AS cd_inq_l6m_diff,
cd_inq_l12m_y-cd_inq_l12m_x AS cd_inq_l12m_diff,
al_inq_l1m_y-al_inq_l1m_x AS al_inq_l1m_diff,
al_inq_l2m_y-al_inq_l2m_x AS al_inq_l2m_diff,
al_inq_l3m_y-al_inq_l3m_x AS al_inq_l3m_diff,
al_inq_l6m_y-al_inq_l6m_x AS al_inq_l6m_diff,
al_inq_l12m_y-al_inq_l12m_x AS al_inq_l12m_diff,
gl_inq_l1m_y-gl_inq_l1m_x AS gl_inq_l1m_diff,
gl_inq_l2m_y-gl_inq_l2m_x AS gl_inq_l2m_diff,
gl_inq_l3m_y-gl_inq_l3m_x AS gl_inq_l3m_diff,
gl_inq_l6m_y-gl_inq_l6m_x AS gl_inq_l6m_diff,
gl_inq_l12m_y-gl_inq_l12m_x AS gl_inq_l12m_diff,
hl_inq_l1m_y-hl_inq_l1m_x AS hl_inq_l1m_diff,
hl_inq_l2m_y-hl_inq_l2m_x AS hl_inq_l2m_diff,
hl_inq_l3m_y-hl_inq_l3m_x AS hl_inq_l3m_diff,
hl_inq_l6m_y-hl_inq_l6m_x AS hl_inq_l6m_diff,
hl_inq_l12m_y-hl_inq_l12m_x AS hl_inq_l12m_diff,
tw_inq_l1m_y-tw_inq_l1m_x AS tw_inq_l1m_diff,
tw_inq_l2m_y-tw_inq_l2m_x AS tw_inq_l2m_diff,
tw_inq_l3m_y-tw_inq_l3m_x AS tw_inq_l3m_diff,
tw_inq_l6m_y-tw_inq_l6m_x AS tw_inq_l6m_diff,
tw_inq_l12m_y-tw_inq_l12m_x AS tw_inq_l12m_diff,

nbr_ever_accounts_y-nbr_ever_accounts_x AS nbr_ever_accounts_diff,
nbr_active_accounts_y-nbr_active_accounts_x AS nbr_active_accounts_diff,
accts_new_l1m_y-accts_new_l1m_x AS accts_new_l1m_diff,
accts_new_l3m_y-accts_new_l3m_x AS accts_new_l3m_diff,
accts_new_l6m_y-accts_new_l6m_x AS accts_new_l6m_diff,
accts_new_l12m_y-accts_new_l12m_x AS accts_new_l12m_diff,
accts_closed_l1m_y-accts_closed_l1m_x AS accts_closed_l1m_diff,
accts_closed_l3m_y-accts_closed_l3m_x AS accts_closed_l3m_diff,
accts_closed_l6m_y-accts_closed_l6m_x AS accts_closed_l6m_diff,
accts_closed_l12m_y-accts_closed_l12m_x AS accts_closed_l12m_diff,
nbr_ever_secured_accounts_y-nbr_ever_secured_accounts_x AS nbr_ever_secured_accounts_diff,
nbr_active_secured_accounts_y-nbr_active_secured_accounts_x AS nbr_active_secured_accounts_diff,
secured_accts_new_l1m_y-secured_accts_new_l1m_x AS secured_accts_new_l1m_diff,
secured_accts_new_l3m_y-secured_accts_new_l3m_x AS secured_accts_new_l3m_diff,
secured_accts_new_l6m_y-secured_accts_new_l6m_x AS secured_accts_new_l6m_diff,
secured_accts_new_l12m_y-secured_accts_new_l12m_x AS secured_accts_new_l12m_diff,
secured_accts_closed_l1m_y-secured_accts_closed_l1m_x AS secured_accts_closed_l1m_diff,
secured_accts_closed_l3m_y-secured_accts_closed_l3m_x AS secured_accts_closed_l3m_diff,
secured_accts_closed_l6m_y-secured_accts_closed_l6m_x AS secured_accts_closed_l6m_diff,
secured_accts_closed_l12m_y-secured_accts_closed_l12m_x AS secured_accts_closed_l12m_diff,
nbr_ever_unsecured_accounts_y-nbr_ever_unsecured_accounts_x AS nbr_ever_unsecured_accounts_diff,
nbr_active_unsecured_accounts_y-nbr_active_unsecured_accounts_x AS nbr_active_unsecured_accounts_diff,
unsecured_accts_new_l1m_y-unsecured_accts_new_l1m_x AS unsecured_accts_new_l1m_diff,
unsecured_accts_new_l3m_y-unsecured_accts_new_l3m_x AS unsecured_accts_new_l3m_diff,
unsecured_accts_new_l6m_y-unsecured_accts_new_l6m_x AS unsecured_accts_new_l6m_diff,
unsecured_accts_new_l12m_y-unsecured_accts_new_l12m_x AS unsecured_accts_new_l12m_diff,
unsecured_accts_closed_l1m_y-unsecured_accts_closed_l1m_x AS unsecured_accts_closed_l1m_diff,
unsecured_accts_closed_l3m_y-unsecured_accts_closed_l3m_x AS unsecured_accts_closed_l3m_diff,
unsecured_accts_closed_l6m_y-unsecured_accts_closed_l6m_x AS unsecured_accts_closed_l6m_diff,
unsecured_accts_closed_l12m_y-unsecured_accts_closed_l12m_x AS unsecured_accts_closed_l12m_diff,
ever_hl_y-ever_hl_x AS ever_hl_diff,
nbr_ever_hl_y-nbr_ever_hl_x AS nbr_ever_hl_diff,
active_hl_y-active_hl_x AS active_hl_diff,
nbr_active_hl_y-nbr_active_hl_x AS nbr_active_hl_diff,
hl_new_l1m_y-hl_new_l1m_x AS hl_new_l1m_diff,
hl_new_l3m_y-hl_new_l3m_x AS hl_new_l3m_diff,
hl_new_l6m_y-hl_new_l6m_x AS hl_new_l6m_diff,
hl_new_l12m_y-hl_new_l12m_x AS hl_new_l12m_diff,
hl_closed_l1m_y-hl_closed_l1m_x AS hl_closed_l1m_diff,
hl_closed_l3m_y-hl_closed_l3m_x AS hl_closed_l3m_diff,
hl_closed_l6m_y-hl_closed_l6m_x AS hl_closed_l6m_diff,
hl_closed_l12m_y-hl_closed_l12m_x AS hl_closed_l12m_diff,
ever_pl_y-ever_pl_x AS ever_pl_diff,
nbr_ever_pl_y-nbr_ever_pl_x AS nbr_ever_pl_diff,
active_pl_y-active_pl_x AS active_pl_diff,
nbr_active_pl_y-nbr_active_pl_x AS nbr_active_pl_diff,
pl_new_l1m_y-pl_new_l1m_x AS pl_new_l1m_diff,
pl_new_l3m_y-pl_new_l3m_x AS pl_new_l3m_diff,
pl_new_l6m_y-pl_new_l6m_x AS pl_new_l6m_diff,
pl_new_l12m_y-pl_new_l12m_x AS pl_new_l12m_diff,
pl_closed_l1m_y-pl_closed_l1m_x AS pl_closed_l1m_diff,
pl_closed_l3m_y-pl_closed_l3m_x AS pl_closed_l3m_diff,
pl_closed_l6m_y-pl_closed_l6m_x AS pl_closed_l6m_diff,
pl_closed_l12m_y-pl_closed_l12m_x AS pl_closed_l12m_diff,
ever_cc_y-ever_cc_x AS ever_cc_diff,
nbr_ever_cc_y-nbr_ever_cc_x AS nbr_ever_cc_diff,
active_cc_y-active_cc_x AS active_cc_diff,
nbr_active_cc_y-nbr_active_cc_x AS nbr_active_cc_diff,
cc_new_l1m_y-cc_new_l1m_x AS cc_new_l1m_diff,
cc_new_l3m_y-cc_new_l3m_x AS cc_new_l3m_diff,
cc_new_l6m_y-cc_new_l6m_x AS cc_new_l6m_diff,
cc_new_l12m_y-cc_new_l12m_x AS cc_new_l12m_diff,
cc_closed_l1m_y-cc_closed_l1m_x AS cc_closed_l1m_diff,
cc_closed_l3m_y-cc_closed_l3m_x AS cc_closed_l3m_diff,
cc_closed_l6m_y-cc_closed_l6m_x AS cc_closed_l6m_diff,
cc_closed_l12m_y-cc_closed_l12m_x AS cc_closed_l12m_diff,
ever_cd_y-ever_cd_x AS ever_cd_diff,
nbr_ever_cd_y-nbr_ever_cd_x AS nbr_ever_cd_diff,
active_cd_y-active_cd_x AS active_cd_diff,
nbr_active_cd_y-nbr_active_cd_x AS nbr_active_cd_diff,
cd_new_l1m_y-cd_new_l1m_x AS cd_new_l1m_diff,
cd_new_l3m_y-cd_new_l3m_x AS cd_new_l3m_diff,
cd_new_l6m_y-cd_new_l6m_x AS cd_new_l6m_diff,
cd_new_l12m_y-cd_new_l12m_x AS cd_new_l12m_diff,
cd_closed_l1m_y-cd_closed_l1m_x AS cd_closed_l1m_diff,
cd_closed_l3m_y-cd_closed_l3m_x AS cd_closed_l3m_diff,
cd_closed_l6m_y-cd_closed_l6m_x AS cd_closed_l6m_diff,
cd_closed_l12m_y-cd_closed_l12m_x AS cd_closed_l12m_diff,
ever_gl_y-ever_gl_x AS ever_gl_diff,
nbr_ever_gl_y-nbr_ever_gl_x AS nbr_ever_gl_diff,
active_gl_y-active_gl_x AS active_gl_diff,
nbr_active_gl_y-nbr_active_gl_x AS nbr_active_gl_diff,
gl_new_l1m_y-gl_new_l1m_x AS gl_new_l1m_diff,
gl_new_l3m_y-gl_new_l3m_x AS gl_new_l3m_diff,
gl_new_l6m_y-gl_new_l6m_x AS gl_new_l6m_diff,
gl_new_l12m_y-gl_new_l12m_x AS gl_new_l12m_diff,
gl_closed_l1m_y-gl_closed_l1m_x AS gl_closed_l1m_diff,
gl_closed_l3m_y-gl_closed_l3m_x AS gl_closed_l3m_diff,
gl_closed_l6m_y-gl_closed_l6m_x AS gl_closed_l6m_diff,
gl_closed_l12m_y-gl_closed_l12m_x AS gl_closed_l12m_diff,
ever_al_y-ever_al_x AS ever_al_diff,
nbr_ever_al_y-nbr_ever_al_x AS nbr_ever_al_diff,
active_al_y-active_al_x AS active_al_diff,
nbr_active_al_y-nbr_active_al_x AS nbr_active_al_diff,
al_new_l1m_y-al_new_l1m_x AS al_new_l1m_diff,
al_new_l3m_y-al_new_l3m_x AS al_new_l3m_diff,
al_new_l6m_y-al_new_l6m_x AS al_new_l6m_diff,
al_new_l12m_y-al_new_l12m_x AS al_new_l12m_diff,
al_closed_l1m_y-al_closed_l1m_x AS al_closed_l1m_diff,
al_closed_l3m_y-al_closed_l3m_x AS al_closed_l3m_diff,
al_closed_l6m_y-al_closed_l6m_x AS al_closed_l6m_diff,
al_closed_l12m_y-al_closed_l12m_x AS al_closed_l12m_diff,
ever_tw_y-ever_tw_x AS ever_tw_diff,
nbr_ever_tw_y-nbr_ever_tw_x AS nbr_ever_tw_diff,
active_tw_y-active_tw_x AS active_tw_diff,
nbr_active_tw_y-nbr_active_tw_x AS nbr_active_tw_diff,
tw_new_l1m_y-tw_new_l1m_x AS tw_new_l1m_diff,
tw_new_l3m_y-tw_new_l3m_x AS tw_new_l3m_diff,
tw_new_l6m_y-tw_new_l6m_x AS tw_new_l6m_diff,
tw_new_l12m_y-tw_new_l12m_x AS tw_new_l12m_diff,
tw_closed_l1m_y-tw_closed_l1m_x AS tw_closed_l1m_diff,
tw_closed_l3m_y-tw_closed_l3m_x AS tw_closed_l3m_diff,
tw_closed_l6m_y-tw_closed_l6m_x AS tw_closed_l6m_diff,
tw_closed_l12m_y-tw_closed_l12m_x AS tw_closed_l12m_diff,
nbr_ever_stpl_y-nbr_ever_stpl_x AS nbr_ever_stpl_diff,
nbr_active_stpl_y-nbr_active_stpl_x AS nbr_active_stpl_diff,
stpl_new_l1m_y-stpl_new_l1m_x AS stpl_new_l1m_diff,
stpl_new_l3m_y-stpl_new_l3m_x AS stpl_new_l3m_diff,
stpl_new_l6m_y-stpl_new_l6m_x AS stpl_new_l6m_diff,
stpl_new_l12m_y-stpl_new_l12m_x AS stpl_new_l12m_diff,
stpl_closed_l1m_y-stpl_closed_l1m_x AS stpl_closed_l1m_diff,
stpl_closed_l3m_y-stpl_closed_l3m_x AS stpl_closed_l3m_diff,
stpl_closed_l6m_y-stpl_closed_l6m_x AS stpl_closed_l6m_diff,
stpl_closed_l12m_y-stpl_closed_l12m_x AS stpl_closed_l12m_diff,
ever_cv_y-ever_cv_x AS ever_cv_diff,
nbr_ever_cv_y-nbr_ever_cv_x AS nbr_ever_cv_diff,
active_cv_y-active_cv_x AS active_cv_diff,
nbr_active_cv_y-nbr_active_cv_x AS nbr_active_cv_diff,
ever_agri_y-ever_agri_x AS ever_agri_diff,
nbr_ever_agri_y-nbr_ever_agri_x AS nbr_ever_agri_diff,
active_agri_y-active_agri_x AS active_agri_diff,
nbr_active_agri_y-nbr_active_agri_x AS nbr_active_agri_diff,
ever_lap_y-ever_lap_x AS ever_lap_diff,
nbr_ever_lap_y-nbr_ever_lap_x AS nbr_ever_lap_diff,
active_lap_y-active_lap_x AS active_lap_diff,
nbr_active_lap_y-nbr_active_lap_x AS nbr_active_lap_diff,
ever_las_y-ever_las_x AS ever_las_diff,
nbr_ever_las_y-nbr_ever_las_x AS nbr_ever_las_diff,
active_las_y-active_las_x AS active_las_diff,
nbr_active_las_y-nbr_active_las_x AS nbr_active_las_diff
FROM fpl_ds_analytics.staging_tmp_table_os_fow_realtime_testing_m2_m1_data_overall 

""")

Executed!


In [14]:
df = wr.s3.read_parquet(path="s3://fpl-datalake-global/l2/Onescore/Score_simulator/staging_tmp_table_os_fow_realtime_testing_m2_m1_diff_data_overall")



/home/kumar/.local/lib/python3.9/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by promote_options='default'.
  return cls.dispatch_func(func)(*args, **kw)


In [15]:
df = df.drop(columns=[col for col in df.columns if col.endswith('_y')])

In [16]:
df1=df.drop(columns=['customer_no',
                     'creation_date_x',
                     'first_product_x',
                     'latest_product_x'
                    ])

In [17]:
df1.to_csv('temp3.csv', index=False)

In [80]:
gc.collect()

31235

In [81]:
import boto3
import pickle

# Initialize S3 client
s3 = boto3.client('s3')

# Define S3 bucket and key (file path in S3)
bucket_name = 'fpl-datalake-global'
s3_key = 'l2/Onescore/Score_simulator/score_change_or_not_model1.pkl'

# Download the pickle file from S3
try:
    response = s3.get_object(Bucket=bucket_name, Key=s3_key)
    model_str = response['Body'].read()

    # Unpickle the file to retrieve the StandardScaler
    xgb_reg = pickle.loads(model_str)
    
    print("Pickle file successfully loaded from S3.")
    
except Exception as e:
    print(f"Error occurred: {e}")


Pickle file successfully loaded from S3.


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [82]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

predictions = xgb_reg.predict(df1)
predictions_prob=xgb_reg.predict_proba(df1)[:, 1]

# auc_score = roc_auc_score(y, predictions_prob)


In [83]:
df['predicted_change']=predictions
df['predicted_change_prob']=predictions_prob


In [84]:
import boto3
import pickle

# Initialize S3 client
s3 = boto3.client('s3')

# Define S3 bucket and key (file path in S3)
bucket_name = 'fpl-datalake-global'
s3_key = 'l2/Onescore/Score_simulator/score_change_or_not_model_wo_1_5_1.pkl'

# Download the pickle file from S3
try:
    response = s3.get_object(Bucket=bucket_name, Key=s3_key)
    model_str = response['Body'].read()

    # Unpickle the file to retrieve the StandardScaler
    xgb_reg = pickle.loads(model_str)
    
    print("Pickle file successfully loaded from S3.")
    
except Exception as e:
    print(f"Error occurred: {e}")


Pickle file successfully loaded from S3.


In [85]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

predictions1 = xgb_reg.predict(df1)
predictions_prob1=xgb_reg.predict_proba(df1)[:, 1]

# auc_score = roc_auc_score(y, predictions_prob)


In [86]:
df['predicted_change_wo_1_5']=predictions1
df['predicted_change_prob_wo_1_5']=predictions_prob1


In [87]:
import boto3
import pickle

# Initialize S3 client
s3 = boto3.client('s3')

# Define S3 bucket and key (file path in S3)
bucket_name = 'fpl-datalake-global'
s3_key = 'l2/Onescore/Score_simulator/score_direction_model3.pkl'

# Download the pickle file from S3
try:
    response = s3.get_object(Bucket=bucket_name, Key=s3_key)
    model_str = response['Body'].read()

    # Unpickle the file to retrieve the StandardScaler
    xgb_reg = pickle.loads(model_str)
    
    print("Pickle file successfully loaded from S3.")
    
except Exception as e:
    print(f"Error occurred: {e}")


Pickle file successfully loaded from S3.


In [88]:
# Create a SHAP explainer
explainer = shap.TreeExplainer(xgb_reg)
# explainer = shap.Explainer(xgb_reg, df1)

# Get SHAP values
shap_values = explainer(df1)

# Convert to DataFrame for better visualization
shap_df = pd.DataFrame(shap_values.values, columns=df1.columns)

# Display SHAP values
shap_df.head()


,risk_score_x,max_dpd_l36m_x,max_dpd_l24m_x,max_dpd_l18m_x,max_dpd_l12m_x,max_dpd_l6m_x,max_dpd_l3m_x,max_dpd_cm_x,max_active_dpd_l36m_x,max_active_dpd_l24m_x,max_active_dpd_l18m_x,max_active_dpd_l12m_x,max_active_dpd_l6m_x,max_active_dpd_l3m_x,max_active_dpd_cm_x,max_active_cc_dpd_l36m_x,max_active_cc_dpd_l24m_x,max_active_cc_dpd_l18m_x,max_active_cc_dpd_l12m_x,max_active_cc_dpd_l6m_x,max_active_cc_dpd_l3m_x,max_active_cc_dpd_cm_x,max_cc_dpd_l36m_x,max_cc_dpd_l24m_x,max_cc_dpd_l18m_x,max_cc_dpd_l12m_x,max_cc_dpd_l6m_x,max_cc_dpd_l3m_x,max_cc_dpd_cm_x,max_active_pl_dpd_l36m_x,max_active_pl_dpd_l24m_x,max_active_pl_dpd_l18m_x,max_active_pl_dpd_l12m_x,max_active_pl_dpd_l6m_x,max_active_pl_dpd_l3m_x,max_active_pl_dpd_cm_x,max_pl_dpd_l36m_x,max_pl_dpd_l24m_x,max_pl_dpd_l18m_x,max_pl_dpd_l12m_x,max_pl_dpd_l6m_x,max_pl_dpd_l3m_x,max_pl_dpd_cm_x,max_active_cd_dpd_l36m_x,max_active_cd_dpd_l24m_x,max_active_cd_dpd_l18m_x,max_active_cd_dpd_l12m_x,max_active_cd_dpd_l6m_x,max_active_cd_dpd_l3m_x,max_active_cd_dpd_cm_x,max_cd_dpd_l36m_x,max_cd_dpd_l24m_x,max_cd_dpd_l18m_x,max_cd_dpd_l12m_x,max_cd_dpd_l6m_x,max_cd_dpd_l3m_x,max_cd_dpd_cm_x,max_active_gl_dpd_l36m_x,max_active_gl_dpd_l24m_x,max_active_gl_dpd_l18m_x,max_active_gl_dpd_l12m_x,max_active_gl_dpd_l6m_x,max_active_gl_dpd_l3m_x,max_active_gl_dpd_cm_x,max_gl_dpd_l36m_x,max_gl_dpd_l24m_x,max_gl_dpd_l18m_x,max_gl_dpd_l12m_x,max_gl_dpd_l6m_x,max_gl_dpd_l3m_x,max_gl_dpd_cm_x,max_active_al_dpd_l36m_x,max_active_al_dpd_l24m_x,max_active_al_dpd_l18m_x,max_active_al_dpd_l12m_x,max_active_al_dpd_l6m_x,max_active_al_dpd_l3m_x,max_active_al_dpd_cm_x,max_al_dpd_l36m_x,max_al_dpd_l24m_x,max_al_dpd_l18m_x,max_al_dpd_l12m_x,max_al_dpd_l6m_x,max_al_dpd_l3m_x,max_al_dpd_cm_x,max_active_tw_dpd_l36m_x,max_active_tw_dpd_l24m_x,max_active_tw_dpd_l18m_x,max_active_tw_dpd_l12m_x,max_active_tw_dpd_l6m_x,max_active_tw_dpd_l3m_x,max_active_tw_dpd_cm_x,max_tw_dpd_l36m_x,max_tw_dpd_l24m_x,max_tw_dpd_l18m_x,max_tw_dpd_l12m_x,max_tw_dpd_l6m_x,max_tw_dpd_l3m_x,max_tw_dpd_cm_x,max_active_hl_dpd_l36m_x,max_active_hl_dpd_l24m_x,max_active_hl_dpd_l18m_x,max_active_hl_dpd_l12m_x,max_active_hl_dpd_l6m_x,max_active_hl_dpd_l3m_x,max_active_hl_dpd_cm_x,max_hl_dpd_l36m_x,max_hl_dpd_l24m_x,max_hl_dpd_l18m_x,max_hl_dpd_l12m_x,max_hl_dpd_l6m_x,max_hl_dpd_l3m_x,max_hl_dpd_cm_x,max_active_unsecured_dpd_l36m_x,max_active_unsecured_dpd_l24m_x,max_active_unsecured_dpd_l18m_x,max_active_unsecured_dpd_l12m_x,max_active_unsecured_dpd_l6m_x,max_active_unsecured_dpd_l3m_x,max_active_unsecured_dpd_cm_x,max_unsecured_dpd_l36m_x,max_unsecured_dpd_l24m_x,max_unsecured_dpd_l18m_x,max_unsecured_dpd_l12m_x,max_unsecured_dpd_l6m_x,max_unsecured_dpd_l3m_x,max_unsecured_dpd_cm_x,max_active_secured_dpd_l36m_x,max_active_secured_dpd_l24m_x,max_active_secured_dpd_l18m_x,max_active_secured_dpd_l12m_x,max_active_secured_dpd_l6m_x,max_active_secured_dpd_l3m_x,max_active_secured_dpd_cm_x,max_secured_dpd_l36m_x,max_secured_dpd_l24m_x,max_secured_dpd_l18m_x,max_secured_dpd_l12m_x,max_secured_dpd_l6m_x,max_secured_dpd_l3m_x,max_secured_dpd_cm_x,overdue_l36m_x,overdue_l24m_x,overdue_l18m_x,overdue_l12m_x,overdue_l9m_x,overdue_l6m_x,overdue_l3m_x,overdue_cm_x,cc_overdue_l36m_x,cc_overdue_l24m_x,cc_overdue_l18m_x,cc_overdue_l12m_x,cc_overdue_l9m_x,cc_overdue_l6m_x,cc_overdue_l3m_x,cc_overdue_cm_x,pl_overdue_l36m_x,pl_overdue_l24m_x,pl_overdue_l18m_x,pl_overdue_l12m_x,pl_overdue_l9m_x,pl_overdue_l6m_x,pl_overdue_l3m_x,pl_overdue_cm_x,cd_overdue_l36m_x,cd_overdue_l24m_x,cd_overdue_l18m_x,cd_overdue_l12m_x,cd_overdue_l9m_x,cd_overdue_l6m_x,cd_overdue_l3m_x,cd_overdue_cm_x,gl_overdue_l36m_x,gl_overdue_l24m_x,gl_overdue_l18m_x,gl_overdue_l12m_x,gl_overdue_l9m_x,gl_overdue_l6m_x,gl_overdue_l3m_x,gl_overdue_cm_x,al_overdue_l36m_x,al_overdue_l24m_x,al_overdue_l18m_x,al_overdue_l12m_x,al_overdue_l9m_x,al_overdue_l6m_x,al_overdue_l3m_x,al_overdue_cm_x,tw_overdue_l36m_x,tw_overdue_l24m_x,tw_overdue_l18m_x,tw_overdue_l12m_x,tw_overdue_l9m_x,tw_overdue_l6m_x,tw_overdue_l3m

In [89]:
shap_df = shap_df.rename(columns=lambda x: f"{x}_shap")

In [90]:
# Compute mean SHAP values for both absolute and non-absolute
shap_abs = shap_df.abs().mean()
shap_non_abs = shap_df.mean()

# Create a new DataFrame to store both
shap_combined = pd.DataFrame({
    'abs_shap': shap_abs,
    'non_abs_shap': shap_non_abs
})

# Sort the DataFrame by absolute SHAP values in descending order
shap_combined = shap_combined.sort_values(by='abs_shap', ascending=False)



In [91]:
shap_combined=shap_combined.reset_index()

In [92]:
df_category=query("""
WITH CTE AS(
SELECT *, 
CASE WHEN index LIKE '%score%' THEN 'score'
WHEN (index LIKE '%balance%' AND index LIKE '%cc%') or (index LIKE '%utilisation%' AND index LIKE '%cc%') 
or (index LIKE '%sanctioned%' AND index LIKE '%cc%') then 'cc_utilisation'
WHEN (index LIKE '%balance%') or (index LIKE '%utilisation%') 
or (index LIKE '%sanctioned%') then 'utilisation'
WHEN (index LIKE '%dpd%') or (index LIKE '%overdue%') then 'delinquency'
WHEN index LIKE '%vintage%' AND index LIKE '%min%' THEN 'product_holdings'
WHEN index LIKE '%enq%' or index LIKE '%inq%' THEN 'enquiries'
WHEN index LIKE '%vintage%' THEN 'vintage'
WHEN index LIKE '%ever%' or index LIKE '%active%' or index LIKE '%closed%' or index LIKE '%new%' THEN 'product_holdings' ELSE 'utilisation' END AS category,
CASE WHEN index LIKE '%diff%' THEN '2' ELSE '1' END AS delta_type

FROM fpl_ds_analytics.variable_categorisation
)

SELECT index, concat(category, delta_type) AS final_category
FROM CTE
""")
df_category.head()

,index,final_category
0,risk_score_x_shap,score1
1,cc_total_balance_amount_diff_shap,cc_utilisation2
2,max_cc_bureau_vintage_x_shap,vintage1
3,cc_utilisation_diff_shap,cc_utilisation2
4,total_balance_amount_diff_shap,utilisation2


In [93]:
df_features=pd.merge(shap_combined, df_category, on='index')

In [94]:
df_features['rn'] = df_features.groupby('final_category')['abs_shap'].rank(method='first', ascending=False).astype(int)

In [95]:
important_features=df_features[df_features['rn']==1]

In [96]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

direction_predictions = xgb_reg.predict(df1)
predictions_prob=xgb_reg.predict_proba(df1)[:, 1]

# auc_score = roc_auc_score(y, predictions_prob)


In [97]:
df['predicted_direction']=direction_predictions
df['predicted_direction_prob']=predictions_prob

In [98]:
import boto3
import pickle

# Initialize S3 client
s3 = boto3.client('s3')

# Define S3 bucket and key (file path in S3)
bucket_name = 'fpl-datalake-global'
s3_key = 'l2/Onescore/Score_simulator/os_negative_score_simulator_model_1.pkl'

# Download the pickle file from S3
try:
    response = s3.get_object(Bucket=bucket_name, Key=s3_key)
    model_str = response['Body'].read()

    # Unpickle the file to retrieve the StandardScaler
    xgb_reg = pickle.loads(model_str)
    
    print("Pickle file successfully loaded from S3.")
    
except Exception as e:
    print(f"Error occurred: {e}")


Pickle file successfully loaded from S3.


In [99]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

negative_predictions = xgb_reg.predict(df1)
# predictions_prob=xgb_reg.predict_proba(X3)[:, 1]

# auc_score = roc_auc_score(y, predictions_prob)


In [100]:
df['negative_prediction']=negative_predictions

In [101]:
import boto3
import pickle

# Initialize S3 client
s3 = boto3.client('s3')

# Define S3 bucket and key (file path in S3)
bucket_name = 'fpl-datalake-global'
s3_key = 'l2/Onescore/Score_simulator/os_postive_score_simulator_model_1.pkl'

# Download the pickle file from S3
try:
    response = s3.get_object(Bucket=bucket_name, Key=s3_key)
    model_str = response['Body'].read()

    # Unpickle the file to retrieve the StandardScaler
    xgb_reg = pickle.loads(model_str)
    
    print("Pickle file successfully loaded from S3.")
    
except Exception as e:
    print(f"Error occurred: {e}")


Pickle file successfully loaded from S3.


In [102]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

positive_predictions = xgb_reg.predict(df1)
# predictions_prob=xgb_reg.predict_proba(df2)[:, 1]

# auc_score = roc_auc_score(y, predictions_prob)


In [103]:
df['positive_prediction']=positive_predictions

In [104]:
df_predicted=df[['customer_no', 'risk_score_x', 'predicted_change', 'predicted_change_prob', 
                 'predicted_change_wo_1_5', 'predicted_change_prob_wo_1_5',
                 'predicted_direction', 'predicted_direction_prob', 'positive_prediction',
                 'negative_prediction']]



In [105]:
df_predicted

,customer_no,risk_score_x,predicted_change,predicted_change_prob,predicted_change_wo_1_5,predicted_change_prob_wo_1_5,predicted_direction,predicted_direction_prob,positive_prediction,negative_prediction
0,2565455133978658509,650,0,0.098516,0,0.292156,1,0.739819,18.550289,-10.025165


In [106]:

df1=query(f"""
SELECT *
FROM(
SELECT customer_no, pay_status_history_x, account_number_x, 
account_type_symbol_x, amount_past_due_x, creditor_name_x, 
current_balance_x, date_opened_x, date_closed_x, 
date_reported_x, app_date_x, high_balance_x, interest_rate_x, 
term_months_x, creation_date_x, risk_score_x, loan_id_x,  diff_sin_open AS diff_sin_open_x, 
diff_rep_pull AS diff_rep_pull_x, credit_limit_x,
final_array[1] AS pay_hist_1_x,
final_array[2] AS pay_hist_2_x,
final_array[3] AS pay_hist_3_x,
final_array[4] AS pay_hist_4_x,
final_array[5] AS pay_hist_5_x,
final_array[6] AS pay_hist_6_x,
final_array[7] AS pay_hist_7_x,
final_array[8] AS pay_hist_8_x,
final_array[9] AS pay_hist_9_x,
final_array[10] AS pay_hist_10_x,
final_array[11] AS pay_hist_11_x,
final_array[12] AS pay_hist_12_x,
final_array[13] AS pay_hist_13_x,
final_array[14] AS pay_hist_14_x,
final_array[15] AS pay_hist_15_x,
final_array[16] AS pay_hist_16_x,
final_array[17] AS pay_hist_17_x,
final_array[18] AS pay_hist_18_x,
final_array[19] AS pay_hist_19_x,
final_array[20] AS pay_hist_20_x,
final_array[21] AS pay_hist_21_x,
final_array[22] AS pay_hist_22_x,
final_array[23] AS pay_hist_23_x,
final_array[24] AS pay_hist_24_x,
final_array[25] AS pay_hist_25_x,
final_array[26] AS pay_hist_26_x,
final_array[27] AS pay_hist_27_x,
final_array[28] AS pay_hist_28_x,
final_array[29] AS pay_hist_29_x,
final_array[30] AS pay_hist_30_x,
final_array[31] AS pay_hist_31_x,
final_array[32] AS pay_hist_32_x,
final_array[33] AS pay_hist_33_x,
final_array[34] AS pay_hist_34_x,
final_array[35] AS pay_hist_35_x,
final_array[36] AS pay_hist_36_x,
CAST(SPLIT(replaced_string, ',') AS ARRAY<INT>)[1] AS latest_payment_dpd_status_x,
IF(string_length_x>=2, TRY_CAST(SPLIT(replaced_string, ',') AS ARRAY<INT>)[2], NULL) AS latest_payment_dpd_status2_x,
IF(string_length_x>=3, TRY_CAST(SPLIT(replaced_string, ',') AS ARRAY<INT>)[3], NULL) AS latest_payment_dpd_status3_x,
string_length_x
FROM(
SELECT *, 
slice(CAST(repeat(-1, CAST(diff_rep_pull AS INT)) AS ARRAY<INT>) || CAST(SPLIT(replaced_string, ',') AS ARRAY<INT>) || CAST(repeat(-1, 36) AS ARRAY<INT>),1,36) AS final_array
FROM(
SELECT *, (YEAR(DATE(app_date_x))-YEAR(DATE(date_opened_x)))*12+(MONTH(DATE(app_date_x))-MONTH(DATE(date_opened_x))) AS diff_sin_open, 
(YEAR(DATE(app_date_x))-YEAR(DATE(date_reported_x)))*12+(MONTH(DATE(app_date_x))-MONTH(DATE(date_reported_x))) AS diff_rep_pull,
REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(
SUBSTRING(temp, 1, length(temp)-1), 'XXX', '-1'), 'DBT', '180'),'LSS', '180'), 'SUB', '90'), 'SMA', '60'), 'STD', '0'), ' XXX', '-1'), '   ', '0'), ',', ',0'), '0-', '-') AS replaced_string,
LENGTH(temp) - LENGTH(REPLACE(temp, ',', '')) AS string_length_x
FROM(
SELECT customer_no, pay_status_history AS pay_status_history_x, account_number AS account_number_x, 
account_type_symbol AS account_type_symbol_x, amount_past_due AS amount_past_due_x, creditor_name AS creditor_name_x, 
current_balance AS current_balance_x, date_opened AS date_opened_x, date_closed AS date_closed_x, 
date_reported AS date_reported_x, high_balance AS high_balance_x, interest_rate AS interest_rate_x, 
term_months AS term_months_x, creation_date AS creation_date_x, risk_score AS risk_score_x, loan_id AS loan_id_x, 
credit_limit AS credit_limit_x,
CASE WHEN SUBSTRING(pay_status_history,1,1)=',' THEN '0'||pay_status_history ELSE pay_status_history END AS temp,
GREATEST(DATE(DATE(DATE_TRUNC('month', DATE_ADD('month', -2, MAX(DATE(creation_date)) over(partition by customer_no, DATE(creation_date)))))-interval '1' day), MAX(DATE(date_reported)) over(partition by customer_no, DATE(creation_date))) AS app_date_x
FROM fpl_bureau.score_cibil_report
WHERE customer_no={customer_no} AND year={y} AND month IN ({m})
)
)
)) a
LEFT JOIN fpl_ds_analytics.lender_mapper b
ON a.creditor_name_x=b.lender
""")

In [107]:
product_scrub=fplathena.get_query_result(
"""select * from fpl_mis.for_cibil_scrub_prod_type_scrub """)
product_scrub=product_scrub.rename(columns={'accounttypecode':'accounttype','accounttype':'accounttypestr'})

df11= pd.merge(df1,product_scrub[['accounttype','priority','priority_1','priority_2','priority_3','product_type','secured_unsecured']],left_on='account_type_symbol_x',right_on='accounttype',how='left')



In [108]:
product_scrub1=fplathena.get_query_result(
"""select * from fpl_ds_onboarding_l2.cibil_bureau_accounttype_mapping """)


df11= pd.merge(df11,product_scrub1[['acct_type_symbol','loan_type']],left_on='account_type_symbol_x',right_on='acct_type_symbol',how='left')



In [109]:
df11['closed_flag_x']=np.where (df11['date_closed_x'].notna(),1,0)
df11['date_closed1_x']=np.where (df11['date_closed_x']>date(2049,12,1),date(1970,12,31),df11['date_closed_x'])
# df11.drop(['date_closed'],inplace=True,axis=1)

df11.drop(df11[df11['date_opened_x']< date(1970,1,1)].index,inplace=True)
# df11.drop(df11[df11['date_reported']> '2022-05-31'].index,inplace=True)
df11.drop(df11[df11['date_reported_x'].isna()].index,inplace=True)
df11.drop(df11[df11['date_opened_x'].isna()].index,inplace=True)
df11['date_opened_x']=pd.to_datetime(df11['date_opened_x'])
df11['date_reported_x']=pd.to_datetime(df11['date_reported_x'])
df11['creation_date_x']=pd.to_datetime(df11['creation_date_x'])
df11['date_closed1_x']=pd.to_datetime(df11['date_closed1_x'])


In [110]:
df11['app_date_x']=pd.to_datetime(df11['app_date_x'])
df11['creation_date_x']=pd.to_datetime(df11['creation_date_x'])
df11['date_closed1_x']=pd.to_datetime(df11['date_closed1_x'])

In [111]:
df11['diff_closed_date_x'] = (df11['app_date_x'].dt.year - df11['date_closed1_x'].dt.year)*12 + (df11['app_date_x'].dt.month - df11['date_closed1_x'].dt.month)

df11['acc_age_x'] = df11['diff_sin_open_x']

# for i in obs2.index:
#     if obs2['DateClosed_3'][i] == date(1970,1,31):
#         obs2['Closed_Flag'][i] = 0
#     elif obs2['DateClosed_3'][i] < obs2['app_date_2'][i]:
#         obs2['Closed_Flag'][i] = 1
#     else:
#         obs2['Closed_Flag'][i] = 0
        
df11['last_active_x'] = df11['diff_rep_pull_x']

df11['Activity_Flag_x'] = np.where((df11['closed_flag_x'] == 1 ) | (df11['last_active_x'] > 12),0,1)





In [112]:
df2=query(f"""
SELECT *
FROM(
SELECT customer_no, pay_status_history_y, account_number_y, 
account_type_symbol_y, amount_past_due_y, creditor_name_y, 
current_balance_y, date_opened_y, date_closed_y, 
date_reported_y, app_date_y, high_balance_y, interest_rate_y, 
term_months_y, creation_date_y, risk_score_y, loan_id_y, diff_sin_open AS diff_sin_open_y, 
diff_rep_pull AS diff_rep_pull_y, credit_limit_y,
final_array[1] AS pay_hist_1_y,
final_array[2] AS pay_hist_2_y,
final_array[3] AS pay_hist_3_y,
final_array[4] AS pay_hist_4_y,
final_array[5] AS pay_hist_5_y,
final_array[6] AS pay_hist_6_y,
final_array[7] AS pay_hist_7_y,
final_array[8] AS pay_hist_8_y,
final_array[9] AS pay_hist_9_y,
final_array[10] AS pay_hist_10_y,
final_array[11] AS pay_hist_11_y,
final_array[12] AS pay_hist_12_y,
final_array[13] AS pay_hist_13_y,
final_array[14] AS pay_hist_14_y,
final_array[15] AS pay_hist_15_y,
final_array[16] AS pay_hist_16_y,
final_array[17] AS pay_hist_17_y,
final_array[18] AS pay_hist_18_y,
final_array[19] AS pay_hist_19_y,
final_array[20] AS pay_hist_20_y,
final_array[21] AS pay_hist_21_y,
final_array[22] AS pay_hist_22_y,
final_array[23] AS pay_hist_23_y,
final_array[24] AS pay_hist_24_y,
final_array[25] AS pay_hist_25_y,
final_array[26] AS pay_hist_26_y,
final_array[27] AS pay_hist_27_y,
final_array[28] AS pay_hist_28_y,
final_array[29] AS pay_hist_29_y,
final_array[30] AS pay_hist_30_y,
final_array[31] AS pay_hist_31_y,
final_array[32] AS pay_hist_32_y,
final_array[33] AS pay_hist_33_y,
final_array[34] AS pay_hist_34_y,
final_array[35] AS pay_hist_35_y,
final_array[36] AS pay_hist_36_y,
CAST(SPLIT(replaced_string, ',') AS ARRAY<INT>)[1] AS latest_payment_dpd_status_y,
IF(string_length_y>=2, TRY_CAST(SPLIT(replaced_string, ',') AS ARRAY<INT>)[2], NULL) AS latest_payment_dpd_status2_y,
IF(string_length_y>=3, TRY_CAST(SPLIT(replaced_string, ',') AS ARRAY<INT>)[3], NULL) AS latest_payment_dpd_status3_y,
string_length_y
FROM(
SELECT *, 
slice(CAST(repeat(-1, CAST(diff_rep_pull AS INT)) AS ARRAY<INT>) || CAST(SPLIT(replaced_string, ',') AS ARRAY<INT>) || CAST(repeat(-1, 36) AS ARRAY<INT>),1,36) AS final_array
FROM(
SELECT *, (YEAR(DATE(app_date_y))-YEAR(DATE(date_opened_y)))*12+(MONTH(DATE(app_date_y))-MONTH(DATE(date_opened_y))) AS diff_sin_open, 
(YEAR(DATE(app_date_y))-YEAR(DATE(date_reported_y)))*12+(MONTH(DATE(app_date_y))-MONTH(DATE(date_reported_y))) AS diff_rep_pull,
REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(
SUBSTRING(temp, 1, length(temp)-1), 'XXX', '-1'), 'DBT', '180'),'LSS', '180'), 'SUB', '90'), 'SMA', '60'), 'STD', '0'), ' XXX', '-1'), '   ', '0'), ',', ',0'), '0-', '-') AS replaced_string,
LENGTH(temp) - LENGTH(REPLACE(temp, ',', '')) AS string_length_y
FROM(
SELECT customer_no, pay_status_history AS pay_status_history_y, account_number AS account_number_y, 
account_type_symbol AS account_type_symbol_y, amount_past_due AS amount_past_due_y, creditor_name AS creditor_name_y, 
current_balance AS current_balance_y, date_opened AS date_opened_y, date_closed AS date_closed_y, 
date_reported AS date_reported_y, high_balance AS high_balance_y, interest_rate AS interest_rate_y, 
credit_limit AS credit_limit_y,
term_months AS term_months_y, creation_date AS creation_date_y, risk_score AS risk_score_y, loan_id AS loan_id_y,
CASE WHEN SUBSTRING(pay_status_history,1,1)=',' THEN '0'||pay_status_history ELSE pay_status_history END AS temp,
GREATEST(DATE(DATE(DATE_TRUNC('month', DATE_ADD('month', -2, MAX(DATE(creation_date)) over(partition by customer_no, DATE(creation_date)))))-interval '1' day), MAX(DATE(date_reported)) over(partition by customer_no, DATE(creation_date))) AS app_date_y
FROM fpl_bureau.score_cibil_report
WHERE customer_no={customer_no} AND year={y_y} AND month IN ({m_y})
)
)
)) a
LEFT JOIN fpl_ds_analytics.lender_mapper b
ON a.creditor_name_y=b.lender
""")

In [113]:
product_scrub=fplathena.get_query_result(
"""select * from fpl_mis.for_cibil_scrub_prod_type_scrub """)
product_scrub=product_scrub.rename(columns={'accounttypecode':'accounttype','accounttype':'accounttypestr'})

df21= pd.merge(df2,product_scrub[['accounttype','priority','priority_1','priority_2','priority_3','product_type','secured_unsecured']],left_on='account_type_symbol_y',right_on='accounttype',how='left')



In [114]:
product_scrub1=fplathena.get_query_result(
"""select * from fpl_ds_onboarding_l2.cibil_bureau_accounttype_mapping """)


df21= pd.merge(df21,product_scrub1[['acct_type_symbol','loan_type']],left_on='account_type_symbol_y',right_on='acct_type_symbol',how='left')



In [115]:
df21['closed_flag_y']=np.where (df21['date_closed_y'].notna(),1,0)
df21['date_closed1_y']=np.where (df21['date_closed_y']>date(2049,12,1),date(1970,12,31),df21['date_closed_y'])
# df21.drop(['date_closed'],inplace=True,axis=1)

df21.drop(df21[df21['date_opened_y']< date(1970,1,1)].index,inplace=True)
# df21.drop(df21[df21['date_reported']> '2022-05-31'].index,inplace=True)
df21.drop(df21[df21['date_reported_y'].isna()].index,inplace=True)
df21.drop(df21[df21['date_opened_y'].isna()].index,inplace=True)
df21['date_opened_y']=pd.to_datetime(df21['date_opened_y'])
df21['date_reported_y']=pd.to_datetime(df21['date_reported_y'])
df21['creation_date_y']=pd.to_datetime(df21['creation_date_y'])
df21['date_closed1_y']=pd.to_datetime(df21['date_closed1_y'])


In [116]:
df21['app_date_y']=pd.to_datetime(df21['app_date_y'])
df21['creation_date_y']=pd.to_datetime(df21['creation_date_y'])
df21['date_closed1_y']=pd.to_datetime(df21['date_closed1_y'])

In [117]:
df21['diff_closed_date_y'] = (df21['app_date_y'].dt.year - df21['date_closed1_y'].dt.year)*12 + (df21['app_date_y'].dt.month - df21['date_closed1_y'].dt.month)

df21['acc_age_y'] = df21['diff_sin_open_y']

# for i in obs2.index:
#     if obs2['DateClosed_3'][i] == date(1970,1,31):
#         obs2['Closed_Flag'][i] = 0
#     elif obs2['DateClosed_3'][i] < obs2['app_date_2'][i]:
#         obs2['Closed_Flag'][i] = 1
#     else:
#         obs2['Closed_Flag'][i] = 0
        
df21['last_active_y'] = df21['diff_rep_pull_y']

df21['Activity_Flag_y'] = np.where((df21['closed_flag_y'] == 1 ) | (df21['last_active_y'] > 12),0,1)





In [118]:
# Step 1: Align the secondary key names (_x and _y)
df11 = df11.rename(columns={'account_number_x': 'account_number', 'creditor_name_x':'creditor_name', 'date_opened_x':'date_opened'})
df21 = df21.rename(columns={'account_number_y': 'account_number', 'creditor_name_y':'creditor_name', 'date_opened_y':'date_opened'})



In [119]:
df11['acc_no']=df11['account_number'].str[-5:]
df21['acc_no']=df21['account_number'].str[-5:]

In [120]:



# Step 1: Primary Join (on key1, key2, and key3)
primary_join = pd.merge(
    df11,
    df21,
    how='outer',
    on=['customer_no', 'acc_no', 'creditor_name', 'date_opened'],
    #suffixes=('_x', '_y'),
    indicator=True  # To track which rows matched
)




In [121]:
unmatched=primary_join[primary_join['_merge']!='both']

In [122]:
if unmatched.isnull().all().all():
    final_result=primary_join
else:
    secondary_join=pd.merge(
    df11,
    df21,
    how='outer',
    on=['customer_no', 'acc_no', 'date_opened'],
    #suffixes=('_x', '_y'),
    indicator=True  # To track which rows matched
    )
    
    secondary_join['creditor_name']=np.where(secondary_join['creditor_name_y'].notnull(),secondary_join['creditor_name_y'],secondary_join['creditor_name_x'])
    #secondary_join.rename(columns={'creditor_name_y':'creditor_name'}, inplace=True)
    
    secondary_join.drop(columns=['creditor_name_x', 'creditor_name_y'], inplace=True)
    
    final_result = pd.concat(
    [primary_join, secondary_join],
    ignore_index=True
    ).drop_duplicates(subset=['customer_no', 'acc_no', 'creditor_name', 'date_opened'], keep='first')  # Avoid duplicate rows

In [123]:
unmatched1=final_result[final_result['_merge']!='both']

In [124]:
if unmatched1.isnull().all().all():
    final_result=final_result
else:
    tertiary_join=pd.merge(
    df11,
    df21,
    how='outer',
    left_on=['customer_no', 'date_opened', 'high_balance_x'],
    right_on=['customer_no', 'date_opened', 'high_balance_y'],
    #suffixes=('_x', '_y'),
    indicator=True  # To track which rows matched
    )
    
    tertiary_join1=tertiary_join[(tertiary_join['account_number_y'].isin(list(unmatched1['account_number_y']))) | (tertiary_join['account_number_x'].isin(list(unmatched1['account_number_x'])))]
    tertiary_join1['creditor_name']=np.where(tertiary_join1['creditor_name_y'].notnull(),tertiary_join1['creditor_name_y'],tertiary_join1['creditor_name_x'])
    tertiary_join1['acc_no']=np.where(tertiary_join1['acc_no_y'].notnull(),tertiary_join1['acc_no_y'],tertiary_join1['acc_no_x'])
        #secondary_join.rename(columns={'creditor_name_y':'creditor_name'}, inplace=True)

    tertiary_join1.drop(columns=['creditor_name_x', 'creditor_name_y', 'acc_no_y', 'acc_no_x'], inplace=True)
    
    final_result = pd.concat(
    [final_result[final_result['_merge']=='both'], tertiary_join1],
    ignore_index=True
    ).drop_duplicates(subset=['customer_no', 'acc_no', 'creditor_name', 'date_opened'], keep='first')  # Avoid duplicate rows
    

In [125]:
data_cols_x = ['pay_hist_1_x','pay_hist_2_x','pay_hist_3_x','pay_hist_4_x','pay_hist_5_x','pay_hist_6_x','pay_hist_7_x','pay_hist_8_x','pay_hist_9_x','pay_hist_10_x','pay_hist_11_x','pay_hist_12_x','pay_hist_13_x','pay_hist_14_x','pay_hist_15_x','pay_hist_16_x','pay_hist_17_x','pay_hist_18_x','pay_hist_19_x','pay_hist_20_x','pay_hist_21_x','pay_hist_22_x','pay_hist_23_x','pay_hist_24_x','pay_hist_25_x','pay_hist_26_x','pay_hist_27_x','pay_hist_28_x','pay_hist_29_x','pay_hist_30_x','pay_hist_31_x','pay_hist_32_x','pay_hist_33_x','pay_hist_34_x','pay_hist_35_x','pay_hist_36_x']



In [126]:
data_cols_y = ['pay_hist_1_y','pay_hist_2_y','pay_hist_3_y','pay_hist_4_y','pay_hist_5_y','pay_hist_6_y','pay_hist_7_y','pay_hist_8_y','pay_hist_9_y','pay_hist_10_y','pay_hist_11_y','pay_hist_12_y','pay_hist_13_y','pay_hist_14_y','pay_hist_15_y','pay_hist_16_y','pay_hist_17_y','pay_hist_18_y','pay_hist_19_y','pay_hist_20_y','pay_hist_21_y','pay_hist_22_y','pay_hist_23_y','pay_hist_24_y','pay_hist_25_y','pay_hist_26_y','pay_hist_27_y','pay_hist_28_y','pay_hist_29_y','pay_hist_30_y','pay_hist_31_y','pay_hist_32_y','pay_hist_33_y','pay_hist_34_y','pay_hist_35_y','pay_hist_36_y']



In [127]:
# max_dpd_all account
final_result['max_dpd_l36m_x'] = final_result[data_cols_x[:36]].max(axis = 1)
final_result['max_dpd_l24m_x'] = final_result[data_cols_x[:24]].max(axis = 1)
final_result['max_dpd_l18m_x'] = final_result[data_cols_x[:18]].max(axis = 1)
final_result['max_dpd_l12m_x'] = final_result[data_cols_x[:12]].max(axis = 1)
final_result['max_dpd_l6m_x'] = final_result[data_cols_x[:6]].max(axis = 1)
final_result['max_dpd_l3m_x'] = final_result[data_cols_x[:3]].max(axis = 1)
final_result['max_dpd_l2m_x'] = final_result[data_cols_x[:2]].max(axis = 1)
final_result['max_dpd_cm_x'] = final_result[data_cols_x[:1]].max(axis = 1)

In [128]:
final_result['max_dpd_l36m_x'] = np.where(final_result['max_dpd_l36m_x'].isnull(),0,final_result['max_dpd_l36m_x'])
final_result['max_dpd_l24m_x'] = np.where(final_result['max_dpd_l24m_x'].isnull(),0,final_result['max_dpd_l24m_x'])
final_result['max_dpd_l18m_x'] = np.where(final_result['max_dpd_l18m_x'].isnull(),0,final_result['max_dpd_l18m_x'])
final_result['max_dpd_l12m_x'] = np.where(final_result['max_dpd_l12m_x'].isnull(),0,final_result['max_dpd_l12m_x'])
final_result['max_dpd_l6m_x'] = np.where(final_result['max_dpd_l6m_x'].isnull(),0,final_result['max_dpd_l6m_x'])
final_result['max_dpd_l3m_x'] = np.where(final_result['max_dpd_l3m_x'].isnull(),0,final_result['max_dpd_l3m_x'])
final_result['max_dpd_l2m_x'] = np.where(final_result['max_dpd_l2m_x'].isnull(),0,final_result['max_dpd_l2m_x'])
final_result['max_dpd_cm_x'] = np.where(final_result['max_dpd_cm_x'].isnull(),0,final_result['max_dpd_cm_x'])

In [129]:
# max_dpd_all account
final_result['max_dpd_l36m_y'] = final_result[data_cols_y[:36]].max(axis = 1)
final_result['max_dpd_l24m_y'] = final_result[data_cols_y[:24]].max(axis = 1)
final_result['max_dpd_l18m_y'] = final_result[data_cols_y[:18]].max(axis = 1)
final_result['max_dpd_l12m_y'] = final_result[data_cols_y[:12]].max(axis = 1)
final_result['max_dpd_l6m_y'] = final_result[data_cols_y[:6]].max(axis = 1)
final_result['max_dpd_l3m_y'] = final_result[data_cols_y[:3]].max(axis = 1)
final_result['max_dpd_l2m_y'] = final_result[data_cols_y[:2]].max(axis = 1)
final_result['max_dpd_cm_y'] = final_result[data_cols_y[:1]].max(axis = 1)

In [130]:
final_result['max_dpd_l36m_y'] = np.where(final_result['max_dpd_l36m_y'].isnull(),0,final_result['max_dpd_l36m_y'])
final_result['max_dpd_l24m_y'] = np.where(final_result['max_dpd_l24m_y'].isnull(),0,final_result['max_dpd_l24m_y'])
final_result['max_dpd_l18m_y'] = np.where(final_result['max_dpd_l18m_y'].isnull(),0,final_result['max_dpd_l18m_y'])
final_result['max_dpd_l12m_y'] = np.where(final_result['max_dpd_l12m_y'].isnull(),0,final_result['max_dpd_l12m_y'])
final_result['max_dpd_l6m_y'] = np.where(final_result['max_dpd_l6m_y'].isnull(),0,final_result['max_dpd_l6m_y'])
final_result['max_dpd_l3m_y'] = np.where(final_result['max_dpd_l3m_y'].isnull(),0,final_result['max_dpd_l3m_y'])
final_result['max_dpd_l2m_y'] = np.where(final_result['max_dpd_l2m_y'].isnull(),0,final_result['max_dpd_l2m_y'])
final_result['max_dpd_cm_y'] = np.where(final_result['max_dpd_cm_y'].isnull(),0,final_result['max_dpd_cm_y'])

In [131]:
# max_dpd_all account
final_result['max_dpd_l36m_diff'] = final_result['max_dpd_l36m_y']- final_result['max_dpd_l36m_x']
final_result['max_dpd_l24m_diff'] = final_result['max_dpd_l24m_y']- final_result['max_dpd_l24m_x']
final_result['max_dpd_l18m_diff'] = final_result['max_dpd_l18m_y']- final_result['max_dpd_l18m_x']
final_result['max_dpd_l12m_diff'] = final_result['max_dpd_l12m_y']- final_result['max_dpd_l12m_x']
final_result['max_dpd_l6m_diff'] = final_result['max_dpd_l6m_y']- final_result['max_dpd_l6m_x']
final_result['max_dpd_l3m_diff'] = final_result['max_dpd_l3m_y']- final_result['max_dpd_l3m_x']
final_result['max_dpd_l2m_diff'] = final_result['max_dpd_l2m_y']- final_result['max_dpd_l2m_x']
final_result['max_dpd_cm_diff'] = final_result['max_dpd_cm_y']- final_result['max_dpd_cm_x']
final_result['latest_payment_dpd_status2_diff'] = np.where(final_result['string_length_y']-final_result['string_length_x']>1, final_result['latest_payment_dpd_status3_y']- final_result['latest_payment_dpd_status2_x'], 
                                                          final_result['latest_payment_dpd_status2_y']- final_result['latest_payment_dpd_status2_x'])
final_result['latest_payment_dpd_status_y_adjusted'] = np.where(final_result['string_length_y']-final_result['string_length_x']>1, final_result['latest_payment_dpd_status2_y'], 
                                                          final_result['latest_payment_dpd_status_y'])
final_result['latest_payment_dpd_status2_y_adjusted'] = np.where(final_result['string_length_y']-final_result['string_length_x']>1, final_result['latest_payment_dpd_status3_y'], 
                                                          final_result['latest_payment_dpd_status2_y'])
final_result['max_delinquency_latest_2_months_y'] = final_result[
    ['latest_payment_dpd_status_y_adjusted', 'latest_payment_dpd_status2_y_adjusted', 'latest_payment_dpd_status_y']
].max(axis=1)
final_result['latest_payment_dpd_status_diff'] = np.where(final_result['string_length_y']-final_result['string_length_x']>1, final_result['max_delinquency_latest_2_months_y']- final_result['latest_payment_dpd_status_x'], 
                                                          final_result['latest_payment_dpd_status_y']- final_result['latest_payment_dpd_status_x'])


In [132]:
# Balance variables

final_result['current_balance_x'] = pd.to_numeric(final_result['current_balance_x'])
final_result['high_balance_x'] = pd.to_numeric(final_result['high_balance_x'])
final_result['credit_limit_x'] = pd.to_numeric(final_result['credit_limit_x'])

# Limit Sanctioned

final_result['lim_disbursed_x'] = np.nanmax(final_result[[ 'high_balance_x', 'credit_limit_x']].values,1)

final_result['active_balance_x'] = np.where(final_result['Activity_Flag_x']==1, final_result['current_balance_x'],0)

final_result['utilisation_x'] = final_result['current_balance_x'] / final_result['lim_disbursed_x']
final_result['utilisation_x'] = final_result['utilisation_x'].replace([np.inf, -np.inf], np.nan)
final_result['utilisation_x'] = final_result['utilisation_x'].fillna(0)

In [133]:
# Initialize the column with NaN or 0 for clarity
final_result['total_lim_disbursed_x'] = 0
final_result['total_active_balance_x'] = 0

# Compute the total where Activity_Flag_x equals 1
condition = final_result['Activity_Flag_x'] == 1
final_result.loc[condition, 'total_lim_disbursed_x'] = final_result[condition].groupby('customer_no')['lim_disbursed_x'].transform('sum')
final_result.loc[condition, 'total_active_balance_x'] = final_result[condition].groupby('customer_no')['active_balance_x'].transform('sum')


In [134]:
# Initialize the column with NaN or 0 for clarity
final_result['total_cc_lim_disbursed_x'] = 0
final_result['total_cc_active_balance_x'] = 0

# Compute the total where Activity_Flag_x equals 1
condition = (final_result['Activity_Flag_x'] == 1) & (final_result['priority_3_x'] == '01.0 CC')
final_result.loc[condition, 'total_cc_lim_disbursed_x'] = final_result[condition].groupby('customer_no')['lim_disbursed_x'].transform('sum')
final_result.loc[condition, 'total_cc_active_balance_x'] = final_result[condition].groupby('customer_no')['active_balance_x'].transform('sum')


In [135]:
final_result['overall_utilisation_x'] = final_result['total_active_balance_x'] / final_result['total_lim_disbursed_x']
final_result['overall_utilisation_x'] = final_result['overall_utilisation_x'].replace([np.inf, -np.inf], np.nan)
final_result['overall_utilisation_x'] = final_result['overall_utilisation_x'].fillna(0)
final_result['overall_utilisation_x'] = final_result['overall_utilisation_x'].max()


In [136]:
final_result.loc[final_result['priority_3_x'] == '01.0 CC', 'overall_cc_utilisation_x'] = (
    final_result['total_cc_active_balance_x'] / final_result['total_cc_lim_disbursed_x']
)

In [137]:
final_result['overall_cc_utilisation_x'] = final_result['overall_cc_utilisation_x'].max()
final_result['total_lim_disbursed_x']=final_result['total_lim_disbursed_x'].max()
final_result['total_active_balance_x']=final_result['total_active_balance_x'].max()
final_result['total_cc_lim_disbursed_x']=final_result['total_cc_lim_disbursed_x'].max()
final_result['total_cc_active_balance_x']=final_result['total_cc_active_balance_x'].max()

In [138]:
# Balance variables

final_result['current_balance_y'] = pd.to_numeric(final_result['current_balance_y'])
final_result['high_balance_y'] = pd.to_numeric(final_result['high_balance_y'])
final_result['credit_limit_y'] = pd.to_numeric(final_result['credit_limit_y'])

# Limit Sanctioned

final_result['lim_disbursed_y'] = np.nanmax(final_result[[ 'high_balance_y', 'credit_limit_y']].values,1)

final_result['active_balance_y'] = np.where(final_result['Activity_Flag_y']==1, final_result['current_balance_y'],0)

final_result['utilisation_y'] = final_result['current_balance_y'] / final_result['lim_disbursed_y']
final_result['utilisation_y'] = final_result['utilisation_y'].replace([np.inf, -np.inf], np.nan)
final_result['utilisation_y'] = final_result['utilisation_y'].fillna(0)

In [139]:
# Initialize the column with NaN or 0 for clarity
final_result['total_lim_disbursed_y'] = 0
final_result['total_active_balance_y'] = 0

# Compute the total where Activity_Flag_y equals 1
condition = final_result['Activity_Flag_y'] == 1
final_result.loc[condition, 'total_lim_disbursed_y'] = final_result[condition].groupby('customer_no')['lim_disbursed_y'].transform('sum')
final_result.loc[condition, 'total_active_balance_y'] = final_result[condition].groupby('customer_no')['active_balance_y'].transform('sum')


In [140]:
# Initialize the column with NaN or 0 for clarity
final_result['total_cc_lim_disbursed_y'] = 0
final_result['total_cc_active_balance_y'] = 0

# Compute the total where Activity_Flag_y equals 1
condition = (final_result['Activity_Flag_y'] == 1) & (final_result['priority_3_y'] == '01.0 CC')
final_result.loc[condition, 'total_cc_lim_disbursed_y'] = final_result[condition].groupby('customer_no')['lim_disbursed_y'].transform('sum')
final_result.loc[condition, 'total_cc_active_balance_y'] = final_result[condition].groupby('customer_no')['active_balance_y'].transform('sum')


In [141]:
final_result['overall_utilisation_y'] = final_result['total_active_balance_y'] / final_result['total_lim_disbursed_y']
final_result['overall_utilisation_y'] = final_result['overall_utilisation_y'].replace([np.inf, -np.inf], np.nan)
final_result['overall_utilisation_y'] = final_result['overall_utilisation_y'].fillna(0)
final_result['overall_utilisation_y'] = final_result['overall_utilisation_y'].max()
final_result.loc[final_result['priority_3_y'] == '01.0 CC', 'overall_cc_utilisation_y'] = (
    final_result['total_cc_active_balance_y'] / final_result['total_cc_lim_disbursed_y']
)

In [142]:
final_result['overall_cc_utilisation_y'] = final_result['overall_cc_utilisation_y'].max()
final_result['total_lim_disbursed_y']=final_result['total_lim_disbursed_y'].max()
final_result['total_active_balance_y']=final_result['total_active_balance_y'].max()
final_result['total_cc_lim_disbursed_y']=final_result['total_cc_lim_disbursed_y'].max()
final_result['total_cc_active_balance_y']=final_result['total_cc_active_balance_y'].max()

In [143]:
final_result['utilisation_diff']=final_result['utilisation_y']-final_result['utilisation_x']
final_result['overall_utilisation_diff']=final_result['overall_utilisation_y'].max()-final_result['overall_utilisation_x'].max()
final_result['overall_cc_utilisation_diff']=final_result['overall_cc_utilisation_y'].max()-final_result['overall_cc_utilisation_x'].max()



In [144]:
final_result['utilisation_percent_diff']=final_result['utilisation_diff']/final_result['utilisation_x']
final_result['utilisation_percent_diff'] = final_result['utilisation_percent_diff'].replace([np.inf, -np.inf], np.nan)
final_result['utilisation_percent_diff'] = final_result['utilisation_percent_diff'].fillna(0)

In [145]:
final_result['overall_utilisation_percent_diff']=final_result['overall_utilisation_diff']/final_result['overall_utilisation_x'].max()
final_result['overall_utilisation_percent_diff'] = final_result['overall_utilisation_percent_diff'].replace([np.inf, -np.inf], np.nan)
final_result['overall_utilisation_percent_diff'] = final_result['overall_utilisation_percent_diff'].fillna(0)

In [146]:
final_result['overall_cc_utilisation_percent_diff']=final_result['overall_cc_utilisation_diff']/final_result['overall_cc_utilisation_x'].max()
final_result['overall_cc_utilisation_percent_diff'] = final_result['overall_cc_utilisation_percent_diff'].replace([np.inf, -np.inf], np.nan)
final_result['overall_cc_utilisation_percent_diff'] = final_result['overall_cc_utilisation_percent_diff'].fillna(0)

In [147]:
data_cols_x_max_dpd = ['max_dpd_l36m_diff','max_dpd_l24m_diff','max_dpd_l18m_diff','max_dpd_l12m_diff','max_dpd_l6m_diff','max_dpd_l3m_diff','max_dpd_cm_diff']



In [148]:
final_result['max_delinquency_detected']=final_result[data_cols_x_max_dpd[:7]].max(axis = 1)
final_result['min_delinquency_detected']=final_result[data_cols_x_max_dpd[:7]].min(axis = 1)

In [149]:
final_result['priority_3_y']=np.where(final_result['priority_3_y'].isnull(), 'NULL', final_result['priority_3_y']
                                     )

In [150]:
# Convert the 'category' column to an ordered categorical type
final_result['priority_3_y'] = final_result['priority_3_y'].astype('category')

final_result['priority_3_y'] = final_result['priority_3_y'].cat.as_ordered()

In [ ]:
# final_result['risk_score_x']=final_result['risk_score_x'].max()
# final_result['risk_score_y']=final_result['risk_score_y'].max()


In [152]:
final_result= final_result.sort_values(by=['priority_3_y', 'date_opened'], ascending=[True, True])

In [153]:
final_result['coalesced_priority']=np.where(final_result['priority_3_y'].notnull(),final_result['priority_3_y'], final_result['priority_3_x'])


In [154]:
final_result['coalesced_loan_type']=np.where(final_result['loan_type_y'].notnull(),final_result['loan_type_y'], final_result['loan_type_x'])


In [155]:
final_result['coalesced_open_date']=final_result['date_opened']


In [156]:
final_result['rn'] = final_result.groupby('coalesced_loan_type')['coalesced_open_date'].rank(method='first').astype(int)
                                          
                                          
                                          

In [157]:
final_result['new_account_flag']=np.where((final_result['account_number_y'].notnull()) & (final_result['account_number_x'].isnull() & 
                                                                                         (final_result['diff_sin_open_y']<=3)),1,0
                                         )


In [158]:
final_result['coalesced_open_date'] = pd.to_datetime(final_result['coalesced_open_date'])
final_result['coalesced_open_date'] = final_result['coalesced_open_date'].dt.strftime('%d %b, %Y') 
final_result['coalesced_open_date']=final_result['coalesced_open_date'].astype(str)
final_result['total_active_accounts_y']=final_result.groupby('customer_no')['Activity_Flag_y'].transform(sum)
final_result['total_active_accounts_x']=final_result.groupby('customer_no')['Activity_Flag_x'].transform(sum)
final_result['total_active_cc_accounts_x'] = np.where((final_result['priority_3_x']=='01.0 CC') & (final_result['Activity_Flag_x']==1),1,0)
final_result['total_active_cc_accounts_y'] = np.where((final_result['priority_3_y']=='01.0 CC') & (final_result['Activity_Flag_y']==1),1,0)

final_result['total_active_cc_accounts_x']=final_result.groupby('customer_no')['total_active_cc_accounts_x'].transform(sum)
final_result['total_active_cc_accounts_y']=final_result.groupby('customer_no')['total_active_cc_accounts_y'].transform(sum)

final_result['temp']=np.where(final_result['latest_payment_dpd_status_y']==0, final_result['max_delinquency_latest_2_months_y'], final_result['latest_payment_dpd_status_y'])
final_result['risk_score_diff']=final_result['risk_score_y']- final_result['risk_score_x']

/tmp/ipykernel_16517/3791458399.py:4: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  final_result['total_active_accounts_y']=final_result.groupby('customer_no')['Activity_Flag_y'].transform(sum)
/tmp/ipykernel_16517/3791458399.py:5: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  final_result['total_active_accounts_x']=final_result.groupby('customer_no')['Activity_Flag_x'].transform(sum)
/tmp/ipykernel_16517/3791458399.py:9: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior 

In [159]:
def fill_nulls(col):
    if col.dtype.name == 'category':  # Handle categorical columns
        # Add the required category for filling
        if col.cat.categories.dtype.kind in 'iu':  # Integer categories
            col = col.cat.add_categories([0])
            return col.fillna(0)
        elif col.cat.categories.dtype.kind in 'O':  # Object/string categories
            col = col.cat.add_categories(['NA'])
            return col.fillna('NA')
    elif col.dtype == 'object':  # Strings
        return col.fillna('NA')
    elif np.issubdtype(col.dtype, np.number):  # Numeric
        return col.fillna(0)
    else:  # Other types
        return col.fillna('Unknown')

# # Apply function to entire DataFrame
final_result=final_result.apply(fill_nulls)


In [160]:
final_result=final_result[['customer_no',
 'pay_status_history_x',
 'pay_status_history_y',
 'account_number_x',
 'account_number_y',
 'account_type_symbol_x',
 'account_type_symbol_y',
 'amount_past_due_x',
 'amount_past_due_y',
 'creditor_name',
 'current_balance_x',
 'current_balance_y',
 'date_opened',
 'date_closed_x',
 'date_closed_y',
 'date_reported_x',
 'date_reported_y',
 'app_date_x',
 'app_date_y',
 'high_balance_x',
 'high_balance_y',
 'interest_rate_x',
 'interest_rate_y',
 'term_months_x',
 'term_months_y',
 'creation_date_x',
 'creation_date_y',
 'risk_score_x',
 'risk_score_y',
 'risk_score_diff',
 'loan_id_x',
 'loan_id_y',
 'diff_sin_open_x',
 'diff_sin_open_y',
 'diff_rep_pull_x',
 'diff_rep_pull_y',
 'credit_limit_x',
 'credit_limit_y',
 'pay_hist_1_x',  'pay_hist_1_y',
'pay_hist_2_x',  'pay_hist_2_y',
'pay_hist_3_x',  'pay_hist_3_y',
'pay_hist_4_x',  'pay_hist_4_y',
'pay_hist_5_x',  'pay_hist_5_y',
'pay_hist_6_x',  'pay_hist_6_y',
'pay_hist_7_x',  'pay_hist_7_y',
'pay_hist_8_x',  'pay_hist_8_y',
'pay_hist_9_x',  'pay_hist_9_y',
'pay_hist_10_x', 'pay_hist_10_y',
'pay_hist_11_x', 'pay_hist_11_y',
'pay_hist_12_x', 'pay_hist_12_y',
'pay_hist_13_x', 'pay_hist_13_y',
'pay_hist_14_x', 'pay_hist_14_y',
'pay_hist_15_x', 'pay_hist_15_y',
'pay_hist_16_x', 'pay_hist_16_y',
'pay_hist_17_x', 'pay_hist_17_y',
'pay_hist_18_x', 'pay_hist_18_y',
'pay_hist_19_x', 'pay_hist_19_y',
'pay_hist_20_x', 'pay_hist_20_y',
'pay_hist_21_x', 'pay_hist_21_y',
'pay_hist_22_x', 'pay_hist_22_y',
'pay_hist_23_x', 'pay_hist_23_y',
'pay_hist_24_x', 'pay_hist_24_y',
'pay_hist_25_x', 'pay_hist_25_y',
'pay_hist_26_x', 'pay_hist_26_y',
'pay_hist_27_x', 'pay_hist_27_y',
'pay_hist_28_x', 'pay_hist_28_y',
'pay_hist_29_x', 'pay_hist_29_y',
'pay_hist_30_x', 'pay_hist_30_y',
'pay_hist_31_x', 'pay_hist_31_y',
'pay_hist_32_x', 'pay_hist_32_y',
'pay_hist_33_x', 'pay_hist_33_y',
'pay_hist_34_x', 'pay_hist_34_y',
'pay_hist_35_x', 'pay_hist_35_y',
'pay_hist_36_x', 'pay_hist_36_y',
 'accounttype_x',
 'accounttype_y',
 'priority_x',
 'priority_y',
 'priority_1_x',
 'priority_1_y',
 'priority_2_x',
 'priority_2_y',
 'priority_3_x',
 'priority_3_y',
 'loan_type_x',
 'loan_type_y',
 'category_x',
 'category_y',                         
 'product_type_x',
 'product_type_y',
 'secured_unsecured_x',
 'secured_unsecured_y',
 'closed_flag_x',
 'closed_flag_y',
 'date_closed1_x',
 'date_closed1_y',
 'diff_closed_date_x',
 'diff_closed_date_y',
 'acc_age_x',
 'acc_age_y',
 'last_active_x',
 'last_active_y',
 'Activity_Flag_x',
 'Activity_Flag_y',
 'acc_no',
 'max_dpd_l36m_x',
 'max_dpd_l36m_y',
 'max_dpd_l36m_diff',
 'max_dpd_l24m_x',
 'max_dpd_l24m_y',
 'max_dpd_l24m_diff',
 'max_dpd_l18m_x',
 'max_dpd_l18m_y',
 'max_dpd_l18m_diff',
 'max_dpd_l12m_x',
 'max_dpd_l12m_y',
 'max_dpd_l12m_diff',
 'max_dpd_l6m_x',
 'max_dpd_l6m_y',
 'max_dpd_l6m_diff',
 'max_dpd_l3m_x',
 'max_dpd_l3m_y',
 'max_dpd_l3m_diff',
 'max_dpd_l2m_x',
 'max_dpd_l2m_y',
 'max_dpd_l2m_diff',
 'max_dpd_cm_x',
 'max_dpd_cm_y',
 'max_dpd_cm_diff',
'latest_payment_dpd_status_x',
'latest_payment_dpd_status_y',
'latest_payment_dpd_status_diff',
'latest_payment_dpd_status2_x',
'latest_payment_dpd_status2_y',
'latest_payment_dpd_status2_diff',
 'latest_payment_dpd_status_y_adjusted', 
 'max_delinquency_latest_2_months_y',                          
 'string_length_x',
 'string_length_y',                          
 'lim_disbursed_x',
 'lim_disbursed_y',
 'active_balance_x',
 'active_balance_y',
 'total_lim_disbursed_x',
 'total_lim_disbursed_y',
 'total_active_balance_x',
 'total_active_balance_y',
 'total_cc_lim_disbursed_x',
 'total_cc_lim_disbursed_y',
 'total_cc_active_balance_x',
 'total_cc_active_balance_y',
 'utilisation_x',
 'utilisation_y',
 'utilisation_diff',
 'utilisation_percent_diff',
 'overall_utilisation_x',
 'overall_utilisation_y',
 'overall_utilisation_diff',
 'overall_utilisation_percent_diff',
 'overall_cc_utilisation_x',
 'overall_cc_utilisation_y',
 'overall_cc_utilisation_diff',
 'overall_cc_utilisation_percent_diff',
 'max_delinquency_detected',
 'min_delinquency_detected',
 'coalesced_priority',
 'coalesced_loan_type',
 'coalesced_open_date',
 'rn',
 'total_active_accounts_x',
 'total_active_accounts_y',
 'total_active_cc_accounts_x',
 'total_active_cc_accounts_y',
 'new_account_flag',
 'temp',
 '_merge']]

In [161]:
df_enq1=query(f"""
SELECT customer_no, inquiry_date, subscriber_name, inquiry_type, 
DATE_DIFF('day', DATE(inquiry_date), DATE(creation_date)) AS diff
FROM fpl_bureau.score_cibil_report_inquiry
WHERE customer_no={customer_no}
AND year={y} AND month={m}
--AND DATE(inquiry_date)>=DATE(creation_date-interval '32' day)
ORDER BY inquiry_date DESC
""")

In [162]:
df_enq2=query(f"""
SELECT customer_no, inquiry_date, subscriber_name, inquiry_type,
DATE_DIFF('day', DATE(inquiry_date), DATE(creation_date)) AS diff
FROM fpl_bureau.score_cibil_report_inquiry
WHERE customer_no={customer_no}
AND year={y_y} AND month={m_y}
--AND DATE(inquiry_date)>=DATE(creation_date-interval '32' day)
ORDER BY inquiry_date DESC
""")

In [163]:
# Find rows in df2 that are not in df1
df_enq = pd.concat([df_enq2, df_enq1]).drop_duplicates(subset=['customer_no', 'inquiry_date', 'subscriber_name', 'inquiry_type'], keep=False)

df_enq=df_enq[df_enq['diff']<90]
df_enq

,customer_no,inquiry_date,subscriber_name,inquiry_type,diff
0,2565455133978658509,2025-02-11,NAVI,5,24


In [164]:
product_scrub1=fplathena.get_query_result(
"""select * from fpl_ds_onboarding_l2.cibil_bureau_accounttype_mapping """)


df_enq= pd.merge(df_enq,product_scrub1[['acct_type_symbol','loan_type']],left_on='inquiry_type',right_on='acct_type_symbol',how='left')

df_enq

,customer_no,inquiry_date,subscriber_name,inquiry_type,diff,acct_type_symbol,loan_type
0,2565455133978658509,2025-02-11,NAVI,5,24,5.0,Personal Loan


In [165]:
import pandas as pd


# Convert to desired format
df_enq['inquiry_date'] = pd.to_datetime(df_enq['inquiry_date'])
df_enq['inquiry_date'] = df_enq['inquiry_date'].dt.strftime('%d %b, %Y') 
df_enq['inquiry_date']=df_enq['inquiry_date'].astype(str)
df_enq

,customer_no,inquiry_date,subscriber_name,inquiry_type,diff,acct_type_symbol,loan_type
0,2565455133978658509,"11 Feb, 2025",NAVI,5,24,5.0,Personal Loan


In [166]:
final_result['new_enquiries_cnt']=df_enq.shape[0]

In [167]:
final_result['bureau_vintage_y']=final_result['acc_age_y'].max()

In [168]:
final_result1=final_result[['customer_no', 'account_number_x', 'account_number_y', 'account_type_symbol_x', 'account_type_symbol_y',
                            'creditor_name', 'risk_score_x',
                           'risk_score_y', 'risk_score_diff', 'priority_3_x', 'priority_3_y', 'loan_type_x', 'loan_type_y', 'category_x', 'category_y',
                            'secured_unsecured_y', 'acc_age_y',
                           'Activity_Flag_x', 'Activity_Flag_y', 'bureau_vintage_y',
                           'acc_no', 'max_dpd_l36m_x', 'max_dpd_l36m_y', 'max_dpd_l36m_diff', 'max_dpd_l24m_x', 'max_dpd_l24m_y', 'max_dpd_l24m_diff',
                           'max_dpd_l18m_x', 'max_dpd_l18m_y', 'max_dpd_l18m_diff', 'max_dpd_l12m_x', 'max_dpd_l12m_y', 'max_dpd_l12m_diff',
                           'max_dpd_l6m_x', 'max_dpd_l6m_y', 'max_dpd_l6m_diff', 'max_dpd_l3m_x', 'max_dpd_l3m_y', 'max_dpd_l3m_diff',
                           'max_dpd_l2m_x', 'max_dpd_l2m_y', 'max_dpd_l2m_diff', 'max_dpd_cm_x', 'max_dpd_cm_y', 'max_dpd_cm_diff',
                           'latest_payment_dpd_status_x', 'latest_payment_dpd_status_y', 'latest_payment_dpd_status_diff', 'latest_payment_dpd_status2_x',
                           'latest_payment_dpd_status2_y', 'latest_payment_dpd_status2_diff', 'latest_payment_dpd_status_y_adjusted', 'max_delinquency_latest_2_months_y',
                           'lim_disbursed_x', 'lim_disbursed_y', 'utilisation_x', 'utilisation_y', 'utilisation_diff', 'utilisation_percent_diff',
                           'total_lim_disbursed_x', 'total_lim_disbursed_y', 'overall_utilisation_x', 'overall_utilisation_y', 'overall_utilisation_diff', 'overall_utilisation_percent_diff',
                           'total_cc_lim_disbursed_x', 'total_cc_lim_disbursed_y', 'overall_cc_utilisation_x', 'overall_cc_utilisation_y', 'overall_cc_utilisation_diff', 'overall_cc_utilisation_percent_diff', 
                           'max_delinquency_detected', 'min_delinquency_detected', 'coalesced_priority', 'coalesced_loan_type', 'rn', 'total_active_accounts_x',
                            'coalesced_open_date',
                           'total_active_accounts_y', 'total_active_cc_accounts_x','total_active_cc_accounts_y', 'new_account_flag', 'new_enquiries_cnt','_merge', 'temp']]



In [169]:
final_result1['secured_unsecured_y']=np.where(final_result1['secured_unsecured_y']=='NA', '2. Unsecured', final_result1['secured_unsecured_y'])
final_result1['lender_type']=np.where(final_result1['category_y'].isin(['PUB_1_G', 'PUB_2', 'PUB_1_B']), 'Public sector',
                                                np.where(final_result1['category_y'].isin(['PVT_1', 'PVT_2']), 'Private sector',
                                                                            np.where(final_result1['category_y'].isin(['NBFC_1_G', 'NBFC_2', 'NBFC_1_B']), 'NBFC',
                                                                                                        np.where(final_result1['category_y'].isin(['CORB']), 'Corporate bank',
                                                                                                                                    np.where(final_result1['category_y'].isin(['FOR']), 'Foreign bank', 'Others')))))


/tmp/ipykernel_16517/2245890024.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result1['secured_unsecured_y']=np.where(final_result1['secured_unsecured_y']=='NA', '2. Unsecured', final_result1['secured_unsecured_y'])
/tmp/ipykernel_16517/2245890024.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result1['lender_type']=np.where(final_result1['category_y'].isin(['PUB_1_G', 'PUB_2', 'PUB_1_B']), 'Public sector',


In [170]:
if final_result1['risk_score_diff'].max()<0:
    drop=-1*final_result1['risk_score_diff'][0]
    print(f"Bad:- User's score has reduced between 2 months by {drop} points.")
    print("\n")
    
if final_result1['risk_score_diff'].max()>0:
    increase=final_result1['risk_score_diff'][0]
    print(f"Good:- User's score has increased between 2 months by {increase} points.")
    print("\n")
print("-----Credit Mix-------")
#credit_mix
if 'secured_unsecured_y' in final_result1.columns:
    import pandas as pd
    import matplotlib.pyplot as plt

    # Count the total number of accounts
    total_loans = len(final_result1[final_result1['account_number_y']!='NA'])

    # Count the total number of active accounts
    total_active_loans = len(final_result1[final_result1['Activity_Flag_y'] == 1])

    # Print the summary message
    print(f"\nUser has a total of {total_loans} accounts, of which {total_active_loans} are active currently.\n")

    # Count the number of active secured and unsecured loans
    active_loan_counts = final_result1[final_result1['Activity_Flag_y'] == 1]['secured_unsecured_y'].value_counts()

    print("Details of active products:")
    print(f"Number of active secured products: {active_loan_counts.get('1. Secured', 0)}")
    print(f"Number of active unsecured products: {active_loan_counts.get('2. Unsecured', 0)}")

    # Optional: Create a pie chart (uncomment to visualize)
    # plt.figure(figsize=(4, 4))
    # active_loan_counts.plot.pie(
    #     autopct='%1.1f%%',
    #     startangle=90,
    #     explode=(0.05, 0),  # Slightly separate the first slice
    #     labels=['Secured', 'Unsecured']
    # )
    # plt.title('Distribution of Active Secured and Unsecured Products')
    # plt.ylabel('')  # Remove the default ylabel
    # plt.show()

if 'lender_type' in final_result1.columns:
    import pandas as pd
    import matplotlib.pyplot as plt

    # Count the number of accounts by lender type
    loan_counts = final_result1[final_result1['Activity_Flag_y'] == 1]['lender_type'].value_counts()

    # Print the counts
    #print("\n")
    print("Number of Public sector bank issued accounts:", loan_counts.get('Public sector', 0))
    print("Number of Private sector bank issued accounts:", loan_counts.get('Private sector', 0))
    print("Number of NBFC issued accounts:", loan_counts.get('NBFC', 0))
    print("Number of Corporate bank issued accounts:", loan_counts.get('Corporate bank', 0))
    print("Number of Foreign bank issued accounts:", loan_counts.get('Foreign bank', 0))

if 'new_account_flag' in final_result1.columns:
    # Filter only users with delinquency
    new_accounts = final_result1[(final_result1['new_account_flag']==1) & (final_result1['rn']!=1) ]

    # Step 2: Aggregate delinquent accounts and days by user
    summary = new_accounts.groupby('customer_no').agg(
        accounts=('account_number_y', list),
        product_types=('loan_type_y', list),
        names=('creditor_name', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "accounts": row['accounts'],
            "product_types": row['product_types'],
            "names": row['names']
        },
        axis=1
    )

    # Drop the intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        product_types = row['details']['product_types']
        accounts = row['details']['accounts']

        # Format the accounts and days into a readable message
        new_accounts = ', '.join(
            [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
        )
        print("\n")
        print(f"Bad:- User {user_id} has opened new following accounts: {new_accounts}.")
if 'account_type_symbol_y' in final_result1.columns and 'account_type_symbol_x' in final_result1.columns:
    # Filter only users with delinquency
    reporting_errors = final_result1[(final_result1['account_type_symbol_y']!=final_result1['account_type_symbol_x']) & (final_result1['Activity_Flag_y']== 1) & (final_result1['Activity_Flag_x']== 1)]

    # Step 2: Aggregate account type changes and creditor names by user
    summary = reporting_errors.groupby('customer_no').agg(
        initials=('account_type_symbol_x', list),
        finals=('account_type_symbol_y', list),
        names=('creditor_name', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "initials": row['initials'],
            "finals": row['finals'],
            "names": row['names']
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        initials = row['details']['initials']
        finals = row['details']['finals']

        # Format the accounts and days into a readable message
        reporting_errors = ', '.join(
            [f"{name} {initial} {final}" for name, initial, final in zip(names, initials, finals)]
        )
        print("\n")
        print(f"Bad:- User {user_id}'s following accounts were reported wrongly between two months: {reporting_errors}.")
        
if 'account_number_y' in final_result1.columns and 'account_number_x' in final_result1.columns:  # Check if df_increase is defined
        # Filter only users with delinquency
        fixed_reporting = final_result1[(final_result1['_merge']=='left_only')]

        # Step 2: Aggregate delinquent accounts and days by user
        summary = fixed_reporting.groupby('customer_no').agg(
            accounts=('account_number_x', list),
            product_types=('coalesced_loan_type', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names":row['names']
            },
            axis=1
        )

        # Drop the intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            product_types = row['details']['product_types']
            accounts = row['details']['accounts']
            names=row['details']['names']

            # Format the accounts and days into a readable message
            fixed_reporting = ', '.join(
                [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
            )
            print("\n")
            print(f"Good:- User {user_id}'s following accounts were removed from their report: {fixed_reporting}.")
if 'account_number_y' in final_result1.columns and 'account_number_x' in final_result1.columns:
        # Filter only users with delinquency
        new_accounts = final_result1[(final_result1['new_account_flag']==1) & (final_result1['rn']==1)]


        # Step 2: Aggregate account closure data by user
        summary = new_accounts.groupby('customer_no').agg(
            accounts=('account_number_y', list),
            product_types=('loan_type_y', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names": row['names']
            },
            axis=1
        )

        # Drop intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            names=row['details']['names']
            product_types=row['details']['product_types']
            accounts = row['details']['accounts']

            # Format the accounts and days into a readable message
            new_accounts = ', '.join(
                [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
            )
            print("\n")
            print(f"Good:- User {user_id} has opened new following accounts: {new_accounts}.")
if 'account_number_y' in final_result1.columns and 'account_number_x' in final_result1.columns:
        # Filter only users with delinquency
        new_accounts_after_dormancy = final_result1[(final_result1['new_account_flag']==1) & (final_result1['total_active_accounts_y']>=1) & (final_result['total_active_accounts_x']==0)]


        # Step 2: Aggregate account closure data by user
        summary = new_accounts_after_dormancy.groupby('customer_no').agg(
            accounts=('account_number_y', list),
            product_types=('loan_type_y', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names": row['names']
            },
            axis=1
        )

        # Drop intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            names=row['details']['names']
            product_types=row['details']['product_types']
            accounts = row['details']['accounts']

            # Format the accounts and days into a readable message
            new_accounts_after_dormancy = ', '.join(
                [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
            )
            print("\n")
            print(f"Good:- User {user_id} has opened new following accounts after dormancy: {new_accounts_after_dormancy}.")
print("\n")
print("-----Credit Age------")
if 'acc_age_y' in final_result1.columns:
    import pandas as pd
    import matplotlib.pyplot as plt

    # Filter only users with delinquency
    final_result2=final_result1[final_result1['account_number_y']!='NA']
    first_account = final_result2[final_result2['acc_age_y'] == final_result2['acc_age_y'].max()]
    #print("\n")
    bureau_vintage=first_account.iloc[0]['acc_age_y']
    print(f"User is {bureau_vintage} months old on bureau.")

    # Step 2: Aggregate delinquent accounts and days by user
    summary = first_account.groupby('customer_no').agg(
        accounts=('account_number_y', list),
        names=('creditor_name', list),
        product_types=('loan_type_y', list),
        open_dates=('coalesced_open_date', list),
        months=('acc_age_y', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "accounts": row['accounts'],
            "names": row['names'],
            "product_types": row['product_types'],
            "open_dates": row['open_dates'],
            "months": row['months']
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        accounts = row['details']['accounts']
        open_dates = row['details']['open_dates']
        product_types = row['details']['product_types']
        months=row['details']['months']

        # Format the accounts and days into a readable message
        first_account = ', '.join(
            [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
        )
        first_account1 = f"{open_dates}"
        first_account2 = f"{months}"
        
        #print("\n")
        print(f"User {user_id}'s oldest account was: {first_account} which was opened on :{first_account1} and is {first_account2} months old.")
if 'acc_age_y' in final_result1.columns:
    import pandas as pd
    import matplotlib.pyplot as plt

    # Filter only users with delinquency
    final_result2=final_result1[final_result1['account_number_y']!='NA']
    latest_account = final_result2[final_result2['acc_age_y'] == final_result2['acc_age_y'].min()]

    # Step 2: Aggregate delinquent accounts and days by user
    summary = latest_account.groupby('customer_no').agg(
        accounts=('account_number_y', list),
        names=('creditor_name', list),
        product_types=('loan_type_y', list),
        open_dates=('coalesced_open_date', list),
        months=('acc_age_y', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "accounts": row['accounts'],
            "names": row['names'],
            "product_types": row['product_types'],
            "open_dates": row['open_dates'],
            "months": row['months']
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        accounts = row['details']['accounts']
        open_dates = row['details']['open_dates']
        product_types = row['details']['product_types']
        months=row['details']['months']

        # Format the accounts and days into a readable message
        latest_account = ', '.join(
            [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
        )
        latest_account1 = f"{open_dates}"
        latest_account2 = f"{months}"
        
        #print("\n")
        print(f"User {user_id}'s most recent account is: {latest_account} which was opened on :{latest_account1} and is {latest_account2} months old.")
print("\n")
print("------Payment behaviour-------")
if 'max_dpd_l36m_y' in final_result1.columns:
    # Filter only users with delinquency
    delinquent_accounts = final_result1[final_result1['max_dpd_l36m_y']> 0]
    delinquent_accounts_cnt=delinquent_accounts.shape[0]
    #print("\n")
    if delinquent_accounts_cnt==0:
        print("User has never gone delinquent in the past.")
    else:
        print(f"User has gone delinquent on {delinquent_accounts_cnt} accounts in the past.")
    highest_delinquency=delinquent_accounts[delinquent_accounts['max_dpd_l36m_y'] == delinquent_accounts['max_dpd_l36m_y'].max()]
     # Step 2: Aggregate delinquent accounts and days by user
    summary = highest_delinquency.groupby('customer_no').agg(
        accounts=('account_number_y', list),
        names=('creditor_name', list),
        product_types=('loan_type_y', list),
        days=('max_dpd_l36m_y', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "accounts": row['accounts'],
            "names": row['names'],
            "product_types": row['product_types'],
            "days": row['days'],
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        accounts = row['details']['accounts']
        product_types = row['details']['product_types']
        days = row['details']['days']

        # Format the accounts and days into a readable message
        delinquent_days = ', '.join(
            [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
        )
        delinquent_days1 = f"{days}"
        
        #print("\n")
        print(f"User {user_id}'s highest delinquency was : {delinquent_days} where user had delinquent by :{delinquent_days1} days")        
if 'max_delinquency_detected' in final_result1.columns:
    # Filter only users with delinquency
    delinquent_accounts = final_result1[((final_result1['latest_payment_dpd_status_y'] > 0) & (final_result1['Activity_Flag_y']== 1))]
    
    # Step 2: Aggregate delinquent accounts and days by user
    summary = delinquent_accounts.groupby('customer_no').agg(
        accounts=('account_number_y', list),
        product_types=('loan_type_y', list),
        names=('creditor_name', list),
        days=('temp', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "accounts": row['accounts'],
            "product_types": row['product_types'],
            "names": row['names'],
            "days": row['days']
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        accounts = row['details']['accounts']
        days = row['details']['days']
        product_types = row['details']['product_types']

        # Format the accounts and days into a readable message
        delinquent_summary = ', '.join(
            [f"{name} {product_type} {account} by ({day} days)" for name, product_type, account, day in zip(names, product_types, accounts, days)]
        )
        print("\n")
        print(f"Bad:- User {user_id} is delinquent on accounts: {delinquent_summary}.")

if 'max_delinquency_detected' in final_result1.columns and 'max_dpd_l3m_x' in final_result1.columns:
    # Filter only users with delinquency
    new_delinquent_accounts = final_result1[((final_result1['temp'] > 0) & (final_result1['Activity_Flag_y']== 1) & (final_result1['max_dpd_l2m_x']<=0) & (final_result1['max_dpd_l3m_x']<=0) & (final_result1['max_dpd_l2m_y'] > 0))]

    # Step 2: Aggregate delinquent accounts and days by user
    summary = new_delinquent_accounts.groupby('customer_no').agg(
        accounts=('account_number_y', list),
        names=('creditor_name', list),
        product_types=('loan_type_y', list),
        days=('temp', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "accounts": row['accounts'],
            "names": row['names'],
            "product_types": row['product_types'],
            "days": row['days']
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        accounts = row['details']['accounts']
        days = row['details']['days']
        product_types = row['details']['product_types']

        # Format the accounts and days into a readable message
        new_delinquent_accounts = ', '.join(
            [f"{name} {product_type} {account} ({day} days)" for name, product_type, account, day in zip(names, product_types, accounts, days)]
        )
        print("\n")
        print(f"Bad:- User {user_id} has become freshly delinquent on accounts: {new_delinquent_accounts}.")
if 'min_delinquency_detected' in final_result1.columns:
        # Filter only users with delinquency
        delinquency1_good = final_result1[((final_result1['latest_payment_dpd_status_diff']<-1)) & (final_result1['Activity_Flag_x']== 1) & (final_result1['max_dpd_l2m_x']> 0)]

        # Step 2: Aggregate delinquent accounts and days by user
        summary = delinquency1_good.groupby('customer_no').agg(
            days= ('latest_payment_dpd_status_diff', list),
            accounts=('account_number_y', list),
            product_types=('loan_type_y', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "days":row['days'],
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names": row['names']
            },
            axis=1
        )

        # Drop the intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            days=row['details']['days']
            names = row['details']['names']
            product_types = row['details']['product_types']
            accounts = row['details']['accounts']

            # Format the accounts and days into a readable message
            delinquency1_good = ', '.join(
                [f"{name} {product_type} {account} by ({day} days)" for name, product_type, account, day in zip(names, product_types, accounts, days)]
            )
            print("\n")
            print(f"Good:- User {user_id}'s delinquency has reduced in the following accounts: {delinquency1_good}.")

if 'min_delinquency_detected' in final_result1.columns and 'max_dpd_l3m_y' in final_result1.columns:
        # Filter only users with delinquency
        delinquency2_good = final_result1[(final_result1['latest_payment_dpd_status_diff']<-1) & (final_result1['latest_payment_dpd_status_y']==0) & (final_result1['max_dpd_l2m_x']> 0) & (final_result1['max_dpd_l3m_x']> 0) & (final_result1['Activity_Flag_x']== 1)]

        # Step 2: Aggregate delinquent accounts and days by user

        summary = delinquency2_good.groupby('customer_no').agg(
            days=('min_delinquency_detected', list),
            accounts=('account_number_x', list),
            product_types=('loan_type_y', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "days": row['days'],
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names": row['names']
            },
            axis=1
        )

        # Drop intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            names = row['details']['names']
            days = row['details']['days']
            product_types = row['details']['product_types']
            accounts = row['details']['accounts']

            # Format the accounts and days into a readable message
            delinquency2_good = ', '.join(
                [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
            )
            print("\n")
            print(f"Good:- User {user_id} is no more delinquent on the following accounts: {delinquency2_good}.")
print("\n")
print("------Utilisation profile------")
if 'lim_disbursed_y' in final_result1.columns:
    # Filter only users with delinquency
    highest_sanctioned = final_result1[final_result1['lim_disbursed_y']==final_result1['lim_disbursed_y'].max()]
    total_lim_disbursed=highest_sanctioned.iloc[0]['total_lim_disbursed_y']
     # Step 2: Aggregate delinquent accounts and days by user
    #print("\n")
    print(f"The overall sanctioned amount to the user currently is {total_lim_disbursed}")
    summary = highest_sanctioned.groupby('customer_no').agg(
        accounts=('account_number_y', list),
        names=('creditor_name', list),
        product_types=('loan_type_y', list),
        amounts=('lim_disbursed_y', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "accounts": row['accounts'],
            "names": row['names'],
            "product_types": row['product_types'],
            "amounts": row['amounts'],
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        accounts = row['details']['accounts']
        product_types = row['details']['product_types']
        amounts = row['details']['amounts']

        # Format the accounts and days into a readable message
        highest_sanctioned1 = ', '.join(
            [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
        )
        highest_sanctioned2 = f"{amounts}"
        
        #print("\n")
        print(f"User {user_id}'s highest amount sanctioned is on : {highest_sanctioned1} in which :{highest_sanctioned2} rupees have been sanctioned to the user.")
    #print("\n")
    overall_utilisation=highest_sanctioned.iloc[0]['overall_utilisation_y']*100
    print(f"User's overall utilisation is {overall_utilisation} percent.")
    if highest_sanctioned.iloc[0]['total_active_cc_accounts_y']>0:
        cc_utilisation=highest_sanctioned.iloc[0]['overall_cc_utilisation_y']*100
        print(f"User's overall utilisation on credit cards is {cc_utilisation} percent.") 
if 'utilisation_y' in final_result1.columns and final_result1['utilisation_y'].max()<=0:
        print("\n")
        print("Bad:- User has become dormant and has zero overall credit utilization.")
if 'overall_utilisation_percent_diff' in final_result1.columns:
#                 print("found")
        print("\n")
        overall_utilisation_change_percent=final_result1['overall_utilisation_percent_diff'].max()*100
        cc_utilisation_change_percent=final_result1['overall_cc_utilisation_percent_diff'].max()*100
        
        if overall_utilisation_change_percent>0 and cc_utilisation_change_percent>0:
            print(f"Bad:- User's overall utilisation has increased by {overall_utilisation_change_percent} percentage points and cc utilisation has increased by {cc_utilisation_change_percent} percentage points.")
        elif overall_utilisation_change_percent>0 and cc_utilisation_change_percent<=0:
            print(f"Bad:- User's overall utilisation has increased by {overall_utilisation_change_percent} percentage points.")
        elif overall_utilisation_change_percent<=0 and cc_utilisation_change_percent>0:
            print(f"Bad:- User's cc utilisation has increased by {cc_utilisation_change_percent} percentage points.")
if 'utilisation_percent_diff' in final_result1.columns:
    # Filter only users with delinquency
    utilisation_percent_diff_accounts = final_result1[(final_result1['utilisation_percent_diff'] > 0.5) & (final_result1['Activity_Flag_y']== 1)]

    # Step 2: Aggregate delinquent accounts and increases by user
    summary = utilisation_percent_diff_accounts.groupby('customer_no').agg(
        accounts=('account_number_x', list),
        names=('creditor_name', list),
        product_types=('loan_type_y', list),
        increases=('utilisation_percent_diff', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "accounts": row['accounts'],
            "names": row['names'],
            "product_types": row['product_types'],
            "increases": row['increases']
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        accounts = row['details']['accounts']
        product_types = row['details']['product_types']
        increases = row['details']['increases']

        # Format the accounts and days into a readable message
        utilisation_percent_diff_accounts = ', '.join(
            [f"{name} {product_type} {account} ({increase*100} percent)" for name, product_type, account, increase in zip(names, product_types, accounts, increases)]
        )
        print("\n")
        print(f"Bad:- User {user_id} has increased utilisation on following accounts by: {utilisation_percent_diff_accounts}.")

if 'overall_cc_utilisation_y' in final_result1.columns:
        print("\n")
        utilisation_all_y=final_result1['overall_utilisation_y'].max()*100
        utilisation_cc_y=final_result1['overall_cc_utilisation_y'].max()*100
        if utilisation_all_y>=30 and utilisation_cc_y>=30:
            print(f"Bad:- User's overall utilisation is {utilisation_all_y} percent and cc utilisation is {utilisation_cc_y} percent.")
        elif utilisation_all_y>=30 and utilisation_cc_y<30:
            print(f"Bad:- User's overall utilisation is {utilisation_all_y} percent.")
        elif utilisation_all_y<30 and utilisation_cc_y>=30:
            print(f"Bad:- User's cc utilisation is {utilisation_cc_y} percent.")
if 'utilisation_y' in final_result1.columns:
    # Filter only users with delinquency
    utilisation_y = final_result1[(final_result1['utilisation_y'] >= 0.3) & (final_result1['Activity_Flag_y']== 1)]

    # Step 2: Aggregate accounts, names, product types, and utilisations by user
    summary = utilisation_y.groupby('customer_no').agg(
        accounts=('account_number_x', list),
        names=('creditor_name', list),
        product_types=('loan_type_y', list),
        utilisations=('utilisation_y', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "accounts": row['accounts'],
            "names": row['names'],
            "product_types": row['product_types'],
            "utilisations": row['utilisations']
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        accounts = row['details']['accounts']
        product_types = row['details']['product_types']
        utilisations = row['details']['utilisations']

        # Format the accounts and days into a readable message
        utilisation_y = ', '.join(
            [f"{name} {product_type} {account} ({utilisation*100} percent)" for name, product_type, account, utilisation in zip(names, product_types, accounts, utilisations)]
        )
        print("\n")
        print(f"Bad:- User {user_id} has higher utilisation(>30%) in following accounts: {utilisation_y}.")
if 'overall_utilisation_percent_diff' in final_result1.columns:  # Check if df_increase is defined
    print("\n")
    overall_utilisation_change_percent=final_result1['overall_utilisation_percent_diff'].max()*100
    cc_utilisation_change_percent=final_result1['overall_cc_utilisation_percent_diff'].max()*100
    if overall_utilisation_change_percent<0 and cc_utilisation_change_percent<0:
        print(f"Good:- User's overall utilisation has seen a percentage decrease by {overall_utilisation_change_percent} percentage points and cc utilisation has seen a percentage decrease by {cc_utilisation_change_percent} percentage points.")
    elif overall_utilisation_change_percent<0 and cc_utilisation_change_percent>=0:
        print(f"Good:- User's overall utilisation has seen a percentage decrease by {overall_utilisation_change_percent} percentage points.")
    elif overall_utilisation_change_percent>=0 and cc_utilisation_change_percent<0:
        print(f"Good:- User's cc utilisation has seen a percentage decrease by {cc_utilisation_change_percent} percentage points.")
if 'utilisation_percent_diff' in final_result1.columns:
        # Filter only users with delinquency
        utilisation1_good = final_result1[(final_result1['utilisation_percent_diff']<-0.1) & (final_result1['Activity_Flag_x']== 1)]

       # Step 2: Aggregate utilisation data by user
        summary = utilisation1_good.groupby('customer_no').agg(
            utilisation_points=('utilisation_percent_diff', list),
            accounts=('account_number_x', list),
            product_types=('loan_type_y', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "utilisation_points": row['utilisation_points'],
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names": row['names']
            },
            axis=1
        )

        # Drop intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            names = row['details']['names']
            utilisation_points = row['details']['utilisation_points']
            product_types = row['details']['product_types']
            accounts = row['details']['accounts']

            # Format the accounts and days into a readable message
            utilisation1_good = ', '.join(
                [f"{name} {product_type} {account} ({utilisation_point*100} percent)" for name, product_type, account, utilisation_point in zip(names, product_types, accounts, utilisation_points)]
            )
            print("\n")
            print(f"Good:- User {user_id} has reduced their utilisation in the following accounts: {utilisation1_good}.")
if 'overall_cc_utilisation_y' in final_result1.columns:
    print("\n")
    utilisation_all_y=final_result1['overall_utilisation_y'].max()*100
    utilisation_cc_y=final_result1['overall_cc_utilisation_y'].max()*100
    if utilisation_all_y<30 and utilisation_cc_y<30:
        print(f"Good:- User's overall utilisation is {utilisation_all_y} percent and cc utilisation is {utilisation_cc_y} percent.")
    elif utilisation_all_y<30 and utilisation_cc_y>=30:
        print(f"Good:- User's overall utilisation is {utilisation_all_y} percent.")
    elif utilisation_all_y>=30 and utilisation_cc_y<30:
        print(f"Good:- User's cc utilisation is {utilisation_cc_y} percent.")
    # Check if df_increase is defined
if 'utilisation_y' in final_result1.columns:
        # Filter only users with delinquency
        utilisation2_good = final_result1[(final_result1['utilisation_y']<0.3) & (final_result1['Activity_Flag_y']== 1)]

        # Step 2: Aggregate utilisation data by user
        summary = utilisation2_good.groupby('customer_no').agg(
            utilisation_points=('utilisation_y', list),
            accounts=('account_number_x', list),
            product_types=('loan_type_y', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "utilisation_points": row['utilisation_points'],
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names": row['names']
            },
            axis=1
        )

        # Drop intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            names = row['details']['names']
            utilisation_points = row['details']['utilisation_points']
            product_types = row['details']['product_types']
            accounts = row['details']['accounts']

            # Format the accounts and days into a readable message
            utilisation2_good = ', '.join(
                [f"{name} {product_type} {account} ({utilisation_point*100} percent)" for name, product_type, account, utilisation_point in zip(names, product_types, accounts, utilisation_points)]
            )
            print("\n")
            print(f"Good:- User {user_id} has utilisation less than 30% in the following accounts: {utilisation2_good}.")        
print("\n")

if 'Activity_Flag_y' in final_result1.columns and 'Activity_Flag_x' in final_result1.columns:
        # Filter only users with delinquency
        account_closed = final_result1[(final_result1['Activity_Flag_y']==0) & (final_result1['Activity_Flag_x']==1)]

        # Step 2: Aggregate account closure data by user
        summary = account_closed.groupby('customer_no').agg(
            accounts=('account_number_x', list),
            product_types=('loan_type_y', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names": row['names']
            },
            axis=1
        )

        # Drop intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            names = row['details']['names']
            product_types = row['details']['product_types']
            accounts = row['details']['accounts']

            # Format the accounts and days into a readable message
            account_closed = ', '.join(
                [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
            )
            print("\n")
            print(f"Good:- User {user_id} has closed the following accounts: {account_closed}.")

print("\n")
# print("------Recent Activity-------")
# if 'closed_l6m_all_y' in df_test.columns and df_test['closed_l6m_all_y'].max()>0:
#     closed_l6m_all_y=df_test['closed_l6m_all_y'].max()
#     #print("\n")
#     print(f"User has closed {closed_l6m_all_y} accounts in the last 6 months.")
# if 'new_acct_l6m_all_y' in df_test.columns and df_test['new_acct_l6m_all_y'].max()>0:
#     new_acct_l6m_all_y=df_test['new_acct_l6m_all_y'].max()
#     #print("\n")
#     print(f"User has opened {new_acct_l6m_all_y} accounts in the last 6 months.")
# if 'enq_l6m_all_y' in df_test.columns and df_test['enq_l6m_all_y'].max()>0:
#     enq_l6m_all_y=df_test['enq_l6m_all_y'].max()
#     #print("\n")
#     print(f"User has done {enq_l6m_all_y} enquiries in the last 6 months.")
if ~df_enq.isnull().all().all():
    # Filter only users with delinquency
    new_enquiries = df_enq

    # Step 2: Aggregate inquiry dates and subscriber names by user
    summary = new_enquiries.groupby('customer_no').agg(
        dates=('inquiry_date', list),
        loan_types=('loan_type', list),
        names=('subscriber_name', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "dates": row['dates'],
            "loan_types": row['loan_types'],
            "names": row['names']
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        loan_types = row['details']['loan_types']
        dates = row['details']['dates']

        # Format the accounts and days into a readable message
        new_enquiries = ', '.join(
            [f"{name} {loan_type} {date} " for name, loan_type, date in zip(names, loan_types, dates)]
        )
        print("\n")
        print(f"Bad:- User {user_id} has done new inquiries with following banks: {new_enquiries}.")
            


-----Credit Mix-------

User has a total of 16 accounts, of which 8 are active currently.

Details of active products:
Number of active secured products: 1
Number of active unsecured products: 7
Number of Public sector bank issued accounts: 0
Number of Private sector bank issued accounts: 5
Number of NBFC issued accounts: 1
Number of Corporate bank issued accounts: 0
Number of Foreign bank issued accounts: 1


-----Credit Age------
User is 79 months old on bureau.
User 2565455133978658509's oldest account was: ICICI BANK  Personal Loan ############6040 which was opened on :['09 Jul, 2018'] and is [79] months old.
User 2565455133978658509's most recent account is: KOTAK BANK  Gold Loan ######1231 which was opened on :['31 Jul, 2024'] and is [7] months old.


------Payment behaviour-------
User has gone delinquent on 9 accounts in the past.
User 2565455133978658509's highest delinquency was : AXIS BANK  Personal Loan ###########1194 where user had delinquent by :[599] days


Bad:- User 2

In [171]:

if final_result1['risk_score_diff'].max()<0:
    drop=-1*final_result1['risk_score_diff'][0]
    print(f"Bad:- User's score has reduced between 2 months by {drop} points.")
    
if final_result1['risk_score_diff'].max()>0:
    increase=final_result1['risk_score_diff'][0]
    print(f"Good:- User's score has increased between 2 months by {increase} points.")

if 'max_delinquency_detected' in final_result1.columns:
    # Filter only users with delinquency
    delinquent_accounts = final_result1[((final_result1['latest_payment_dpd_status_y'] > 0) & (final_result1['Activity_Flag_y']== 1))]
    
    # Step 2: Aggregate delinquent accounts and days by user
    summary = delinquent_accounts.groupby('customer_no').agg(
        accounts=('account_number_y', list),
        product_types=('loan_type_y', list),
        names=('creditor_name', list),
        days=('temp', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "accounts": row['accounts'],
            "product_types": row['product_types'],
            "names": row['names'],
            "days": row['days']
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        accounts = row['details']['accounts']
        days = row['details']['days']
        product_types = row['details']['product_types']

        # Format the accounts and days into a readable message
        delinquent_summary = ', '.join(
            [f"{name} {product_type} {account} by ({day} days)" for name, product_type, account, day in zip(names, product_types, accounts, days)]
        )
        print("\n")
        print(f"Bad:- User {user_id} is delinquent on accounts: {delinquent_summary}.")

if 'max_delinquency_detected' in final_result1.columns and 'max_dpd_l3m_x' in final_result1.columns:
    # Filter only users with delinquency
    new_delinquent_accounts = final_result1[((final_result1['temp'] > 0) & (final_result1['Activity_Flag_y']== 1) & (final_result1['max_dpd_l2m_x']<=0) & (final_result1['max_dpd_l3m_x']<=0) & (final_result1['max_dpd_l2m_y'] > 0))]

    # Step 2: Aggregate delinquent accounts and days by user
    summary = new_delinquent_accounts.groupby('customer_no').agg(
        accounts=('account_number_y', list),
        names=('creditor_name', list),
        product_types=('loan_type_y', list),
        days=('temp', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "accounts": row['accounts'],
            "names": row['names'],
            "product_types": row['product_types'],
            "days": row['days']
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        accounts = row['details']['accounts']
        days = row['details']['days']
        product_types = row['details']['product_types']

        # Format the accounts and days into a readable message
        new_delinquent_accounts = ', '.join(
            [f"{name} {product_type} {account} ({day} days)" for name, product_type, account, day in zip(names, product_types, accounts, days)]
        )
        print("\n")
        print(f"Bad:- User {user_id} has become freshly delinquent on accounts: {new_delinquent_accounts}.")

if 'utilisation_y' in final_result1.columns and final_result1['utilisation_y'].max()<=0:
        print("\n")
        print("Bad:- User has become dormant and has zero overall credit utilization.")
if 'overall_utilisation_percent_diff' in final_result1.columns:
#                 print("found")
        print("\n")
        overall_utilisation_change_percent=final_result1['overall_utilisation_percent_diff'].max()*100
        cc_utilisation_change_percent=final_result1['overall_cc_utilisation_percent_diff'].max()*100
        
        if overall_utilisation_change_percent>0 and cc_utilisation_change_percent>0:
            print(f"Bad:- User's overall utilisation has increased by {overall_utilisation_change_percent} percentage points and cc utilisation has increased by {cc_utilisation_change_percent} percentage points.")
        elif overall_utilisation_change_percent>0 and cc_utilisation_change_percent<=0:
            print(f"Bad:- User's overall utilisation has increased by {overall_utilisation_change_percent} percentage points.")
        elif overall_utilisation_change_percent<=0 and cc_utilisation_change_percent>0:
            print(f"Bad:- User's cc utilisation has increased by {cc_utilisation_change_percent} percentage points.")
if 'utilisation_percent_diff' in final_result1.columns:
    # Filter only users with delinquency
    utilisation_percent_diff_accounts = final_result1[(final_result1['utilisation_percent_diff'] > 0.5) & (final_result1['Activity_Flag_y']== 1)]

    # Step 2: Aggregate delinquent accounts and increases by user
    summary = utilisation_percent_diff_accounts.groupby('customer_no').agg(
        accounts=('account_number_x', list),
        names=('creditor_name', list),
        product_types=('loan_type_y', list),
        increases=('utilisation_percent_diff', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "accounts": row['accounts'],
            "names": row['names'],
            "product_types": row['product_types'],
            "increases": row['increases']
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        accounts = row['details']['accounts']
        product_types = row['details']['product_types']
        increases = row['details']['increases']

        # Format the accounts and days into a readable message
        utilisation_percent_diff_accounts = ', '.join(
            [f"{name} {product_type} {account} ({increase*100} percent)" for name, product_type, account, increase in zip(names, product_types, accounts, increases)]
        )
        print("\n")
        print(f"Bad:- User {user_id} has increased utilisation on following accounts by: {utilisation_percent_diff_accounts}.")

if 'overall_cc_utilisation_y' in final_result1.columns:
        print("\n")
        utilisation_all_y=final_result1['overall_utilisation_y'].max()*100
        utilisation_cc_y=final_result1['overall_cc_utilisation_y'].max()*100
        if utilisation_all_y>=30 and utilisation_cc_y>=30:
            print(f"Bad:- User's overall utilisation is {utilisation_all_y} percent and cc utilisation is {utilisation_cc_y} percent.")
        elif utilisation_all_y>=30 and utilisation_cc_y<30:
            print(f"Bad:- User's overall utilisation is {utilisation_all_y} percent.")
        elif utilisation_all_y<30 and utilisation_cc_y>=30:
            print(f"Bad:- User's cc utilisation is {utilisation_cc_y} percent.")
if 'utilisation_y' in final_result1.columns:
    # Filter only users with delinquency
    utilisation_y = final_result1[(final_result1['utilisation_y'] >= 0.3) & (final_result1['Activity_Flag_y']== 1)]

    # Step 2: Aggregate accounts, names, product types, and utilisations by user
    summary = utilisation_y.groupby('customer_no').agg(
        accounts=('account_number_x', list),
        names=('creditor_name', list),
        product_types=('loan_type_y', list),
        utilisations=('utilisation_y', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "accounts": row['accounts'],
            "names": row['names'],
            "product_types": row['product_types'],
            "utilisations": row['utilisations']
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        accounts = row['details']['accounts']
        product_types = row['details']['product_types']
        utilisations = row['details']['utilisations']

        # Format the accounts and days into a readable message
        utilisation_y = ', '.join(
            [f"{name} {product_type} {account} ({utilisation*100} percent)" for name, product_type, account, utilisation in zip(names, product_types, accounts, utilisations)]
        )
        print("\n")
        print(f"Bad:- User {user_id} has higher utilisation(>30%) in following accounts: {utilisation_y}.")
        
if 'new_account_flag' in final_result1.columns:
    # Filter only users with delinquency
    new_accounts = final_result1[(final_result1['new_account_flag']==1) & (final_result1['rn']!=1) ]

    # Step 2: Aggregate delinquent accounts and days by user
    summary = new_accounts.groupby('customer_no').agg(
        accounts=('account_number_y', list),
        product_types=('loan_type_y', list),
        names=('creditor_name', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "accounts": row['accounts'],
            "product_types": row['product_types'],
            "names": row['names']
        },
        axis=1
    )

    # Drop the intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        product_types = row['details']['product_types']
        accounts = row['details']['accounts']

        # Format the accounts and days into a readable message
        new_accounts = ', '.join(
            [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
        )
        print("\n")
        print(f"Bad:- User {user_id} has opened new following accounts: {new_accounts}.")
if ~df_enq.isnull().all().all():
    # Filter only users with delinquency
    new_enquiries = df_enq

    # Step 2: Aggregate inquiry dates and subscriber names by user
    summary = new_enquiries.groupby('customer_no').agg(
        dates=('inquiry_date', list),
        loan_types=('loan_type', list),
        names=('subscriber_name', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "dates": row['dates'],
            "loan_types": row['loan_types'],
            "names": row['names']
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        loan_types = row['details']['loan_types']
        dates = row['details']['dates']

        # Format the accounts and days into a readable message
        new_enquiries = ', '.join(
            [f"{name} {loan_type} {date} " for name, loan_type, date in zip(names, loan_types, dates)]
        )
        print("\n")
        print(f"Bad:- User {user_id} has done new inquiries with following banks: {new_enquiries}.")

if 'account_type_symbol_y' in final_result1.columns and 'account_type_symbol_x' in final_result1.columns:
    # Filter only users with delinquency
    reporting_errors = final_result1[(final_result1['account_type_symbol_y']!=final_result1['account_type_symbol_x']) & (final_result1['Activity_Flag_y']== 1) & (final_result1['Activity_Flag_x']== 1)]

    # Step 2: Aggregate account type changes and creditor names by user
    summary = reporting_errors.groupby('customer_no').agg(
        initials=('account_type_symbol_x', list),
        finals=('account_type_symbol_y', list),
        names=('creditor_name', list)
    ).reset_index()

    # Convert the aggregated columns into the desired structure
    summary['details'] = summary.apply(
        lambda row: {
            "initials": row['initials'],
            "finals": row['finals'],
            "names": row['names']
        },
        axis=1
    )

    # Drop intermediate columns if not needed
    summary = summary[['customer_no', 'details']]

    # Step 3: Generate user-specific messages
    for _, row in summary.iterrows():
        user_id = row['customer_no']
        names = row['details']['names']
        initials = row['details']['initials']
        finals = row['details']['finals']

        # Format the accounts and days into a readable message
        reporting_errors = ', '.join(
            [f"{name} {initial} {final}" for name, initial, final in zip(names, initials, finals)]
        )
        print("\n")
        print(f"Bad:- User {user_id}'s following accounts were reported wrongly between two months: {reporting_errors}.")

if 'account_number_y' in final_result1.columns and 'account_number_x' in final_result1.columns:  # Check if df_increase is defined
        # Filter only users with delinquency
        fixed_reporting = final_result1[(final_result1['_merge']=='left_only')]

        # Step 2: Aggregate delinquent accounts and days by user
        summary = fixed_reporting.groupby('customer_no').agg(
            accounts=('account_number_x', list),
            product_types=('coalesced_priority', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names":row['names']
            },
            axis=1
        )

        # Drop the intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            product_types = row['details']['product_types']
            accounts = row['details']['accounts']
            names=row['details']['names']

            # Format the accounts and days into a readable message
            fixed_reporting = ', '.join(
                [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
            )
            print("\n")
            print(f"Good:- User {user_id}'s following accounts were removed from their report: {fixed_reporting}.")


if 'min_delinquency_detected' in final_result1.columns:
        # Filter only users with delinquency
        delinquency1_good = final_result1[((final_result1['latest_payment_dpd_status_diff']<-1)) & (final_result1['Activity_Flag_x']== 1) & (final_result1['max_dpd_l2m_x']> 0)]

        # Step 2: Aggregate delinquent accounts and days by user
        summary = delinquency1_good.groupby('customer_no').agg(
            days= ('latest_payment_dpd_status_diff', list),
            accounts=('account_number_y', list),
            product_types=('loan_type_y', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "days":row['days'],
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names": row['names']
            },
            axis=1
        )

        # Drop the intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            days=row['details']['days']
            names = row['details']['names']
            product_types = row['details']['product_types']
            accounts = row['details']['accounts']

            # Format the accounts and days into a readable message
            delinquency1_good = ', '.join(
                [f"{name} {product_type} {account} by ({day} days)" for name, product_type, account, day in zip(names, product_types, accounts, days)]
            )
            print("\n")
            print(f"Good:- User {user_id}'s delinquency has reduced in the following accounts: {delinquency1_good}.")

if 'min_delinquency_detected' in final_result1.columns and 'max_dpd_l3m_y' in final_result1.columns:
        # Filter only users with delinquency
        delinquency2_good = final_result1[(final_result1['latest_payment_dpd_status_diff']<-1) & (final_result1['latest_payment_dpd_status_y']==0) & (final_result1['max_dpd_l2m_x']> 0) & (final_result1['max_dpd_l3m_x']> 0) & (final_result1['Activity_Flag_x']== 1)]

        # Step 2: Aggregate delinquent accounts and days by user

        summary = delinquency2_good.groupby('customer_no').agg(
            days=('min_delinquency_detected', list),
            accounts=('account_number_x', list),
            product_types=('loan_type_y', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "days": row['days'],
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names": row['names']
            },
            axis=1
        )

        # Drop intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            names = row['details']['names']
            days = row['details']['days']
            product_types = row['details']['product_types']
            accounts = row['details']['accounts']

            # Format the accounts and days into a readable message
            delinquency2_good = ', '.join(
                [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
            )
            print("\n")
            print(f"Good:- User {user_id} is no more delinquent on the following accounts: {delinquency2_good}.")
if 'overall_utilisation_percent_diff' in final_result1.columns:  # Check if df_increase is defined
    print("\n")
    overall_utilisation_change_percent=final_result1['overall_utilisation_percent_diff'].max()*100
    cc_utilisation_change_percent=final_result1['overall_cc_utilisation_percent_diff'].max()*100
    if overall_utilisation_change_percent<0 and cc_utilisation_change_percent<0:
        print(f"Good:- User's overall utilisation has seen a percentage decrease by {overall_utilisation_change_percent} percentage points and cc utilisation has seen a percentage decrease by {cc_utilisation_change_percent} percentage points.")
    elif overall_utilisation_change_percent<0 and cc_utilisation_change_percent>=0:
        print(f"Good:- User's overall utilisation has seen a percentage decrease by {overall_utilisation_change_percent} percentage points.")
    elif overall_utilisation_change_percent>=0 and cc_utilisation_change_percent<0:
        print(f"Good:- User's cc utilisation has seen a percentage decrease by {cc_utilisation_change_percent} percentage points.")
if 'utilisation_percent_diff' in final_result1.columns:
        # Filter only users with delinquency
        utilisation1_good = final_result1[(final_result1['utilisation_percent_diff']<-0.1) & (final_result1['Activity_Flag_x']== 1)]

       # Step 2: Aggregate utilisation data by user
        summary = utilisation1_good.groupby('customer_no').agg(
            utilisation_points=('utilisation_percent_diff', list),
            accounts=('account_number_x', list),
            product_types=('loan_type_y', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "utilisation_points": row['utilisation_points'],
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names": row['names']
            },
            axis=1
        )

        # Drop intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            names = row['details']['names']
            utilisation_points = row['details']['utilisation_points']
            product_types = row['details']['product_types']
            accounts = row['details']['accounts']

            # Format the accounts and days into a readable message
            utilisation1_good = ', '.join(
                [f"{name} {product_type} {account} ({utilisation_point*100} percent)" for name, product_type, account, utilisation_point in zip(names, product_types, accounts, utilisation_points)]
            )
            print("\n")
            print(f"Good:- User {user_id} has reduced their utilisation in the following accounts: {utilisation1_good}.")
if 'overall_cc_utilisation_y' in final_result1.columns:
    print("\n")
    utilisation_all_y=final_result1['overall_utilisation_y'].max()*100
    utilisation_cc_y=final_result1['overall_cc_utilisation_y'].max()*100
    if utilisation_all_y<30 and utilisation_cc_y<30:
        print(f"Good:- User's overall utilisation is {utilisation_all_y} percent and cc utilisation is {utilisation_cc_y} percent.")
    elif utilisation_all_y<30 and utilisation_cc_y>=30:
        print(f"Good:- User's overall utilisation is {utilisation_all_y} percent.")
    elif utilisation_all_y>=30 and utilisation_cc_y<30:
        print(f"Good:- User's cc utilisation is {utilisation_cc_y} percent.")
    # Check if df_increase is defined
if 'utilisation_y' in final_result1.columns:
        # Filter only users with delinquency
        utilisation2_good = final_result1[(final_result1['utilisation_y']<0.3) & (final_result1['Activity_Flag_y']== 1)]

        # Step 2: Aggregate utilisation data by user
        summary = utilisation2_good.groupby('customer_no').agg(
            utilisation_points=('utilisation_y', list),
            accounts=('account_number_x', list),
            product_types=('loan_type_y', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "utilisation_points": row['utilisation_points'],
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names": row['names']
            },
            axis=1
        )

        # Drop intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            names = row['details']['names']
            utilisation_points = row['details']['utilisation_points']
            product_types = row['details']['product_types']
            accounts = row['details']['accounts']

            # Format the accounts and days into a readable message
            utilisation2_good = ', '.join(
                [f"{name} {product_type} {account} ({utilisation_point*100} percent)" for name, product_type, account, utilisation_point in zip(names, product_types, accounts, utilisation_points)]
            )
            print("\n")
            print(f"Good:- User {user_id} has utilisation less than 30% in the following accounts: {utilisation2_good}.")
    
if 'Activity_Flag_y' in final_result1.columns and 'Activity_Flag_x' in final_result1.columns:
        # Filter only users with delinquency
        account_closed = final_result1[(final_result1['Activity_Flag_y']==0) & (final_result1['Activity_Flag_x']==1)]

        # Step 2: Aggregate account closure data by user
        summary = account_closed.groupby('customer_no').agg(
            accounts=('account_number_x', list),
            product_types=('loan_type_y', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names": row['names']
            },
            axis=1
        )

        # Drop intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            names = row['details']['names']
            product_types = row['details']['product_types']
            accounts = row['details']['accounts']

            # Format the accounts and days into a readable message
            account_closed = ', '.join(
                [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
            )
            print("\n")
            print(f"Good:- User {user_id} has closed the following accounts: {account_closed}.")
if 'account_number_y' in final_result1.columns and 'account_number_x' in final_result1.columns:
        # Filter only users with delinquency
        new_accounts = final_result1[(final_result1['new_account_flag']==1) & (final_result1['rn']==1)]


        # Step 2: Aggregate account closure data by user
        summary = new_accounts.groupby('customer_no').agg(
            accounts=('account_number_y', list),
            product_types=('loan_type_y', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names": row['names']
            },
            axis=1
        )

        # Drop intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            names=row['details']['names']
            product_types=row['details']['product_types']
            accounts = row['details']['accounts']

            # Format the accounts and days into a readable message
            new_accounts = ', '.join(
                [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
            )
            print("\n")
            print(f"Good:- User {user_id} has opened new following accounts: {new_accounts}.")
if 'account_number_y' in final_result1.columns and 'account_number_x' in final_result1.columns:
        # Filter only users with delinquency
        new_accounts_after_dormancy = final_result1[(final_result1['new_account_flag']==1) & (final_result1['total_active_accounts_y']>=1) & (final_result['total_active_accounts_x']==0)]


        # Step 2: Aggregate account closure data by user
        summary = new_accounts_after_dormancy.groupby('customer_no').agg(
            accounts=('account_number_y', list),
            product_types=('loan_type_y', list),
            names=('creditor_name', list)
        ).reset_index()

        # Convert the aggregated columns into the desired structure
        summary['details'] = summary.apply(
            lambda row: {
                "accounts": row['accounts'],
                "product_types": row['product_types'],
                "names": row['names']
            },
            axis=1
        )

        # Drop intermediate columns if not needed
        summary = summary[['customer_no', 'details']]

        # Step 3: Generate user-specific messages
        for _, row in summary.iterrows():
            user_id = row['customer_no']
            names=row['details']['names']
            product_types=row['details']['product_types']
            accounts = row['details']['accounts']

            # Format the accounts and days into a readable message
            new_accounts_after_dormancy = ', '.join(
                [f"{name} {product_type} {account}" for name, product_type, account in zip(names, product_types, accounts)]
            )
            print("\n")
            print(f"Good:- User {user_id} has opened new following accounts after dormancy: {new_accounts_after_dormancy}.")
        






Bad:- User 2565455133978658509 is delinquent on accounts: SMICC  Personal Loan ##########8943 by (74 days), KOTAK BANK  Personal Loan ########5168 by (11 days), YES BANK  Personal Loan ###########2578 by (74 days), KOTAK BANK  Gold Loan ######1231 by (17 days).




Bad:- User's overall utilisation is 38.35704280883425 percent and cc utilisation is 63.96875 percent.


Bad:- User 2565455133978658509 has higher utilisation(>30%) in following accounts: SBMBKINDIA  Credit Card ###############8509 (63.96875 percent), IDFC FIRST BANK  Personal Loan ####2153 (39.30292864388078 percent), SMICC  Personal Loan ##########8943 (33.77953270417355 percent), KOTAK BANK  Personal Loan ########5168 (39.7134980696918 percent), YES BANK  Personal Loan ###########2578 (49.400618199046335 percent), KOTAK BANK  Gold Loan ######1231 (100.38965517241378 percent).


Bad:- User 2565455133978658509 has done new inquiries with following banks: NAVI  Personal Loan 11 Feb, 2025 .


Good:- User 2565455133978658509'

In [172]:
final_result1[final_result1['Activity_Flag_y']==1]

,customer_no,account_number_x,account_number_y,account_type_symbol_x,account_type_symbol_y,creditor_name,risk_score_x,risk_score_y,risk_score_diff,priority_3_x,priority_3_y,loan_type_x,loan_type_y,category_x,category_y,secured_unsecured_y,acc_age_y,Activity_Flag_x,Activity_Flag_y,bureau_vintage_y,acc_no,max_dpd_l36m_x,max_dpd_l36m_y,max_dpd_l36m_diff,max_dpd_l24m_x,max_dpd_l24m_y,max_dpd_l24m_diff,max_dpd_l18m_x,max_dpd_l18m_y,max_dpd_l18m_diff,max_dpd_l12m_x,max_dpd_l12m_y,max_dpd_l12m_diff,max_dpd_l6m_x,max_dpd_l6m_y,max_dpd_l6m_diff,max_dpd_l3m_x,max_dpd_l3m_y,max_dpd_l3m_diff,max_dpd_l2m_x,max_dpd_l2m_y,max_dpd_l2m_diff,max_dpd_cm_x,max_dpd_cm_y,max_dpd_cm_diff,latest_payment_dpd_status_x,latest_payment_dpd_status_y,latest_payment_dpd_status_diff,latest_payment_dpd_status2_x,latest_payment_dpd_status2_y,latest_payment_dpd_status2_diff,latest_payment_dpd_status_y_adjusted,max_delinquency_latest_2_months_y,lim_disbursed_x,lim_disbursed_y,utilisation_x,utilisation_y,utilisation_diff,utilisation_percent_diff,total_lim_disbursed_x,total_lim_disbursed_y,overall_utilisation_x,overall_utilisation_y,overall_utilisation_diff,overall_utilisation_percent_diff,total_cc_lim_disbursed_x,total_cc_lim_disbursed_y,overall_cc_utilisation_x,overall_cc_utilisation_y,overall_cc_utilisation_diff,overall_cc_utilisation_percent_diff,max_delinquency_detected,min_delinquency_detected,coalesced_priority,coalesced_loan_type,rn,total_active_accounts_x,coalesced_open_date,total_active_accounts_y,total_active_cc_accounts_x,total_active_cc_accounts_y,new_account_flag,new_enquiries_cnt,_merge,temp,lender_type
12,2565455133978658509,###############8509,###############8509,10,10,SBMBKINDIA,650,650,0,01.0 CC,01.0 CC,Credit Card,Credit Card,FOR,FOR,2. Unsecured,43,1,1,79,#8509,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16000,16000,0.948125,0.639687,-0.308438,-0.325313,4704870,4704870,0.40153,0.38357,-0.01796,-0.044728,16000,16000,0.948125,0.639687,-0.308438,-0.325313,0,0,01.0 CC,Credit Card,1,8,"31 Jul, 2021",8,1,1,0,1,both,0,Foreign bank
3,2565455133978658509,####2153,####2153,5,5,IDFC FIRST BANK,650,650,0,03.0 PL,03.0 PL,Personal Loan,Personal Loan,PVT_2,PVT_2,2. Unsecured,62,1,1,79,#2153,30,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,898129,898129,0.406153,0.393029,-0.013124,-0.032313,4704870,4704870,0.40153,0.38357,-0.01796,-0.044728,16000,16000,0.948125,0.639687,-0.308438,-0.325313,0,0,03.0 PL,Personal Loan,4,8,"30 Dec, 2019",8,1,1,0,1,both,0,Private sector
10,2565455133978658509,############8258,############8258,5,5,ICICI BANK,650,650,0,03.0 PL,03.0 PL,Personal Loan,Personal Loan,PVT_1,PVT_1,2. Unsecured,62,1,1,79,#8258,85,85,0,85,55,-30,55,55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,500000,500000,0.204712,0.208130,0.003418,0.016697,4704870,4704870,0.40153,0.38357,-0.01796,-0.044728,16000,16000,0.948125,0.639687,-0.308438,-0.325313,0,-30,03.0 PL,Personal Loan,6,8,"30 Dec, 2019",8,1,1,0,1,both,0,Private sector
13,2565455133978658509,##########8943,##########8943,5,5,SMICC,650,650,0,03.0 PL,03.0 PL,Personal Loan,Personal Loan,NA,NA,2. Unsecured,62,1,1,79,#8943,89,89,0,89,89,0,89,89,0,89,89,0,89,89,0,73,74,1,73,74,1,73,74,1,73,74,1,58,59,1,74,74,749889,749889,0.358806,0.337795,-0.021011,-0.058558,4704870,4704870,0.40153,0.38357,-0.01796,-0.044728,16000,16000,0.948125,0.639687,-0.308438,-0.325313,1,0,03.0 PL,Personal Loan,10,8,"31 Dec, 2019",8,1,1,0,1,both,74,Others
9,2565455133978658509,################7867,################7867,5,5,INCRED FINANCE,650,650,0,03.0 PL,03.0 PL,Personal Loan,Personal Loan,NBFC_2,NBFC_2,2. Unsecured,54,1,1,79,#7867,27,27,0,27,27,0,11,0,-11,11,0,-11,11,0,-11,11,0,-11,11,0,-11,11,0,-11,11,0,-11,0,0,0,0,0,428577,428577,0.340254,0.294414,-0.045840,-0.134723,4704870,4704870,0.40153,0.38357,-0.01796,-0.044728,16000,16000,0.948125,0.639687,-0.308438,-0.325313,0,-11,03.0 PL,Personal Loan,12,8,"19 Aug, 2020",8,1,1,0,1,both,0,NBFC
7,2565455133978658509,########5168,########5168,5,5,KOTAK BANK,6

In [173]:
important_features

,index,abs_shap,non_abs_shap,final_category,rn
0,risk_score_x_shap,1.767853,1.767853,score1,1
1,overdue_cm_diff_shap,0.368895,-0.368895,delinquency2,1
6,total_balance_amount_diff_shap,0.138606,0.138606,utilisation2,1
7,min_gl_bureau_vintage_x_shap,0.136545,0.136545,product_holdings1,1
9,cc_total_balance_amount_diff_shap,0.128685,0.128685,cc_utilisation2,1
11,max_active_dpd_l3m_x_shap,0.122838,-0.122838,delinquency1,1
16,nbr_ever_accounts_diff_shap,0.078337,-0.078337,product_holdings2,1
42,max_bureau_vintage_x_shap,0.032814,0.032814,vintage1,1
57,total_sanctioned_amount_l12m_x_shap,0.021607,-0.021607,utilisation1,1
60,avg_cc_utilisation_x_shap,0.020951,-0.020951,cc_utilisation1,1


In [174]:
#  res = wr.s3.to_parquet(
#         df=df,
#         index=False,
#         path=f"s3://fpl-datalake-global/l2/Anugrah/Onescore/Score_simulator/os_fow_all_models_oot_validation_realtime_testing",
#         dataset=True,
#         database="fpl_ds_analytics",
#         table=f"os_fow_all_models_oot_validation_realtime_testing",
#         mode="overwrite"   
#         )

In [175]:
df_predicted

,customer_no,risk_score_x,predicted_change,predicted_change_prob,predicted_change_wo_1_5,predicted_change_prob_wo_1_5,predicted_direction,predicted_direction_prob,positive_prediction,negative_prediction
0,2565455133978658509,650,0,0.098516,0,0.292156,1,0.739819,18.550289,-10.025165
